In [1]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import pickle
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from diffusers.training_utils import EMAModel
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm
import submodules.data_filter as _df
import diffusion_pipline.data_processing as dproc
import diffusion_pipline.model as md
import submodules.cleaned_file_parser as cfp


In [2]:
# observation and action dimensions corrsponding to
# the output of PushTEnv
# obs_dim = 25
# action_dim = 13

obs_dim = 57
action_dim = 12
# parameters
pred_horizon = 16
obs_horizon = 2
action_horizon = 8
target_fps = 120.0

action_item = ['chisel', 'gripper']
obs_item = ['battery']

# create network object
noise_pred_net = md.ConditionalUnet1D(
    input_dim=action_dim,
    global_cond_dim=obs_dim*obs_horizon
)

# example inputs
noised_action = torch.randn((1, pred_horizon, action_dim))
obs = torch.zeros((1, obs_horizon, obs_dim))
diffusion_iter = torch.zeros((1,))

# the noise prediction network
# takes noisy action, diffusion iteration and observation as input
# predicts the noise added to action
noise = noise_pred_net(
    sample=noised_action,
    timestep=diffusion_iter,
    global_cond=obs.flatten(start_dim=1))

# illustration of removing noise
# the actual noise removal is performed by NoiseScheduler
# and is dependent on the diffusion noise schedule
denoised_action = noised_action - noise

# for this demo, we use DDPMScheduler with 100 diffusion iterations
num_diffusion_iters = 100
noise_scheduler = DDPMScheduler(
    num_train_timesteps=num_diffusion_iters,
    # the choise of beta schedule has big impact on performance
    # we found squared cosine works the best
    beta_schedule='squaredcos_cap_v2',
    # clip output to [-1,1] to improve stability
    clip_sample=True,
    # our network predicts noise (instead of denoised action)
    prediction_type='epsilon'
)

# device transfer
device = torch.device('cuda')
_ = noise_pred_net.to(device)


number of parameters: 6.686209e+07


In [4]:
# create dataset from file
# path_name = "/home/cam/Downloads/Supporting Data - Sheet1.csv"
base_path = "no_sync/data_chisel_task/2-cleaned_interpolation_with_offset/offset_interpolated_training_traj/"

# Load data
dict_of_df_rigid = {}
dict_of_df_rigid_velocity = {}
dict_of_df_marker = {}


for file in os.listdir(base_path):

    if file.endswith(".csv"):
        path_name = base_path + file
        data = cfp.DataParser.from_euler_file(file_path = path_name, target_fps=target_fps, filter=True, window_size=15, polyorder=3)

        marker_data = data.get_marker_Txyz()
        data_time = data.get_time().astype(float)
        data_state_dict = data.get_rigid_TxyzRxyz()

        # use the time and state data to get the velocity data
        data_velocity_state_dict = {}
        data_velocity_dict = {}
        for key in data_state_dict.keys():
            if key != 'battery':
                data_velocity = []
                data_velocity_state = []
                for i in range(0, len(data_time) -1):
                    veloctiy_val = (data_state_dict[key][i + 1] - data_state_dict[key][i]) / (data_time[i + 1] - data_time[i])
                    data_velocity.append(veloctiy_val)
                    data_velocity_state.append(np.concatenate((data_state_dict[key][i], veloctiy_val), axis=0).tolist())
                velocity_state_data = pd.DataFrame(data_velocity_state, columns= [f'{key}_X', f'{key}_Y', f'{key}_Z', f'{key}_x', f'{key}_y', f'{key}_z', f'{key}_Xv', f'{key}_Yv', f'{key}_Zv', f'{key}_xv', f'{key}_yv', f'{key}_zv'])
                filtered_velocity_state = _df.apply_savgol_filter(velocity_state_data, window_size = 15, polyorder = 3, time_frame= False)
                data_velocity_state_dict[key] = filtered_velocity_state.values
                velocity_data = pd.DataFrame(data_velocity, columns= [f'{key}_Xv', f'{key}_Yv', f'{key}_Zv', f'{key}_xv', f'{key}_yv', f'{key}_zv'])
                filtered_velocity = _df.apply_savgol_filter(velocity_data, window_size = 15, polyorder = 3, time_frame= False)
                data_velocity_dict[key] = filtered_velocity.values
            else:
                data_velocity_state_dict[key] = data_state_dict[key]


        dicts = [data_velocity_state_dict, data_velocity_dict, marker_data]
        trimmed_dicts = _df.trim_lists_in_dicts(dicts)

        
        dict_of_df_rigid[file] = trimmed_dicts[0]
        dict_of_df_rigid_velocity[file] = trimmed_dicts[1]
        dict_of_df_marker[file] = trimmed_dicts[2]


item_name = data.rigid_bodies
marker_name = data.markers

if len(dict_of_df_rigid) == len(dict_of_df_marker) == len(dict_of_df_rigid_velocity):

    rigiddataset, index = _df.episode_combiner(dict_of_df_rigid, item_name)
    velocitydataset, _ = _df.episode_combiner(dict_of_df_rigid_velocity, action_item)
    markerdataset, _ = _df.episode_combiner(dict_of_df_marker, marker_name)
    print(index[action_item[0]])
    # print(index[action_item[0]])

#### if you don't want battery info then just do obs_item = None and also do clear all outputs and restart the kernal before that and satrt from the top 


dataset = dproc.TaskStateDataset(Rigiddataset=rigiddataset, Velocitydataset = velocitydataset, Markerdataset= markerdataset, index= index[action_item[0]], 
                                 action_item = action_item, obs_item = obs_item,
                                 marker_item= marker_name,
                                 pred_horizon=pred_horizon,
                                 obs_horizon=obs_horizon,
                                 action_horizon=action_horizon)

torch.save(dataset, 'no_sync/data_chisel_task/visualization_for_datasets/2-cleaned_interpolation_with_offset/cleaned/11_(SV)(SV)SSS/csvs/dataset_train.pth')

# create dataloader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    num_workers=1,
    shuffle=True,
    # accelerate cpu-gpu transfer
    pin_memory=True,
    # don't kill worker process afte each epoch
    persistent_workers=True
)

batch = next(iter(dataloader))
print("batch['obs'].shape:", batch['obs'].shape)
print("batch['action'].shape", batch['action'].shape)

# Save dataloader to a file using pickle
# with open('no_sync/data_chisel_task/visualization_for_datasets/2-cleaned_interpolation_with_offset/pred/11_(SV)(SV)SSS/csvs/dataloader_train.pkl', 'wb') as f:
#     pickle.dump(dataloader, f)

[1090, 2309, 3706, 4606, 5620, 7270, 8210, 9424, 10366, 11041, 12227, 13068, 14115, 15639, 16657, 17672, 18694, 20002, 20831, 21776, 22609, 23786, 24749, 25578, 27025, 28136, 28962, 29743, 30575, 32330, 33534, 34697, 35648, 36888, 37900, 38926, 40317, 41694, 42706, 43845, 45386, 46258, 47223, 48098, 49064, 49903, 50809, 52246, 53907, 54860, 56890, 57935, 58994, 59920, 60771, 62118, 63086, 64314, 65209, 67502, 68586, 69723, 70573, 71503, 72655, 73748, 74616, 75587, 77021, 77941, 78862, 80285, 81148, 82180, 83060, 84011, 84967, 85992, 86759, 87685, 88617, 89641, 90500, 91427, 92324, 93591, 94835, 95614, 96555, 97839, 99021, 100103, 101367, 102511, 103854, 104704, 105669, 106852, 107667, 108688, 109859, 110557, 111535, 113111, 114101, 115048, 115959, 116935, 118264, 119159, 120035, 121242, 122138, 123008, 123987, 125141, 126706, 127616, 128484]
batch['obs'].shape: torch.Size([256, 2, 57])
batch['action'].shape torch.Size([256, 16, 12])


In [3]:
# create dataset from file
# path_name = "/home/cam/Downloads/Supporting Data - Sheet1.csv"
base_path = "no_sync/data_chisel_task/2-cleaned_interpolation_with_offset/offset_interpolated_test_traj/"

# Load data
dict_of_df_rigid_test = {}
dict_of_df_rigid_velocity_test = {}
dict_of_df_marker_test = {}
name = []

for file in os.listdir(base_path):
    if file.endswith(".csv"):
        name.append(file)
        path_name = base_path + file
        data_test = cfp.DataParser.from_euler_file(file_path = path_name, target_fps=target_fps, filter=True, window_size=15, polyorder=3)

        marker_data = data_test.get_marker_Txyz()
        data_time = data_test.get_time().astype(float)
        data_state_dict = data_test.get_rigid_TxyzRxyz()

        # use the time and state data to get the velocity data
        data_velocity_state_dict = {}
        data_velocity_dict = {}
        for key in data_state_dict.keys():
            if key != 'battery':
                data_velocity = []
                data_velocity_state = []
                for i in range(0, len(data_time) -1):
                    veloctiy_val = (data_state_dict[key][i + 1] - data_state_dict[key][i]) / (data_time[i + 1] - data_time[i])
                    data_velocity.append(veloctiy_val)
                    data_velocity_state.append(np.concatenate((data_state_dict[key][i], veloctiy_val), axis=0).tolist())
                velocity_state_data = pd.DataFrame(data_velocity_state, columns= [f'{key}_X', f'{key}_Y', f'{key}_Z', f'{key}_x', f'{key}_y', f'{key}_z', f'{key}_Xv', f'{key}_Yv', f'{key}_Zv', f'{key}_xv', f'{key}_yv', f'{key}_zv'])
                filtered_velocity_state = _df.apply_savgol_filter(velocity_state_data, window_size = 15, polyorder = 3, time_frame= False)
                data_velocity_state_dict[key] = filtered_velocity_state.values
                velocity_data = pd.DataFrame(data_velocity, columns= [f'{key}_Xv', f'{key}_Yv', f'{key}_Zv', f'{key}_xv', f'{key}_yv', f'{key}_zv'])
                filtered_velocity = _df.apply_savgol_filter(velocity_data, window_size = 15, polyorder = 3, time_frame= False)
                data_velocity_dict[key] = filtered_velocity.values
            else:
                data_velocity_state_dict[key] = data_state_dict[key]


        dicts = [data_velocity_state_dict, data_velocity_dict, marker_data]
        trimmed_dicts = _df.trim_lists_in_dicts(dicts)

        
        dict_of_df_rigid_test[file] = trimmed_dicts[0]
        dict_of_df_rigid_velocity_test[file] = trimmed_dicts[1]
        dict_of_df_marker_test[file] = trimmed_dicts[2]


item_name_test = data_test.rigid_bodies
marker_name_test = data_test.markers

if len(dict_of_df_rigid_test) == len(dict_of_df_marker_test) == len(dict_of_df_rigid_velocity_test):

    rigiddataset_test, index_test = _df.episode_combiner(dict_of_df_rigid_test, item_name_test)
    velocitydataset_test, _ = _df.episode_combiner(dict_of_df_rigid_velocity_test, action_item)
    markerdataset_test, _ = _df.episode_combiner(dict_of_df_marker_test, marker_name_test)
    print(index_test[action_item[0]])


indexes = index_test[action_item[0]]
action = []
obs = []
for i in range(indexes[-1]):
    # a = []
    a = np.concatenate([velocitydataset_test[item][i] for item in action_item])
    # print(a)

    b = np.concatenate([rigiddataset_test[item][i] for item in action_item] + [rigiddataset_test[item][i] for item in obs_item] + [markerdataset_test[item][i] for item in marker_name_test])
    # print(b)
    
    action.append(a)
    obs.append(b)
    
# All demonstration episodes are concatinated in the first dimension N
action = np.array(action, dtype=np.float64)
obs = np.array(obs, dtype=np.float64)

# Initialize lists to store segmented data
splits_obs = []
splits_action = []
previous_index = 0

# Iterate through episode_ends and slice action and obs accordingly
for index in indexes:
    splits_obs.append(obs[previous_index:index + 1])  # Include index itself in the slice
    splits_action.append(action[previous_index:index + 1])
    previous_index = index + 1

[844, 1678, 2671, 3526, 4328, 5589, 6376, 7315, 8390, 9282, 10313, 11342, 12405, 13297, 14214, 15464, 16524, 17466, 18442, 19539, 20473, 21319, 22118, 23050]


In [5]:

num_epochs =200
# Exponential Moving Average
# accelerates training and improves stability
# holds a copy of the model weights
ema = EMAModel(
    parameters=noise_pred_net.parameters(),
    power=0.75)

# Standard ADAM optimizer
# Note that EMA parametesr are not optimized
optimizer = torch.optim.AdamW(
    params=noise_pred_net.parameters(),
    lr=1e-4, weight_decay=1e-6)

# Cosine LR schedule with linear warmup
lr_scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=200,
    num_training_steps=len(dataloader) * num_epochs
)


In [10]:
lr_scheduler.state_dict()

{'base_lrs': [0.0001],
 'last_epoch': 0,
 'verbose': False,
 '_step_count': 1,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0],
 'lr_lambdas': [None]}

In [13]:
ema_wights = {'decay': 0.9999, 'min_decay': 0.0, 'optimirzation_step': 99800, 'update_after_step': 0, 'use_ema_warmup': False, 'inv_gamma': 1.0, 'power': 0.75, 'shadow_params': [toch.tensor([[[-0.0257, -0.0194, -0.0476, -0.0182,  0.0025],
         [-0.0223,  0.0277, -0.0176, -0.0001,  0.0041],
         [ 0.0222,  0.0457,  0.0284, -0.0087,  0.0012],
         ...,
         [ 0.0179,  0.0109,  0.0017, -0.0079,  0.0226],
         [ 0.0062, -0.0166, -0.0218, -0.0118, -0.0125],
         [-0.0211, -0.0018,  0.0194,  0.0129, -0.0015]],

        [[ 0.0163, -0.0066, -0.0102,  0.0015,  0.0097],
         [-0.0024,  0.0254, -0.0083,  0.0206,  0.0117],
         [-0.0085, -0.0101, -0.0045, -0.0006,  0.0262],
         ...,
         [-0.0151, -0.0206, -0.0047,  0.0113,  0.0241],
         [-0.0017,  0.0199,  0.0179, -0.0138, -0.0175],
         [ 0.0046,  0.0192,  0.0134,  0.0031,  0.0257]],

        [[ 0.0008, -0.0021, -0.0072, -0.0026, -0.0137],
         [ 0.0365,  0.0148,  0.0179, -0.0077, -0.0052],
         [ 0.0053,  0.0265,  0.0206,  0.0098,  0.0034],
         ...,
         [ 0.0322,  0.0262,  0.0161,  0.0320, -0.0080],
         [ 0.0152,  0.0044,  0.0092, -0.0135, -0.0006],
         [ 0.0066, -0.0207, -0.0117, -0.0137, -0.0043]],

        ...,

        [[-0.0100,  0.0106,  0.0171, -0.0052,  0.0246],
         [ 0.0107,  0.0217,  0.0157,  0.0165, -0.0090],
         [-0.0250,  0.0018, -0.0056, -0.0155, -0.0194],
         ...,
         [ 0.0051,  0.0176,  0.0258,  0.0084,  0.0190],
         [ 0.0335,  0.0364,  0.0255,  0.0211,  0.0256],
         [ 0.0420,  0.0233,  0.0381,  0.0233,  0.0366]],

        [[ 0.0383,  0.0180,  0.0168,  0.0186,  0.0128],
         [-0.0413, -0.0015, -0.0119, -0.0225,  0.0101],
         [ 0.0047,  0.0087, -0.0249, -0.0165, -0.0240],
         ...,
         [-0.0073, -0.0087, -0.0009, -0.0007,  0.0024],
         [ 0.0159,  0.0231,  0.0189,  0.0261,  0.0517],
         [-0.0225,  0.0007,  0.0258, -0.0068, -0.0316]],

        [[ 0.0046,  0.0161,  0.0139,  0.0058, -0.0023],
         [-0.0196, -0.0121,  0.0050,  0.0163,  0.0148],
         [ 0.0134,  0.0061,  0.0057,  0.0306,  0.0283],
         ...,
         [-0.0161, -0.0010,  0.0170,  0.0108,  0.0127],
         [-0.0120, -0.0048, -0.0060,  0.0088,  0.0118],
         [ 0.0067,  0.0155, -0.0054,  0.0088,  0.0087]]], device='cuda:0'), torch.tensor([-0.0150,  0.0123,  0.0522,  ..., -0.0091,  0.0385,  0.0286],
       device='cuda:0'), torch.tensor([1.0977, 1.0340, 0.9584,  ..., 1.0736, 1.0163, 0.r9569], device='cuda:0'), toch.tensor([-0.0878, -0.0429, -0.0574,  ..., -0.0447, -0.0616, -0.0523],
       device='cuda:0'), torch.tensor([[[ 0.0010,  0.0330,  0.0203,  0.0055, -0.0090],
         [ 0.0083,  0.0380,  0.0210,  0.0131,  0.0229],
         [-0.0451, -0.0132, -0.0280, -0.0161,  0.0175],
         ...,
         [-0.0022, -0.0107, -0.0181, -0.0083,  0.0087],
         [ 0.0065, -0.0019,  0.0035,  0.0071, -0.0067],
         [-0.0086, -0.0029,  0.0229,  0.0165, -0.0048]],

        [[ 0.0039,  0.0116,  0.0062, -0.0063, -0.0071],
         [ 0.0156,  0.0168,  0.0246,  0.0224,  0.0093],
         [ 0.0052,  0.0160, -0.0054, -0.0188, -0.0013],
         ...,
         [-0.0210, -0.0416, -0.0317, -0.0461, -0.0402],
         [ 0.0093, -0.0052,  0.0054,  0.0041,  0.0131],
         [-0.0054, -0.0197,  0.0036, -0.0148,  0.0192]],

        [[ 0.0293,  0.0122,  0.0059,  0.0155,  0.0168],
         [ 0.0160, -0.0184,  0.0005,  0.0091, -0.0208],
         [-0.0142, -0.0263, -0.0142,  0.0017, -0.0170],
         ...,
         [ 0.0038,  0.0045, -0.0029,  0.0203,  0.0128],
         [ 0.0399,  0.0060,  0.0102,  0.0145,  0.0090],
         [ 0.0061,  0.0004,  0.0009,  0.0160,  0.0113]],

        ...,

        [[-0.0063,  0.0119,  0.0186, -0.0064, -0.0125],
         [ 0.0279, -0.0041, -0.0043,  0.0145,  0.0167],
         [ 0.0187,  0.0205,  0.0091,  0.0232,  0.0336],
         ...,
         [-0.0345, -0.0416, -0.0161,  0.0067,  0.0026],
         [-0.0159, -0.0125, -0.0279, -0.0329, -0.0162],
         [ 0.0033, -0.0008, -0.0100,  0.0010, -0.0168]],

        [[ 0.0039, -0.0194, -0.0056, -0.0003, -0.0201],
         [-0.0068,  0.0136,  0.0116,  0.0181, -0.0031],
         [-0.0068,  0.0016,  0.0174,  0.0024,  0.0094],
         ...,
         [ 0.0238,  0.0116, -0.0204, -0.0047,  0.0004],
         [ 0.0195,  0.0022, -0.0195, -0.0125, -0.0221],
         [-0.0030,  0.0020, -0.0135,  0.0105,  0.0077]],

        [[ 0.0070,  0.0051, -0.0064, -0.0338, -0.0205],
         [ 0.0015, -0.0030, -0.0069, -0.0167, -0.0117],
         [-0.0181, -0.0101, -0.0022, -0.0092,  0.0021],
         ...,
         [-0.0037, -0.0048, -0.0005,  0.0314,  0.0099],
         [ 0.0123,  0.0341,  0.0142,  0.0059, -0.0010],
         [-0.0125, -0.0063, -0.0013,  0.0012,  0.0104]]], device='cuda:0'), torch.tensor([ 0.0042, -0.0114,  0.0111,  ..., -0.0080, -0.0083,  0.0115],
       device='cuda:0'), torch.tensor([1.1154, 1.0642, 0.9987,  ..., 1.0415, 1.0692, 1.0217], dervice='cuda:0'), toch.tensor([-0.0981, -0.0830, -0.0859,  ..., -0.0774, -0.0870, -0.0634],
       device='cuda:0'), tensor([[-0.0581, -0.0335, -0.0089,  ...,  0.0802, -0.0649,  0.0187],
        [-0.0053, -0.0266, -0.0230,  ..., -0.0517, -0.0369, -0.0076],
        [-0.0221,  0.0301,  0.0085,  ...,  0.0176, -0.0409, -0.0570],
        ...,
        [-0.0206, -0.0464, -0.0152,  ..., -0.0518, -0.0483, -0.0369],
        [-0.0073, -0.0004,  0.0201,  ..., -0.0201,  0.0024, -0.0251],
        [-0.0174,  0.0389, -0.0419,  ...,  0.0150, -0.0158,  0.0337]],
       device='cuda:0'), tensor([-0.0134,  0.0447, -0.0115,  ...,  0.0200, -0.0295, -0.0191],
       device='cuda:0'), tensor([[[ 1.8075e-02,  2.8184e-02,  3.3582e-02,  2.0567e-02, -5.9600e-03],
         [-2.6379e-02, -5.1976e-03, -1.1443e-02, -1.9092e-02, -1.6005e-02],
         [ 8.0268e-03, -1.5169e-02,  1.4732e-02,  1.4050e-02, -8.0296e-03],
         ...,
         [-9.0662e-03,  8.1207e-03,  5.7761e-03,  1.0731e-02,  1.2055e-02],
         [ 4.6954e-03,  1.3739e-02,  1.3561e-02, -3.4443e-03, -1.9309e-03],
         [ 4.2356e-02,  4.0732e-03, -1.7968e-02, -1.4108e-02,  1.1185e-02]],

        [[ 2.5024e-02,  2.8039e-02,  1.9650e-02,  1.6227e-02,  2.2831e-02],
         [ 5.6003e-03, -4.9406e-03, -1.1715e-03,  1.1222e-03,  1.1550e-02],
         [ 3.3003e-02,  2.9520e-02,  2.9797e-02,  2.3521e-02,  1.5206e-02],
         ...,
         [-5.2469e-03, -1.0885e-02,  1.4823e-02, -2.9962e-04, -2.6456e-03],
         [ 9.9415e-03,  2.3881e-02,  1.6615e-03,  7.7831e-03,  8.5294e-03],
         [ 1.4780e-04, -1.1112e-02, -1.8064e-02, -2.1428e-02, -6.7559e-03]],

        [[-6.6251e-02, -4.6463e-02, -4.9627e-02, -4.2167e-02,  2.1190e-02],
         [ 3.7058e-03, -3.9640e-02, -4.5177e-02,  8.8730e-04,  1.7980e-02],
         [-5.2500e-03,  3.2589e-04, -3.1463e-02, -9.5871e-03,  3.0163e-03],
         ...,
         [ 3.6795e-03, -2.1611e-02, -1.5530e-02, -1.9020e-02, -1.3795e-02],
         [ 1.8629e-02, -5.5085e-03, -7.2778e-03, -2.9213e-03,  1.6040e-03],
         [ 2.0403e-04,  1.4259e-03,  4.7985e-03,  1.5089e-02,  6.5113e-03]],

        ...,

        [[ 3.4013e-03,  1.6655e-02,  3.7900e-02,  1.9150e-02,  6.0010e-03],
         [ 6.2189e-04, -1.0923e-02, -1.0564e-02,  8.1396e-03,  4.6509e-03],
         [-1.7397e-02, -2.4686e-02, -5.6283e-03,  4.2974e-03,  1.3997e-02],
         ...,
         [ 4.4072e-03,  6.9222e-03,  1.6194e-02, -6.0732e-03, -8.7347e-03],
         [-7.6460e-03, -2.0145e-02,  3.2734e-04, -1.7146e-02, -1.6762e-02],
         [ 7.7556e-03,  1.0294e-02,  7.9527e-03,  4.6968e-03, -3.5614e-03]],

        [[-3.4436e-03,  8.8882e-03, -2.2659e-03, -2.0931e-02, -2.4121e-02],
         [-7.3052e-05, -5.7382e-03, -1.2617e-03,  1.7694e-02,  5.5027e-03],
         [-2.9363e-03, -2.2681e-02, -1.0760e-02, -2.2655e-02,  3.5839e-03],
         ...,
         [ 7.7765e-03, -1.9319e-02, -1.4679e-02, -1.3017e-02,  1.0137e-02],
         [ 2.8342e-02,  1.4247e-02, -6.7195e-03,  8.5901e-03,  9.9584e-03],
         [ 2.1663e-02, -3.9918e-03,  1.1820e-02,  1.3673e-02,  6.1672e-03]],

        [[ 2.0142e-03,  2.2972e-02, -4.7230e-03,  8.5136e-03,  1.5865e-02],
         [-1.3682e-03, -2.0531e-02, -1.3737e-02,  1.6817e-02,  1.3567e-02],
         [ 2.5724e-02,  3.2831e-02,  1.9578e-02, -4.3074e-03, -1.5320e-02],
         ...,
         [ 2.1630e-02,  2.0931e-02,  1.8720e-02,  2.4819e-04, -2.3367e-02],
         [ 1.1607e-02,  6.7099e-03,  1.3150e-02, -4.1457e-04,  1.0398e-02],
         [ 2.1512e-02,  8.1948e-03,  1.7410e-02, -2.8209e-03, -1.7086e-02]]],
       device='cuda:0'), tensor([-0.0267,  0.0108, -0.0079,  ...,  0.0215,  0.0106,  0.0020],
       device='cuda:0'), tensor([1.0942, 0.9404, 1.1026,  ..., 0.9368, 0.9628, 1.0196], device='cuda:0'), tensor([-0.0606, -0.0781, -0.0523,  ..., -0.0676, -0.0583, -0.0070],
       device='cuda:0'), tensor([[[-1.7198e-03, -3.2573e-03, -2.4022e-04, -6.6379e-03, -1.8849e-02],
         [-1.7158e-02, -1.9526e-02, -1.7083e-02, -1.5088e-02, -1.0172e-02],
         [ 1.3900e-02,  1.6912e-02,  2.0821e-02,  3.1257e-02,  1.3655e-02],
         ...,
         [ 3.8035e-02,  8.2086e-03,  1.7190e-02,  4.4760e-03,  1.2069e-02],
         [-3.3002e-03, -1.5296e-02, -2.9773e-02, -2.3431e-02, -3.9120e-02],
         [ 1.9136e-02,  8.6728e-03,  8.0657e-03,  8.1224e-04, -8.0909e-03]],

        [[ 7.2724e-03, -2.8181e-02,  8.6047e-05,  3.1839e-02,  2.7608e-02],
         [ 2.5064e-03,  1.7479e-03,  2.8324e-03, -1.4572e-03,  2.7148e-03],
         [-1.7499e-02, -1.7043e-02, -2.0701e-02, -8.4644e-03,  7.0091e-04],
         ...,
         [-2.8742e-03,  6.7639e-03, -1.8267e-02, -8.8647e-03,  8.0417e-03],
         [-6.2802e-03,  3.2563e-02,  2.3604e-02,  3.9340e-02,  3.3441e-02],
         [ 2.0381e-03, -5.4636e-03,  4.2622e-03, -1.3501e-02, -8.2195e-03]],

        [[ 1.0911e-02,  1.4660e-03,  6.1724e-03,  2.2626e-02,  1.1222e-02],
         [-1.4519e-02,  7.0558e-03,  1.0523e-02, -2.8205e-04, -6.3529e-03],
         [ 2.1967e-02, -1.4357e-02, -3.3043e-02, -1.2429e-02, -2.2178e-02],
         ...,
         [ 2.7933e-02,  1.0185e-02,  1.6410e-02,  4.4064e-03,  3.6288e-03],
         [ 3.9862e-03,  5.8839e-03, -5.9015e-03,  1.9343e-02,  5.8023e-03],
         [ 7.2765e-03, -1.5045e-02, -1.5070e-02, -3.6220e-02,  2.8093e-03]],

        ...,

        [[-1.0958e-02,  3.4862e-02,  2.1289e-02,  4.8823e-03,  5.8585e-03],
         [ 6.8140e-03, -1.4777e-02, -2.0793e-03,  1.2419e-02,  7.5245e-03],
         [ 3.0510e-02,  1.2724e-02, -1.1111e-02,  9.6046e-03, -4.8385e-03],
         ...,
         [-1.0455e-02, -1.0261e-02,  1.2213e-02,  4.2826e-03, -1.4027e-03],
         [-1.2483e-02, -8.0324e-03, -1.9954e-02,  1.1958e-02, -4.7831e-03],
         [ 7.3043e-03, -1.4607e-03,  4.9277e-04, -4.9750e-04, -3.2741e-03]],

        [[-7.0000e-03,  8.5449e-03, -2.8791e-02, -3.9519e-03,  2.4527e-02],
         [ 7.4683e-03,  5.8214e-03,  4.4192e-03,  9.5003e-03,  4.9443e-03],
         [ 2.5205e-02,  2.4628e-03, -6.4742e-03, -8.0335e-03, -2.4475e-02],
         ...,
         [ 1.5499e-02, -1.2562e-02, -2.0950e-03,  2.7772e-03,  1.0769e-02],
         [ 7.9209e-03,  1.1329e-02, -1.6705e-03,  1.0757e-02,  1.0929e-02],
         [ 5.9799e-03,  2.8213e-02, -1.6365e-03, -1.9229e-03, -2.8194e-02]],

        [[-3.1544e-02, -2.4316e-02,  8.3113e-04,  7.8306e-03,  3.9064e-02],
         [ 8.6619e-03,  8.5884e-03, -4.2193e-03, -1.4567e-02, -2.9292e-03],
         [ 9.3206e-03, -1.6234e-02,  8.0883e-03, -8.6698e-03, -8.8800e-03],
         ...,
         [-2.4944e-03, -1.1088e-02,  7.6400e-05,  3.2218e-03,  3.6528e-03],
         [-5.4326e-03,  5.4107e-03, -1.6634e-03,  6.4499e-05,  1.2454e-02],
         [-1.4839e-02,  1.7594e-02,  1.1343e-02,  1.6754e-02,  7.7185e-03]]],
       device='cuda:0'), tensor([-0.0887,  0.0185, -0.0003,  ..., -0.0147,  0.0103,  0.0067],
       device='cuda:0'), tensor([1.0721, 1.0520, 0.9835,  ..., 1.0296, 0.9996, 1.0459], device='cuda:0'), tensor([ 0.0117, -0.0626, -0.0667,  ..., -0.0473, -0.0557, -0.0586],
       device='cuda:0'), tensor([[-0.0297, -0.0728, -0.0104,  ..., -0.0221,  0.0565, -0.0014],
        [ 0.0116, -0.0484, -0.0088,  ...,  0.0175, -0.0375,  0.0203],
        [ 0.0146,  0.0194,  0.0165,  ..., -0.0059,  0.0133, -0.0231],
        ...,
        [-0.0419,  0.0131, -0.0340,  ...,  0.0032,  0.0020, -0.0335],
        [-0.0212, -0.0328,  0.0427,  ..., -0.0217,  0.0025, -0.0528],
        [ 0.0170, -0.0509, -0.0013,  ...,  0.0172,  0.0095, -0.0098]],
       device='cuda:0'), tensor([-0.0422,  0.0282, -0.0278,  ..., -0.0194, -0.0046, -0.0194],
       device='cuda:0'), tensor([[ 0.0400, -0.0140, -0.0466,  ...,  0.0616, -0.0492,  0.0593],
        [-0.0239, -0.0141,  0.0150,  ...,  0.0051,  0.0380,  0.0360],
        [ 0.0395,  0.0498,  0.0349,  ...,  0.0377,  0.0224, -0.0661],
        ...,
        [-0.0356,  0.0012, -0.0068,  ..., -0.0480, -0.0580, -0.0025],
        [ 0.0367, -0.0043, -0.0079,  ..., -0.0260,  0.0185,  0.0263],
        [-0.0024, -0.0236,  0.0307,  ..., -0.0516, -0.0038, -0.0553]],
       device='cuda:0'), tensor([ 0.0586, -0.0554, -0.0112,  ...,  0.0141,  0.0390,  0.0527],
       device='cuda:0'), tensor([[ 0.0260,  0.0189,  0.0024,  ..., -0.0016,  0.0046,  0.0086],
        [ 0.0206, -0.0186, -0.0147,  ...,  0.0035,  0.0164, -0.0147],
        [-0.0249, -0.0049,  0.0116,  ...,  0.0086, -0.0039, -0.0139],
        ...,
        [ 0.0196,  0.0074,  0.0153,  ..., -0.0039, -0.0135, -0.0195],
        [-0.0352, -0.0183, -0.0111,  ...,  0.0086, -0.0093, -0.0125],
        [ 0.0050,  0.0022,  0.0274,  ...,  0.0186,  0.0155,  0.0039]],
       device='cuda:0'), tensor([ 1.7039e-02,  1.0859e-02,  4.3616e-03, -2.7989e-02,  1.1959e-02,
        -1.2921e-02, -2.1082e-02,  1.3650e-02, -7.9260e-03, -1.0789e-02,
        -2.0287e-02, -4.3864e-03, -3.6678e-03,  2.2095e-02, -2.2495e-02,
        -3.1688e-03, -5.6508e-03, -2.1615e-02,  2.4607e-02,  7.2655e-03,
        -4.4351e-03, -1.6427e-02, -1.9033e-02,  5.9051e-03,  3.6496e-02,
         1.3493e-02, -5.9433e-03, -2.8689e-02,  3.9312e-03, -1.4779e-03,
        -4.1138e-03, -3.0733e-02, -1.5144e-02, -3.1785e-03,  7.5485e-03,
         2.5715e-02,  2.0393e-02,  2.1755e-03, -1.8421e-02,  1.9931e-02,
         3.1361e-02, -7.1072e-03,  3.8672e-03,  2.8166e-03,  1.3291e-02,
        -2.6969e-02,  6.3981e-03, -2.0849e-02, -1.9836e-02, -9.6085e-03,
         1.7299e-02, -2.2625e-03,  3.1734e-02, -4.8538e-04,  3.7569e-03,
         3.9434e-03, -3.0443e-03,  3.1440e-03, -2.1934e-02, -2.5277e-02,
         1.1631e-02, -1.1008e-02, -9.8331e-03, -1.9421e-02, -6.6984e-03,
        -4.9508e-03, -2.0087e-02, -2.6035e-02, -6.7477e-02,  1.4974e-02,
         7.9678e-04, -5.1804e-03, -1.4599e-02, -3.1096e-02,  1.8211e-03,
         2.8304e-02, -1.3111e-02, -9.3585e-03,  2.8506e-02, -1.5655e-02,
         1.4340e-02,  2.4438e-02,  1.7709e-03, -2.5468e-02, -2.9014e-02,
        -1.0664e-02, -2.5356e-02,  2.4786e-02, -1.5383e-02,  2.2915e-02,
        -1.1476e-02,  2.7774e-03, -1.9178e-02,  2.4028e-03, -7.1332e-03,
         2.5746e-02, -6.3935e-03, -1.3854e-03, -2.4441e-02, -2.1027e-02,
         3.0772e-02, -1.7907e-02, -4.1431e-03,  2.7365e-02, -1.1260e-02,
        -2.2772e-02,  2.0046e-02,  6.8870e-03, -1.1143e-02, -1.0499e-02,
         1.5927e-02,  4.6556e-03,  4.8045e-03, -1.0782e-02, -1.5560e-02,
         2.1544e-02, -1.9672e-02, -8.7595e-03,  1.4560e-02, -1.8866e-02,
         1.3724e-03,  1.7687e-02, -2.6553e-02, -2.7143e-02,  2.0952e-02,
         5.3355e-03,  2.8622e-02, -1.8516e-02, -1.4377e-02,  1.0242e-02,
        -2.5544e-02, -2.2757e-02,  2.1007e-02,  7.3692e-03,  8.1346e-03,
         4.4296e-04,  1.6311e-02, -7.8781e-04,  2.8033e-02, -1.5852e-02,
        -2.2363e-02,  1.0086e-02, -5.3066e-03,  1.4400e-03, -6.1660e-03,
         1.4290e-02, -5.1264e-03, -2.3836e-02, -2.8475e-02, -6.5111e-03,
        -1.6107e-02, -2.0638e-02, -2.0180e-02,  1.4206e-02, -1.3665e-02,
         4.9332e-03,  2.7228e-02,  2.0215e-02,  2.5269e-02,  1.0901e-02,
         1.0712e-02, -2.9743e-02, -1.3911e-02,  1.6737e-02, -1.7205e-02,
         2.2181e-02,  2.8265e-03, -9.3333e-04, -2.2145e-05,  2.6601e-02,
         1.4589e-02,  1.7678e-03,  1.6117e-02, -1.9602e-02,  2.1411e-03,
        -1.8659e-02, -2.8390e-02, -3.6531e-03,  6.8265e-03,  2.2564e-02,
        -2.5053e-02, -1.7404e-03, -2.2244e-02, -1.0866e-02, -1.2319e-02,
        -1.9904e-02, -2.1059e-02, -1.8940e-02,  7.2478e-03, -2.4692e-02,
         4.0767e-03,  3.0017e-02,  5.1109e-03, -1.8901e-02,  1.2936e-02,
         1.8516e-02,  2.3400e-02, -2.2546e-02, -5.9397e-03, -1.8230e-02,
         4.9180e-04,  1.4298e-02,  1.5742e-02,  1.1808e-03, -2.1034e-02,
        -5.4047e-03,  2.4734e-02,  2.3518e-02, -2.0092e-03, -2.5274e-02,
        -2.7433e-03,  2.6843e-02,  1.8144e-02,  1.0385e-03,  4.2338e-03,
        -6.7924e-03, -2.7037e-02, -2.9655e-02, -1.5731e-02, -2.8551e-02,
        -1.1537e-02, -3.3800e-02, -1.5628e-02,  1.7121e-02, -5.7517e-03,
        -3.8374e-03,  1.2117e-03, -5.8815e-03, -3.9083e-03, -3.0027e-02,
        -1.3216e-02,  1.4335e-02,  1.1361e-03,  1.1653e-02, -5.6476e-03,
        -2.8341e-02, -1.4209e-02, -2.4033e-02,  1.1111e-02,  1.0645e-03,
         1.4485e-02,  2.8518e-02,  2.9778e-02,  9.0654e-03,  2.6306e-02,
        -5.4318e-02,  6.4425e-03,  1.6891e-02,  1.8688e-03, -4.9243e-04,
         2.7770e-02, -2.8505e-02, -2.3978e-02,  1.0772e-02,  1.8933e-02,
        -3.1312e-03], device='cuda:0'), tensor([[[ 0.0099, -0.0160, -0.0195, -0.0147,  0.0065],
         [ 0.0127, -0.0096, -0.0061, -0.0151,  0.0113],
         [ 0.0084, -0.0014,  0.0134,  0.0047, -0.0056],
         ...,
         [-0.0001, -0.0060, -0.0008, -0.0074, -0.0062],
         [ 0.0066, -0.0184, -0.0132,  0.0068, -0.0004],
         [ 0.0006, -0.0010, -0.0080,  0.0011, -0.0052]],

        [[ 0.0183,  0.0101,  0.0122, -0.0141, -0.0052],
         [ 0.0031, -0.0023, -0.0035, -0.0003,  0.0070],
         [-0.0054, -0.0080,  0.0005, -0.0119, -0.0221],
         ...,
         [-0.0111, -0.0161, -0.0094, -0.0160, -0.0011],
         [-0.0021, -0.0015, -0.0083,  0.0001, -0.0048],
         [ 0.0007, -0.0059, -0.0010, -0.0178, -0.0067]],

        [[-0.0029, -0.0244, -0.0106, -0.0057, -0.0156],
         [-0.0092, -0.0044, -0.0055,  0.0146,  0.0026],
         [-0.0041,  0.0151,  0.0175, -0.0101, -0.0034],
         ...,
         [-0.0006, -0.0007,  0.0015, -0.0077, -0.0044],
         [ 0.0160, -0.0018,  0.0014,  0.0107, -0.0059],
         [-0.0040, -0.0061, -0.0103,  0.0011, -0.0033]],

        ...,

        [[ 0.0006, -0.0008, -0.0068, -0.0111,  0.0031],
         [ 0.0212,  0.0136,  0.0334,  0.0245,  0.0073],
         [ 0.0028,  0.0043,  0.0033,  0.0151,  0.0045],
         ...,
         [ 0.0070, -0.0053, -0.0003, -0.0082, -0.0078],
         [ 0.0144,  0.0080,  0.0354,  0.0150,  0.0211],
         [ 0.0021, -0.0070, -0.0140, -0.0054,  0.0252]],

        [[-0.0131, -0.0039,  0.0042,  0.0103, -0.0094],
         [ 0.0030, -0.0055, -0.0178, -0.0054, -0.0003],
         [ 0.0104,  0.0176,  0.0129,  0.0090,  0.0098],
         ...,
         [-0.0051,  0.0113,  0.0142,  0.0046, -0.0046],
         [ 0.0058, -0.0157, -0.0100, -0.0058, -0.0096],
         [ 0.0028, -0.0012,  0.0018,  0.0083, -0.0115]],

        [[ 0.0096,  0.0043, -0.0163,  0.0050,  0.0007],
         [ 0.0157, -0.0082,  0.0093, -0.0117, -0.0114],
         [-0.0031,  0.0095, -0.0018, -0.0075,  0.0066],
         ...,
         [ 0.0004, -0.0031,  0.0080,  0.0137,  0.0132],
         [ 0.0090,  0.0157,  0.0173, -0.0073,  0.0124],
         [ 0.0128, -0.0100, -0.0027,  0.0032,  0.0066]]], device='cuda:0'), tensor([ 1.0926e-02,  2.7743e-03,  1.6377e-03, -4.9596e-03, -8.4186e-03,
         1.1818e-02, -2.6501e-03,  1.3931e-02,  3.7500e-03,  3.2759e-03,
         1.4636e-02,  1.2950e-02,  1.7562e-03, -2.3944e-04,  9.0530e-03,
        -2.9698e-03,  9.0584e-04, -4.3332e-03,  3.9312e-03, -1.3703e-04,
         1.0942e-02, -1.1611e-02, -3.6284e-03, -2.5351e-03, -6.2971e-03,
         1.5858e-03,  5.9686e-03,  1.4117e-02,  1.3074e-02,  1.5879e-04,
         1.7060e-03,  8.9278e-03,  1.0911e-02,  7.7378e-03,  2.8901e-03,
        -6.4325e-03,  1.3112e-02,  1.3460e-02, -6.3611e-03,  1.3731e-02,
        -1.9952e-03,  2.1174e-03, -1.5216e-03, -6.1519e-03,  4.3747e-03,
         7.3810e-03,  8.3616e-03,  5.4978e-04,  1.5600e-03,  1.2044e-02,
         6.9655e-03, -3.1887e-03,  9.5530e-03, -1.3552e-02,  1.3675e-02,
         1.0626e-02,  8.0817e-03, -5.5531e-04,  1.2941e-02,  1.0502e-02,
         6.7357e-03,  5.2133e-03,  1.5644e-02, -6.8092e-03, -2.3271e-03,
         1.9499e-03,  6.7222e-03,  7.6853e-03, -1.3148e-02,  1.8854e-03,
        -8.4935e-03, -3.8351e-03, -2.9789e-03, -1.3026e-04,  8.9549e-03,
        -4.3013e-03,  1.2347e-02,  7.4427e-03, -1.4472e-02,  1.8240e-02,
         6.3114e-03,  1.5347e-02,  1.2161e-02,  9.6558e-03,  3.7590e-03,
         4.8144e-03,  1.1819e-02,  3.8413e-03,  1.2698e-02, -1.3311e-02,
         1.3519e-03,  8.2291e-03,  1.2901e-02,  3.5538e-03,  3.8367e-03,
         1.0349e-02, -1.9611e-03, -5.4638e-03, -8.2381e-03,  1.8991e-02,
         4.1175e-03,  4.3726e-03, -2.9949e-04,  3.8518e-03,  7.5960e-03,
         7.7908e-04, -5.4308e-03,  1.0562e-02, -3.8797e-03, -1.3195e-02,
        -2.3500e-03,  1.0453e-02,  1.7760e-02,  1.3294e-02, -2.2483e-02,
         1.5411e-02,  1.3581e-02, -3.2037e-04, -1.2605e-02,  5.6024e-03,
         7.8200e-03, -1.2014e-02,  4.7726e-03,  1.6580e-03,  6.2523e-03,
        -8.0964e-03, -4.5403e-03, -1.3723e-02,  1.7463e-03, -2.2007e-03,
         2.6220e-03, -1.5094e-03,  1.2765e-03,  6.6423e-03, -2.4843e-03,
        -1.4026e-02,  6.6165e-03,  3.9899e-03,  7.4837e-03, -4.0310e-03,
         6.9464e-03,  2.2158e-03,  7.3886e-03,  3.3556e-03, -2.9498e-03,
         1.5559e-02,  9.2852e-03,  3.0626e-03,  3.9900e-03, -5.2853e-04,
         2.5803e-04, -5.6822e-03,  4.3284e-04,  1.4913e-02,  1.4271e-03,
         8.9397e-03,  1.6610e-02,  7.7365e-03,  3.4010e-03,  9.8879e-03,
         1.1639e-02, -1.4560e-02,  6.4711e-04, -7.0179e-03, -8.1228e-03,
        -1.3487e-02, -9.1460e-03, -6.1366e-04, -8.3372e-03,  1.0277e-02,
        -4.3305e-03, -5.0700e-03,  9.1976e-05,  1.5144e-02,  5.5580e-03,
         3.1117e-03, -1.0733e-03,  4.8537e-03,  5.4650e-04, -2.1453e-03,
        -8.9475e-03, -1.7803e-03,  1.3426e-02, -2.3613e-03,  2.9144e-03,
        -2.2973e-03,  4.0671e-04,  6.4771e-04,  9.3332e-03,  3.8827e-04,
         3.0040e-03, -4.1681e-03,  1.3349e-04,  1.4502e-02,  8.8800e-03,
        -3.2626e-03, -1.0352e-03,  2.0737e-03,  9.0921e-03, -7.6649e-04,
         1.1264e-02,  1.0831e-03, -1.5172e-03,  4.1497e-03, -1.8051e-03,
        -6.3216e-03, -9.8367e-03,  1.8013e-04,  5.8720e-03, -7.8992e-03,
        -2.0210e-04, -1.6945e-03,  4.6416e-04, -1.0260e-02, -7.5240e-03,
         3.8303e-03, -1.0680e-02, -2.1131e-03, -3.6054e-03,  7.3139e-03,
         2.2331e-03,  5.5406e-04,  1.5951e-02,  6.1691e-03,  1.2688e-02,
         1.8030e-02,  1.0896e-02, -9.0514e-03,  1.3950e-02,  4.2458e-03,
         9.3456e-03,  2.2363e-03, -8.4166e-03,  8.1367e-03, -2.1351e-03,
         5.5937e-03,  1.4408e-02,  1.0056e-02, -9.9910e-05, -1.2380e-03,
         2.1737e-03,  3.6492e-03, -3.1698e-03,  8.5672e-03,  1.2847e-03,
         6.1779e-03,  6.7409e-03,  1.1170e-02, -2.4970e-03,  8.9781e-03,
         5.4494e-03,  5.5828e-03,  5.8547e-03,  5.4468e-03,  1.4715e-02,
         1.2610e-02,  3.1637e-03, -7.0603e-04,  5.2545e-03, -2.0945e-04,
        -8.8401e-03, -6.0621e-03,  1.1331e-02, -1.0923e-03,  9.5126e-03,
         2.5272e-03,  1.1091e-02,  1.0710e-03,  1.1303e-03,  4.5439e-03,
        -4.0385e-03,  1.4676e-03, -5.5702e-03,  9.2952e-03,  1.2256e-02,
        -4.7363e-03,  9.1791e-03,  2.9552e-03,  2.7354e-03, -7.0130e-03,
         4.5619e-04,  2.0272e-03,  2.4410e-03,  2.0830e-03, -3.9527e-03,
        -8.6351e-03,  5.4185e-03,  1.1514e-04,  2.9125e-03,  2.6930e-03,
        -4.1950e-03, -5.8794e-03,  2.8758e-04,  9.1512e-04, -7.2150e-04,
         1.2322e-02,  9.6250e-03,  5.9452e-03,  3.8822e-03, -5.9832e-04,
        -1.1064e-03,  1.0898e-02, -1.5763e-02,  6.9203e-03,  6.2162e-04,
         4.2195e-04,  9.6657e-03,  9.0312e-04,  1.1878e-02,  1.0824e-02,
         9.2885e-03, -1.3179e-03, -6.6095e-03,  2.3094e-03,  1.2315e-02,
         4.0149e-03,  7.0855e-04, -4.9192e-03, -2.1874e-03,  1.7412e-02,
        -1.1587e-03,  8.1540e-03,  1.5931e-02, -1.9530e-02,  6.0814e-03,
         1.2961e-02,  8.5776e-03, -2.8771e-03,  1.4702e-02,  1.5418e-04,
        -7.4152e-03,  5.5007e-03, -9.1869e-04,  8.4929e-03,  1.4903e-02,
         1.0456e-02,  9.2954e-03, -5.9813e-03,  1.9830e-03,  5.9283e-03,
         1.7679e-03,  1.2361e-02,  6.6907e-03,  1.0365e-02,  1.3118e-02,
         8.6293e-03,  7.0311e-03,  1.6782e-02, -2.8759e-03,  6.9450e-03,
         6.7070e-03,  9.1948e-03, -2.7455e-03,  7.4213e-03,  1.0931e-02,
         2.0134e-02, -5.0940e-04, -4.2344e-03,  9.8343e-03,  1.0534e-02,
         1.4310e-02,  1.2182e-02,  7.1065e-03,  3.8730e-03,  1.2803e-02,
        -9.3140e-03,  1.6793e-02,  7.8013e-03, -5.9416e-03,  1.2935e-02,
        -3.0392e-03,  9.2964e-03, -9.4185e-03, -1.1102e-05, -7.9766e-03,
         1.8360e-03, -2.0880e-05, -1.1120e-03,  1.6592e-02,  1.3269e-02,
         4.3646e-03,  7.5784e-04,  1.4950e-02,  1.2339e-02,  2.3046e-03,
         6.1286e-04,  3.8847e-03, -3.4787e-03,  1.4234e-02,  7.0160e-03,
         1.1264e-02,  1.0240e-02, -3.1337e-03,  1.1131e-02,  6.5924e-03,
         1.6986e-03, -7.3521e-03,  1.3184e-02, -6.6646e-03,  8.9837e-04,
         1.0067e-02,  4.0830e-03, -5.3526e-03,  1.6443e-02,  1.2696e-04,
         9.1261e-03,  2.3275e-03,  8.2359e-03,  1.9075e-03,  5.9630e-03,
         1.5175e-02,  9.3769e-03,  3.6816e-03, -2.5556e-03,  2.8089e-03,
         3.6695e-03,  4.5870e-03,  6.4198e-03, -3.8819e-03,  4.0440e-03,
         7.6519e-03, -6.4795e-03,  5.9261e-03, -5.5158e-03, -6.7698e-04,
        -4.6662e-03,  2.9623e-03, -5.7947e-04,  7.9443e-03,  8.8506e-03,
         8.4214e-03,  9.4161e-03,  8.5732e-04,  6.6451e-03, -5.0452e-03,
        -6.8819e-03,  1.8948e-02,  1.2351e-02,  1.2169e-02, -4.4474e-03,
         1.3137e-02, -9.9011e-03, -5.0446e-03,  6.4811e-04, -8.1916e-03,
        -2.5602e-03, -1.4505e-02, -2.0409e-03, -1.9180e-03,  1.4022e-03,
         1.9720e-02, -7.3371e-03,  3.7312e-03,  1.0828e-02,  5.9200e-03,
         2.1046e-03,  7.6867e-03,  6.6317e-03, -1.4122e-03,  9.9183e-03,
         1.8962e-03, -4.2119e-03,  1.5170e-02, -3.3519e-03, -2.2009e-03,
         3.4802e-03,  9.9633e-03,  3.8037e-03,  1.6231e-02,  3.1709e-03,
         1.8563e-03, -2.0992e-04, -3.0883e-03, -1.6147e-04,  8.6351e-04,
         1.6203e-02,  8.8323e-03,  6.2799e-03, -4.6511e-03,  8.1050e-03,
        -6.2267e-03,  1.8118e-02, -1.8962e-03, -7.4808e-03,  1.8057e-02,
        -2.0177e-03,  5.5468e-03,  2.2076e-03,  9.6825e-03,  6.3410e-03,
         1.2396e-02, -3.6951e-04,  1.0692e-02,  6.7314e-03,  8.6040e-03,
         1.0369e-02,  1.6528e-02, -2.6666e-04, -3.4298e-03, -1.7756e-03,
         6.3642e-03, -3.7847e-03,  8.0264e-03,  8.1102e-03,  1.1599e-02,
         7.8631e-03, -5.9889e-03,  6.4379e-03, -5.7992e-03,  5.8984e-03,
         9.6794e-03,  3.2934e-03], device='cuda:0'), tensor([0.9890, 0.9177, 1.0443, 1.1372, 1.1052, 1.0230, 1.0401, 0.9989, 1.0591,
        1.0465, 1.0179, 1.0022, 1.0247, 0.9618, 1.0819, 1.0801, 0.9755, 1.0159,
        0.9773, 1.0309, 1.0404, 1.0795, 0.9782, 1.0713, 1.0814, 1.0043, 1.1246,
        1.0619, 0.9983, 0.9879, 0.9948, 0.9796, 0.9993, 1.0089, 0.9754, 1.0512,
        0.9892, 0.9629, 0.9956, 0.9223, 0.9746, 0.9825, 1.0593, 1.0682, 1.0543,
        1.0632, 0.9856, 0.9673, 0.9466, 0.9912, 1.0352, 1.0420, 1.1109, 1.0824,
        1.0144, 1.0009, 0.9597, 0.9849, 0.9646, 0.9503, 1.0764, 0.9862, 1.0066,
        1.0213, 1.1039, 1.0382, 0.9912, 1.0020, 1.0584, 1.0683, 1.0582, 0.9926,
        0.9868, 1.0129, 1.0800, 1.0515, 0.9728, 0.9660, 1.0493, 1.0588, 0.9341,
        1.0054, 0.9734, 0.9623, 0.9902, 0.9800, 0.9550, 0.9477, 0.9925, 1.0871,
        1.0550, 0.9848, 0.9980, 1.0833, 1.0012, 0.9780, 0.9997, 1.0885, 1.0520,
        0.9889, 1.0340, 0.9643, 1.0615, 1.0727, 0.9851, 0.9825, 1.0084, 1.0260,
        0.9961, 1.1151, 1.0477, 1.0049, 0.9596, 0.9827, 1.0917, 0.9731, 0.9780,
        1.0539, 1.0719, 1.0737, 0.9607, 1.0741, 1.0221, 1.0191, 1.0024, 1.0655,
        1.0804, 1.0764, 1.0306, 1.0070, 0.9848, 1.0522, 1.0862, 1.0046, 1.0801,
        1.0949, 1.0560, 0.9827, 1.0786, 0.9968, 1.0358, 1.0262, 1.0259, 1.0831,
        0.9720, 1.0027, 1.0278, 1.0000, 0.9780, 0.9818, 0.9562, 0.9390, 1.0167,
        1.0310, 0.9957, 0.9484, 1.0916, 0.9982, 1.0440, 1.0325, 1.0014, 1.1120,
        1.0792, 1.0666, 1.0777, 1.0268, 1.1030, 1.0868, 1.0859, 1.0997, 1.0713,
        1.0982, 1.0763, 0.9641, 1.0244, 1.0827, 1.0258, 1.0216, 1.0864, 0.9983,
        1.0218, 1.0611, 0.9939, 1.0400, 0.9838, 1.0357, 0.9876, 1.0521, 1.0971,
        1.0118, 1.0559, 1.0811, 1.0792, 0.9629, 1.0928, 1.0219, 1.0655, 1.0701,
        1.0098, 1.0040, 1.0297, 1.0692, 0.9341, 1.0609, 0.9927, 1.0419, 1.0901,
        1.0608, 1.0582, 1.0994, 0.9877, 1.0082, 1.0243, 1.0476, 1.0327, 0.9859,
        1.0607, 1.1194, 1.0105, 0.9493, 1.0227, 1.0424, 1.0447, 1.0244, 0.9918,
        0.9749, 0.9388, 1.0907, 1.0181, 1.0466, 0.9095, 1.0122, 1.1249, 1.0212,
        1.0194, 1.0331, 0.9858, 0.9938, 0.9562, 0.9611, 1.0602, 0.9895, 0.9840,
        0.9978, 0.9349, 1.0633, 1.0722, 0.9829, 1.0681, 0.9638, 1.0764, 1.1155,
        0.9607, 0.9539, 1.0025, 1.0582, 1.0071, 1.0081, 1.0510, 1.0265, 1.0503,
        1.0769, 1.0633, 1.0273, 1.0060, 0.9795, 0.9979, 1.0329, 1.0868, 0.9922,
        1.0183, 0.9882, 1.0260, 0.9695, 1.0410, 1.0505, 1.0250, 1.0503, 1.0134,
        1.0931, 1.0775, 1.0321, 0.9840, 1.0430, 1.1014, 1.0864, 1.0611, 1.0295,
        0.9881, 0.9639, 0.9886, 1.1043, 1.0663, 1.1318, 0.9466, 0.9893, 1.0665,
        0.9778, 0.9673, 1.0915, 1.0359, 0.9882, 1.0901, 1.0548, 1.1018, 0.9708,
        1.0764, 1.1357, 0.9827, 1.0052, 0.9830, 0.9977, 1.0978, 0.9737, 0.9763,
        0.9963, 1.0520, 1.0136, 0.9819, 0.9702, 1.0668, 0.9840, 0.9837, 1.1098,
        0.9555, 0.9930, 0.9426, 1.0309, 1.0062, 0.9967, 1.0399, 1.0624, 1.0033,
        0.9918, 0.9706, 0.9470, 1.0074, 1.0472, 0.9867, 0.9828, 0.9573, 0.9691,
        1.0157, 0.9877, 1.0031, 0.9398, 1.0043, 0.9410, 0.9888, 1.0536, 1.0025,
        0.9927, 1.0633, 1.0503, 0.9899, 0.9704, 1.0641, 1.1413, 0.9856, 0.9222,
        1.0264, 0.9315, 1.0045, 0.9740, 0.9578, 1.0668, 0.9506, 1.0278, 1.0189,
        0.9712, 0.9987, 1.0027, 1.0711, 1.0395, 1.0199, 0.9746, 1.0563, 0.9918,
        0.9628, 0.9816, 0.9652, 1.0390, 1.0830, 0.9804, 0.9988, 0.9836, 0.9824,
        1.1016, 0.9444, 0.9954, 0.9639, 0.9331, 1.0458, 0.9463, 1.0819, 1.0709,
        1.1083, 1.0512, 1.0585, 0.9808, 0.9740, 1.0000, 1.0504, 1.0073, 1.0574,
        0.9818, 0.9821, 0.9668, 1.0553, 0.9867, 1.0062, 1.0317, 1.0329, 1.0225,
        1.0015, 1.0914, 1.0184, 1.0630, 1.0746, 1.0278, 1.0447, 1.0479, 0.9810,
        1.0684, 0.9903, 1.0354, 1.0126, 1.0163, 1.0271, 1.0096, 1.1124, 0.9912,
        0.9896, 0.9543, 1.1064, 1.0657, 0.9929, 0.9256, 0.9519, 0.9871, 0.9877,
        1.0469, 1.0685, 1.0098, 1.0309, 1.0635, 1.0734, 1.0596, 1.0100, 1.0427,
        0.9565, 1.0402, 1.0729, 1.0374, 0.9915, 1.0104, 0.9734, 1.0119, 0.9812,
        0.9588, 1.0245, 1.0501, 1.0205, 1.0588, 1.0561, 1.0127, 1.0233, 0.9975,
        1.0026, 1.0829, 1.0505, 0.9778, 0.9815, 0.9677, 0.9763, 1.0501, 1.0196,
        1.0040, 1.1077, 0.9288, 1.1097, 0.9859, 1.0720, 1.0522, 1.0032, 1.1051,
        1.0275, 0.9976, 1.0019, 0.9558, 0.9747, 1.0192, 1.0080, 0.9374, 1.0164,
        1.0102, 0.9631, 1.0087, 0.9471, 1.0042, 0.9506, 1.0877, 1.0371, 1.0633,
        0.9970, 0.9554, 1.0742, 0.9797, 1.0890, 1.0803, 1.0478, 0.9982],
       device='cuda:0'), tensor([-0.0766, -0.0885,  0.0379, -0.0348, -0.0677, -0.0311, -0.0498, -0.0463,
        -0.0364, -0.0539, -0.0510, -0.0369, -0.0591, -0.0608, -0.0622, -0.0513,
        -0.0520, -0.0245, -0.0618, -0.0653, -0.0185, -0.0733, -0.0334, -0.0382,
        -0.0492, -0.0267, -0.0403, -0.0341, -0.0261, -0.0578, -0.0254, -0.0631,
        -0.0288, -0.0448, -0.0454, -0.0456, -0.0451, -0.0773, -0.0343, -0.0838,
        -0.0299, -0.0377, -0.0365, -0.0598, -0.0409,  0.0221, -0.0660, -0.0441,
        -0.0814, -0.0482, -0.0323, -0.0077, -0.0468, -0.0468, -0.0409, -0.0482,
        -0.0638, -0.0350, -0.0454, -0.0763, -0.0402, -0.0460, -0.0243, -0.0191,
        -0.0310, -0.0736, -0.0600, -0.0447, -0.0595, -0.0332, -0.0453, -0.0519,
        -0.0782, -0.0203, -0.0223, -0.0604, -0.0500, -0.0556, -0.0460, -0.0513,
        -0.0630, -0.0534, -0.0317, -0.0579, -0.0305, -0.0391, -0.0626, -0.0644,
        -0.0201, -0.0378, -0.0485, -0.0637, -0.0186, -0.0396, -0.0413, -0.0585,
        -0.0576, -0.0591, -0.0448, -0.0426, -0.0424, -0.0481, -0.0264,  0.0494,
        -0.0840, -0.0566, -0.0237, -0.0342, -0.0476, -0.0388, -0.0732, -0.0042,
        -0.0709, -0.0323, -0.0609, -0.0538, -0.0291, -0.0587, -0.0516, -0.0566,
        -0.0638, -0.0680, -0.0408, -0.0424, -0.0263, -0.0523, -0.0546, -0.0498,
        -0.0381, -0.0343, -0.0955,  0.0224, -0.0516, -0.0410, -0.0418, -0.0439,
        -0.0347, -0.0345, -0.0761, -0.0435, -0.0346, -0.0049, -0.0466, -0.0699,
        -0.0654, -0.0307, -0.0372, -0.0568, -0.0402, -0.0436, -0.0642, -0.0524,
        -0.0438, -0.0468, -0.0602, -0.0711,  0.0545, -0.0389, -0.0327, -0.0295,
        -0.0526, -0.0657, -0.0481, -0.0435, -0.0600, -0.0385, -0.0650, -0.0464,
        -0.0520, -0.0574, -0.0560, -0.0443, -0.0709, -0.0484, -0.0356, -0.0295,
         0.0058, -0.0451, -0.0401, -0.0325, -0.0475, -0.0461, -0.0453, -0.0580,
        -0.0447, -0.0275, -0.0168, -0.0291,  0.0246, -0.0602, -0.0194, -0.0492,
        -0.0321, -0.0764, -0.0605, -0.0655, -0.0580, -0.0147, -0.0408, -0.0196,
        -0.0103, -0.0343, -0.0878, -0.0670, -0.0577, -0.0273, -0.0558, -0.0410,
        -0.0302, -0.0422, -0.0302, -0.0420, -0.0524, -0.0379, -0.0364, -0.0325,
        -0.0546, -0.0456, -0.0793, -0.0663, -0.0272, -0.0190, -0.0167, -0.0238,
        -0.0284, -0.0507, -0.0684, -0.0513,  0.0139, -0.0390, -0.0766, -0.0256,
        -0.0548, -0.0149, -0.0376, -0.0125, -0.0639, -0.0368, -0.0621, -0.0568,
        -0.0518, -0.0367, -0.0408, -0.0502, -0.0658, -0.0110, -0.0507, -0.0468,
        -0.0577, -0.0787, -0.0291, -0.0570, -0.0650, -0.0774, -0.0262, -0.0609,
        -0.0522, -0.0113, -0.0242, -0.0131, -0.0475, -0.0703, -0.0236, -0.0224,
        -0.0402, -0.0527, -0.0563, -0.0360, -0.0708, -0.0116, -0.0675, -0.0500,
        -0.0337, -0.0371, -0.0402, -0.0355, -0.0298, -0.0345, -0.0283, -0.0450,
        -0.0543, -0.0370, -0.0286, -0.0530, -0.0657, -0.0563, -0.0319, -0.0198,
        -0.0457, -0.0476, -0.0599, -0.0543, -0.0465,  0.0541, -0.0634, -0.0265,
        -0.0413, -0.0511, -0.0441, -0.0515, -0.0324, -0.0340, -0.0466, -0.0254,
        -0.0392, -0.0641, -0.0301, -0.0615, -0.0514, -0.0380, -0.0304, -0.0263,
        -0.0658, -0.0448, -0.0757, -0.0465, -0.0536, -0.0170, -0.0311, -0.0755,
        -0.0750, -0.0527, -0.0446, -0.0454, -0.0505, -0.0652, -0.0615, -0.0282,
        -0.0457, -0.0369, -0.0207, -0.0378, -0.0356, -0.0484, -0.0588, -0.0890,
        -0.0524, -0.0281, -0.0279, -0.0490, -0.0869, -0.0732, -0.0409, -0.0536,
        -0.0534, -0.0753, -0.0471, -0.0668, -0.0438, -0.0144, -0.0341, -0.0471,
        -0.0585, -0.0492, -0.0445, -0.0552, -0.0602, -0.0349, -0.0422, -0.0662,
        -0.0194, -0.0755, -0.0393, -0.0319, -0.0556, -0.0223, -0.0544, -0.0433,
        -0.0233, -0.0627, -0.0415,  0.0014, -0.0338, -0.0513, -0.0377, -0.0159,
        -0.0130, -0.0224, -0.0814, -0.0436, -0.0576, -0.0576, -0.0838, -0.0148,
        -0.0619, -0.0522, -0.0488, -0.0416, -0.0514, -0.0220, -0.0694, -0.0642,
        -0.0275, -0.0479, -0.0244, -0.0433, -0.0592, -0.0092, -0.0718, -0.0606,
        -0.0303, -0.0446, -0.0322, -0.0408, -0.0388, -0.0386, -0.0419, -0.0376,
        -0.0444, -0.0325, -0.0054, -0.0413, -0.0311, -0.0352, -0.0475, -0.0369,
        -0.0146,  0.0224, -0.0326, -0.0242, -0.0663, -0.0462, -0.0473, -0.0441,
        -0.0408, -0.0300, -0.0219, -0.0478, -0.0133, -0.0295, -0.0253, -0.0407,
        -0.0341, -0.0706, -0.0562, -0.0718, -0.0429, -0.0799, -0.0699, -0.0511,
        -0.0229, -0.0452, -0.0326, -0.0291, -0.0577, -0.0288, -0.0521, -0.0547,
        -0.0352, -0.0113, -0.0564, -0.0661, -0.0339, -0.0380, -0.0818, -0.0348,
        -0.0585, -0.0462, -0.0417, -0.0629, -0.0290, -0.0485, -0.0177, -0.0138,
        -0.0560, -0.0315, -0.0515, -0.0536, -0.0022, -0.0664, -0.0382, -0.0397,
        -0.0358, -0.0643, -0.0532, -0.0103, -0.0343, -0.0201, -0.0662, -0.0918,
        -0.0968, -0.0492,  0.0284, -0.0477, -0.0229, -0.0424, -0.0345, -0.0447,
        -0.0632, -0.0994, -0.0341, -0.0210, -0.0268, -0.0632, -0.0279, -0.0425,
        -0.0552, -0.0443, -0.0642, -0.0311, -0.0805, -0.0534, -0.0898, -0.0392,
        -0.0427, -0.0755, -0.0421, -0.0474, -0.0335, -0.0556, -0.0513, -0.0357],
       device='cuda:0'), tensor([[[ 2.2564e-02,  1.7209e-02, -4.1218e-03,  1.6538e-02,  1.5179e-02],
         [-2.1301e-02,  8.6959e-03,  5.8902e-03, -1.4474e-02,  3.9017e-02],
         [ 1.4872e-02, -1.4338e-02, -8.3143e-03, -2.7739e-02, -2.1211e-02],
         ...,
         [-7.7281e-03, -4.6014e-02,  5.9318e-03, -3.3977e-02, -1.7008e-02],
         [ 4.6774e-02,  2.0225e-02, -1.1474e-02, -2.3984e-02,  1.7350e-03],
         [-1.4866e-02, -4.5475e-03,  1.4570e-02,  1.2513e-02, -1.6056e-02]],

        [[ 2.6385e-02,  1.0697e-02, -4.9698e-03, -1.2263e-02, -3.7291e-02],
         [-7.0129e-03, -3.5433e-03,  2.5722e-02,  1.5672e-02, -1.0225e-02],
         [-3.0720e-02,  7.4810e-03, -6.1580e-03, -9.7257e-04, -1.5503e-02],
         ...,
         [ 2.2172e-02,  9.1018e-03,  1.4344e-03, -2.0415e-02,  2.0567e-03],
         [ 2.5604e-03,  3.8623e-03,  4.4284e-03,  2.9717e-02,  1.9207e-02],
         [ 1.1963e-02, -9.4566e-03,  1.9366e-02,  1.4568e-02,  7.7999e-05]],

        [[ 4.0111e-02,  9.0118e-03, -1.8719e-02, -2.2945e-03, -7.6626e-03],
         [-7.2129e-03,  4.8405e-03,  4.8606e-03, -1.3529e-02,  2.0030e-02],
         [-1.1243e-02,  5.9011e-03, -1.8580e-02,  2.2676e-02,  2.3846e-03],
         ...,
         [ 3.2310e-02, -8.7999e-03,  2.7502e-02,  1.8872e-02,  2.8757e-03],
         [-7.2016e-03, -1.0738e-02,  2.4222e-02,  3.9469e-03,  9.2563e-03],
         [ 1.7469e-03,  1.3889e-02,  2.4723e-02, -3.2373e-03, -5.3551e-03]],

        ...,

        [[ 1.5918e-02,  1.4249e-02, -2.9260e-02, -1.5970e-02, -1.2173e-02],
         [ 2.2122e-02,  2.6150e-02, -1.4276e-02, -2.5061e-02, -3.3544e-02],
         [ 8.9417e-03,  1.2674e-02, -7.5719e-03,  9.7824e-04,  1.7159e-02],
         ...,
         [-1.7491e-02, -1.9668e-02, -4.1895e-03,  4.5311e-03, -1.5926e-05],
         [ 6.2815e-03, -4.5440e-03, -1.7511e-02, -1.5476e-02,  6.6410e-03],
         [ 2.2873e-02,  4.0945e-03, -2.6768e-03, -7.2284e-04,  7.7177e-03]],

        [[-1.4706e-02,  1.4111e-02, -1.9413e-02, -2.8759e-02, -1.3355e-03],
         [ 1.3273e-02,  1.4688e-02, -1.5995e-02, -1.2995e-02,  2.0266e-02],
         [-2.1978e-02, -1.1079e-02, -3.3634e-02, -2.3805e-02, -4.0191e-02],
         ...,
         [-4.6318e-04,  1.5112e-02, -3.3436e-02,  3.8992e-03, -1.2992e-02],
         [ 1.8139e-02,  1.6533e-02,  1.2809e-02,  2.8857e-03, -1.3191e-02],
         [ 2.2158e-02,  1.5989e-02, -1.5519e-02,  6.8452e-03,  1.2867e-03]],

        [[-8.5339e-03, -1.7407e-02, -2.5989e-02, -3.4686e-02, -3.6646e-02],
         [ 4.1600e-03, -5.5326e-03,  6.4744e-03, -8.7825e-03, -3.3736e-02],
         [ 1.1280e-02, -8.5076e-03,  4.7054e-03, -3.3626e-03,  3.1146e-02],
         ...,
         [ 6.5776e-03, -1.0204e-02, -2.3189e-02, -2.0793e-02, -2.7834e-02],
         [-2.3629e-02, -6.4842e-03, -1.6903e-02, -2.4740e-02, -3.1236e-02],
         [-2.9579e-02, -1.9222e-02, -3.5761e-02, -1.9807e-02, -1.3797e-02]]],
       device='cuda:0'), tensor([-3.5588e-03, -9.2938e-03, -7.6105e-03, -1.1219e-02,  1.5264e-02,
         2.0832e-02, -1.4667e-02,  8.6698e-03,  4.1960e-03,  3.0643e-03,
        -5.6197e-03, -1.8761e-02,  9.0395e-03,  1.3644e-02,  2.4921e-03,
        -8.0119e-03,  1.0639e-02, -3.3901e-03, -2.1532e-02, -3.4978e-02,
         7.4460e-05, -6.8943e-03,  1.6557e-02, -3.9107e-03,  1.3395e-02,
         2.5668e-03, -6.7934e-02, -2.7049e-02,  1.1852e-02, -1.4560e-02,
        -4.7458e-03,  2.3662e-02,  7.6464e-03,  2.2335e-02, -1.1591e-02,
         8.2876e-03,  8.0367e-03,  9.0056e-03,  1.4360e-02,  1.8553e-02,
        -1.5423e-02, -6.0968e-02,  1.7994e-02, -1.1702e-02, -8.2140e-03,
        -5.0717e-03, -1.0453e-02,  4.8333e-04, -8.4115e-02, -7.0582e-03,
        -4.4239e-03, -1.0630e-02,  3.0306e-03, -3.8378e-03, -1.4476e-02,
         1.2297e-03, -1.8506e-02,  1.5023e-03,  2.2835e-02,  1.3149e-03,
        -5.5705e-03, -3.4415e-03,  1.3428e-02, -6.1019e-03,  7.8691e-03,
        -3.0507e-02, -7.9815e-03,  1.4801e-02, -1.6375e-02,  1.7701e-02,
         1.8378e-02,  4.6165e-03, -1.1226e-02, -8.8013e-03,  5.9127e-03,
         1.5071e-02,  3.7825e-03, -1.2189e-02, -1.3374e-02,  7.0456e-03,
        -3.5737e-02,  1.3580e-02, -6.1288e-02, -2.2134e-02, -1.2110e-02,
        -3.6639e-04,  2.2589e-03, -1.1685e-02, -7.7144e-03,  1.8582e-02,
         1.4580e-02, -1.8044e-02,  1.6799e-02,  4.2822e-03, -3.1948e-03,
         2.2020e-02,  5.1970e-03,  1.7097e-02,  1.7383e-02,  1.9205e-02,
        -5.1843e-03,  1.6529e-02,  1.4560e-02,  1.7765e-02,  4.7482e-03,
        -6.3125e-03,  6.7984e-03,  1.4739e-03, -2.8281e-02,  2.2698e-02,
        -9.5774e-03, -4.1279e-03, -1.1919e-02, -4.1372e-04,  8.7991e-04,
         3.3532e-03, -9.2860e-03,  6.1312e-03,  1.3456e-02,  1.6568e-03,
         2.3923e-02,  7.5829e-03, -2.2164e-02, -7.1189e-02, -3.5834e-03,
        -5.0110e-03, -1.7855e-02, -6.7937e-03, -1.4112e-02,  6.4903e-03,
        -9.1086e-04, -1.1556e-02,  9.0666e-03,  5.3608e-03, -1.3203e-02,
        -3.8671e-02, -7.0046e-03,  2.3290e-03,  1.0744e-04,  1.2691e-02,
         1.2949e-02, -6.1642e-03,  1.9031e-03,  2.8144e-02,  4.4718e-03,
         1.7239e-03, -9.4073e-03,  1.5644e-02,  2.3595e-03,  7.8524e-03,
        -1.0322e-02,  1.0273e-02, -2.7240e-02, -1.3172e-02, -1.4721e-02,
         2.9583e-02, -2.4690e-03, -6.1199e-03, -2.5377e-03, -6.1872e-03,
         2.0130e-02, -5.3799e-03,  1.4817e-02, -8.0344e-02, -1.1546e-02,
        -1.2470e-02, -1.2037e-02,  2.6108e-02, -1.3072e-03,  1.1384e-02,
        -1.1527e-02, -9.0402e-03, -9.7340e-04, -1.0570e-02, -2.0952e-02,
        -8.6767e-03,  5.5476e-03,  1.7608e-02, -6.8916e-02,  2.5254e-04,
         1.6286e-02, -2.0949e-02, -5.1987e-03, -5.0076e-02, -2.8975e-03,
         2.9364e-04,  4.2975e-03,  3.0693e-02, -1.4047e-02,  8.9245e-03,
         1.3481e-02,  5.1538e-03, -8.4940e-03,  1.1752e-02, -1.7208e-02,
         3.0690e-03, -6.5715e-04, -4.1745e-02,  7.4373e-03, -2.9109e-03,
         2.0830e-02, -7.4152e-02, -7.2838e-03, -5.8522e-02,  6.1857e-03,
        -7.7467e-03,  2.0005e-02,  3.7731e-03,  1.1891e-03,  1.0806e-02,
         1.0584e-02,  6.5899e-03,  6.9719e-03,  2.2871e-02,  1.3007e-02,
        -1.2060e-02,  2.9371e-02,  1.7548e-02,  1.1626e-02,  2.1738e-02,
        -3.5865e-02, -1.4852e-02, -9.7220e-03,  7.6956e-03,  1.1224e-04,
         2.1914e-02,  8.6998e-03,  5.4196e-04, -8.5405e-03, -2.9282e-02,
        -1.3698e-02,  1.6274e-02, -2.2704e-02,  2.2839e-02,  2.0928e-03,
        -3.7816e-02,  2.4368e-02,  1.3327e-02, -2.3890e-02, -1.0604e-02,
         2.4612e-02, -2.6058e-02,  4.0693e-03,  1.5136e-02, -3.2607e-03,
         1.6044e-03, -2.4837e-02,  8.5009e-03,  7.8254e-03, -4.7107e-03,
        -1.1621e-02,  2.7425e-02, -1.5582e-02,  9.0759e-03, -1.1337e-02,
        -2.1941e-03,  1.6173e-02,  2.9246e-02, -7.3539e-02,  6.2302e-03,
        -4.0450e-03,  5.1961e-03,  1.3098e-02, -6.9818e-03,  1.3375e-02,
        -2.3238e-03, -2.6198e-03,  2.0309e-02,  1.3671e-02,  1.3973e-02,
         2.2867e-02,  1.8762e-02,  2.0410e-02,  1.5744e-02, -1.1099e-02,
         1.6402e-02, -2.0569e-02,  1.8226e-02, -6.1082e-03,  4.2640e-03,
         3.3691e-02, -2.8753e-02,  1.0464e-02,  2.9425e-03,  1.3954e-02,
        -6.1150e-03,  2.0178e-02, -2.8063e-02,  9.5749e-03,  3.9215e-03,
         2.5312e-02,  6.7793e-03,  1.4788e-02, -6.8991e-03, -3.8158e-03,
        -3.1429e-02, -5.1564e-03, -1.9997e-03, -1.4626e-03,  1.1339e-02,
        -4.3617e-03,  1.1606e-02, -1.4970e-02,  1.8885e-02, -1.1198e-02,
         2.1413e-02,  8.3234e-04, -3.0536e-02,  1.7566e-02,  2.1628e-02,
        -6.7969e-02,  1.7925e-02, -1.3043e-02,  1.1445e-02,  3.4803e-03,
        -5.1278e-02, -6.6413e-02,  1.1140e-02, -3.2887e-03,  1.3723e-02,
         6.6353e-03,  2.4701e-04,  1.7303e-02, -6.0119e-03, -9.9447e-03,
        -1.5940e-03,  1.5706e-02,  1.4680e-02, -7.1708e-03,  1.2472e-02,
         1.8258e-02,  8.2629e-03,  1.4420e-02,  8.2188e-03, -2.4309e-03,
        -8.4160e-02, -2.4396e-03,  1.9605e-03, -4.8410e-03,  1.5729e-02,
        -4.1665e-02, -1.0844e-01,  5.3561e-04, -1.2669e-02, -7.8359e-03,
         3.3896e-03, -4.7310e-03, -8.6897e-03,  1.6337e-02, -1.5338e-02,
        -2.0916e-02, -1.5052e-02, -1.0863e-02, -4.8358e-03,  5.4503e-03,
         2.3419e-03, -2.6418e-03,  4.1683e-03,  2.7298e-02, -8.6620e-03,
        -1.6806e-02, -4.6855e-03, -8.4442e-03,  9.9320e-03,  1.5831e-02,
        -1.5461e-02,  2.0149e-02,  1.3222e-03, -8.5670e-03,  1.3152e-02,
         5.2938e-03,  1.6486e-02,  1.4895e-02,  4.6776e-03, -1.1854e-02,
        -9.7989e-03, -1.2492e-02, -8.7972e-02,  1.4092e-02, -4.0004e-03,
         4.6014e-03,  1.1849e-03,  1.3558e-02,  1.8842e-02,  1.2358e-02,
        -2.1236e-02,  1.3342e-02, -1.7519e-02,  1.3858e-02, -1.0092e-01,
        -1.5012e-03, -3.5387e-02, -1.0347e-02, -1.1974e-02,  1.9502e-02,
        -3.4657e-03,  5.2581e-03, -1.9870e-02,  7.5596e-03, -6.2404e-03,
        -1.5479e-02,  2.6632e-02,  9.6135e-03,  8.3584e-03,  9.3126e-03,
        -1.5825e-02, -2.3505e-02, -1.3642e-02, -1.2557e-02, -7.9412e-03,
        -1.1024e-02,  1.7094e-03, -7.6946e-03, -1.8573e-02,  1.9212e-02,
        -1.9429e-02,  3.7565e-03,  4.3291e-03, -1.5035e-02, -4.0875e-03,
         1.9556e-02,  2.2250e-02,  1.6336e-02,  2.2641e-02, -2.8857e-02,
         1.2983e-02,  1.7764e-02,  2.0350e-02,  1.1062e-02,  3.8658e-03,
        -1.6349e-02, -2.5803e-03, -2.3740e-02,  1.2937e-03,  1.4275e-02,
        -6.2921e-02,  1.2331e-02,  7.9416e-03, -1.7805e-03, -7.9444e-04,
         2.8398e-03, -1.4605e-02,  1.1777e-02, -6.4831e-03, -6.0942e-03,
         2.2971e-02, -4.8923e-02, -1.2157e-02, -3.2523e-03,  1.9082e-02,
        -1.2231e-02, -1.6872e-03, -9.0025e-03, -1.2344e-02, -1.0559e-02,
        -7.6680e-03,  1.5157e-02,  1.0869e-02, -1.2661e-03, -9.1784e-02,
        -8.2062e-03, -1.8617e-02,  1.2787e-02, -4.1933e-03,  2.5784e-03,
        -3.2506e-03,  9.7328e-03, -5.2336e-03,  7.9576e-03,  3.6689e-03,
        -1.5656e-02, -1.5080e-02,  1.1942e-02,  1.9433e-02, -7.1819e-03,
         9.7471e-03, -7.2712e-03,  6.9282e-03, -1.0936e-02, -1.4416e-02,
        -7.0740e-03, -5.4769e-03, -6.3985e-03, -6.5028e-03, -6.4257e-03,
         6.6215e-03, -1.1989e-03,  1.0542e-02, -3.5208e-04,  2.1585e-02,
        -4.8592e-03, -3.7429e-03, -4.4717e-03,  8.7136e-04, -7.4781e-02,
        -5.4836e-03,  1.5649e-02,  1.6408e-02,  9.5669e-04,  8.0450e-04,
        -3.7784e-03,  1.4044e-02,  6.2852e-03,  1.9258e-02,  4.4296e-03,
         1.1387e-02,  2.0176e-02, -7.3227e-02, -3.8382e-03, -4.5296e-04,
        -9.6788e-03, -7.3049e-02], device='cuda:0'), tensor([1.0246, 1.0295, 0.9815, 1.0277, 1.0217, 1.0002, 1.0691, 0.9921, 0.9970,
        1.0426, 0.9993, 1.0615, 1.0026, 1.0274, 0.9642, 1.0091, 1.0189, 1.0156,
        1.0787, 1.0719, 0.9926, 1.0195, 1.0134, 0.9993, 0.9955, 0.9965, 1.0251,
        1.0050, 1.0159, 1.0003, 0.9943, 0.9850, 1.0000, 1.0028, 0.9809, 0.9853,
        0.9977, 1.0009, 1.0042, 1.0046, 1.0693, 1.0159, 0.9993, 1.0599, 1.0593,
        0.9995, 1.0298, 1.0048, 1.0307, 0.9989, 0.9981, 0.9857, 1.0175, 1.0021,
        1.0438, 1.0387, 1.0414, 1.0129, 0.9987, 0.9929, 0.9981, 0.9937, 1.0167,
        0.9853, 1.0105, 1.0402, 1.0000, 0.9970, 1.0348, 0.9851, 1.0184, 1.0084,
        1.0193, 1.0078, 1.0218, 0.9944, 1.0108, 1.0234, 0.9816, 1.0280, 1.0383,
        0.9871, 1.0423, 1.0455, 0.9911, 0.9858, 0.9793, 1.0274, 1.0027, 1.0056,
        0.9892, 1.0590, 0.9873, 1.0082, 0.9993, 0.9868, 0.9873, 1.0000, 1.0057,
        0.9900, 0.9927, 0.9942, 1.0039, 0.9906, 1.0051, 0.9837, 0.9883, 1.0160,
        1.1405, 0.9866, 1.0888, 1.0033, 0.9876, 1.0686, 1.0170, 1.0705, 1.0055,
        1.0459, 0.9928, 0.9991, 1.0059, 1.0002, 1.0374, 1.0067, 0.9922, 0.9896,
        1.0175, 1.0870, 1.0284, 0.9977, 1.0269, 1.0779, 0.9958, 1.0045, 1.0272,
        1.0686, 1.0190, 1.0241, 0.9871, 1.0010, 0.9938, 0.9915, 0.9946, 0.9883,
        1.0087, 1.0725, 1.0363, 1.0056, 1.0037, 0.9949, 1.0625, 1.0069, 1.1498,
        1.0034, 0.9912, 1.0001, 0.9996, 1.0089, 1.0384, 1.0074, 1.0008, 1.0115,
        1.0029, 1.0248, 0.9991, 0.9775, 0.9979, 0.9929, 0.9967, 0.9792, 1.0277,
        0.9909, 0.9813, 1.0000, 1.1569, 0.9945, 0.9894, 0.9948, 1.0295, 1.0000,
        0.9868, 1.0568, 1.0103, 1.0320, 1.0012, 1.0142, 1.0303, 0.9935, 0.9911,
        1.0118, 1.0215, 1.0447, 0.9875, 0.9925, 0.9856, 1.0051, 1.0407, 1.0227,
        1.0190, 1.0075, 0.9876, 1.0274, 0.9890, 1.0185, 1.0408, 0.9969, 0.9959,
        1.0074, 1.0011, 1.0073, 1.0228, 1.0171, 1.0457, 0.9858, 0.9694, 0.9972,
        0.9789, 0.9992, 1.0152, 0.9918, 1.1141, 1.0269, 0.9816, 0.9967, 1.0007,
        0.9912, 1.0058, 1.0507, 1.0165, 1.0813, 1.0771, 0.9993, 1.0618, 1.0039,
        1.0448, 1.1229, 0.9927, 1.0170, 1.1198, 1.0225, 1.0120, 1.0789, 0.9930,
        0.9956, 1.0015, 1.0439, 1.0202, 1.0460, 0.9871, 1.0145, 1.0395, 0.9898,
        1.0397, 1.0146, 1.0022, 0.9941, 1.0009, 0.9947, 1.0336, 1.0099, 0.9981,
        0.9896, 0.9901, 1.0227, 0.9876, 1.0171, 1.0186, 1.0072, 0.9896, 1.0082,
        0.9915, 1.0022, 0.9978, 1.0227, 1.0424, 0.9945, 1.0352, 1.0028, 0.9915,
        0.9888, 1.0080, 1.0916, 0.9990, 0.9984, 0.9975, 0.9738, 1.0076, 1.1153,
        1.0117, 1.0144, 0.9959, 1.0016, 0.9993, 1.0056, 1.0923, 1.0774, 0.9933,
        1.0135, 1.0060, 1.0019, 1.0504, 1.0068, 1.0935, 1.0158, 1.0289, 0.9939,
        0.9827, 1.1007, 0.9898, 0.9929, 1.0200, 0.9907, 1.0037, 0.9974, 1.0191,
        1.0430, 1.0293, 0.9883, 1.0325, 1.0000, 0.9974, 1.0026, 1.0277, 1.0288,
        1.0245, 1.0386, 0.9977, 0.9773, 1.0485, 1.0051, 1.0093, 1.0288, 1.0218,
        1.0230, 0.9949, 1.0124, 0.9917, 1.0081, 1.0897, 0.9947, 1.1046, 1.0110,
        1.0002, 0.9874, 1.0340, 1.0120, 1.0068, 1.0190, 1.0004, 1.0479, 1.0716,
        1.0302, 1.0053, 1.0240, 1.0462, 0.9981, 1.0137, 1.0308, 1.0022, 1.0023,
        1.0085, 1.0106, 1.0033, 0.9860, 0.9765, 1.0506, 1.0038, 1.0167, 0.9767,
        1.0150, 0.9936, 0.9952, 0.9964, 1.0344, 1.0234, 1.0193, 1.0092, 1.0608,
        0.9986, 1.0076, 1.0014, 1.0052, 0.9893, 0.9976, 0.9869, 1.1204, 1.0097,
        0.9968, 0.9936, 1.0210, 0.9899, 1.1084, 0.9811, 1.0353, 0.9956, 0.9925,
        1.0225, 1.0323, 0.9874, 0.9910, 1.1019, 0.9991, 0.9983, 0.9861, 1.0117,
        1.0018, 1.1216, 1.0137, 1.0051, 1.0058, 1.0214, 1.0145, 1.0098, 0.9894,
        0.9905, 1.0217, 1.0002, 1.0146, 1.0191, 1.0117, 0.9843, 0.9865, 1.0081,
        0.9947, 1.0671, 0.9808, 0.9866, 1.0126, 0.9883, 1.0027, 1.1526, 1.0243,
        1.0350, 0.9996, 0.9661, 1.0346, 0.9892, 1.0056, 0.9991, 0.9822, 1.0052,
        0.9947, 1.0029, 1.0120, 1.0147, 1.0058, 1.0424, 1.0049, 0.9921, 0.9862,
        1.0186, 1.0130, 0.9982, 1.0002, 0.9872, 0.9987, 1.0117, 1.0232, 1.0147,
        1.0178, 0.9876, 1.1017, 0.9986, 1.0691, 1.0300, 1.0000, 1.0041, 1.0554,
        0.9942, 0.9854, 1.0232, 1.0481, 1.0028, 0.9882, 0.9988, 0.9962, 0.9991,
        0.9834, 1.0288, 1.0021, 1.0492, 1.0013, 1.0910, 1.0129, 0.9997, 0.9970,
        0.9966, 1.0013, 0.9978, 0.9982, 0.9943, 1.0104, 1.0597, 1.0304, 1.0339,
        0.9821, 0.9835, 1.0198, 1.0333, 1.0053, 1.0374, 0.9861, 0.9900, 0.9842,
        1.0016, 0.9953, 1.0185, 1.0449, 1.0191, 1.0253, 0.9783, 1.0202],
       device='cuda:0'), tensor([-2.4086e-03, -4.5294e-03, -6.0163e-03, -1.1489e-02, -1.0704e-02,
        -6.2152e-03, -9.4971e-03, -7.4140e-03, -7.4818e-03, -3.3693e-03,
        -5.8222e-03, -5.3504e-03, -6.6480e-03, -3.5490e-03, -1.1740e-02,
        -4.7104e-03, -9.0387e-03, -1.0040e-02, -9.6269e-03, -1.1555e-02,
        -4.8488e-03,  1.2039e-03, -7.2811e-03, -5.0171e-03, -3.6106e-03,
        -5.0294e-03, -4.2364e-03, -9.2772e-03, -7.3962e-03, -4.6460e-03,
        -3.0757e-03, -5.7278e-03, -8.9051e-03, -1.1504e-03, -1.1961e-02,
        -3.8519e-03, -8.1783e-03, -6.3682e-03, -3.8800e-03, -6.3120e-03,
        -6.5673e-03,  5.5385e-04, -5.1165e-03,  3.2868e-04, -1.2904e-02,
         3.4782e-04, -1.0919e-02, -1.8761e-03, -1.8577e-02, -3.3677e-03,
        -2.9027e-03, -8.2503e-03, -4.7434e-03, -2.3895e-03, -8.8714e-03,
        -7.6424e-03, -3.2806e-03, -5.0008e-04, -3.0795e-03, -4.2387e-03,
        -9.6133e-03, -3.8697e-03, -7.4930e-03, -1.2065e-02, -4.3980e-03,
         1.3344e-02, -4.3029e-03, -5.3845e-03, -1.2024e-02, -4.8342e-03,
        -6.0670e-03, -6.3420e-03, -8.7498e-03, -5.8519e-03, -8.5239e-03,
        -3.8689e-03, -1.0209e-02, -5.0483e-03, -7.6840e-03, -5.1441e-03,
         1.2478e-03, -6.9427e-03,  9.6166e-03, -1.5115e-02, -9.3809e-03,
        -6.6761e-03, -8.8613e-03, -7.1808e-04, -6.0286e-03,  3.4605e-03,
        -1.1599e-02, -4.5028e-03, -4.5876e-03, -1.0433e-02, -5.5600e-03,
        -7.3731e-03, -1.0618e-02, -2.5954e-03, -6.0465e-03, -7.6066e-03,
        -3.2732e-03, -6.0876e-03, -2.8680e-03, -6.1653e-03, -1.0162e-02,
        -1.0353e-02, -8.4882e-03, -1.4846e-02, -5.9474e-03, -4.9829e-03,
        -3.2276e-03, -3.2254e-03, -5.0133e-03, -1.2560e-02, -2.8407e-03,
        -2.1490e-02, -7.0161e-03, -9.2349e-03, -7.5373e-03, -5.9445e-03,
        -5.3948e-04, -5.3950e-03, -5.5579e-03, -1.4275e-02, -5.9047e-03,
        -5.7740e-03, -3.8202e-03, -1.1971e-02, -5.7483e-03, -6.2308e-03,
        -8.0253e-03, -1.0361e-02, -5.8503e-03, -3.7406e-03, -1.1384e-03,
         2.0575e-03, -1.0153e-02, -5.3186e-03, -8.5111e-03, -3.9791e-03,
        -7.6806e-03, -6.0268e-03, -2.8057e-03, -7.2344e-03, -1.5890e-03,
        -1.0864e-02, -1.2630e-02, -5.9813e-03, -6.0134e-03, -7.2609e-03,
        -9.6904e-03, -7.5556e-03, -5.5479e-03, -7.5305e-03, -9.5839e-03,
        -6.6766e-03, -5.3971e-03, -6.2656e-03, -5.4585e-03, -4.4569e-03,
        -4.3744e-03, -6.3039e-03, -4.0757e-03,  1.5727e-02, -4.1757e-03,
        -1.0809e-02, -6.7213e-03, -5.7181e-03, -6.2664e-03, -9.0702e-03,
        -6.5939e-03, -4.2506e-03, -9.0906e-03, -3.8410e-03, -3.4884e-03,
        -6.4561e-03, -1.2698e-02, -5.3860e-03,  2.0494e-02, -4.5670e-03,
        -4.1559e-03, -1.0445e-02, -7.7077e-03,  6.8275e-03, -1.4864e-03,
        -6.1481e-03, -4.1464e-03, -8.4836e-03, -8.5118e-03, -8.3798e-03,
        -5.3154e-03, -9.6501e-03, -4.7245e-03, -6.7890e-03, -9.0947e-03,
        -8.9140e-03, -9.3395e-03,  6.1600e-03, -1.2179e-02, -3.8383e-03,
        -4.1886e-03,  2.3370e-03, -7.2302e-03,  1.0983e-02, -7.1755e-03,
        -1.0181e-02, -6.0269e-03, -5.1514e-03, -6.4452e-03, -2.5166e-03,
        -9.2418e-03, -1.6441e-03, -8.9390e-03, -7.7133e-03, -1.1259e-02,
        -4.3199e-03, -8.0759e-03, -7.0938e-03, -3.1353e-03, -4.0263e-03,
        -1.6847e-02, -1.0911e-02, -9.3652e-03, -4.1319e-03, -2.6809e-03,
        -4.2487e-03, -8.8114e-03, -1.2574e-02, -1.0724e-02, -6.5113e-03,
        -5.1838e-03, -3.5455e-03, -5.1986e-03, -6.9814e-03, -7.5277e-03,
        -3.2611e-03, -5.7317e-03, -8.5391e-03, -9.7297e-03, -2.2263e-03,
         1.1263e-04, -1.6629e-02, -5.7802e-03, -5.3014e-03, -3.1819e-03,
        -7.8404e-03, -9.9947e-03, -3.1159e-03, -7.4612e-03, -7.9935e-03,
        -1.2421e-02, -6.0794e-03, -9.0868e-03, -8.1200e-03, -1.4503e-02,
        -5.7418e-03, -2.4239e-03, -3.4771e-03,  9.4803e-03, -4.0417e-03,
        -1.9406e-03, -7.2182e-03, -8.2909e-03, -6.3698e-03, -7.4040e-03,
        -1.6239e-03, -1.2475e-02, -5.9929e-03, -9.9252e-03, -6.1632e-03,
        -7.9655e-03, -7.8665e-03, -1.1072e-02, -5.4149e-03, -7.6196e-03,
        -3.4150e-03, -1.1969e-02, -3.2552e-03, -7.1038e-03, -8.1380e-03,
        -1.1230e-03, -1.0382e-02, -8.0769e-03, -3.8747e-03, -3.9776e-03,
        -8.5475e-03, -3.7725e-04, -1.6961e-02, -5.9576e-03, -8.4134e-03,
        -4.1792e-03, -1.8609e-03, -6.1933e-03, -5.3272e-03, -5.9284e-03,
        -2.3813e-02, -1.6591e-03, -5.1099e-03, -4.6569e-03, -1.7515e-04,
        -1.2833e-02, -1.2227e-02, -9.1682e-03, -4.8400e-03, -3.1968e-04,
        -4.2210e-03, -6.0769e-03,  1.5971e-02, -1.0586e-02, -2.9083e-03,
        -1.1159e-03, -6.0216e-03, -1.5035e-03, -3.4346e-03, -6.5554e-03,
        -3.6089e-04,  1.9225e-02, -1.0827e-02, -7.7615e-03, -4.2305e-03,
        -3.2750e-04, -3.8647e-03, -5.9016e-03, -4.5058e-03, -4.3736e-03,
        -1.1386e-02, -6.9714e-04, -9.4006e-03, -1.8199e-03, -5.6767e-03,
        -4.0858e-03, -6.9480e-03, -4.4764e-03, -2.1245e-03, -5.0400e-03,
         4.0410e-03, -6.0476e-03, -1.0223e-02, -4.3984e-03, -4.4193e-03,
         6.0162e-03, -4.5935e-03, -5.4845e-03, -5.2932e-03, -3.6130e-03,
        -1.1149e-02, -6.7660e-03, -2.8166e-03, -5.2570e-03, -5.3997e-03,
        -7.8951e-03, -3.7502e-03, -5.0745e-03, -7.2201e-04, -4.1422e-03,
        -1.1317e-02, -4.2220e-03, -8.8522e-03, -1.4401e-03, -1.7871e-03,
        -2.7880e-03, -1.1467e-03, -1.1269e-03, -5.9282e-03, -7.1369e-03,
        -3.0000e-03,  1.4807e-03, -5.6897e-03, -8.1180e-03, -6.0499e-03,
        -5.3611e-03, -3.6421e-03, -3.7619e-03, -5.9425e-03, -8.3856e-04,
        -8.6233e-03, -2.6749e-03,  3.0732e-02, -4.5607e-03, -1.7646e-03,
         7.3425e-04, -6.5662e-03, -5.6755e-03, -6.1662e-03, -2.8164e-03,
        -4.1756e-04, -4.0272e-03, -5.6300e-03, -4.2776e-03,  2.0291e-02,
        -1.0601e-02, -1.0404e-02, -4.5671e-03, -4.2093e-03, -5.7224e-03,
        -1.0187e-02, -5.2934e-03, -6.7052e-03, -5.2734e-03, -3.5449e-03,
        -1.6788e-02, -5.4777e-03, -2.8210e-03, -8.5722e-03, -1.1621e-03,
        -5.3920e-03, -1.0188e-02, -8.0875e-03, -4.1777e-03, -1.3621e-03,
        -3.3027e-03, -2.2597e-03, -7.0201e-03, -1.0360e-02, -6.3935e-03,
        -2.0039e-03, -7.2524e-03, -7.9240e-04, -5.1666e-03, -3.4608e-03,
        -7.0402e-03, -5.1995e-03, -5.9115e-03, -4.7182e-03, -6.3920e-03,
        -9.5161e-03, -8.1549e-03, -7.6351e-04, -7.0509e-03, -4.8794e-03,
         8.4825e-03, -5.9002e-03, -5.0070e-03, -8.1315e-03, -1.2902e-02,
         1.3907e-02, -4.7004e-03,  6.3721e-04, -3.6541e-03, -8.2052e-03,
        -5.9198e-03, -4.8125e-03, -6.5923e-03, -2.9045e-03, -9.1517e-03,
        -4.5165e-03,  6.8392e-03, -1.3877e-02, -6.3753e-03, -4.8646e-03,
        -3.8524e-03, -2.3009e-03, -5.0370e-03, -5.5002e-03, -3.8729e-03,
        -2.4887e-04, -1.1587e-03, -6.1622e-03, -2.4155e-03,  8.0589e-03,
        -1.3623e-02, -1.2350e-02, -3.2016e-03, -6.0456e-03, -8.6466e-03,
        -2.2827e-03, -7.5602e-03, -7.8851e-03, -5.8794e-03, -9.3579e-03,
        -3.8616e-04, -5.1159e-03, -7.6230e-03, -8.2100e-03, -3.9998e-03,
        -4.0594e-03, -4.6071e-03, -1.1524e-02, -7.7427e-03, -2.6890e-03,
        -1.0427e-02, -2.5607e-03, -1.4515e-02, -4.0624e-03, -8.3257e-03,
        -2.3034e-03, -2.8007e-03, -7.6968e-05, -5.5099e-03, -3.9885e-03,
        -3.2941e-03, -2.7890e-03, -5.5415e-03, -5.2859e-03, -4.5767e-04,
        -1.4307e-02, -5.7890e-03, -3.8454e-03, -4.5570e-03, -9.7833e-03,
        -5.4215e-03, -7.2760e-03, -7.5150e-03, -1.0412e-02, -6.4473e-03,
        -6.8089e-03, -4.4504e-03,  1.3070e-02, -5.3702e-03, -1.0388e-02,
        -9.7006e-03,  1.3850e-03], device='cuda:0'), tensor([[ 0.0066, -0.0260, -0.0319,  ...,  0.0142,  0.0048, -0.0264],
        [-0.0036,  0.0033, -0.0113,  ..., -0.0411, -0.0034, -0.0469],
        [-0.0005,  0.0250, -0.0436,  ...,  0.0354,  0.0283,  0.0156],
        ...,
        [-0.0313,  0.0389, -0.0228,  ...,  0.0447,  0.0091, -0.0201],
        [ 0.0112, -0.0472, -0.0534,  ...,  0.0058, -0.0157,  0.0199],
        [ 0.0117,  0.0241,  0.0033,  ...,  0.0279,  0.0500,  0.0017]],
       device='cuda:0'), tensor([ 0.0053, -0.0394,  0.0337,  ...,  0.0260,  0.0330, -0.0225],
       device='cuda:0'), tensor([[[-0.0278],
         [-0.0152],
         [-0.0116],
         ...,
         [ 0.0039],
         [-0.0037],
         [ 0.0103]],

        [[-0.0071],
         [ 0.0107],
         [-0.0034],
         ...,
         [ 0.0040],
         [-0.0102],
         [-0.0204]],

        [[ 0.0046],
         [ 0.0099],
         [-0.0086],
         ...,
         [-0.0233],
         [-0.0036],
         [ 0.0037]],

        ...,

        [[ 0.0110],
         [ 0.0013],
         [ 0.0097],
         ...,
         [ 0.0063],
         [ 0.0073],
         [ 0.0054]],

        [[-0.0176],
         [-0.0132],
         [-0.0083],
         ...,
         [-0.0044],
         [ 0.0138],
         [-0.0317]],

        [[ 0.0076],
         [-0.0156],
         [ 0.0166],
         ...,
         [ 0.0123],
         [-0.0106],
         [ 0.0021]]], device='cuda:0'), tensor([ 1.2660e-02,  6.6085e-03, -8.4171e-04, -2.0778e-02,  5.8848e-03,
         1.4427e-02, -4.2727e-03, -8.8945e-03, -2.2461e-02,  1.9220e-02,
         1.6868e-02, -2.1889e-02, -2.0192e-02, -1.1001e-02, -1.7938e-02,
        -3.0924e-03, -6.9195e-04, -1.6780e-02, -1.6510e-03, -1.3957e-02,
         4.7053e-03,  4.7667e-03, -1.7495e-02, -2.1193e-02, -9.5333e-03,
         1.4979e-02,  6.1405e-03, -1.8470e-02, -4.6514e-03,  1.7617e-02,
        -1.4832e-02,  1.7679e-02, -8.1606e-03, -2.2981e-02, -1.6938e-02,
         1.3919e-02,  1.1166e-02,  1.5164e-02,  1.4268e-02,  1.9761e-02,
         1.3855e-02, -2.0431e-02,  2.0835e-02,  1.4089e-03, -2.1019e-02,
        -3.0934e-03, -1.6553e-02, -8.0402e-03, -1.9200e-03,  9.7546e-03,
         1.3667e-03,  1.2308e-02, -2.2261e-02, -1.2148e-02,  2.4961e-03,
         2.8551e-03, -2.4023e-03, -1.6545e-02, -6.8022e-03, -1.5308e-02,
         4.9234e-03, -5.1234e-03, -2.1197e-02, -2.1971e-02, -2.1476e-02,
         4.1512e-03,  7.6979e-03,  1.0989e-03,  9.6771e-03, -1.0852e-02,
         1.0424e-02, -1.8245e-02,  6.8194e-03, -5.2942e-05, -2.1151e-02,
         5.9740e-03,  6.7912e-03,  9.9072e-03, -1.1100e-02,  2.4090e-03,
         1.4451e-03, -2.3757e-02, -9.3247e-03, -2.2143e-03, -9.0823e-03,
        -2.1191e-02, -8.4362e-03, -3.2052e-03,  2.4628e-03,  9.6058e-03,
         1.0370e-02,  4.0409e-03,  1.7726e-02, -2.6468e-02, -1.2035e-02,
         1.8873e-02, -2.5510e-02,  1.1485e-02,  1.5768e-02, -6.0877e-03,
         1.0547e-02, -9.9367e-04, -5.2943e-03, -6.0520e-03,  2.8519e-03,
         1.5409e-03, -3.1774e-03,  1.8540e-02,  6.2238e-03, -1.3560e-02,
         3.8089e-03, -5.1020e-03, -1.5835e-02,  7.4629e-03, -9.3509e-03,
         5.0746e-03, -2.4708e-02, -6.3526e-03,  1.1000e-02, -1.5944e-02,
        -1.9479e-02,  7.3837e-04, -5.3303e-03,  7.3933e-03, -1.1627e-03,
        -2.3001e-02, -2.2293e-02,  6.7774e-03, -7.6076e-03, -6.6589e-03,
        -7.8533e-03, -6.9339e-03,  1.6447e-02, -8.8917e-03,  8.1750e-03,
         3.0101e-03,  1.4916e-02, -1.8720e-02, -2.2616e-02, -2.1305e-04,
         3.4767e-03,  9.4769e-03, -1.4014e-02, -2.4358e-02, -1.6949e-03,
         7.1853e-03,  7.0763e-03, -8.6783e-03, -2.2712e-02, -2.4514e-02,
         1.6216e-02, -1.3557e-02,  3.4990e-03,  1.6805e-02,  1.1454e-02,
        -8.9996e-03, -1.9878e-02, -6.9795e-03,  1.8868e-02,  1.2801e-02,
        -8.2331e-03, -9.0973e-03, -1.8181e-02, -6.1293e-03, -1.2197e-02,
        -2.4391e-02, -1.7264e-02,  4.6847e-03,  2.7989e-03,  7.7562e-03,
         1.1648e-02, -3.3293e-03,  1.1218e-02,  1.3160e-02,  4.4359e-03,
         1.0491e-02, -8.1010e-03, -1.4832e-02, -8.9502e-03, -2.3276e-02,
        -6.8905e-03,  1.7923e-02, -6.2568e-03,  1.9198e-02, -1.1596e-02,
        -1.3563e-02, -1.1319e-02,  1.1930e-02, -2.0862e-02, -2.8787e-03,
        -5.3541e-03,  1.8835e-02, -8.1321e-03, -2.5902e-02,  1.0185e-02,
        -1.8640e-02,  1.7213e-02, -1.1722e-02, -1.4869e-02, -1.9829e-02,
        -2.3022e-02,  1.4036e-02,  1.5743e-02, -4.1760e-03, -1.3739e-02,
        -7.5299e-04,  8.4464e-03, -1.8605e-02, -2.1832e-03, -1.2209e-02,
        -7.8238e-03,  1.3709e-02,  1.0724e-02, -1.3146e-02, -2.1723e-02,
         1.4763e-02,  7.2205e-03, -1.9539e-02,  1.6990e-02, -1.4152e-02,
         1.0407e-02,  8.1810e-03, -2.0784e-03,  1.0607e-02,  9.7569e-03,
        -1.5051e-02,  8.9546e-03, -1.7106e-02,  9.9276e-03, -9.5929e-03,
         1.0955e-02,  1.2212e-03,  1.1147e-02,  2.2326e-04,  7.5210e-03,
         1.3159e-02, -1.8076e-02, -1.4898e-02, -4.4422e-03,  3.1751e-03,
         1.6232e-02,  5.7683e-04, -2.4292e-02, -1.7145e-02, -7.5962e-03,
        -2.3239e-02,  1.9778e-02, -1.4224e-02,  5.7267e-03,  1.5495e-03,
        -1.8656e-02,  7.9740e-03, -1.1296e-02, -1.5111e-02,  1.1894e-02,
        -2.2930e-02,  1.9876e-03, -9.0178e-03, -1.2396e-02, -4.3120e-03,
         1.0773e-02,  7.7695e-03,  1.2040e-02, -1.2277e-02, -1.4063e-02,
        -2.1534e-03, -1.1526e-03,  6.7218e-03,  1.4356e-02, -2.1989e-02,
        -6.7862e-03, -2.3085e-02, -2.0576e-02,  7.4116e-03,  6.4981e-03,
         1.2706e-03,  7.9285e-04,  1.7732e-02, -3.2949e-03,  3.5201e-04,
        -1.9643e-02, -2.4622e-02, -1.5746e-03,  1.3558e-02, -2.4375e-02,
        -6.8020e-03, -5.2083e-03,  1.4111e-02,  2.8181e-05,  1.0305e-02,
         6.6020e-03,  1.4737e-02,  7.9854e-03,  9.2743e-03,  4.6948e-03,
        -3.1039e-04, -1.4816e-02, -1.2270e-02,  1.3332e-02, -3.4112e-03,
         7.3686e-03,  1.1735e-03, -1.4411e-02,  4.5364e-03, -1.7431e-03,
        -2.4430e-02,  9.4358e-03, -1.2412e-02,  3.2020e-04,  5.7275e-03,
         8.2841e-03, -1.9580e-02,  1.1212e-02, -8.3116e-03,  1.6468e-04,
        -2.8461e-04,  2.0954e-02, -1.6090e-02,  6.7183e-03, -2.5086e-02,
         6.7670e-04, -1.8339e-02,  7.0718e-04,  1.6258e-02,  9.8732e-03,
         1.7395e-02, -6.7759e-03, -1.2968e-02,  1.1836e-02,  1.2704e-02,
        -1.8856e-02,  1.5298e-02, -3.8602e-03, -2.0099e-02, -6.8989e-03,
         6.3467e-03,  1.6166e-02, -2.0927e-02, -1.3306e-03, -1.5375e-02,
         1.3499e-02, -7.8724e-03, -1.3831e-02,  6.4419e-03, -1.1742e-02,
         7.5713e-03, -8.3579e-05,  5.3298e-03,  9.9117e-04,  1.9427e-02,
        -2.3586e-02, -1.2288e-02, -1.7035e-02, -1.1450e-02, -8.3461e-03,
        -2.1169e-02,  1.6112e-03, -2.4969e-02,  1.7133e-02,  9.1803e-03,
         2.7794e-03,  6.0270e-03, -3.5062e-03,  7.9930e-03, -1.7388e-02,
        -2.5526e-02, -8.9292e-03, -4.1005e-03,  8.1051e-03, -2.2960e-02,
        -2.4206e-02,  7.4314e-03, -1.5528e-02, -2.0248e-02, -5.7430e-03,
         1.2380e-02, -8.6048e-03,  4.7087e-03, -1.3683e-02,  1.8549e-02,
        -1.0600e-02, -5.3548e-03, -8.7164e-03, -8.6344e-03, -1.1307e-02,
         6.1636e-03, -1.8984e-02, -2.5304e-03,  6.6693e-03, -1.4520e-03,
        -2.2703e-02, -2.1575e-02, -1.8158e-02, -1.8825e-02, -8.8710e-03,
        -1.1304e-02,  1.0623e-02, -1.4357e-02,  1.1547e-02, -1.3775e-02,
         9.6775e-03, -1.3283e-02,  1.4942e-02, -7.0904e-03,  8.0426e-03,
         1.0120e-02, -1.2318e-02,  1.5835e-02,  1.2616e-02, -5.8322e-03,
         1.0414e-02, -3.6171e-03,  2.4615e-03,  1.1226e-02,  1.1261e-02,
         1.0680e-02, -9.1434e-03, -1.6645e-02,  1.8374e-02,  1.7452e-02,
        -4.3342e-03,  9.4678e-03, -1.7086e-02, -1.5491e-02,  5.5033e-03,
         8.8593e-03, -2.5659e-03,  1.7836e-02, -1.7228e-03, -1.8367e-02,
        -1.9705e-02, -1.9872e-02,  2.2491e-02, -1.7598e-02, -2.2321e-02,
        -9.1388e-03,  1.3041e-02,  5.7004e-03,  2.0451e-03,  1.5028e-02,
         1.2786e-02, -9.9478e-03,  7.6054e-03, -3.2855e-03, -1.8910e-02,
         7.1066e-03,  9.6750e-03, -2.3880e-02, -1.5267e-02, -1.1075e-02,
        -8.4231e-03,  6.3093e-03,  4.8235e-03,  1.0304e-02,  6.5708e-03,
        -4.0959e-03,  1.2572e-02, -2.3896e-02, -5.0974e-03, -4.2039e-03,
        -1.3932e-02, -2.1861e-02,  1.6509e-02, -2.1754e-02, -3.2409e-03,
        -1.9764e-03, -9.9848e-04, -1.7841e-02,  1.5840e-02, -2.1865e-02,
         2.9433e-03, -2.3891e-02,  3.9327e-03, -1.4169e-02,  8.8965e-03,
        -1.5316e-02, -1.1726e-02, -2.2947e-02, -2.2074e-02, -1.1015e-02,
        -4.9743e-03, -7.4397e-03, -2.0103e-02, -1.3775e-02, -1.2309e-02,
        -8.3742e-04, -5.0092e-03,  3.4891e-03, -1.4943e-02,  1.0247e-02,
        -3.3117e-03,  1.3122e-02,  4.3025e-04,  1.0788e-02, -1.8728e-02,
         2.2182e-03, -1.5147e-02,  1.0340e-02,  1.6400e-02,  1.0017e-03,
        -1.3588e-02, -3.2627e-03,  1.5266e-02,  1.1106e-02, -1.2466e-02,
        -1.9833e-02,  1.2215e-02, -1.8035e-02,  1.1107e-02, -2.4205e-03,
         1.2034e-02,  3.0295e-03], device='cuda:0'), tensor([[[-3.6589e-03,  1.7713e-02,  7.8951e-03, -8.4124e-03,  1.7135e-02],
         [-4.4011e-02, -4.8717e-03,  1.3115e-02,  1.2034e-02, -4.7132e-03],
         [-3.0882e-02,  2.3190e-02, -1.6174e-03, -1.0685e-02,  4.7784e-02],
         ...,
         [-2.5687e-02,  1.3616e-02,  2.4543e-03,  3.7588e-03,  1.4609e-02],
         [ 2.5235e-02,  1.4860e-02,  1.9830e-02,  3.0525e-02,  1.1528e-02],
         [ 1.4725e-02,  6.5948e-03, -1.3691e-02, -2.3228e-02, -1.8456e-02]],

        [[ 7.1999e-03,  2.5169e-03, -6.8954e-05,  1.5148e-02, -2.4780e-02],
         [ 4.4048e-03,  6.2199e-03, -1.2407e-02, -1.1501e-02,  5.6292e-03],
         [-1.3697e-02,  3.7282e-02,  1.7409e-02, -7.4840e-03, -4.2501e-03],
         ...,
         [-1.2937e-02,  2.0651e-02, -1.8991e-02, -9.6645e-03,  1.2869e-02],
         [ 4.0281e-02,  4.1555e-03, -1.5641e-02,  4.0575e-03,  2.7888e-03],
         [ 4.6156e-03,  1.5924e-02,  1.6799e-02, -5.2218e-03,  2.9403e-02]],

        [[ 1.4014e-02,  1.6697e-02, -4.1694e-03, -1.9343e-02,  4.7939e-03],
         [-2.5390e-02, -1.5551e-02, -3.6557e-02, -4.0309e-02, -2.3748e-02],
         [-2.2507e-02, -1.7957e-02,  2.6016e-03,  4.0928e-03,  8.4496e-03],
         ...,
         [ 1.1571e-02, -2.1753e-03, -1.3091e-02, -3.0883e-03, -1.1253e-02],
         [-3.0459e-02,  2.7151e-02,  2.2516e-02, -9.5283e-03,  1.4322e-02],
         [-1.6216e-02, -7.6629e-04,  2.3229e-02,  2.0875e-02,  2.8528e-02]],

        ...,

        [[-2.2809e-02,  1.9617e-02, -3.6366e-03, -3.6226e-02, -8.1283e-03],
         [ 8.2752e-03,  3.0124e-02,  2.6441e-03, -3.5465e-03, -2.0442e-02],
         [-2.6842e-02,  3.2241e-04,  2.1471e-03, -1.8402e-03, -1.7803e-02],
         ...,
         [-2.5880e-03,  1.7702e-02,  2.4189e-02, -4.3017e-03,  1.5580e-02],
         [-5.4703e-03, -1.0507e-02,  1.0760e-02, -1.1025e-02,  3.6867e-03],
         [-1.0103e-03,  5.1325e-03, -1.1302e-02,  9.6002e-03,  8.4797e-03]],

        [[ 2.1611e-02, -2.0265e-02, -1.4192e-02, -7.7096e-04,  6.5407e-03],
         [ 3.2892e-02,  2.8149e-02,  2.6212e-02, -1.1051e-02, -1.9374e-03],
         [-2.8924e-06, -5.2820e-02, -9.1501e-03, -3.7912e-02, -1.4207e-02],
         ...,
         [-3.3866e-03,  1.8090e-02,  2.0575e-02,  2.2657e-02, -1.6046e-03],
         [-2.7069e-03,  5.7211e-03,  1.0735e-03,  7.7712e-03,  8.8292e-03],
         [ 1.2905e-02, -5.4171e-03,  1.4597e-03,  1.8226e-02,  2.8525e-03]],

        [[ 8.3800e-03, -4.4013e-03,  1.2737e-02,  4.9685e-03,  6.4803e-03],
         [ 4.0311e-02,  3.5171e-02,  4.5616e-02,  4.1416e-02, -8.4260e-03],
         [-2.4190e-03,  5.1611e-04, -9.9609e-03,  3.8971e-02,  2.2167e-02],
         ...,
         [-1.2649e-02,  1.1489e-02, -9.8603e-03, -1.4867e-02,  5.4729e-03],
         [-4.8222e-03,  3.6931e-02,  1.5876e-02,  3.5387e-02, -1.5790e-02],
         [ 7.5025e-04,  1.4505e-02, -2.6567e-03, -1.9283e-03, -2.6118e-02]]],
       device='cuda:0'), tensor([ 0.0040,  0.0420, -0.0044,  0.0168,  0.0199, -0.0196,  0.0270, -0.0004,
        -0.0159,  0.0124, -0.0110, -0.0152, -0.0235, -0.0208, -0.0300,  0.0230,
        -0.0184,  0.0103,  0.0213, -0.0088,  0.0066, -0.0229,  0.0119,  0.0170,
        -0.0301, -0.0771, -0.0249,  0.0100, -0.0019,  0.0209,  0.0319,  0.0167,
         0.0012, -0.0098,  0.0243,  0.0354, -0.0251,  0.0459,  0.0072, -0.0038,
         0.0172,  0.0194, -0.0128,  0.0027,  0.0360, -0.0032,  0.0110, -0.0028,
         0.0167, -0.0162,  0.0441, -0.0049, -0.0002,  0.0106,  0.0016,  0.0250,
         0.0445, -0.0058,  0.0019, -0.0052, -0.0194, -0.0115, -0.0151,  0.0097,
         0.0083, -0.0030, -0.0084, -0.0241,  0.0089, -0.0086, -0.0242,  0.0201,
        -0.0016, -0.0005, -0.0050, -0.0238,  0.0104, -0.0536,  0.0134, -0.0030,
         0.0186, -0.0037,  0.0205,  0.0166, -0.0178,  0.0219, -0.0176,  0.0335,
         0.0443,  0.0002, -0.0099,  0.0082, -0.0093, -0.0093,  0.0012, -0.0169,
        -0.0117, -0.0149,  0.0020,  0.0153,  0.0035, -0.0175, -0.0209, -0.0045,
        -0.0307,  0.0026, -0.0057,  0.0081,  0.0049,  0.0238, -0.0061, -0.0047,
        -0.0045,  0.0096,  0.0004, -0.0039,  0.0385, -0.0109, -0.0223,  0.0169,
         0.0209,  0.0013, -0.0038, -0.0117, -0.0170,  0.0135, -0.0009, -0.0378,
         0.0032, -0.0012, -0.0385,  0.0230,  0.0177, -0.0065, -0.0181,  0.0128,
         0.0140,  0.0105,  0.0004, -0.0052, -0.0191,  0.0105,  0.0108,  0.0022,
         0.0272, -0.0101,  0.0140,  0.0068,  0.0182, -0.0224, -0.0159,  0.0465,
        -0.0248, -0.0014,  0.0146, -0.0035, -0.0137, -0.0089, -0.0082,  0.0154,
        -0.0146,  0.0198,  0.0211,  0.0162,  0.0093, -0.0181, -0.0331,  0.0112,
        -0.0317, -0.0202, -0.0137, -0.0030,  0.0157, -0.0017, -0.0162, -0.0216,
         0.0160, -0.0029,  0.0026,  0.0104,  0.0062,  0.0115,  0.0320,  0.0049,
        -0.0165,  0.0171, -0.0120,  0.0148, -0.0402, -0.0269,  0.0295, -0.0192,
         0.0094, -0.0121,  0.0100,  0.0137,  0.0112, -0.0138,  0.0070, -0.0142,
         0.0176,  0.0050, -0.0134,  0.0425,  0.0087, -0.0175,  0.0167, -0.0360,
         0.0057, -0.0339, -0.0041,  0.0205, -0.0284, -0.0102,  0.0198, -0.0023,
        -0.0141, -0.0067,  0.0230, -0.0297, -0.0294, -0.0376, -0.0151, -0.0267,
        -0.0160,  0.0059, -0.0173,  0.0159, -0.0076, -0.0231, -0.0087,  0.0017,
        -0.0036, -0.0099,  0.0070, -0.0105, -0.0078, -0.0315, -0.0032,  0.0254,
        -0.0290,  0.0233,  0.0164,  0.0036,  0.0133, -0.0026,  0.0209, -0.0138,
        -0.0362,  0.0013,  0.0004, -0.0233, -0.0399, -0.0152, -0.0165, -0.0306,
         0.0154, -0.0377, -0.0149, -0.0113, -0.0093, -0.0126, -0.0176, -0.0078,
         0.0193,  0.0297, -0.0084, -0.0161, -0.0595, -0.0108,  0.0020, -0.0165,
        -0.0142, -0.0007,  0.0162, -0.0011, -0.0245, -0.0179, -0.0172, -0.0223,
         0.0057,  0.0033,  0.0381, -0.0103,  0.0296,  0.0087,  0.0189, -0.0028,
         0.0031,  0.0248,  0.0144,  0.0083,  0.0038,  0.0061,  0.0261,  0.0265,
         0.0412,  0.0014,  0.0175, -0.0057,  0.0146,  0.0068,  0.0114, -0.0060,
         0.0117,  0.0007, -0.0122,  0.0098, -0.0234, -0.0049, -0.0034, -0.0233,
         0.0213,  0.0077, -0.0112,  0.0004,  0.0115,  0.0179,  0.0069,  0.0098,
        -0.0036, -0.0082, -0.0230,  0.0036, -0.0160, -0.0439, -0.0439,  0.0051,
        -0.0020,  0.0134, -0.0141, -0.0177, -0.0278, -0.0028,  0.0041, -0.0333,
         0.0123,  0.0050, -0.0196, -0.0082, -0.0032, -0.0170, -0.0176, -0.0348,
         0.0170, -0.0064, -0.0319, -0.0420, -0.0686,  0.0004,  0.0046, -0.0408,
         0.0079, -0.0089, -0.0064, -0.0083,  0.0376,  0.0382, -0.0132,  0.0103,
        -0.0118,  0.0081,  0.0050,  0.0191,  0.0030, -0.0368, -0.0085,  0.0125,
        -0.0162, -0.0132, -0.0335,  0.0186, -0.0019, -0.0172,  0.0052,  0.0018,
         0.0119, -0.0005, -0.0060, -0.0070, -0.0019, -0.0081, -0.0309, -0.0155,
        -0.0117, -0.0169, -0.0112,  0.0158, -0.0087,  0.0099,  0.0024, -0.0088,
         0.0122, -0.0126, -0.0007, -0.0115,  0.0013,  0.0280, -0.0506,  0.0013,
        -0.0188,  0.0098,  0.0190, -0.0065,  0.0146,  0.0164,  0.0022,  0.0236,
        -0.0187,  0.0116, -0.0050, -0.0103,  0.0089, -0.0103,  0.0271, -0.0243,
         0.0531, -0.0190, -0.0082,  0.0009,  0.0088,  0.0445,  0.0022, -0.0183,
         0.0139, -0.0125, -0.0102,  0.0155, -0.0128, -0.0060, -0.0158, -0.0065,
         0.0056,  0.0322, -0.0344,  0.0486, -0.0753, -0.0022,  0.0035, -0.0127,
        -0.0075,  0.0253,  0.0131,  0.0043,  0.0123,  0.0052, -0.0211, -0.0033,
        -0.0203, -0.0140,  0.0121,  0.0046, -0.0012, -0.0615, -0.0084, -0.0054,
         0.0213,  0.0167,  0.0374, -0.0085, -0.0495, -0.0248,  0.0084,  0.0085,
        -0.0268, -0.0262, -0.0320, -0.0270, -0.0157, -0.0139,  0.0380,  0.0018,
        -0.0173, -0.0072,  0.0191, -0.0037, -0.0092, -0.0428, -0.0140,  0.0005,
        -0.0168, -0.0078, -0.0173,  0.0070, -0.0023, -0.0260,  0.0113, -0.0142,
        -0.0138,  0.0023, -0.0452, -0.0487, -0.0285,  0.0016, -0.0265,  0.0045,
        -0.0079, -0.0082, -0.0467,  0.0172,  0.0037,  0.0031,  0.0142, -0.0298,
        -0.0309, -0.0167, -0.0331, -0.0361,  0.0072,  0.0113, -0.0409, -0.0100],
       device='cuda:0'), tensor([0.9995, 1.0054, 1.0371, 1.0407, 1.0199, 1.0123, 0.9913, 1.0359, 1.0279,
        1.0251, 1.0289, 1.0214, 1.0208, 0.9776, 1.0220, 1.0372, 1.0691, 0.9989,
        1.0050, 1.0137, 1.0204, 1.0652, 1.0087, 1.0263, 1.0393, 1.0699, 1.0793,
        0.9921, 0.9903, 1.0008, 1.0152, 1.0216, 0.9965, 1.0220, 0.9958, 1.0186,
        1.0368, 1.0153, 0.9703, 1.0314, 1.0177, 1.0426, 1.0297, 1.0767, 1.0305,
        1.0251, 1.0034, 1.0125, 0.9982, 1.0111, 1.0151, 1.0617, 1.0452, 1.0349,
        1.0081, 1.0215, 1.0121, 1.0243, 1.0290, 1.0140, 1.0603, 1.0379, 1.0323,
        1.0245, 1.0196, 1.0232, 1.0346, 1.0545, 1.0284, 1.0187, 1.0117, 0.9985,
        1.0213, 1.0347, 1.0305, 1.0065, 1.0211, 1.0955, 1.0273, 1.0050, 1.0292,
        0.9916, 1.0297, 1.0378, 1.0398, 1.0256, 1.0383, 1.0189, 1.0114, 0.9951,
        0.9900, 1.0254, 1.0229, 0.9889, 1.0188, 1.0190, 1.0279, 1.0294, 1.0259,
        1.0249, 1.0000, 1.0022, 1.0322, 1.0383, 1.0379, 0.9724, 1.0435, 0.9944,
        1.0191, 0.9686, 1.0215, 1.0177, 1.0395, 1.0289, 1.0582, 1.0202, 0.9892,
        1.0082, 1.0490, 1.0174, 1.0192, 1.0012, 1.0321, 1.0224, 1.0270, 1.0284,
        1.0132, 1.0421, 1.0151, 1.0252, 1.0268, 0.9774, 1.0209, 1.0299, 1.0413,
        0.9894, 1.0151, 1.0197, 0.9798, 1.0521, 1.0278, 1.0272, 1.0288, 1.0191,
        1.0369, 0.9522, 1.0301, 1.0332, 1.0259, 1.0398, 1.0018, 1.0135, 1.0742,
        0.9779, 0.9631, 1.0018, 1.0544, 1.0341, 1.0050, 1.0059, 1.0546, 1.0132,
        1.0382, 0.9817, 1.0000, 1.0264, 1.0326, 1.0427, 1.0230, 1.0769, 1.0126,
        1.0306, 1.0244, 1.0137, 1.0259, 1.0085, 1.0762, 1.0755, 1.0055, 1.0105,
        1.0157, 1.0223, 0.9953, 1.0172, 1.0227, 1.0290, 1.0620, 0.9770, 1.0946,
        1.0708, 1.0313, 0.9478, 1.0265, 1.0231, 1.0221, 1.0123, 1.0161, 1.0151,
        0.9789, 1.0367, 1.0206, 1.0101, 0.9965, 1.0617, 0.9968, 1.0248, 1.0424,
        0.9638, 1.0000, 1.0313, 0.9898, 1.0346, 1.0342, 1.0382, 1.0092, 1.0205,
        0.9989, 0.9942, 1.0339, 1.0256, 0.9990, 1.0032, 1.0316, 1.0055, 1.0875,
        1.0281, 1.0284, 1.0150, 1.0080, 1.0465, 1.0359, 0.9901, 1.0478, 1.0004,
        1.0209, 1.0072, 0.9708, 1.0801, 1.0085, 1.0235, 0.9755, 0.9966, 1.0160,
        1.0167, 1.0222, 1.0117, 1.0304, 1.0090, 1.0163, 1.0412, 1.0253, 0.9973,
        1.0819, 1.0116, 1.0397, 1.0155, 0.9924, 1.1180, 1.0024, 1.0085, 1.0089,
        1.0172, 1.0174, 1.0642, 1.0037, 1.0158, 0.9907, 0.9775, 1.0656, 1.0096,
        1.0319, 1.0336, 1.0300, 1.0916, 1.0227, 0.9864, 1.0216, 1.0143, 1.0221,
        1.0439, 1.0185, 1.0367, 1.0293, 1.0215, 1.0096, 1.0441, 0.9818, 0.9861,
        1.0148, 1.0109, 1.0203, 1.0170, 0.9547, 1.0644, 1.0407, 1.0242, 1.0119,
        0.9987, 0.9736, 1.0130, 1.0496, 0.9741, 0.9888, 1.0283, 1.0090, 0.9793,
        1.0262, 0.9821, 1.0310, 1.0112, 1.0362, 1.0663, 1.0254, 1.0138, 1.1071,
        1.0005, 0.9909, 1.0177, 0.9797, 0.9679, 1.0363, 1.0167, 1.0226, 1.0100,
        1.0151, 1.0886, 1.0786, 1.0237, 1.0010, 1.0287, 0.9957, 1.0169, 1.0667,
        1.0264, 1.0507, 1.0392, 1.0514, 1.0374, 1.0116, 1.0329, 1.0241, 0.9856,
        1.0830, 0.9744, 1.0114, 1.0713, 1.0380, 1.0100, 1.0748, 1.0208, 1.0517,
        1.0267, 1.0499, 1.0462, 1.0291, 1.0032, 1.0216, 1.0265, 1.0123, 0.9986,
        1.0033, 1.0116, 1.0155, 1.0425, 1.0296, 1.0704, 1.0182, 1.0250, 1.0150,
        1.0217, 1.0749, 1.0179, 1.0093, 1.0496, 1.0382, 1.0197, 1.0256, 1.0303,
        1.0229, 1.0005, 1.0351, 1.0273, 1.0446, 1.0416, 1.0271, 1.0213, 1.0377,
        1.0048, 1.0470, 1.0061, 1.0175, 1.0355, 1.0273, 0.9422, 1.0191, 1.0005,
        1.0074, 0.9962, 1.0850, 1.0213, 1.0703, 0.9934, 1.0191, 1.0212, 1.0193,
        1.0123, 0.9955, 1.0273, 1.0224, 0.9798, 0.9883, 1.0217, 1.0036, 1.0522,
        1.0153, 1.0528, 1.0041, 1.0013, 0.9808, 1.0372, 1.0096, 1.0069, 1.0044,
        1.0245, 1.0092, 0.9898, 1.0198, 0.9908, 1.0232, 0.9633, 1.0226, 1.0265,
        1.0313, 1.0204, 1.0728, 1.0031, 1.0685, 1.0427, 1.0096, 1.0185, 1.0315,
        0.9570, 1.0243, 0.9904, 1.0189, 1.0064, 1.0148, 1.0121, 0.9928, 1.0378,
        1.0178, 1.0303, 1.0088, 1.0602, 1.0108, 1.0297, 1.0042, 1.0406, 1.0470,
        1.0454, 1.0318, 1.0048, 1.0352, 1.0192, 1.0000, 1.0317, 1.0059, 1.0377,
        1.0272, 1.0351, 1.0370, 1.0213, 1.0311, 1.0315, 1.0229, 1.0113, 1.0498,
        1.0412, 1.0086, 1.0351, 0.9671, 1.0164, 1.0144, 1.0261, 1.0288, 1.0364,
        1.0155, 1.0018, 0.9922, 1.0244, 1.0353, 0.9993, 1.0496, 0.9976, 0.9874,
        1.0157, 1.0121, 1.0479, 0.9897, 1.0105, 1.0119, 1.0226, 1.0263, 1.0638,
        1.0228, 1.0120, 1.0523, 1.0318, 1.0071, 1.0095, 0.9948, 1.0152],
       device='cuda:0'), tensor([-1.7860e-02, -3.0405e-02, -4.0520e-03, -1.6891e-02, -3.4080e-02,
        -1.1972e-02, -4.1383e-02, -1.7212e-02, -2.7822e-02, -3.2490e-02,
        -5.2645e-03, -2.6497e-02, -9.9547e-03, -5.1469e-02, -1.3124e-02,
        -2.7331e-02, -8.5420e-02, -2.0470e-02, -5.2377e-02,  5.9573e-03,
        -4.9048e-03, -4.7141e-02, -3.7558e-02, -5.7566e-02, -1.4401e-02,
        -7.5239e-02, -6.0000e-02, -2.8270e-02, -1.9921e-02,  1.6471e-04,
        -1.4485e-02, -1.6623e-02, -2.2159e-02, -4.2372e-03, -4.8700e-02,
         7.4996e-03, -9.4354e-03, -1.1857e-02, -3.3870e-02, -1.9871e-02,
        -2.4509e-02, -4.0295e-02, -1.2535e-02, -6.9912e-02, -1.3861e-02,
        -1.6896e-02, -9.7108e-03, -2.1842e-02, -5.9081e-02, -3.5944e-02,
        -3.8372e-02, -4.5000e-02, -8.9158e-04, -9.8078e-03, -2.7216e-02,
        -5.6950e-03, -2.1359e-02, -2.1330e-02, -1.4947e-02, -1.7106e-02,
        -1.3600e-02, -6.6556e-05, -1.1832e-02, -1.3354e-02, -3.8707e-02,
        -8.5334e-03, -3.1002e-02, -4.3622e-02, -3.2225e-02, -1.2231e-02,
        -8.2485e-02, -3.0398e-02, -3.8119e-02, -2.1786e-02, -1.9913e-02,
        -2.2401e-02, -1.7080e-02, -8.7907e-02, -4.1193e-02, -2.7691e-02,
        -3.2778e-02, -1.1881e-02, -1.4066e-02, -1.2419e-02, -2.1911e-02,
        -1.6390e-02, -2.3585e-02, -6.2124e-03, -1.7473e-02, -1.7753e-02,
        -6.8097e-02, -2.1424e-02, -8.7146e-04, -3.8138e-02, -1.0237e-02,
        -1.3161e-02, -1.0176e-02, -3.6771e-02,  4.7635e-03, -3.1225e-02,
        -3.9947e-02, -6.2540e-02, -3.1002e-02,  6.0081e-04, -5.2418e-03,
        -4.5124e-02, -7.4708e-02, -1.6401e-02, -3.0907e-03, -4.0509e-02,
        -4.9342e-03, -6.2964e-03, -1.3901e-02,  9.2310e-04, -6.3322e-02,
        -8.4249e-04, -1.2874e-02, -4.6291e-03, -4.2895e-02, -2.6979e-02,
        -4.5743e-03, -3.5979e-02,  1.2279e-03, -3.7248e-02, -1.3859e-02,
        -1.7837e-02, -1.5961e-02, -2.5871e-02, -2.5326e-02, -7.1837e-03,
        -2.2801e-02, -4.7815e-02, -2.5315e-02, -4.3928e-02, -3.7448e-02,
        -4.2048e-02, -1.8938e-02, -2.8172e-02, -5.8810e-02, -2.5585e-02,
        -9.1136e-03, -4.5343e-02, -1.7753e-02, -8.7060e-03, -1.2318e-02,
        -5.7328e-02, -1.8317e-02, -2.8328e-02, -2.6297e-02, -6.6763e-02,
        -2.9848e-02, -3.2543e-02, -6.9790e-02, -3.5778e-02, -5.5990e-02,
        -3.1015e-02, -3.3698e-02, -2.3586e-02, -2.0285e-02, -2.4570e-02,
        -5.9818e-02, -1.6496e-02, -4.7939e-02, -3.4391e-02,  8.4574e-04,
        -1.9155e-04, -5.4210e-02, -1.0988e-02, -2.7697e-02, -7.2736e-02,
        -1.7185e-02, -1.9778e-02, -4.8573e-03, -9.2737e-03, -1.9648e-02,
        -2.6752e-03,  3.2572e-02, -7.4059e-02, -2.5503e-02, -3.4795e-02,
        -1.5432e-02, -1.4706e-03, -2.4717e-02, -1.9944e-02, -1.9132e-02,
        -9.7510e-03, -4.5312e-02, -3.6959e-02, -5.3223e-02, -7.1601e-02,
        -6.5095e-03, -6.4268e-02, -1.2437e-02, -2.3549e-02, -2.6970e-02,
        -4.7233e-03, -8.7550e-04, -1.8606e-02, -2.7531e-02, -2.3777e-02,
        -1.3118e-02, -2.1768e-02, -3.8825e-02,  7.4153e-03, -2.5719e-02,
        -3.4394e-02,  1.7501e-02, -9.9409e-02, -2.3470e-02, -6.6892e-02,
        -4.6426e-02, -4.5747e-03, -8.9558e-03, -1.3207e-02, -2.8680e-03,
        -1.8913e-02, -6.6327e-02, -1.4931e-02, -1.0014e-02, -2.7623e-02,
        -1.7775e-02, -6.4122e-02, -5.8606e-02, -2.2589e-02, -6.9333e-02,
        -2.5968e-02, -1.1047e-02, -4.7506e-03, -4.4084e-02, -3.6430e-02,
        -3.0504e-02, -2.4690e-02, -2.5334e-02, -3.8243e-02, -2.3068e-02,
        -1.7952e-02, -4.7059e-02, -7.9688e-02, -1.4376e-05, -2.2084e-02,
        -4.0552e-02, -2.8862e-02, -1.3514e-02, -1.5210e-02, -9.1633e-03,
        -1.1612e-02, -6.0639e-03, -5.2326e-02, -3.1776e-02, -4.2409e-02,
        -8.1834e-03, -2.9182e-02, -9.9172e-02,  3.9657e-03, -6.1403e-03,
        -3.7474e-03, -2.4285e-02, -8.7812e-02, -7.3317e-02, -3.5961e-02,
        -3.0795e-02, -3.3089e-02, -1.0242e-02, -5.6588e-02, -3.6548e-02,
        -1.5131e-02, -5.5411e-02, -2.4205e-02, -9.7273e-02, -1.6327e-02,
        -1.2767e-04, -2.5962e-02, -1.4742e-02, -8.7343e-02,  7.2710e-03,
        -2.6548e-02, -2.4201e-02, -2.1455e-02, -1.8143e-02, -3.6030e-02,
        -2.7430e-02, -3.6716e-02, -9.4121e-04, -1.6586e-02, -1.5644e-02,
        -1.0422e-02, -2.9443e-02, -4.7405e-02, -1.3001e-02, -5.0305e-02,
        -1.3065e-02, -1.5839e-02, -6.7593e-02, -7.0747e-02, -1.0380e-02,
         1.6026e-04, -1.8709e-02, -1.3443e-02, -4.7843e-02, -3.5128e-02,
        -1.8204e-02, -2.2577e-02, -4.0210e-02, -3.4107e-02, -8.1321e-03,
        -4.1554e-02,  8.1662e-03, -3.4652e-02, -2.8349e-02, -1.9349e-02,
        -8.1574e-03, -8.0067e-02, -9.2449e-03, -3.9932e-02, -5.7057e-02,
        -3.7497e-02, -1.4164e-02, -8.3173e-03, -2.4968e-02, -3.1126e-02,
        -2.1444e-02, -1.1250e-02, -1.3677e-02, -9.9818e-03, -4.8703e-02,
        -1.1099e-01, -9.6748e-02, -1.3189e-02, -3.1521e-02, -2.0744e-02,
        -3.4954e-02, -1.4470e-02, -5.0896e-02,  4.5658e-03, -2.0887e-02,
        -3.3595e-02, -3.0513e-02, -1.7400e-02, -3.0952e-02, -2.4653e-02,
        -3.5979e-02, -5.0843e-02, -5.9149e-02, -5.4061e-02, -1.6857e-02,
        -3.6301e-02, -1.0324e-02, -2.2085e-02, -7.3491e-02, -1.4873e-02,
        -1.3428e-02, -1.8711e-02, -2.2068e-02, -7.9806e-03, -1.5913e-02,
        -2.1931e-02, -2.0903e-02, -4.0699e-03, -3.1932e-02, -2.6091e-02,
         1.5446e-02, -2.6033e-02, -2.0618e-02,  1.8960e-02, -6.0222e-03,
        -5.8958e-02, -1.5573e-02, -7.8904e-03, -1.9980e-02, -1.6356e-02,
        -7.8596e-02,  2.3368e-02, -1.1724e-02, -4.2443e-02, -3.1509e-02,
        -1.0751e-02,  2.6831e-03, -7.5167e-03, -2.4733e-03, -2.6975e-02,
        -9.8123e-03, -1.3896e-02, -1.3064e-02, -1.8547e-02, -1.0177e-02,
        -3.9363e-02, -9.7725e-03, -1.3521e-02, -3.5713e-02, -1.9507e-02,
        -1.3201e-02, -3.9109e-02, -1.2093e-02, -5.3174e-02, -2.2279e-02,
        -2.5697e-02, -4.6796e-02, -2.1889e-02, -9.2023e-02, -2.7854e-02,
        -4.9664e-02, -3.1967e-02, -1.5201e-02, -5.6687e-02, -7.8145e-03,
        -1.7987e-02, -3.5910e-02, -3.9583e-02, -4.0323e-02, -2.8825e-02,
        -3.2283e-02, -3.1884e-02, -2.5237e-02, -3.6700e-02, -1.9867e-02,
        -2.2588e-02, -9.1600e-03, -3.7881e-03, -6.3173e-02, -1.2719e-02,
        -3.0595e-02, -1.4852e-02, -1.7020e-02, -1.6913e-02, -2.1505e-02,
        -2.5547e-02, -1.5434e-02, -4.7074e-03, -3.3496e-02, -5.1984e-02,
        -3.5669e-02, -1.0131e-02, -8.1506e-03, -1.5830e-03, -6.8730e-02,
        -1.9816e-02, -6.3229e-02, -2.3936e-02, -1.3253e-02, -3.0543e-02,
        -5.5190e-02, -7.9418e-02, -1.3142e-02, -5.1903e-02, -3.9411e-03,
        -2.4298e-02, -2.5618e-02, -1.1598e-02, -6.6130e-02, -6.7403e-03,
        -3.6781e-02, -2.0902e-02, -8.1434e-03, -7.6420e-02, -1.5935e-02,
        -2.5263e-03, -2.7632e-02, -2.2592e-02, -1.5922e-02, -3.8670e-02,
        -7.4867e-02, -2.0875e-02,  5.1085e-03,  5.0451e-03, -2.5128e-02,
        -2.0304e-02, -6.4364e-02, -3.7629e-02, -1.4749e-02, -4.6224e-02,
        -3.4993e-03, -2.1804e-02, -3.0278e-02, -2.7654e-02,  1.1024e-03,
        -1.9274e-02,  7.9610e-03, -4.1876e-02, -1.2529e-02, -9.4868e-03,
        -2.7522e-02, -2.3386e-02, -5.3085e-03, -1.3677e-02, -1.4001e-02,
        -1.0817e-02, -1.0831e-02, -1.3238e-02, -1.8500e-02, -2.0010e-02,
        -8.4722e-02, -1.4964e-02, -5.6441e-02, -2.4969e-02, -2.6623e-02,
        -6.8169e-03, -1.0866e-02, -2.3636e-02, -6.1239e-02, -2.9181e-02,
        -1.7253e-02, -1.4702e-02, -1.2399e-02, -6.8436e-02, -1.9479e-02,
        -1.4920e-02, -2.9622e-03, -3.8465e-02, -2.3288e-02, -2.8712e-02,
        -3.7974e-02, -8.4999e-03], device='cuda:0'), tensor([[[-2.9956e-02,  5.6065e-03,  9.1897e-03,  2.0309e-02, -1.5310e-02],
         [ 2.6979e-02, -1.7472e-02, -8.3559e-05, -5.6035e-03,  5.1226e-03],
         [ 3.0282e-03,  1.7571e-02, -3.0964e-03, -2.5073e-02, -2.3574e-03],
         ...,
         [ 9.3401e-03, -8.7923e-03, -1.0328e-02, -1.3415e-02,  1.0315e-02],
         [ 6.3105e-05, -3.6197e-03,  1.2892e-02, -1.4447e-02,  2.7503e-03],
         [ 1.2337e-02, -1.1810e-02, -1.0387e-02, -9.8488e-03,  5.7258e-03]],

        [[-2.2253e-03, -2.1550e-02, -5.9435e-03, -3.9070e-02, -2.7274e-02],
         [-1.8242e-02,  1.5717e-02, -1.8116e-03,  3.4525e-03, -5.7501e-03],
         [-1.5005e-02,  6.9269e-04,  1.4498e-02,  1.6210e-02, -5.5239e-03],
         ...,
         [-1.2348e-02,  7.8506e-04,  6.2499e-03, -2.2324e-02, -2.4252e-02],
         [ 1.3363e-02,  2.4029e-02, -2.7848e-03,  5.4629e-03, -1.7434e-02],
         [ 1.3316e-02,  2.0786e-03, -4.2457e-02,  2.5621e-03,  1.3005e-02]],

        [[ 9.8243e-03,  4.9940e-03,  3.8880e-02,  3.3970e-03, -5.6915e-03],
         [ 1.1787e-02, -6.1957e-03,  2.4138e-02, -1.9255e-02, -2.3864e-02],
         [-1.2851e-02, -3.0262e-02, -3.5581e-03,  6.9812e-03,  7.5142e-03],
         ...,
         [-1.1672e-02,  5.8218e-03,  2.6877e-03,  8.3095e-03,  3.0024e-02],
         [-2.8128e-02, -3.2689e-02, -2.4458e-03, -2.9050e-02,  1.9572e-02],
         [ 1.4925e-02, -6.1826e-03,  6.9019e-03,  1.3286e-02, -1.3731e-02]],

        ...,

        [[-3.2219e-02, -2.5037e-02, -1.0791e-02, -1.7408e-03, -7.2252e-03],
         [-2.4480e-02,  2.6176e-02,  1.6396e-02, -2.1570e-02,  3.3198e-03],
         [ 7.3743e-04,  1.1797e-02, -2.5057e-02, -2.7459e-02,  5.7465e-03],
         ...,
         [ 1.3378e-02, -2.2789e-02, -4.3444e-02, -2.1892e-02,  6.9137e-03],
         [-5.8179e-03,  6.4869e-03, -4.8744e-04, -7.8169e-04, -1.4702e-02],
         [ 1.4864e-02,  1.3699e-03,  1.8421e-02,  2.4831e-02,  2.1414e-02]],

        [[ 2.6368e-02,  3.3423e-03,  1.0855e-02, -6.1981e-03, -1.4780e-02],
         [-1.3551e-02, -1.6898e-02, -2.6204e-03, -1.2930e-02, -8.7207e-03],
         [-1.2746e-03,  2.2605e-02, -3.1857e-02, -8.1735e-03, -1.9784e-02],
         ...,
         [-6.9070e-03,  2.3498e-02, -1.8048e-02,  2.9666e-02,  2.1238e-03],
         [-2.9940e-02,  1.2364e-02, -1.5332e-02, -6.4677e-03, -1.5773e-02],
         [-1.1372e-02, -1.4859e-02,  1.3830e-02, -6.6460e-03, -7.9397e-03]],

        [[-9.3871e-03, -1.5145e-02,  1.9425e-02, -1.2475e-02, -6.0237e-03],
         [-7.8017e-03, -2.2249e-02, -1.9860e-02,  1.1135e-02,  2.0343e-02],
         [ 1.6184e-03, -1.0591e-02, -2.5609e-02, -2.0860e-02, -3.6626e-02],
         ...,
         [ 1.6058e-02, -1.5184e-02, -1.9888e-02, -2.2743e-03, -3.4119e-02],
         [-9.3618e-03,  1.1659e-02, -1.3615e-02, -4.6391e-03, -8.9910e-03],
         [ 1.7549e-02, -5.9880e-03, -1.3627e-02, -2.2613e-02, -1.7729e-03]]],
       device='cuda:0'), tensor([-6.5025e-03, -6.7928e-04,  6.6480e-03, -6.8711e-04, -1.1227e-02,
         1.0999e-02,  7.1594e-04,  1.3394e-02, -6.1787e-04,  2.6368e-02,
        -8.0400e-03, -2.1126e-03, -2.0577e-02,  9.8363e-03,  1.1291e-02,
        -5.4843e-03, -7.6238e-03, -1.5936e-02, -1.2710e-02,  1.1831e-02,
        -1.4226e-03,  1.5501e-02,  1.3037e-02, -1.1984e-03, -1.3950e-02,
        -8.2598e-02, -8.9253e-03,  1.4771e-02, -1.1896e-03,  4.8444e-03,
         8.6888e-03,  2.4121e-02,  1.5187e-02,  8.4074e-03,  2.6108e-02,
        -1.0844e-03,  8.4229e-03,  7.8083e-03,  4.6929e-03,  1.9951e-02,
        -5.4973e-03, -1.2093e-02,  1.6975e-02,  8.5931e-03,  1.5256e-02,
         4.3494e-03, -7.5202e-03, -1.9260e-02, -1.3418e-02,  1.3278e-02,
         1.6927e-02, -9.4504e-03,  1.5715e-02,  1.7494e-02, -1.1422e-03,
         2.1728e-02, -1.7152e-03, -7.7462e-02, -1.2350e-02, -7.4463e-03,
        -1.3501e-02, -1.3628e-02,  1.4897e-02, -2.8967e-03, -7.2169e-03,
         2.4505e-03,  1.1332e-02, -6.5483e-02,  1.7556e-02,  1.1501e-02,
        -2.1153e-02,  2.3830e-02,  7.8540e-04,  1.4160e-02,  8.2070e-03,
        -3.7678e-03, -7.2271e-02, -2.4167e-03,  5.9322e-03,  1.7202e-02,
        -7.6107e-03,  1.4586e-02, -6.4747e-03,  6.0994e-03,  8.8922e-03,
        -1.3681e-02,  4.4445e-03, -7.0510e-03,  1.3201e-03, -2.0704e-02,
         1.0086e-02,  1.2721e-02, -1.3032e-02, -1.3091e-02,  2.0564e-02,
         1.7932e-02,  1.1923e-02, -6.4721e-02, -1.2276e-02,  1.7382e-02,
         7.0119e-03, -1.1950e-02,  1.5928e-02,  1.5178e-02,  1.9459e-02,
        -5.9940e-03,  1.2813e-02, -1.4016e-03, -1.1241e-02, -1.0852e-02,
        -3.3568e-03,  2.1557e-03,  1.9303e-02,  1.3271e-02, -1.8209e-02,
        -1.6428e-03, -5.4587e-03, -6.9365e-03, -1.5460e-02, -7.4107e-03,
         4.2607e-03, -1.2984e-02,  2.6387e-03,  2.4152e-02,  4.7421e-03,
        -2.8374e-03,  5.3965e-03, -1.6432e-02, -4.0774e-03,  2.0654e-02,
         9.1462e-03, -1.9053e-02,  1.5287e-02,  7.5503e-03,  4.2782e-03,
        -9.8705e-03,  1.6263e-02, -5.5385e-02, -6.2715e-03,  7.5131e-03,
        -9.4075e-03,  2.1717e-02, -1.9421e-03,  5.0254e-04, -1.7469e-03,
        -3.7183e-03, -7.1543e-03, -1.8838e-03, -2.8712e-04,  1.8825e-02,
        -8.6509e-02,  2.8569e-03, -5.8317e-04, -6.0344e-04,  1.3301e-03,
         1.9483e-02, -1.1923e-02,  1.4653e-02,  8.4871e-03, -1.2571e-02,
         1.9844e-02, -1.2030e-02,  1.9681e-02, -5.0157e-03, -1.1625e-02,
        -7.1707e-03, -8.1198e-03, -1.2826e-02, -9.5399e-03, -1.9881e-02,
         5.2361e-03,  1.3638e-02,  1.9936e-02,  9.8858e-04,  5.5786e-03,
         1.3754e-03, -2.1184e-03, -1.1338e-02, -1.4706e-02, -2.8917e-03,
        -6.9116e-03, -8.1095e-03, -4.3792e-03, -2.3130e-02, -7.2598e-03,
         1.1407e-02, -1.9027e-02,  9.8834e-03, -1.3375e-02,  4.9929e-03,
        -7.4527e-03,  7.4827e-03, -8.6816e-02,  3.3964e-03,  1.7229e-02,
        -1.7251e-03, -1.1837e-02, -1.4028e-02, -1.1530e-02,  6.4191e-03,
         1.5968e-03,  1.2254e-02,  8.1233e-03,  1.1649e-02,  2.1433e-02,
        -1.5161e-02, -7.5124e-03,  1.5706e-02, -4.4647e-02,  4.5948e-03,
         2.4102e-02, -4.3320e-02, -2.3226e-03,  1.1803e-02,  2.5166e-02,
        -1.3747e-02,  1.6455e-02,  1.6297e-02,  9.6849e-04, -5.4381e-03,
         8.4522e-03,  1.5260e-02,  6.3946e-06, -9.0469e-03, -1.7814e-03,
        -7.5715e-03,  1.0586e-03,  1.0437e-02,  1.0462e-02,  9.7196e-03,
        -2.1051e-03, -1.7511e-02, -1.5638e-02,  7.3036e-03, -6.8998e-03,
        -3.6947e-03,  6.5306e-03, -1.9825e-02, -1.4451e-02, -6.6237e-03,
        -1.2153e-02,  1.1281e-02,  5.5005e-03,  1.8725e-02,  1.4730e-02,
        -4.5151e-03,  2.8188e-03,  6.1804e-03, -3.0127e-03,  5.8459e-03,
         1.8881e-02, -6.4400e-02,  1.3812e-02, -8.3794e-02,  2.4540e-02,
        -2.7239e-03, -6.8092e-02, -7.5798e-02,  1.3683e-02,  1.8554e-03,
         7.6723e-03, -1.4146e-02,  2.9423e-03, -1.8184e-03,  3.8559e-03,
        -1.8105e-02, -2.4476e-02,  4.0709e-03, -1.4963e-02,  2.4555e-03,
         1.3206e-02, -2.0661e-03,  3.0525e-03,  2.1694e-03, -3.1683e-03,
        -4.7141e-03, -1.1277e-02,  1.3304e-02,  8.6260e-03,  1.6304e-02,
         1.0098e-02,  2.3149e-03,  2.3766e-02,  9.9616e-03,  9.2198e-03,
        -1.1663e-02, -2.3449e-02,  1.6941e-03, -4.8788e-03,  4.8674e-03,
         5.9602e-03, -2.0743e-02,  2.1569e-03,  1.3530e-02, -1.5512e-03,
        -4.9826e-03,  2.5194e-04,  2.6030e-02,  1.1742e-02, -7.8476e-02,
         1.4888e-02,  6.4400e-03, -9.2998e-03,  1.1741e-02,  5.8383e-03,
         1.1827e-02, -1.4247e-02, -1.1253e-02,  1.9872e-02,  1.9921e-03,
         2.1058e-02, -1.7127e-02,  8.3752e-04, -6.1654e-03, -5.6062e-03,
         1.3771e-02,  7.4999e-03,  8.1036e-03, -4.0703e-03, -1.8357e-04,
         4.5984e-04,  1.6673e-02, -7.4478e-03,  1.0168e-02, -1.8342e-02,
         6.7052e-03, -1.2022e-02,  1.8905e-03,  4.0190e-03,  1.0663e-02,
        -9.0114e-03,  4.8515e-03, -2.1095e-02,  1.2667e-03, -6.7305e-02,
         7.1875e-03, -6.1409e-03, -2.0454e-03,  1.9935e-02,  1.2016e-02,
        -1.4861e-02, -1.1943e-02, -1.3549e-02, -1.7745e-02, -1.6775e-02,
         1.4492e-02,  1.0739e-03,  5.8287e-03, -6.4200e-03, -3.2221e-03,
         2.6271e-03, -1.1766e-02, -1.6649e-02, -1.0678e-03,  9.2090e-03,
        -1.1661e-02,  9.5278e-03,  1.1922e-02,  1.2503e-02, -9.5418e-03,
        -2.9301e-02, -9.2007e-03,  1.0571e-02, -1.4695e-02,  1.3963e-02,
         6.8832e-03, -2.3600e-02,  1.1633e-02,  2.3277e-02, -1.1135e-02,
         2.9979e-02,  9.6256e-03,  1.5386e-02, -1.5850e-02, -8.9958e-03,
         1.4691e-03, -6.8495e-02, -1.4842e-02,  1.5840e-02, -1.6413e-02,
        -7.7060e-03,  1.4409e-03,  1.0352e-02, -1.5405e-02, -8.0354e-03,
         1.6910e-02, -5.9836e-03,  5.6733e-03, -5.6877e-03, -1.1317e-03,
         1.8186e-03, -7.8047e-03,  1.7505e-02,  2.3331e-02, -1.4390e-03,
         6.5038e-03,  1.1132e-02, -1.0011e-01,  1.9025e-02,  1.8445e-02,
         2.1054e-02,  2.9830e-03,  9.0701e-03, -1.0541e-02, -1.7781e-02,
        -7.4854e-03,  1.3020e-02, -1.4185e-02, -1.0373e-02,  1.8480e-02,
        -1.0748e-02, -1.3422e-02,  1.2633e-02, -5.4729e-03,  2.4463e-03,
        -1.2690e-02,  1.4521e-02,  1.4161e-02,  1.0054e-02, -2.1617e-03,
        -6.0263e-03,  4.4537e-04,  2.0927e-02,  1.1735e-02, -2.6384e-02,
         2.4777e-03,  5.7616e-03,  9.4455e-03,  9.1827e-03,  3.0901e-02,
         7.8937e-03,  6.8507e-03, -6.7057e-02,  8.3214e-03,  2.0804e-02,
         2.1203e-02,  8.4586e-03, -3.3873e-04,  5.1764e-03,  2.0586e-02,
         1.4248e-02, -7.7578e-04,  2.9357e-03,  3.0015e-03, -7.4592e-03,
         8.6659e-03, -1.4865e-03,  1.4464e-02, -3.0321e-03,  3.6600e-03,
        -8.9136e-03,  1.8233e-02,  1.6350e-02,  5.9717e-03, -2.1954e-02,
         1.0492e-02, -1.0440e-03, -1.1061e-02, -3.2007e-04, -1.8460e-03,
         1.6774e-02,  5.1696e-04,  8.7386e-03,  4.5096e-03,  1.9194e-02,
        -1.4814e-02,  9.6992e-03,  3.4379e-03, -1.2778e-02,  7.3942e-04,
        -1.8766e-02, -1.1577e-02, -1.2428e-03,  4.4861e-03,  4.5510e-03,
        -1.3896e-02,  1.6461e-02, -3.4679e-03,  2.2784e-03, -2.5587e-03,
        -1.8893e-04, -5.6361e-03,  1.9152e-02,  1.1999e-02, -4.2652e-03,
         7.2031e-03,  1.0272e-02, -3.8120e-02,  7.1869e-05, -3.4837e-03,
        -1.4363e-03, -2.3825e-02,  1.3402e-02,  5.8710e-03,  2.3960e-03,
        -1.1901e-02,  1.9832e-02,  5.2426e-03, -4.0728e-02, -9.6876e-04,
        -8.5038e-02,  8.7724e-03,  3.0430e-03, -8.3200e-03, -8.1369e-03,
        -1.0203e-02, -1.6975e-02,  9.2932e-03, -1.0557e-02,  3.2911e-02,
        -6.3113e-03,  6.6448e-03], device='cuda:0'), tensor([0.9945, 0.9977, 1.0052, 1.0068, 0.9979, 0.9934, 0.9838, 0.9943, 1.0130,
        0.9848, 0.9968, 0.9951, 0.9821, 0.9978, 0.9959, 0.9873, 0.9954, 1.0008,
        0.9971, 0.9828, 1.0209, 0.9717, 1.0178, 0.9976, 0.9994, 1.0110, 0.9973,
        0.9872, 1.0071, 0.9967, 0.9873, 0.9938, 0.9995, 0.9868, 0.9978, 0.9888,
        1.0143, 0.9986, 0.9883, 1.0046, 0.9846, 0.9884, 1.0240, 0.9940, 0.9973,
        1.0067, 0.9948, 1.0004, 0.9868, 1.0359, 1.0037, 0.9802, 0.9920, 0.9979,
        0.9972, 1.0158, 0.9781, 1.0240, 1.0386, 1.0006, 0.9950, 0.9653, 0.9869,
        0.9930, 1.0002, 1.0091, 0.9940, 1.0138, 0.9929, 1.0003, 0.9947, 0.9857,
        0.9959, 1.0173, 0.9968, 1.0038, 1.0140, 0.9981, 0.9765, 0.9944, 0.9981,
        0.9850, 0.9966, 0.9878, 1.0072, 0.9945, 0.9875, 0.9815, 0.9951, 1.0465,
        0.9871, 0.9888, 0.9877, 0.9965, 1.0066, 1.0132, 0.9834, 1.0117, 0.9856,
        0.9805, 0.9969, 0.9926, 1.0005, 0.9927, 0.9998, 1.0131, 0.9929, 0.9744,
        0.9897, 1.0014, 0.9886, 1.0053, 0.9799, 0.9950, 1.0486, 0.9995, 0.9839,
        0.9875, 1.0197, 1.0133, 0.9912, 0.9967, 0.9784, 0.9833, 1.0205, 1.0039,
        0.9948, 0.9679, 0.9843, 1.0067, 0.9913, 0.9829, 0.9954, 0.9702, 0.9844,
        0.9923, 0.9849, 1.0247, 1.0082, 1.0002, 1.0078, 0.9921, 0.9826, 0.9746,
        0.9889, 1.0655, 0.9970, 0.9966, 1.0054, 0.9789, 1.0094, 1.0105, 0.9998,
        1.0139, 0.9994, 1.0004, 0.9974, 0.9989, 0.9840, 0.9906, 0.9989, 1.0168,
        0.9899, 0.9959, 1.0039, 1.0010, 1.0242, 0.9894, 1.0109, 1.0269, 0.9873,
        0.9947, 0.9997, 0.9857, 0.9945, 1.0124, 0.9839, 1.0072, 0.9833, 0.9972,
        0.9838, 1.0113, 1.0111, 0.9746, 0.9765, 0.9845, 1.1000, 0.9978, 1.0007,
        0.9780, 0.9943, 1.0057, 1.0152, 0.9842, 1.0005, 0.9985, 0.9849, 0.9930,
        1.0183, 0.9871, 1.0200, 0.9868, 0.9852, 0.9834, 0.9949, 0.9986, 1.0053,
        0.9884, 1.0608, 0.9886, 1.0059, 1.0344, 0.9772, 1.0065, 0.9930, 0.9847,
        0.9791, 0.9950, 0.9989, 0.9784, 1.0001, 0.9822, 0.9897, 1.0024, 1.0124,
        1.0009, 1.0010, 0.9923, 0.9909, 0.9980, 0.9929, 1.0031, 1.0026, 1.0055,
        0.9934, 0.9805, 0.9884, 0.9870, 0.9995, 0.9873, 0.9895, 0.9852, 1.0018,
        0.9821, 0.9805, 0.9850, 1.0079, 0.9893, 0.9924, 0.9913, 0.9963, 1.0241,
        1.0087, 1.0253, 0.9841, 1.0071, 1.0111, 1.0203, 0.9850, 1.0021, 0.9969,
        0.9859, 0.9988, 1.0061, 0.9789, 1.0041, 0.9800, 0.9963, 0.9951, 0.9930,
        0.9883, 0.9936, 0.9856, 0.9934, 0.9968, 1.0119, 0.9982, 0.9985, 0.9722,
        0.9888, 1.0053, 0.9851, 0.9999, 0.9739, 0.9884, 1.0030, 1.0180, 1.0049,
        0.9995, 0.9967, 1.0026, 1.0342, 0.9809, 1.0008, 0.9902, 0.9912, 1.0123,
        0.9925, 0.9911, 1.0062, 0.9844, 0.9866, 0.9978, 0.9996, 0.9915, 0.9960,
        1.0256, 0.9894, 0.9979, 1.0388, 0.9831, 0.9921, 0.9944, 0.9815, 1.0085,
        0.9983, 0.9866, 0.9737, 1.0053, 1.0059, 0.9947, 1.0315, 0.9867, 0.9954,
        1.0329, 0.9980, 0.9602, 1.0075, 0.9964, 0.9851, 0.9896, 0.9852, 0.9970,
        1.0202, 1.0160, 0.9880, 0.9918, 0.9909, 0.9784, 1.0123, 1.0025, 0.9865,
        0.9929, 0.9685, 0.9833, 0.9837, 0.9884, 0.9883, 0.9987, 0.9936, 0.9910,
        0.9968, 0.9905, 0.9929, 0.9883, 1.0085, 1.0065, 0.9846, 0.9866, 1.0104,
        1.0224, 0.9987, 0.9843, 1.0266, 1.0150, 1.0082, 1.0537, 0.9891, 0.9934,
        0.9828, 0.9866, 0.9913, 0.9882, 0.9787, 1.0007, 0.9977, 1.0111, 0.9876,
        0.9965, 1.0075, 1.0789, 0.9962, 0.9947, 0.9919, 0.9842, 0.9894, 1.0163,
        0.9897, 1.0394, 0.9794, 1.0242, 0.9847, 1.0102, 0.9827, 1.0123, 0.9825,
        1.0118, 1.0053, 0.9797, 0.9999, 0.9869, 0.9951, 0.9917, 1.0051, 1.0039,
        1.0106, 0.9733, 0.9919, 0.9898, 0.9846, 0.9878, 1.0251, 0.9815, 1.0006,
        0.9856, 0.9923, 0.9802, 1.0000, 1.0119, 0.9888, 1.0071, 0.9984, 0.9869,
        0.9983, 1.0213, 0.9971, 0.9970, 1.0191, 0.9814, 0.9889, 0.9974, 0.9804,
        1.0245, 1.0037, 0.9992, 0.9979, 0.9912, 0.9838, 0.9828, 0.9898, 0.9908,
        0.9982, 1.0029, 1.0029, 1.0018, 0.9965, 0.9518, 0.9852, 0.9917, 0.9997,
        0.9878, 0.9841, 1.0017, 0.9818, 1.0574, 0.9909, 0.9965, 0.9779, 0.9957,
        0.9907, 0.9889, 0.9971, 0.9929, 0.9844, 0.9820, 0.9992, 0.9891, 0.9800,
        1.0102, 0.9993, 0.9983, 0.9882, 1.0057, 0.9882, 0.9858, 1.0009, 0.9923,
        0.9940, 0.9821, 1.0008, 1.0036, 0.9947, 0.9812, 0.9933, 1.0022, 0.9927,
        1.0065, 1.0263, 1.0021, 0.9952, 0.9819, 0.9857, 0.9919, 1.0344, 0.9797,
        0.9833, 0.9905, 1.0091, 1.0232, 1.0219, 1.0162, 0.9983, 0.9959, 0.9973,
        0.9992, 0.9977, 0.9793, 0.9809, 1.0148, 0.9928, 1.0034, 1.0044],
       device='cuda:0'), tensor([-0.0049, -0.0067, -0.0142, -0.0090, -0.0097, -0.0076, -0.0164, -0.0142,
        -0.0081, -0.0130, -0.0085, -0.0082, -0.0098, -0.0114, -0.0177, -0.0150,
        -0.0072, -0.0055, -0.0039, -0.0157, -0.0157, -0.0075, -0.0100, -0.0176,
        -0.0102, -0.0134, -0.0063, -0.0039, -0.0046, -0.0093, -0.0142, -0.0170,
        -0.0151, -0.0165, -0.0191, -0.0186, -0.0192, -0.0162, -0.0060, -0.0125,
        -0.0061, -0.0146, -0.0097, -0.0092, -0.0124, -0.0130, -0.0177, -0.0033,
        -0.0056, -0.0128, -0.0130, -0.0154, -0.0135, -0.0139, -0.0148, -0.0107,
        -0.0090,  0.0133, -0.0198, -0.0059, -0.0143, -0.0120, -0.0107, -0.0155,
        -0.0129, -0.0119, -0.0171, -0.0043, -0.0186, -0.0133, -0.0103, -0.0210,
        -0.0048, -0.0149, -0.0098, -0.0146,  0.0035, -0.0103, -0.0159, -0.0202,
        -0.0022, -0.0123, -0.0188, -0.0073, -0.0113, -0.0192, -0.0154, -0.0052,
        -0.0125, -0.0167, -0.0156, -0.0155, -0.0202, -0.0019, -0.0113, -0.0186,
        -0.0217, -0.0071, -0.0126, -0.0154, -0.0142, -0.0156, -0.0107, -0.0174,
        -0.0064, -0.0227, -0.0123, -0.0223, -0.0041, -0.0228, -0.0012, -0.0162,
        -0.0182, -0.0050, -0.0178, -0.0124, -0.0099, -0.0128, -0.0150, -0.0190,
        -0.0169, -0.0198, -0.0080, -0.0170, -0.0110, -0.0209, -0.0089, -0.0160,
        -0.0147, -0.0177, -0.0129, -0.0079, -0.0142, -0.0084, -0.0060, -0.0052,
        -0.0164, -0.0073, -0.0179, -0.0080, -0.0075, -0.0131, -0.0178, -0.0200,
        -0.0075, -0.0139, -0.0096, -0.0100, -0.0080, -0.0117,  0.0068, -0.0161,
        -0.0082, -0.0155, -0.0100, -0.0167, -0.0183, -0.0155, -0.0049, -0.0052,
        -0.0117, -0.0136, -0.0106, -0.0086, -0.0098, -0.0112, -0.0121, -0.0167,
        -0.0123, -0.0143, -0.0161, -0.0065, -0.0160, -0.0143, -0.0051, -0.0153,
        -0.0065, -0.0121, -0.0155, -0.0066, -0.0083, -0.0109, -0.0094, -0.0078,
        -0.0177, -0.0097, -0.0184, -0.0169, -0.0131, -0.0140, -0.0083, -0.0100,
        -0.0154, -0.0150, -0.0018, -0.0072, -0.0157, -0.0166, -0.0092, -0.0100,
        -0.0165, -0.0076, -0.0101, -0.0169, -0.0042, -0.0096, -0.0110, -0.0125,
        -0.0148, -0.0142, -0.0115, -0.0177, -0.0058, -0.0223, -0.0205, -0.0081,
        -0.0229, -0.0145, -0.0076, -0.0107, -0.0030, -0.0077, -0.0210, -0.0127,
        -0.0180, -0.0146, -0.0099, -0.0101, -0.0097, -0.0040, -0.0068, -0.0103,
        -0.0004, -0.0112, -0.0079, -0.0057, -0.0174, -0.0048, -0.0146, -0.0091,
        -0.0078, -0.0089, -0.0098, -0.0139, -0.0206, -0.0122, -0.0053, -0.0125,
        -0.0160, -0.0102, -0.0142, -0.0130, -0.0084, -0.0027, -0.0181, -0.0146,
         0.0016,  0.0132, -0.0099, -0.0148, -0.0165, -0.0108, -0.0239, -0.0089,
        -0.0160, -0.0139, -0.0117, -0.0122, -0.0190, -0.0170, -0.0124, -0.0185,
        -0.0153, -0.0066, -0.0155, -0.0174, -0.0075, -0.0161, -0.0229, -0.0194,
        -0.0167, -0.0092, -0.0134, -0.0062, -0.0153, -0.0165, -0.0118, -0.0026,
        -0.0115, -0.0150, -0.0137, -0.0126, -0.0146, -0.0101, -0.0019, -0.0055,
        -0.0167, -0.0013, -0.0150, -0.0083, -0.0092, -0.0122, -0.0206, -0.0155,
        -0.0071, -0.0200, -0.0234, -0.0014, -0.0197, -0.0148, -0.0226, -0.0202,
        -0.0079, -0.0109, -0.0161, -0.0210, -0.0119, -0.0153, -0.0121, -0.0172,
        -0.0233, -0.0173, -0.0110, -0.0115, -0.0134, -0.0121, -0.0085, -0.0100,
        -0.0150, -0.0076, -0.0151, -0.0117, -0.0134, -0.0059,  0.0163, -0.0192,
        -0.0142, -0.0068, -0.0148, -0.0109, -0.0090, -0.0127, -0.0110, -0.0100,
        -0.0039, -0.0100, -0.0089, -0.0142, -0.0128, -0.0102, -0.0061, -0.0134,
        -0.0112, -0.0165, -0.0151, -0.0121, -0.0076, -0.0063, -0.0182, -0.0086,
        -0.0090, -0.0110, -0.0130, -0.0131, -0.0121, -0.0067, -0.0230, -0.0183,
        -0.0009, -0.0144, -0.0146, -0.0174, -0.0136, -0.0037, -0.0095, -0.0141,
        -0.0080, -0.0075, -0.0064, -0.0129, -0.0085, -0.0108, -0.0089, -0.0176,
        -0.0058, -0.0055, -0.0124, -0.0156, -0.0151, -0.0161, -0.0094, -0.0182,
        -0.0192, -0.0096, -0.0161, -0.0173, -0.0077, -0.0106, -0.0173, -0.0172,
        -0.0098, -0.0163, -0.0088, -0.0117, -0.0096, -0.0097, -0.0099, -0.0171,
        -0.0100, -0.0136, -0.0099, -0.0117, -0.0133, -0.0174, -0.0159, -0.0061,
        -0.0150, -0.0078, -0.0095, -0.0129, -0.0160, -0.0144, -0.0079, -0.0108,
        -0.0093, -0.0124, -0.0103, -0.0087, -0.0184, -0.0220, -0.0116, -0.0060,
         0.0016, -0.0100, -0.0163, -0.0136, -0.0175, -0.0087, -0.0140, -0.0257,
        -0.0155, -0.0026, -0.0081, -0.0147, -0.0123, -0.0079, -0.0078, -0.0104,
        -0.0198, -0.0204, -0.0156, -0.0241, -0.0094, -0.0144, -0.0209, -0.0118,
        -0.0063, -0.0137, -0.0079, -0.0169, -0.0191, -0.0045, -0.0174, -0.0042,
        -0.0063, -0.0154, -0.0154, -0.0095, -0.0135, -0.0142, -0.0062, -0.0126,
        -0.0104, -0.0166, -0.0178, -0.0111, -0.0155, -0.0153, -0.0139, -0.0184,
        -0.0074, -0.0189, -0.0095, -0.0122, -0.0163, -0.0188, -0.0118, -0.0045,
        -0.0139, -0.0239, -0.0091, -0.0105, -0.0035, -0.0086, -0.0090, -0.0208,
        -0.0213, -0.0098, -0.0058, -0.0105, -0.0006, -0.0229, -0.0096, -0.0180,
        -0.0129, -0.0209, -0.0040, -0.0175, -0.0084, -0.0075, -0.0098, -0.0079],
       device='cuda:0'), tensor([[ 0.0287, -0.0505, -0.0202,  ...,  0.0290,  0.0027, -0.0049],
        [ 0.0059, -0.0483,  0.0047,  ..., -0.0169, -0.0481, -0.0221],
        [-0.0092, -0.0340,  0.0324,  ..., -0.0247, -0.0276,  0.0350],
        ...,
        [ 0.0231,  0.0165, -0.0261,  ..., -0.0236, -0.0061,  0.0446],
        [ 0.0267, -0.0447,  0.0164,  ..., -0.0358,  0.0264,  0.0297],
        [ 0.0421,  0.0241,  0.0306,  ...,  0.0136, -0.0251, -0.0496]],
       device='cuda:0'), tensor([-0.0397,  0.0306, -0.0116,  ...,  0.0156, -0.0392, -0.0114],
       device='cuda:0'), tensor([[[ 1.8955e-02, -1.2269e-02,  4.1964e-05,  1.7806e-02],
         [ 2.1445e-02,  7.3310e-03,  1.2623e-02, -2.6293e-02],
         [-1.0088e-02,  1.1302e-02, -1.0575e-02, -2.2775e-03],
         ...,
         [-1.6456e-02,  7.3830e-03, -1.3074e-02,  1.8497e-02],
         [ 1.6365e-03,  6.8668e-03,  7.8282e-04, -1.1289e-02],
         [-7.3922e-03, -4.2743e-03,  1.4058e-02,  1.6231e-02]],

        [[ 6.2822e-03,  1.6701e-02, -3.0573e-03, -5.3121e-03],
         [ 5.2277e-03,  5.7068e-03, -1.6470e-02, -6.6922e-03],
         [ 3.0411e-03,  8.3123e-03, -5.9526e-03,  3.1403e-03],
         ...,
         [-1.2056e-02, -1.8375e-02, -2.5433e-02,  9.7125e-03],
         [ 1.0489e-02,  1.3494e-02,  4.0134e-03,  8.5019e-03],
         [-1.5901e-02,  3.6992e-03,  2.6307e-03,  4.3676e-03]],

        [[-8.8856e-04, -1.6988e-03,  7.2762e-03,  3.4070e-03],
         [ 6.9873e-03,  8.0169e-03, -1.9443e-03,  1.2343e-02],
         [-9.9376e-03, -8.9456e-03, -1.1518e-02,  4.8492e-05],
         ...,
         [-2.3939e-02, -9.7439e-03, -3.4042e-03,  7.5176e-03],
         [ 7.3669e-03, -9.5604e-03, -1.9867e-02, -2.2846e-02],
         [ 5.3927e-04,  1.4206e-02,  1.5012e-02,  5.4441e-03]],

        ...,

        [[-6.9416e-03, -3.7144e-03,  2.5971e-02, -3.3895e-03],
         [ 1.6440e-02,  1.6852e-03,  2.0087e-02, -7.4061e-03],
         [ 1.6304e-02,  8.8075e-03, -1.3773e-02, -2.3626e-02],
         ...,
         [ 1.4216e-02, -1.9249e-02,  1.5970e-02, -5.3802e-03],
         [-2.8392e-03,  6.1540e-03, -2.8690e-02,  1.0857e-02],
         [ 1.1620e-02, -2.1363e-02, -1.4481e-02, -1.2800e-02]],

        [[-1.3177e-02,  3.1254e-04, -2.2000e-02,  1.7160e-02],
         [ 1.0530e-02, -7.7991e-03, -9.5888e-03,  8.9804e-04],
         [ 9.6946e-04,  1.0612e-02, -3.8547e-03,  8.5995e-03],
         ...,
         [ 2.2347e-02, -6.4090e-03, -6.1554e-03, -6.2121e-04],
         [ 2.7744e-03, -1.5382e-02, -1.0228e-02, -1.6706e-02],
         [-1.9854e-04, -1.7243e-02,  1.3138e-02,  7.8084e-03]],

        [[-2.0206e-02, -2.2931e-02, -1.3896e-02,  1.5466e-02],
         [ 6.7571e-04, -1.6732e-02,  3.5520e-03, -7.2556e-03],
         [-3.8221e-03, -9.2430e-03, -1.7581e-02, -1.1987e-03],
         ...,
         [-1.6306e-02,  8.2519e-03, -4.5570e-03, -5.6920e-04],
         [ 1.2855e-02, -2.4964e-02, -1.1570e-03,  2.8737e-02],
         [ 2.0361e-02,  7.3357e-03, -7.9200e-03, -5.5228e-03]]],
       device='cuda:0'), tensor([-1.6316e-03,  1.1785e-02,  1.1243e-02,  1.4186e-02, -9.7940e-03,
         1.4581e-02,  1.0307e-02, -1.6906e-03, -2.7542e-03, -2.2034e-02,
         1.6504e-02,  1.1402e-02,  2.2769e-03,  3.4043e-03, -1.3011e-02,
        -1.8587e-04, -1.9248e-02,  1.4815e-02, -1.9934e-02, -2.0164e-02,
         1.1181e-02,  7.5446e-04,  2.8086e-04, -2.6351e-02, -7.7234e-03,
         2.2288e-02,  1.9419e-02, -1.2970e-02,  1.3396e-02,  1.5364e-02,
         1.6694e-02, -2.3771e-03,  2.9610e-03,  1.3214e-02, -1.2649e-02,
         1.8322e-02, -7.1636e-03,  9.7513e-03,  5.9899e-03,  3.1572e-03,
         6.4649e-04, -1.6266e-02, -2.4271e-02, -1.9894e-02, -1.5913e-02,
         4.7792e-03,  1.4778e-02, -9.4636e-03,  4.4645e-03, -1.6508e-02,
         3.8906e-03,  9.7611e-03, -5.7172e-03, -2.0716e-02, -2.4205e-03,
        -2.8394e-02,  2.3463e-02,  8.8064e-03, -1.1414e-02, -2.9814e-02,
         1.5290e-02, -1.0935e-02,  4.3539e-03, -1.2551e-02, -1.1944e-02,
         9.5026e-03,  2.9009e-03,  1.9060e-02, -2.8463e-02,  1.2288e-02,
        -1.0482e-02,  9.7816e-03, -2.5684e-02,  1.7717e-02, -1.1591e-02,
        -5.8359e-03, -1.0082e-02, -1.6014e-02, -1.6874e-02,  3.4636e-04,
        -2.6735e-02,  6.1264e-03, -7.5708e-03,  7.6556e-03,  1.2817e-02,
        -1.4253e-02,  2.8471e-03, -3.6852e-03, -1.4639e-02,  3.1294e-03,
         5.1118e-03,  1.0513e-02, -2.9727e-03,  5.5283e-03,  1.2146e-02,
        -6.1957e-03,  3.0380e-03, -3.6376e-03,  3.7923e-03,  9.7442e-03,
         1.3267e-02, -2.4469e-02, -7.3804e-03,  1.2707e-02,  2.3502e-02,
        -6.5240e-03, -2.0157e-02, -1.5283e-02,  1.9164e-03,  4.8293e-03,
        -9.5856e-03, -8.0909e-03, -5.0857e-03,  1.2174e-02, -5.6524e-03,
        -2.2124e-03, -1.0881e-04, -1.8357e-02,  1.0653e-02,  9.7804e-03,
        -9.3332e-03, -1.6232e-02,  8.6300e-04,  6.2746e-03,  2.3145e-02,
         5.6833e-04, -2.7082e-03,  5.8049e-03,  1.6364e-02,  8.2308e-03,
         1.2012e-02,  1.7084e-02, -2.1984e-02, -1.0451e-02,  1.8775e-02,
        -1.7128e-02,  1.0685e-02,  4.5688e-03,  4.4340e-03, -1.4580e-02,
         1.4453e-02, -1.8983e-02, -1.1525e-02,  1.8668e-03,  5.5544e-03,
        -3.1515e-03,  2.1821e-02,  6.9290e-03,  2.5409e-02,  1.0114e-02,
         2.5227e-02,  1.7570e-02,  3.2620e-03,  1.4735e-03,  1.0300e-03,
        -7.7270e-04,  7.4899e-03, -7.5941e-03, -1.2576e-02, -2.4882e-02,
        -4.0096e-03, -1.3527e-02,  2.2686e-02,  4.6166e-03,  1.6844e-02,
        -1.1812e-02, -6.4348e-03,  1.6754e-02,  1.5965e-02,  2.2794e-02,
        -4.1649e-03, -3.6176e-03,  2.0345e-03, -8.0251e-03, -1.1521e-02,
        -1.2785e-02,  1.1890e-03, -8.1404e-03,  1.0557e-02,  1.0461e-02,
         1.0855e-02, -5.3422e-03,  1.1845e-02, -1.7931e-02,  1.7529e-02,
        -6.1153e-03, -9.5174e-04,  6.1184e-03, -1.3341e-02, -5.3308e-03,
         1.3168e-02, -3.1756e-03,  7.8384e-03, -8.6569e-03, -2.0172e-02,
         9.2409e-03,  9.7835e-03,  2.1737e-02,  1.6723e-02, -7.7320e-03,
         2.0661e-02,  1.0129e-02, -1.0150e-02,  4.1956e-03, -7.0191e-03,
        -6.4541e-03,  1.2232e-03, -7.9646e-03, -2.0041e-02,  8.6341e-04,
        -4.9318e-03,  5.6336e-03, -1.2522e-02,  1.5400e-02,  5.5319e-03,
         1.8740e-02, -1.2583e-02, -6.2556e-03,  1.1608e-02,  1.2774e-02,
        -1.3928e-02,  1.2628e-02, -1.8447e-03,  2.3092e-02, -4.0754e-03,
         2.1132e-02,  2.1922e-02, -5.1907e-03, -1.7896e-02,  1.1048e-02,
        -1.3770e-02, -4.7458e-04, -1.2632e-02,  4.8806e-03,  2.5369e-02,
         1.8125e-02,  1.3284e-02, -5.9761e-03,  4.7634e-03,  2.1912e-02,
         1.8576e-02,  1.4406e-02,  7.0115e-03,  1.9497e-02,  1.0831e-02,
         7.2290e-03,  1.6728e-02,  1.7972e-03, -2.0708e-03,  4.3599e-03,
         1.1381e-02, -7.9531e-03, -1.8408e-02,  2.2719e-03,  1.0379e-02,
        -6.4590e-03,  1.4792e-02, -1.4472e-02, -1.5949e-02, -8.5296e-03,
        -2.1070e-03,  1.7560e-02, -1.4812e-02, -1.2220e-02,  1.5577e-02,
        -1.1307e-02,  1.1847e-02, -2.9170e-02,  1.5642e-02,  2.3384e-03,
        -5.0604e-03,  1.7215e-02, -5.5310e-03,  1.7789e-02, -3.8172e-03,
         2.0882e-03, -1.5586e-02, -1.9848e-03,  7.0958e-03, -2.1935e-02,
         1.5503e-02,  5.9819e-03,  6.3686e-04, -2.3423e-03, -2.1516e-02,
        -1.2364e-02, -1.9615e-02, -9.5064e-03,  8.3440e-03,  1.9808e-02,
         9.1489e-03, -8.4258e-03,  4.7999e-03,  5.0976e-03,  6.4126e-03,
        -1.6243e-02,  6.2840e-04,  2.2920e-03,  2.1916e-02,  9.9640e-03,
        -1.7158e-02, -1.0661e-02, -1.6273e-02,  9.0954e-03, -8.8054e-03,
         5.1744e-03,  1.1335e-02, -2.5999e-02,  1.6242e-02,  2.3031e-02,
        -2.2289e-02,  2.1065e-03,  1.4525e-03, -6.9190e-03,  1.7525e-02,
        -1.6533e-04,  1.0637e-02, -1.0765e-02,  1.6663e-02,  9.0390e-03,
        -2.3199e-02, -2.3403e-03,  1.5122e-02, -1.8655e-02,  2.9117e-03,
        -4.2481e-03, -1.1448e-02,  2.4683e-02,  1.4885e-02,  5.5456e-03,
        -3.0207e-03,  1.3363e-02,  4.5736e-03,  1.1309e-02,  1.3719e-02,
         3.3649e-03, -1.9977e-02,  1.7781e-02, -4.7449e-03, -8.5974e-03,
        -5.5528e-03, -1.2694e-02, -1.4137e-02,  9.6868e-03,  4.5382e-05,
        -5.5562e-03,  3.2713e-03, -8.5780e-03,  1.3476e-02, -5.1457e-03,
        -6.9356e-03,  1.1924e-02, -2.7736e-02, -4.0591e-03,  1.8000e-02,
         1.1946e-03,  1.0004e-02,  5.0999e-03,  6.4347e-03, -6.7580e-03,
         5.0506e-03, -1.2958e-02, -1.6623e-02, -1.1312e-02,  2.0606e-02,
         2.8163e-02,  9.6279e-03,  6.8020e-03,  1.0191e-02, -1.5912e-02,
         2.9657e-03,  6.5147e-04,  2.6538e-03,  4.6585e-03, -1.7312e-02,
        -1.1575e-02,  2.0262e-02, -4.6975e-03,  1.1836e-02, -1.4028e-02,
        -1.1953e-02,  5.9616e-03,  2.1593e-02,  2.2003e-02, -2.4916e-03,
        -3.8817e-03, -6.0981e-03,  4.5098e-03, -9.5036e-03,  5.2333e-03,
         1.3447e-02, -9.8189e-04, -9.4803e-03,  1.9331e-02,  1.4541e-03,
        -2.7614e-03,  3.9887e-02, -5.4875e-03, -2.1581e-02,  1.5071e-02,
        -4.0009e-03,  9.9837e-03, -7.3580e-03,  1.3110e-02, -1.1047e-02,
         8.0212e-03,  1.5168e-02, -1.4055e-02, -1.4767e-02, -1.0989e-02,
         2.7221e-03,  1.1518e-02,  1.2886e-02,  5.8794e-03, -6.5741e-03,
         8.3907e-03, -1.0222e-02, -2.9587e-03,  1.5725e-02, -1.3776e-02,
        -1.9547e-03, -1.2570e-02, -8.2667e-03,  1.2533e-02, -1.2899e-02,
        -3.4358e-03, -5.6349e-03,  1.1624e-03, -1.2026e-02,  1.0182e-02,
        -1.9125e-02,  1.6347e-02, -1.5444e-02,  1.1478e-03, -5.9768e-03,
        -7.2418e-03,  1.3538e-02, -1.9255e-02,  1.0869e-02,  3.2065e-03,
         1.4557e-02, -4.9074e-03,  2.0342e-03, -1.2132e-02, -1.9891e-02,
         6.7242e-04,  6.3886e-03,  1.0344e-03, -1.7610e-03, -4.0773e-03,
         7.7609e-03,  9.0907e-03,  1.5358e-02,  1.0136e-02,  1.0641e-02,
         1.9167e-02,  6.0717e-03,  9.7403e-03, -1.5892e-02,  7.7836e-03,
        -4.8800e-03,  2.0548e-02,  2.0872e-02, -1.9863e-02, -3.9146e-03,
        -1.6496e-02, -1.6777e-03, -1.4294e-02, -3.3497e-03, -3.8785e-03,
        -2.3545e-02,  1.0845e-02, -1.3265e-02,  1.1569e-02,  2.2322e-02,
         8.0553e-03,  1.1781e-03,  6.8246e-04, -1.7245e-02,  6.3215e-03,
        -1.6413e-02, -2.6943e-02, -1.3507e-02, -2.9613e-03, -5.3906e-04,
         8.6539e-03,  2.0652e-03,  4.4348e-03,  2.0957e-03, -4.5746e-04,
        -1.5353e-02,  5.1910e-03, -6.6725e-03,  9.2959e-03, -1.6648e-02,
         3.5933e-03,  7.0094e-03,  1.1512e-02, -2.6380e-02, -1.5549e-02,
         3.4340e-03,  1.2028e-02, -9.6686e-03, -1.5313e-02, -1.3462e-02,
        -2.3815e-02, -5.9382e-03, -1.0678e-02,  6.4010e-03, -2.8793e-03,
        -2.5831e-02, -2.3223e-02], device='cuda:0'), tensor([[[ 0.0263,  0.0099, -0.0163, -0.0234, -0.0069],
         [ 0.0101,  0.0106,  0.0166,  0.0021,  0.0056],
         [ 0.0366,  0.0256,  0.0060, -0.0002, -0.0019],
         ...,
         [-0.0037,  0.0027,  0.0146,  0.0275,  0.0114],
         [-0.0313, -0.0326,  0.0013, -0.0032, -0.0233],
         [-0.0134, -0.0093, -0.0076, -0.0116, -0.0201]],

        [[-0.0111,  0.0051,  0.0177,  0.0159,  0.0068],
         [-0.0339, -0.0101,  0.0186, -0.0272, -0.0081],
         [ 0.0215,  0.0469, -0.0300, -0.0154, -0.0151],
         ...,
         [-0.0112,  0.0069,  0.0112,  0.0154, -0.0024],
         [ 0.0021,  0.0243, -0.0029, -0.0168,  0.0098],
         [ 0.0015, -0.0203,  0.0049, -0.0050,  0.0105]],

        [[ 0.0306,  0.0110,  0.0171,  0.0043, -0.0129],
         [-0.0007, -0.0172, -0.0040, -0.0034,  0.0054],
         [ 0.0112, -0.0098, -0.0133, -0.0339, -0.0275],
         ...,
         [ 0.0108,  0.0041, -0.0015, -0.0050, -0.0211],
         [-0.0098,  0.0019, -0.0042,  0.0020, -0.0118],
         [ 0.0082,  0.0158, -0.0025,  0.0155, -0.0005]],

        ...,

        [[-0.0110,  0.0214,  0.0610,  0.0392,  0.0054],
         [-0.0476, -0.0620, -0.1103, -0.0544, -0.0057],
         [-0.0046, -0.0017,  0.0012, -0.0196,  0.0003],
         ...,
         [-0.0126, -0.0084,  0.0010, -0.0141,  0.0071],
         [ 0.0119,  0.0013, -0.0085, -0.0307,  0.0020],
         [ 0.0028, -0.0068,  0.0200,  0.0112, -0.0019]],

        [[ 0.0010, -0.0088,  0.0159, -0.0115, -0.0095],
         [ 0.0096, -0.0015,  0.0073, -0.0082,  0.0279],
         [-0.0193, -0.0070, -0.0037, -0.0319, -0.0162],
         ...,
         [ 0.0104,  0.0116, -0.0088,  0.0119, -0.0097],
         [-0.0164, -0.0291, -0.0158, -0.0176, -0.0133],
         [-0.0048, -0.0075, -0.0268, -0.0188, -0.0327]],

        [[ 0.0051,  0.0178,  0.0019,  0.0184,  0.0179],
         [-0.0002, -0.0082,  0.0033, -0.0059, -0.0026],
         [ 0.0223,  0.0060,  0.0101,  0.0219, -0.0070],
         ...,
         [-0.0049,  0.0176,  0.0129,  0.0025,  0.0062],
         [-0.0317, -0.0043, -0.0012, -0.0150,  0.0156],
         [ 0.0065, -0.0038,  0.0106,  0.0049, -0.0039]]], device='cuda:0'), tensor([ 4.3378e-03,  4.2616e-03, -2.0443e-02, -1.3165e-02, -7.6492e-03,
         2.6769e-02, -3.6371e-02, -1.1015e-02, -6.4436e-03, -1.3830e-02,
         1.4872e-02,  1.4144e-02,  9.7287e-03,  1.2729e-02,  9.6543e-04,
        -4.5542e-03,  3.0513e-03, -2.8470e-02,  2.0385e-02,  7.9598e-03,
        -2.8998e-03,  1.9728e-02, -6.3156e-03,  1.7559e-02,  6.9967e-03,
        -1.1805e-02,  2.5482e-03,  8.3004e-04, -2.2516e-03,  6.5111e-03,
        -1.4245e-02,  7.7334e-03,  6.5868e-03,  6.8291e-03,  1.0517e-04,
        -1.1589e-02,  6.9799e-03, -5.1097e-03,  1.6130e-02, -8.2617e-03,
        -5.5307e-03,  2.4277e-03,  1.6835e-03, -3.8537e-02, -3.8634e-02,
         1.0730e-02,  2.1796e-02, -2.6278e-04, -8.1594e-03,  1.1296e-02,
        -7.4160e-03, -6.2717e-03, -2.3879e-03,  3.3017e-04,  2.1312e-03,
        -1.1888e-02,  1.8153e-03,  1.5321e-03, -2.6842e-03, -8.1071e-03,
         1.7281e-03,  1.2028e-02,  1.7986e-04, -5.5911e-02,  1.1664e-02,
        -9.2536e-04, -1.9949e-02, -7.2211e-03,  1.0142e-02, -4.8219e-02,
        -6.8331e-04, -4.1464e-02,  1.1851e-02,  8.7968e-03, -2.0652e-02,
        -1.1746e-02,  6.1172e-03, -4.9825e-03, -1.1023e-02,  3.2538e-03,
         1.3331e-02, -1.2120e-02, -3.5532e-03,  9.6281e-03,  1.6156e-02,
        -7.4766e-04, -1.1574e-03, -1.8069e-02,  1.3125e-02,  2.7303e-03,
        -4.4371e-02, -1.2548e-02, -7.9794e-04,  9.9530e-03,  4.8760e-03,
        -8.7747e-03,  1.8438e-02,  1.5318e-03, -6.2381e-03,  2.8251e-02,
        -5.8882e-03,  2.7541e-03,  2.3471e-02, -5.0797e-02, -2.2007e-02,
        -1.3782e-02, -2.3278e-02, -4.3705e-03,  2.0083e-02,  6.9198e-03,
        -1.1067e-02,  6.0966e-04,  2.0425e-03,  2.2845e-03,  1.2493e-02,
        -6.8672e-03, -8.8659e-03, -1.0671e-03,  3.4549e-03,  1.5817e-02,
         9.5048e-03, -5.8530e-03,  4.1878e-03, -1.2171e-02, -1.2795e-03,
         1.3359e-02, -4.5748e-03,  1.9916e-05,  5.7644e-03,  8.5937e-03,
         5.8641e-03,  9.3836e-03,  1.5159e-03, -1.4938e-02, -3.5480e-02,
         6.8049e-03, -7.4724e-03, -6.9665e-03,  1.1712e-02, -5.2890e-02,
        -1.0518e-02, -2.4432e-02,  1.1375e-02,  1.2232e-02,  1.2801e-03,
         1.2339e-02,  7.4764e-03,  1.0593e-02,  1.6120e-02, -7.0505e-03,
         6.2401e-03, -9.9784e-03, -1.7791e-02, -1.1335e-02,  1.5501e-02,
        -1.4218e-02,  2.0694e-03, -4.8571e-03, -7.6213e-04,  4.3449e-03,
        -2.7493e-03, -1.9465e-02,  1.5661e-02, -1.0147e-02,  8.8099e-03,
         3.9998e-04, -7.8400e-03,  1.8006e-03, -1.7745e-02,  1.3659e-03,
         1.7407e-02,  1.9437e-02,  1.2594e-02,  1.1684e-02,  3.8810e-03,
         1.4445e-02, -5.3555e-03,  1.3487e-02,  4.1285e-04, -2.7644e-02,
         2.2035e-03, -3.7691e-02,  6.4323e-03, -2.3175e-02,  8.0070e-03,
        -6.0937e-02,  7.9594e-03,  6.5573e-03,  2.0520e-02,  1.5193e-02,
        -1.6805e-03, -1.4732e-02, -2.9020e-02,  1.0222e-02, -9.5219e-03,
        -1.0997e-03,  1.8321e-03, -1.8825e-02, -3.7741e-02,  8.6140e-03,
        -9.9464e-05,  1.6610e-02, -3.0924e-02, -6.9593e-03, -4.6928e-03,
        -1.7008e-02, -1.7764e-02, -5.2325e-03, -9.9380e-03, -1.0149e-02,
        -4.7906e-04,  2.0607e-03, -1.9554e-02,  1.6601e-02, -3.4687e-03,
        -4.9273e-03,  6.4829e-03, -1.6865e-02,  7.4323e-03, -1.9548e-02,
         6.2415e-03, -2.0139e-02, -2.0705e-02, -1.5173e-02, -2.1138e-02,
         3.8177e-03, -1.1114e-02,  3.8516e-03, -1.5758e-02, -1.5154e-02,
        -1.8268e-02, -8.8588e-03, -2.1508e-02, -4.6401e-03, -2.1691e-02,
         5.1643e-03,  5.1700e-03, -1.6518e-02, -1.0648e-02, -1.2155e-02,
         8.5762e-03, -8.3925e-03,  2.2783e-02,  1.2943e-02,  1.1686e-02,
        -1.5669e-02, -9.7508e-03, -3.8261e-03, -1.2414e-02, -1.3961e-02,
         1.5210e-03, -1.4129e-02, -1.3773e-03, -3.3622e-03, -1.3221e-02,
        -5.9241e-03], device='cuda:0'), tensor([0.9845, 1.0288, 1.0061, 0.9464, 1.0130, 0.9999, 1.0738, 0.9668, 0.9671,
        1.0299, 0.9663, 1.0023, 1.0026, 0.9833, 0.9244, 0.9886, 0.9595, 1.0593,
        0.9433, 0.9831, 0.9986, 0.9396, 0.9447, 0.9909, 0.9731, 1.0616, 1.0022,
        0.9396, 0.9799, 0.9461, 0.9824, 0.9943, 0.9978, 1.0258, 0.9959, 0.9891,
        0.9953, 0.9882, 0.9932, 0.9975, 1.0065, 0.9966, 1.0309, 1.0902, 1.0647,
        0.9781, 0.9927, 0.9817, 0.9824, 0.9906, 0.9992, 1.0220, 0.9972, 0.9815,
        1.0058, 0.9747, 1.0364, 0.9995, 0.9822, 1.0483, 1.0194, 1.0093, 0.9891,
        1.1051, 1.0222, 0.9683, 1.0853, 1.0137, 1.0007, 1.0987, 0.9881, 1.0848,
        0.9764, 0.9894, 1.1015, 0.9935, 0.9879, 0.9888, 0.9933, 0.9829, 0.9976,
        1.0079, 1.0061, 1.0375, 0.9938, 0.9796, 1.0178, 1.0783, 0.9977, 0.9904,
        1.0719, 0.9972, 0.9998, 0.9882, 0.9560, 1.0025, 0.9924, 1.0168, 0.9298,
        1.0048, 1.0874, 1.0588, 1.0053, 1.0746, 1.0619, 1.0506, 1.0669, 0.9909,
        0.9901, 0.9988, 0.9864, 0.9798, 0.9711, 1.0116, 1.0100, 1.0238, 1.0017,
        1.0666, 1.0041, 0.9984, 1.0085, 1.0169, 1.0304, 0.9771, 0.9919, 0.9912,
        1.0216, 1.0085, 0.9997, 0.9895, 0.9891, 1.0039, 0.9755, 1.0706, 1.1108,
        1.0406, 0.9960, 0.9791, 0.9948, 1.0785, 1.0506, 1.0515, 0.9964, 0.9918,
        1.0120, 1.0071, 1.0194, 0.9929, 0.9977, 0.9970, 0.9998, 0.9924, 1.0868,
        1.0587, 0.9935, 1.0039, 0.9984, 1.0190, 1.0008, 1.0642, 0.9569, 1.0545,
        0.9900, 0.9838, 1.0598, 1.0761, 1.0337, 0.9072, 1.0881, 0.9867, 1.0071,
        1.0006, 0.9866, 0.9615, 1.0286, 0.9771, 1.0015, 0.9740, 0.9904, 1.0692,
        0.9932, 1.1120, 0.9978, 1.0608, 0.9993, 1.1269, 0.9787, 0.9924, 0.9998,
        0.9714, 1.0022, 1.0039, 1.0639, 0.9594, 0.9868, 0.9955, 0.9781, 0.8669,
        1.0615, 1.0132, 0.9731, 1.0161, 1.0351, 0.9752, 0.9325, 0.9790, 0.9998,
        1.0286, 0.9321, 0.9963, 0.9630, 0.9943, 1.0341, 0.9996, 1.0060, 1.0191,
        0.9222, 0.9730, 0.9873, 1.0106, 0.9680, 1.0243, 1.0460, 1.0068, 1.0181,
        1.0281, 0.9047, 0.9870, 0.9445, 0.9649, 0.9210, 0.9405, 0.9241, 0.9633,
        0.9924, 0.9549, 0.9330, 0.9632, 0.9986, 0.9346, 0.9504, 0.9957, 1.0014,
        0.9531, 0.9958, 1.0292, 1.0649, 1.0056, 0.8812, 1.0287, 0.9804, 0.9967,
        1.0059, 1.0071, 0.9878, 0.9745], device='cuda:0'), tensor([-9.5036e-03,  1.5336e-02, -1.4506e-02, -2.9544e-03, -2.6296e-02,
         9.1812e-03, -9.7680e-02, -1.1061e-02, -9.5269e-03, -2.8560e-02,
        -1.1007e-02, -4.6688e-03,  9.0809e-03, -2.8281e-03, -1.9029e-02,
        -1.1187e-02, -5.0811e-03, -1.0195e-01, -3.7772e-02, -8.9001e-03,
        -1.8853e-03, -1.9466e-02, -2.5384e-02, -4.8990e-03, -1.7826e-03,
        -8.0742e-02,  7.9444e-03, -1.7056e-02,  3.9765e-03, -2.6307e-02,
         9.3573e-04, -3.7824e-03, -9.2989e-03, -8.2333e-03,  7.5868e-04,
        -3.9957e-03,  7.7117e-03, -3.2785e-03, -9.8293e-03, -7.8217e-03,
        -7.3305e-03, -2.6462e-03, -2.6604e-02, -9.2530e-02, -5.4386e-02,
         4.5337e-03,  6.2403e-03, -3.3354e-03, -1.8169e-02, -4.3667e-03,
        -1.3586e-04, -2.3734e-02, -5.7288e-03,  1.1870e-03,  2.4220e-03,
        -1.8550e-02, -7.3247e-03, -5.4238e-03,  1.9429e-03, -6.8931e-02,
         2.9602e-03, -8.1798e-03, -3.2159e-03, -8.2736e-02,  4.9538e-03,
         3.9622e-03, -7.3519e-02, -2.4199e-02,  6.6168e-05, -6.3385e-02,
        -2.8249e-02, -6.7732e-02, -2.8570e-02, -7.0944e-03, -4.4615e-02,
         6.0394e-04, -2.4005e-02, -2.0923e-03, -6.5502e-03, -1.1475e-02,
        -4.9388e-03, -6.4846e-03, -3.4408e-03, -2.2309e-02,  9.4368e-04,
        -1.8603e-03, -9.9243e-04, -3.9966e-02,  8.7925e-04, -2.0134e-02,
        -7.0525e-02, -1.4783e-02, -2.7850e-03, -3.2139e-03, -2.3253e-02,
         2.6335e-03,  1.9691e-03, -1.0213e-02, -5.5797e-03,  3.5718e-03,
        -3.7164e-02, -3.1452e-02, -9.8494e-04, -4.9722e-02, -4.0852e-02,
        -3.2789e-03, -6.0992e-02, -3.4815e-03, -1.5187e-02, -1.2480e-02,
        -1.2665e-02,  3.7606e-03,  8.4047e-03, -9.4436e-03, -1.2253e-03,
        -1.3301e-02,  6.5940e-03, -1.0432e-02, -1.2513e-02, -1.1605e-03,
         4.1065e-04, -1.6894e-02, -5.3542e-03, -6.0264e-03, -6.0463e-04,
        -9.9448e-03, -5.5021e-03,  3.2761e-03, -7.1761e-03, -8.6950e-03,
        -6.6745e-03, -2.6171e-03, -6.6687e-03, -1.7574e-02, -1.4357e-01,
         2.9543e-03, -1.1671e-02, -3.3413e-03, -4.7446e-03, -6.6149e-02,
        -3.9462e-02, -8.3568e-02, -9.6152e-04, -1.0458e-02, -1.5114e-02,
         8.4151e-04, -7.0405e-03, -1.1204e-02, -4.1479e-04, -1.0649e-03,
        -2.1090e-02,  7.6379e-03, -1.3026e-01, -7.3141e-02,  1.1223e-03,
        -1.0351e-02,  2.6211e-03, -2.7487e-03,  5.7486e-03, -2.7377e-02,
        -1.0054e-02, -5.4649e-02, -5.3393e-03, -1.4219e-02, -1.1368e-02,
        -2.3236e-02, -1.2012e-02, -2.3926e-02, -3.3705e-02, -1.4668e-02,
         4.2632e-03, -2.5188e-03, -5.1882e-03, -1.8129e-02, -3.9437e-02,
        -6.2880e-03, -1.2097e-02, -2.6520e-02, -7.1488e-03, -4.7880e-02,
        -6.9925e-03, -9.1878e-02, -2.1139e-03, -1.5142e-02,  5.2607e-03,
        -1.0519e-01, -3.0868e-02,  1.5998e-03,  5.0315e-03, -2.1512e-02,
        -2.8896e-03, -2.0841e-04, -4.9248e-02,  2.7457e-03, -8.5846e-03,
        -4.6118e-02, -1.0770e-02, -4.5930e-02, -6.0023e-02,  2.4513e-03,
        -1.1324e-02,  8.3384e-03, -6.2232e-02, -2.2444e-03, -2.7314e-03,
         9.4544e-04, -1.0518e-02, -7.1350e-03, -5.3077e-03, -5.7472e-03,
        -1.5810e-02, -9.4708e-04, -4.2535e-02,  5.7449e-03,  4.5915e-03,
         6.4953e-03, -2.3488e-03, -1.4423e-02, -3.7134e-03, -1.8159e-02,
         3.4974e-04, -1.7323e-02, -3.3651e-02, -6.4705e-03, -4.4812e-02,
        -8.8715e-04, -2.2025e-02, -1.4196e-02, -6.4776e-03, -1.1025e-02,
        -1.6119e-02, -1.2342e-02,  4.5670e-03, -1.4814e-02, -2.6157e-03,
        -1.3450e-02,  2.5177e-02, -1.9424e-02, -2.3866e-02, -5.7597e-04,
        -2.8305e-02, -4.8181e-03,  1.1290e-02, -1.5969e-02, -4.3961e-03,
         7.2877e-03, -7.4399e-02,  5.3620e-04, -2.2979e-02, -3.3428e-03,
        -1.2445e-02, -1.4631e-02,  5.7721e-03, -4.3885e-03,  2.3256e-03,
        -1.5282e-02], device='cuda:0'), tensor([[[ 7.6264e-03, -1.8771e-02, -2.1543e-02,  7.4648e-03,  4.5316e-03],
         [-9.8427e-04,  4.9893e-03, -7.3698e-03, -1.4633e-02,  3.4640e-02],
         [ 8.5051e-03, -1.1515e-02, -5.5846e-03, -3.9074e-03,  1.6172e-02],
         ...,
         [ 2.0008e-02,  8.8639e-03,  2.9645e-02,  2.8057e-02,  2.8814e-02],
         [-1.2465e-02,  2.7261e-02,  1.1676e-02,  1.3167e-02, -3.5372e-03],
         [ 1.9587e-03,  1.8126e-02, -1.0793e-03, -2.2396e-02,  6.5685e-03]],

        [[ 1.9585e-02,  3.1320e-02,  1.0805e-02,  2.8334e-02,  5.2162e-03],
         [ 1.1528e-02,  1.5975e-02,  2.3826e-02,  1.9448e-03, -9.4026e-03],
         [-2.4856e-02, -1.8744e-02, -3.3769e-02, -2.5915e-02,  8.1912e-03],
         ...,
         [ 5.8149e-03, -1.0384e-02,  8.8848e-03,  8.2525e-03, -3.5576e-02],
         [ 2.5484e-02,  2.5853e-02,  1.6939e-02,  2.0967e-02,  2.2241e-02],
         [-4.7445e-03, -2.2864e-02, -2.7229e-02, -1.9668e-02,  2.1575e-03]],

        [[-2.5306e-02, -1.6626e-02, -2.0821e-02,  1.8882e-03,  2.2216e-02],
         [ 8.1385e-03, -3.1928e-03,  5.2064e-02, -1.3102e-02, -6.7655e-03],
         [-2.7459e-02, -9.7841e-03, -4.5821e-03,  3.5295e-03, -1.4446e-02],
         ...,
         [-6.9746e-03,  4.2814e-02,  3.5363e-02,  3.6950e-02,  8.1459e-03],
         [-6.1977e-03,  1.4195e-03, -2.7626e-02, -2.4672e-02, -2.0505e-02],
         [ 3.1689e-02,  2.8625e-02,  3.2986e-02, -1.4186e-02,  1.0931e-02]],

        ...,

        [[ 1.0958e-02, -2.3676e-02,  2.3426e-02, -1.0116e-02, -2.3302e-02],
         [ 1.1887e-03,  1.9435e-02,  1.2881e-02,  1.5760e-02, -1.3955e-02],
         [-3.0328e-02,  2.7319e-02, -2.0511e-02, -2.5124e-02, -2.5548e-02],
         ...,
         [ 1.0592e-02,  2.6236e-02,  6.2356e-03,  7.9096e-03,  8.5695e-03],
         [-1.5337e-02,  8.2248e-03, -3.4181e-02, -8.9959e-03, -3.5181e-02],
         [-1.4288e-02, -6.8393e-03, -7.6481e-03,  2.1875e-02,  2.3696e-02]],

        [[ 1.4109e-02, -1.3084e-02, -1.1080e-02, -2.7265e-02, -2.1059e-02],
         [-2.2706e-02,  2.8967e-02, -3.9577e-02, -6.0579e-03,  5.5864e-03],
         [-5.3700e-03,  1.9656e-02, -2.3565e-02,  2.1838e-02, -2.6978e-03],
         ...,
         [-1.1163e-02,  2.2924e-02,  3.3628e-02,  1.2764e-02,  3.1622e-03],
         [ 1.3699e-02,  1.0734e-02, -1.6839e-03, -5.0340e-05, -2.9420e-02],
         [-3.0621e-03, -2.1117e-02,  1.1429e-02,  9.6279e-03, -1.0973e-02]],

        [[ 2.1359e-02, -1.9028e-02,  4.4429e-03, -1.5085e-02, -1.1076e-02],
         [ 2.1029e-02, -7.0964e-03, -1.6316e-02, -2.4382e-02, -1.2641e-02],
         [-1.6217e-02, -2.6621e-02,  1.5412e-02,  7.5088e-03,  2.8210e-02],
         ...,
         [-9.0446e-03,  2.4556e-02,  2.6940e-04,  1.1911e-02, -1.0079e-02],
         [-5.5892e-03,  1.6296e-02, -2.2866e-02,  1.6431e-02, -8.4906e-03],
         [-7.2228e-03,  1.1742e-02,  2.5462e-02, -1.5397e-02, -1.7727e-02]]],
       device='cuda:0'), tensor([-3.5432e-03, -3.7807e-02,  2.3368e-02,  2.6713e-02, -5.0832e-02,
        -1.4658e-02,  3.7139e-03, -6.4744e-02,  1.4651e-02, -1.5864e-02,
        -2.4016e-02, -1.2121e-04, -2.7081e-02,  2.9427e-02,  1.5735e-02,
        -4.7051e-03, -1.6519e-02, -1.0759e-02, -5.5495e-02, -2.4150e-02,
        -1.9630e-03,  7.6569e-03, -2.4740e-02, -7.2432e-03,  1.7117e-02,
         2.5599e-02,  2.4872e-02,  2.6902e-03,  6.9416e-03, -2.6218e-02,
         9.4523e-03,  8.7958e-03,  1.4654e-02,  1.6310e-02,  1.1890e-02,
        -6.0748e-03,  2.1127e-02, -5.1095e-03, -2.3185e-02,  2.3837e-02,
        -4.6383e-04, -2.1507e-02, -3.1370e-02,  7.1286e-04, -1.8080e-02,
         3.1894e-03, -5.9254e-02,  2.3876e-02, -5.6360e-03, -3.9146e-03,
         1.8508e-03, -4.7092e-03,  3.3867e-03,  5.9617e-03,  2.1672e-02,
        -2.6451e-02,  2.8679e-02,  1.3066e-02, -2.4522e-02, -1.6690e-02,
         1.2575e-02, -3.5880e-02,  1.9486e-02, -1.6978e-02, -1.9144e-02,
        -2.0194e-02, -2.1723e-02,  1.9907e-03, -3.1344e-02, -1.3442e-02,
         2.5521e-02,  1.3123e-02, -2.6216e-02,  2.4110e-02,  2.3044e-02,
        -6.7706e-03, -3.1828e-02, -3.1811e-03, -1.3422e-02,  2.4096e-02,
        -6.9935e-02, -1.1708e-02,  1.4319e-02,  1.5315e-02,  4.6389e-03,
        -7.6427e-03, -1.5482e-02, -1.5719e-02, -4.1499e-02, -9.9416e-03,
        -2.2408e-02, -1.4405e-02, -1.2331e-02, -2.5867e-02,  2.4308e-02,
        -1.2973e-02, -4.9293e-02,  1.3809e-02,  2.4227e-02,  8.3105e-03,
         1.9969e-02, -2.8207e-02,  3.3352e-03,  1.9900e-02,  1.5605e-02,
         4.7895e-03, -1.1796e-02, -2.2346e-02, -2.5581e-02, -3.2409e-03,
        -1.3094e-02,  7.0566e-04, -6.7139e-02,  1.6512e-02,  4.2324e-03,
         7.8006e-03,  2.3407e-02, -4.3124e-03, -9.9207e-04, -1.2706e-02,
         6.3120e-03, -2.6506e-02,  7.8371e-03,  4.4062e-03,  1.7445e-02,
        -1.2191e-02,  1.7718e-02,  3.0204e-02,  2.1523e-02,  4.5868e-03,
         2.3879e-02,  1.9958e-02,  1.7408e-02,  2.4428e-02,  2.9381e-02,
        -2.8336e-02, -4.9438e-03,  1.0543e-02,  2.2017e-02,  2.1720e-03,
        -4.5460e-02,  2.0364e-02,  1.3454e-02,  2.3822e-02, -3.5532e-02,
         3.3397e-03, -2.4816e-02,  2.5810e-02,  4.8891e-03,  1.0516e-02,
        -6.0481e-03,  8.0512e-03,  1.8692e-03,  2.6160e-02,  4.1234e-03,
         2.9741e-02, -1.2093e-02,  1.4097e-02, -6.6004e-02, -7.4115e-03,
         2.2670e-02,  9.3087e-03,  9.6061e-03,  1.7858e-02, -1.1111e-02,
        -3.8446e-02, -7.2417e-03,  1.6086e-02,  2.1206e-02, -2.0781e-03,
        -4.7387e-02, -4.5789e-03,  2.1330e-02,  2.0707e-02,  1.8840e-02,
        -1.6426e-02,  9.4981e-03, -2.1514e-02,  2.3918e-02, -5.6441e-02,
         3.2236e-02,  3.9546e-03,  2.3081e-02,  3.0767e-04, -3.3504e-02,
         1.1828e-02,  1.7767e-02, -7.7662e-03,  1.5484e-02, -5.1875e-03,
         2.5656e-02,  9.0699e-04, -2.6324e-03, -4.4234e-03, -5.0692e-03,
         8.3964e-03,  1.8969e-02, -2.5360e-02, -5.7012e-02, -1.3463e-02,
         7.1433e-03,  5.8430e-03,  7.7970e-03, -2.1235e-02,  2.2778e-02,
        -2.6373e-02, -4.6779e-03, -1.5232e-02, -5.6201e-02, -8.2048e-05,
        -1.7557e-03, -3.2382e-02, -9.4617e-03,  2.0954e-03, -2.2075e-02,
         1.2780e-02,  1.5182e-02,  6.2374e-03, -1.5848e-02,  1.2272e-02,
        -1.5419e-02,  6.9342e-03, -9.7140e-03,  2.0440e-02, -1.1864e-02,
        -2.0848e-02, -1.8672e-02,  2.2980e-02,  2.3538e-02,  2.8073e-02,
        -1.2498e-02, -2.3556e-02, -2.7184e-02,  2.4737e-02, -1.2813e-02,
        -1.0794e-02, -1.7893e-02,  2.1785e-02, -2.3267e-02,  3.4811e-03,
         4.9729e-03, -3.4294e-04, -1.1023e-02,  3.1853e-02,  2.3517e-02,
        -4.4382e-03, -1.2280e-02,  1.2008e-02, -6.3766e-02,  6.9265e-03,
        -2.0853e-02,  2.6228e-02, -3.3812e-02, -2.1759e-02,  2.1051e-02,
        -2.5177e-02], device='cuda:0'), tensor([0.9900, 0.9987, 0.9779, 0.9856, 0.9998, 0.9812, 0.9775, 0.9977, 0.9896,
        0.9868, 1.0053, 1.0145, 0.9725, 0.9835, 0.9594, 0.9960, 0.9693, 0.9891,
        0.9949, 0.9836, 0.9732, 0.9738, 0.9587, 0.9885, 0.9854, 0.9849, 0.9868,
        0.9902, 0.9861, 0.9641, 0.9768, 0.9813, 0.9864, 0.9764, 0.9895, 0.9879,
        0.9917, 0.9640, 0.9869, 0.9850, 0.9866, 0.9622, 1.0082, 0.9991, 0.9953,
        0.9840, 0.9910, 0.9800, 0.9907, 0.9912, 0.9872, 0.9885, 0.9774, 1.0001,
        0.9875, 0.9957, 0.9890, 0.9459, 0.9692, 0.9851, 0.9597, 0.9987, 0.9866,
        0.9807, 0.9979, 0.9822, 0.9851, 0.9868, 0.9988, 0.9801, 0.9938, 0.9888,
        1.0036, 0.9897, 0.9812, 0.9657, 1.0027, 0.9755, 0.9824, 0.9808, 1.0001,
        0.9733, 0.9924, 0.9852, 0.9878, 0.9417, 1.0003, 0.9873, 0.9960, 1.0386,
        0.9882, 0.9837, 0.9840, 0.9988, 0.9759, 0.9806, 1.0043, 0.9683, 0.9928,
        0.9850, 0.9844, 0.9976, 0.9940, 0.9892, 0.9890, 0.9877, 0.9892, 0.9806,
        0.9869, 1.0187, 0.9782, 0.9695, 0.9985, 0.9885, 0.9830, 0.9938, 1.0009,
        0.9460, 0.9998, 0.9700, 0.9717, 1.0002, 0.9807, 0.9638, 0.9541, 0.9832,
        0.9821, 0.9808, 0.9511, 0.9997, 0.9838, 0.9782, 0.9758, 0.9949, 0.9806,
        0.9723, 0.9824, 0.9904, 0.9929, 0.9527, 0.9930, 0.9918, 0.9732, 0.9818,
        0.9937, 0.9736, 0.9589, 0.9990, 0.9860, 0.9872, 1.0055, 0.9894, 0.9819,
        0.9972, 0.9891, 0.9908, 1.0005, 0.9605, 0.9933, 0.9922, 0.9782, 0.9847,
        0.9941, 0.9847, 0.9704, 0.9971, 0.9793, 0.9674, 0.9660, 1.0258, 0.9946,
        0.9737, 0.9882, 0.9623, 0.9897, 0.9958, 0.9873, 0.9864, 0.9842, 1.0032,
        0.9849, 0.9773, 0.9772, 0.9841, 1.0111, 0.9908, 0.9911, 0.9969, 0.9913,
        0.9793, 1.0129, 0.9850, 0.9873, 0.9813, 0.9873, 0.9877, 0.9889, 0.9976,
        0.9971, 0.9821, 0.9620, 0.9718, 0.9799, 0.9789, 0.9866, 0.9795, 0.9600,
        1.0009, 0.9973, 1.0027, 0.9744, 0.9967, 0.9443, 0.9820, 0.9907, 0.9643,
        0.9930, 0.9890, 0.9877, 0.9503, 0.9720, 0.9821, 1.0250, 0.9757, 0.9892,
        0.9872, 0.9826, 0.9883, 0.9560, 0.9994, 0.9772, 0.9931, 0.9947, 0.9859,
        0.9939, 0.9491, 0.9595, 0.9862, 0.9891, 0.9804, 0.9753, 0.9695, 0.9773,
        0.9829, 0.9704, 0.9825, 1.0155, 0.9885, 1.0005, 0.9773, 0.9870, 0.9863,
        0.9946, 0.9163, 0.9930, 0.9681], device='cuda:0'), tensor([-0.0092, -0.0030, -0.0117, -0.0079, -0.0011, -0.0036, -0.0082,  0.0013,
        -0.0073, -0.0073, -0.0130, -0.0103, -0.0078, -0.0108, -0.0078, -0.0035,
        -0.0105, -0.0113,  0.0032, -0.0065, -0.0100, -0.0114, -0.0121, -0.0064,
        -0.0086, -0.0060, -0.0089, -0.0069, -0.0052, -0.0080, -0.0090, -0.0059,
        -0.0081, -0.0084, -0.0065, -0.0053, -0.0086, -0.0061, -0.0094, -0.0066,
        -0.0075, -0.0087, -0.0120, -0.0005, -0.0105, -0.0080, -0.0022, -0.0075,
        -0.0103, -0.0070, -0.0121, -0.0037, -0.0114, -0.0008, -0.0044, -0.0065,
        -0.0037, -0.0095, -0.0059, -0.0057, -0.0093,  0.0050, -0.0061, -0.0070,
        -0.0111, -0.0069, -0.0051, -0.0045, -0.0033, -0.0067, -0.0044, -0.0030,
         0.0027, -0.0074, -0.0079, -0.0041, -0.0085, -0.0073, -0.0072, -0.0086,
        -0.0087, -0.0088, -0.0055, -0.0050, -0.0029, -0.0110, -0.0083, -0.0121,
        -0.0032, -0.0074, -0.0047, -0.0054, -0.0042, -0.0058, -0.0052, -0.0057,
        -0.0095, -0.0077, -0.0084, -0.0056, -0.0070, -0.0067, -0.0087, -0.0057,
        -0.0100, -0.0074, -0.0068, -0.0030, -0.0093, -0.0024, -0.0075, -0.0083,
        -0.0029, -0.0056, -0.0099, -0.0033, -0.0084, -0.0057,  0.0006, -0.0074,
        -0.0081, -0.0057, -0.0072, -0.0086, -0.0058, -0.0067, -0.0062, -0.0090,
        -0.0089, -0.0013, -0.0079, -0.0095, -0.0058, -0.0025, -0.0045, -0.0107,
        -0.0063, -0.0059, -0.0040, -0.0067,  0.0002, -0.0064, -0.0103, -0.0072,
         0.0038, -0.0121, -0.0076, -0.0004, -0.0067, -0.0128, -0.0118, -0.0089,
        -0.0102, -0.0023, -0.0047, -0.0087, -0.0070, -0.0076,  0.0064, -0.0057,
        -0.0072, -0.0067, -0.0118, -0.0089, -0.0050,  0.0013, -0.0054, -0.0084,
        -0.0080, -0.0146,  0.0051, -0.0087, -0.0076, -0.0058, -0.0053, -0.0018,
        -0.0093, -0.0049, -0.0067, -0.0075, -0.0136, -0.0069, -0.0097, -0.0100,
        -0.0171, -0.0085, -0.0012, -0.0047, -0.0064, -0.0087, -0.0162, -0.0036,
        -0.0050, -0.0055, -0.0105, -0.0073, -0.0029, -0.0145, -0.0049, -0.0041,
        -0.0090, -0.0092, -0.0137, -0.0038, -0.0090, -0.0081, -0.0090, -0.0082,
        -0.0077,  0.0006, -0.0084, -0.0118, -0.0096, -0.0081, -0.0053, -0.0103,
        -0.0046, -0.0051, -0.0081, -0.0093, -0.0053, -0.0056, -0.0076, -0.0046,
        -0.0081, -0.0057, -0.0040, -0.0092, -0.0093,  0.0002, -0.0093, -0.0014,
        -0.0124, -0.0068, -0.0103, -0.0098, -0.0063, -0.0102, -0.0039, -0.0053,
        -0.0120, -0.0094, -0.0085, -0.0086, -0.0118, -0.0074, -0.0126, -0.0066,
        -0.0072, -0.0048, -0.0054, -0.0044,  0.0027, -0.0062, -0.0131, -0.0112],
       device='cuda:0'), tensor([[-0.0264, -0.0442,  0.0126,  ..., -0.0082,  0.0192,  0.0284],
        [-0.0042,  0.0113, -0.0253,  ..., -0.0196, -0.0295,  0.0192],
        [ 0.0200,  0.0023,  0.0102,  ..., -0.0102, -0.0014, -0.0131],
        ...,
        [ 0.0072,  0.0493, -0.0232,  ...,  0.0420,  0.0049, -0.0030],
        [-0.0051,  0.0263,  0.0121,  ..., -0.0376,  0.0403,  0.0114],
        [ 0.0259,  0.0221, -0.0482,  ...,  0.0063,  0.0077, -0.0252]],
       device='cuda:0'), tensor([-3.7106e-02, -3.8298e-02,  2.2738e-02, -1.5983e-02,  2.6195e-02,
         3.5053e-02,  7.2507e-02, -6.7104e-03, -7.0900e-04, -3.0073e-02,
         2.9317e-02,  2.3603e-02,  5.5713e-02, -2.6601e-02,  3.4168e-02,
        -3.6878e-02, -3.1206e-02, -7.3537e-03, -3.7581e-02,  4.9358e-02,
        -6.1123e-02, -4.2634e-02, -4.5621e-02,  1.9980e-02,  1.8624e-02,
        -2.1004e-02,  5.2654e-03, -4.4296e-02, -2.1966e-02,  3.9067e-02,
         8.4092e-03,  2.5279e-02, -1.5815e-02,  1.5060e-02,  3.0975e-02,
        -5.4215e-02, -5.2080e-02, -3.2289e-03, -3.1385e-02, -1.6251e-02,
        -4.0056e-02,  3.4076e-02,  5.1041e-03, -3.7053e-02, -3.8213e-02,
        -1.4253e-02,  3.8324e-02,  9.9040e-06, -3.6971e-02,  2.2091e-02,
         2.1728e-03,  1.4337e-02,  2.5231e-02,  1.7998e-02, -2.2629e-02,
         1.3583e-02,  3.1392e-02,  5.3728e-02, -2.8727e-02,  3.6097e-02,
         1.5734e-02, -2.7951e-02, -1.4544e-02,  1.5085e-03, -5.5546e-02,
         5.9274e-04, -7.4222e-02, -9.6284e-03,  1.2603e-02, -1.4933e-02,
        -3.2034e-02,  4.1448e-02, -4.8698e-03,  2.5958e-02, -6.6716e-02,
        -4.6397e-03,  2.9179e-02,  2.6380e-03, -5.5553e-02,  1.5181e-02,
        -3.6774e-02,  4.9253e-02, -6.3079e-02, -2.1218e-02, -4.9277e-02,
        -2.5225e-03,  5.5556e-02,  2.9899e-02,  3.9506e-02, -1.4859e-02,
        -6.7129e-02,  2.6883e-02, -2.6089e-03, -4.6939e-02, -4.5395e-02,
         1.4933e-02,  7.0037e-03, -7.6230e-02,  1.5387e-02, -3.5656e-02,
        -3.6378e-02, -1.0220e-02,  5.5407e-02,  7.2776e-02,  2.7591e-02,
        -1.0925e-02,  1.6650e-02, -3.6408e-02,  3.6307e-02,  1.6867e-02,
        -1.1138e-03, -2.2755e-03,  1.2556e-02, -3.6944e-02, -3.1265e-02,
         4.3087e-02,  1.1210e-02, -5.8535e-02, -1.0100e-02,  3.6369e-02,
        -3.8723e-02,  2.0173e-03,  2.7525e-03, -4.6899e-02, -5.5307e-04,
        -4.6865e-02,  6.0757e-02, -4.0747e-02, -1.3755e-02,  2.7556e-02,
         1.5817e-02,  5.3031e-02, -2.3412e-02,  1.6965e-02,  3.9090e-02,
        -7.2085e-02, -1.6043e-02, -3.0489e-02,  2.8043e-02,  4.9480e-02,
         2.1708e-02,  8.9911e-03, -2.0966e-02,  4.9453e-03, -7.0110e-03,
         5.7311e-02,  5.2924e-02,  2.4336e-02,  5.1614e-02, -5.8336e-02,
        -5.4714e-02, -1.4785e-02,  2.2964e-02,  6.7604e-02, -3.9676e-02,
        -7.5394e-02,  4.5195e-02, -7.6189e-02,  4.4498e-02, -5.4550e-02,
        -1.2810e-02,  1.4084e-02,  3.2349e-02, -1.1240e-02,  7.8732e-03,
         1.5936e-02,  4.7638e-02,  2.0082e-02, -3.6974e-02,  2.8223e-02,
        -3.5356e-02,  6.5774e-03,  4.6503e-02, -3.0177e-02, -1.9279e-02,
         1.2104e-02, -8.1136e-02,  6.4310e-03,  5.3233e-03, -2.0777e-02,
         2.2614e-02, -1.6543e-02, -1.3802e-02,  4.4545e-02,  3.1367e-02,
        -5.9508e-02, -2.8564e-02, -4.8776e-02,  1.9812e-02,  4.4818e-02,
         5.1382e-02,  2.4600e-02, -4.9744e-02, -1.6641e-02,  1.6502e-02,
        -4.2914e-02, -4.4056e-02,  2.5455e-02,  6.5647e-02,  3.1739e-02,
         3.1444e-02,  5.2102e-02,  2.0762e-04,  1.2783e-03,  4.3638e-02,
         2.8012e-02, -2.3049e-02,  4.8755e-02, -5.1435e-02,  4.6029e-02,
         2.1812e-02, -4.1821e-02,  2.0271e-02, -4.2604e-02,  6.1228e-02,
         2.5648e-02, -1.9781e-02,  1.6959e-02, -3.6983e-02,  3.7245e-02,
         3.5026e-02, -2.1238e-02,  5.6526e-03,  3.5015e-02,  5.2052e-02,
         4.2322e-03,  1.8731e-02, -3.8688e-02,  5.9948e-03, -3.0058e-02,
         1.5050e-03,  2.9641e-02,  2.2288e-02, -1.8757e-04,  1.5444e-04,
        -2.5584e-02, -4.6454e-02,  1.3231e-02,  5.4193e-02,  4.8122e-02,
         4.7476e-02, -4.9596e-02, -5.2443e-02,  3.2932e-02,  5.8556e-02,
         1.9111e-02,  7.2199e-02,  3.4455e-02, -3.0776e-02,  7.6131e-04,
        -9.6514e-03, -3.5049e-02, -4.8517e-02, -3.3843e-02,  5.2535e-02,
         3.3187e-02,  3.7967e-04, -1.8782e-03, -3.0098e-02,  4.9800e-02,
        -3.6184e-02,  2.8606e-04, -4.0024e-02,  4.4720e-02,  2.5665e-03,
        -2.6148e-02,  4.5876e-02, -1.5127e-02,  2.8441e-02,  5.7165e-02,
        -8.1022e-03, -2.5657e-02, -4.7317e-02, -3.6597e-03,  1.7415e-02,
         1.7218e-02, -9.8587e-03, -9.4040e-03, -1.0072e-02, -5.0671e-02,
        -1.8050e-02, -1.5815e-02, -1.6300e-03,  6.7171e-03, -2.1609e-02,
         3.9857e-02,  1.0019e-03,  1.9891e-02, -3.8477e-02,  5.1114e-02,
         3.4241e-02,  2.9794e-02, -4.5677e-02, -3.5526e-02, -2.4006e-02,
         9.6954e-03,  1.1910e-02, -1.1273e-02, -3.9381e-02,  9.4785e-03,
        -2.3739e-02, -7.0809e-03, -5.5980e-02,  2.9319e-02,  1.3296e-02,
        -3.3120e-02, -4.2718e-02,  4.1413e-02, -2.3445e-04, -2.2346e-02,
         1.4755e-02, -2.7411e-02,  3.9559e-02,  1.6365e-02,  1.4798e-02,
         5.3453e-02,  2.9396e-03, -3.1321e-02, -3.5818e-02,  3.2793e-02,
        -2.1485e-02,  1.2058e-02,  2.5778e-02, -4.0290e-03, -3.8279e-02,
         2.6563e-02, -8.3392e-03,  7.2586e-03,  1.4948e-02, -2.3102e-02,
        -1.5121e-02, -5.9347e-02, -4.1406e-04,  1.0466e-04,  1.5705e-02,
        -5.5872e-02, -2.4038e-02,  3.2989e-02,  4.0504e-02, -6.9754e-03,
        -3.4969e-02, -5.4281e-03, -7.9741e-03,  3.1044e-02, -1.3245e-02,
         7.5436e-03, -4.1159e-02,  8.5322e-03, -1.7364e-02, -4.2958e-02,
         2.6583e-02, -4.6617e-02, -5.1139e-03, -6.8704e-03,  4.5742e-02,
        -5.4714e-02,  1.5619e-02,  4.2888e-02,  4.4277e-02, -1.2018e-02,
        -4.4488e-02, -1.3197e-02, -9.2389e-03, -4.0524e-02,  3.2505e-02,
         1.2024e-02,  1.3842e-03,  2.8428e-03,  3.5987e-02, -4.6257e-02,
         4.1796e-02, -1.9098e-02,  1.8700e-02, -1.2002e-02, -4.5695e-02,
         5.7857e-02, -4.7821e-02, -3.9615e-02, -9.6361e-03, -1.0752e-02,
         5.4182e-03,  5.3829e-02, -4.9427e-02, -3.4510e-02,  7.2753e-03,
         1.2928e-02, -2.3651e-02,  2.8956e-02,  2.2122e-02, -1.7180e-02,
         4.1501e-02,  7.0769e-03,  2.2471e-02, -1.8273e-02,  3.5087e-02,
         1.0396e-02,  2.0923e-02, -1.4491e-02, -1.4949e-02, -5.3081e-02,
         4.4042e-02, -1.5900e-02,  4.3928e-02, -5.7945e-03,  1.2870e-02,
         4.2674e-02,  3.8964e-02, -5.1914e-03, -3.3962e-02, -4.1443e-02,
         2.4580e-02,  3.2245e-02,  4.4243e-02,  8.6954e-03,  1.1563e-02,
        -3.9847e-02, -2.0648e-02,  5.6670e-03, -1.2986e-03, -5.6044e-03,
         2.5490e-02,  3.2631e-02, -2.9533e-03,  1.8830e-02,  1.8642e-02,
         7.7439e-03,  2.4829e-02, -4.0193e-02, -1.2115e-02,  6.1136e-03,
        -3.6980e-03, -3.5057e-02, -2.1551e-03, -2.2404e-02,  2.7580e-02,
         8.3618e-03, -5.3568e-02,  1.6547e-02, -4.8411e-02,  5.0274e-02,
        -1.3279e-02,  5.4472e-02, -5.8165e-03, -3.5160e-04,  3.2193e-02,
        -1.6499e-02, -1.0831e-02, -1.8476e-02, -4.6578e-02, -1.3949e-02,
        -7.7899e-03,  2.2760e-02,  5.4021e-04, -3.3657e-02,  2.5686e-02,
         1.3437e-02, -2.2892e-02,  3.5308e-02, -2.5172e-02,  4.1342e-02,
        -1.0627e-02,  3.3881e-02, -5.1563e-02, -1.2953e-02, -1.5611e-03,
        -5.0883e-02, -2.7522e-02,  1.9420e-02,  3.2691e-02, -5.6115e-02,
         2.9772e-02,  3.0679e-02,  2.4149e-02,  5.4010e-02, -4.9790e-02,
        -3.8277e-02,  5.0777e-02, -1.9097e-02, -4.8491e-02,  4.4638e-02,
         2.5564e-02, -4.4241e-02, -1.1899e-02,  2.6200e-02, -5.1198e-02,
        -3.1619e-02, -3.1102e-02,  3.5558e-02, -3.6456e-02,  4.4130e-02,
         1.4646e-02,  5.6622e-02,  2.3572e-02, -3.4646e-02, -4.7010e-02,
         1.9585e-02, -5.2600e-02, -8.7185e-03,  1.6992e-03,  8.8474e-03,
        -9.2541e-03,  3.0496e-02, -1.2505e-02, -3.3509e-02, -3.9648e-02,
         1.7239e-02, -1.1110e-02, -9.2456e-03, -4.4366e-02,  4.7216e-02,
        -5.8286e-02,  3.4190e-02], device='cuda:0'), tensor([[[-0.0023],
         [-0.0006],
         [-0.0290],
         ...,
         [-0.0121],
         [ 0.0367],
         [-0.0341]],

        [[ 0.0358],
         [-0.0079],
         [-0.0223],
         ...,
         [ 0.0234],
         [-0.0166],
         [ 0.0337]],

        [[-0.0340],
         [-0.0010],
         [-0.0025],
         ...,
         [-0.0360],
         [ 0.0635],
         [-0.0359]],

        ...,

        [[-0.0039],
         [ 0.0072],
         [-0.0133],
         ...,
         [-0.0048],
         [-0.0482],
         [ 0.0111]],

        [[-0.0037],
         [-0.0219],
         [ 0.0089],
         ...,
         [-0.0522],
         [-0.0095],
         [-0.0272]],

        [[ 0.0239],
         [-0.0088],
         [-0.0380],
         ...,
         [-0.0151],
         [-0.0100],
         [ 0.0254]]], device='cuda:0'), tensor([-0.0100, -0.0001, -0.0121,  0.0119, -0.0242,  0.0003, -0.0167,  0.0189,
        -0.0162, -0.0033, -0.0075, -0.0084, -0.0312,  0.0154, -0.0073,  0.0062,
        -0.0084, -0.0147, -0.0232,  0.0266, -0.0242, -0.0329,  0.0144, -0.0166,
         0.0122,  0.0132,  0.0206,  0.0031, -0.0203, -0.0366, -0.0133, -0.0136,
        -0.0089, -0.0021,  0.0066,  0.0190, -0.0063, -0.0158, -0.0322, -0.0346,
        -0.0319, -0.0209,  0.0254, -0.0111, -0.0267, -0.0211,  0.0121, -0.0169,
         0.0060,  0.0141, -0.0347, -0.0086,  0.0076, -0.0079, -0.0316,  0.0164,
         0.0151,  0.0231, -0.0070, -0.0103,  0.0135, -0.0060, -0.0314, -0.0156,
        -0.0044,  0.0169, -0.0203, -0.0063, -0.0061, -0.0112,  0.0242, -0.0322,
         0.0313, -0.0268, -0.0074, -0.0258,  0.0074, -0.0062, -0.0100, -0.0121,
        -0.0287,  0.0037, -0.0122, -0.0101,  0.0081,  0.0157, -0.0328,  0.0205,
        -0.0072,  0.0059, -0.0299, -0.0108, -0.0078, -0.0341, -0.0156, -0.0048,
         0.0155, -0.0238, -0.0250,  0.0071, -0.0369, -0.0289, -0.0047,  0.0251,
        -0.0013,  0.0208, -0.0375, -0.0210, -0.0136, -0.0353, -0.0077, -0.0097,
         0.0015, -0.0362,  0.0159,  0.0174,  0.0144, -0.0276,  0.0212,  0.0071,
         0.0184,  0.0051, -0.0244,  0.0218,  0.0261,  0.0156, -0.0066, -0.0104,
        -0.0241,  0.0192,  0.0038, -0.0138, -0.0134,  0.0062,  0.0236,  0.0235,
        -0.0026, -0.0263, -0.0182, -0.0115,  0.0102, -0.0333,  0.0010, -0.0286,
         0.0250, -0.0377, -0.0166,  0.0236, -0.0120, -0.0014,  0.0289, -0.0074,
         0.0135, -0.0310, -0.0213, -0.0197, -0.0088, -0.0009, -0.0123, -0.0343,
        -0.0060,  0.0136, -0.0451, -0.0114,  0.0016, -0.0038, -0.0283,  0.0124,
         0.0251, -0.0284, -0.0284, -0.0227, -0.0156, -0.0157, -0.0189, -0.0139,
         0.0229,  0.0155, -0.0334, -0.0147,  0.0084,  0.0018,  0.0035,  0.0228,
        -0.0068,  0.0003,  0.0221,  0.0104,  0.0051, -0.0241,  0.0094, -0.0173,
        -0.0247,  0.0214, -0.0159, -0.0006,  0.0017, -0.0039, -0.0062,  0.0279,
         0.0164, -0.0051,  0.0112, -0.0023,  0.0215,  0.0054, -0.0078, -0.0134,
        -0.0127,  0.0101, -0.0167, -0.0248, -0.0132, -0.0067, -0.0207, -0.0184,
        -0.0228, -0.0091,  0.0227,  0.0194, -0.0256,  0.0254,  0.0048,  0.0107,
        -0.0326,  0.0017, -0.0261,  0.0184,  0.0042, -0.0205, -0.0292, -0.0131,
        -0.0335, -0.0100, -0.0118, -0.0224,  0.0244, -0.0167,  0.0173,  0.0038,
         0.0085, -0.0045, -0.0191, -0.0354, -0.0021, -0.0295, -0.0132, -0.0119,
        -0.0274,  0.0177,  0.0199, -0.0135, -0.0092,  0.0048,  0.0214, -0.0023],
       device='cuda:0'), tensor([[[-1.9681e-02,  2.5644e-03,  2.9600e-02,  2.9113e-02,  2.6786e-03],
         [ 1.2823e-02,  1.9893e-02,  1.2966e-02, -2.2760e-02, -1.6560e-03],
         [ 2.1457e-02, -1.6772e-02, -2.9785e-03,  2.5547e-02, -5.6965e-03],
         ...,
         [ 4.3122e-02, -3.3034e-02, -1.7550e-02, -1.2566e-02,  2.7031e-02],
         [-1.7386e-02,  1.9852e-02, -9.9822e-03,  1.6694e-02, -8.3877e-03],
         [-6.2856e-03, -2.9515e-03,  5.9230e-03, -1.0908e-02,  1.7491e-02]],

        [[ 2.8746e-02,  1.0395e-02, -1.0376e-02,  1.1638e-02,  5.6849e-03],
         [-1.0218e-02, -1.4109e-02,  5.1873e-03,  2.0006e-02,  2.8999e-03],
         [ 6.0476e-03,  7.7014e-04, -2.4514e-02, -2.2086e-02,  2.8120e-02],
         ...,
         [ 3.4711e-03, -1.6858e-02, -5.6284e-03,  1.1079e-02, -2.8059e-02],
         [ 3.6959e-05,  4.0522e-03, -4.6045e-03,  2.2127e-02, -1.3916e-03],
         [ 1.0108e-02, -2.8442e-02, -6.5068e-03, -2.2737e-02,  3.6666e-02]],

        [[-1.5714e-02,  2.9944e-03, -1.0285e-02, -1.0330e-02,  4.1789e-04],
         [-3.5136e-03,  2.1480e-03, -1.1202e-02,  1.3435e-02, -2.6878e-02],
         [-7.0278e-03, -1.9662e-02,  1.5621e-02, -3.1543e-02, -3.3832e-02],
         ...,
         [ 1.0349e-04,  1.9887e-02,  1.7343e-02, -3.3331e-02,  1.3188e-03],
         [-2.8313e-03,  4.5045e-03, -1.6503e-02, -2.7172e-02, -2.9527e-02],
         [-2.3359e-02,  9.5442e-03, -1.4628e-02, -1.7879e-02,  8.7669e-03]],

        ...,

        [[ 8.6419e-03,  1.3979e-02, -1.4309e-02,  1.5309e-03, -3.2040e-02],
         [-1.0742e-02, -5.9149e-03,  1.4703e-02, -3.5353e-03,  6.6051e-04],
         [-1.7453e-03, -3.6583e-03,  2.8786e-02, -2.3837e-02,  1.9327e-02],
         ...,
         [-5.2696e-03, -1.6188e-02,  4.0162e-03, -1.0085e-02, -6.3031e-03],
         [ 3.4317e-03,  2.2255e-02,  2.4672e-02,  1.5422e-02,  2.1343e-03],
         [-1.1617e-02, -1.4454e-02,  2.1274e-02, -1.8744e-02,  1.0381e-03]],

        [[-2.3754e-04, -1.0660e-02,  3.5795e-03, -1.9878e-02, -2.7186e-03],
         [-2.4535e-02, -1.9013e-02, -9.3606e-03,  1.6891e-02,  2.3931e-02],
         [ 2.5258e-02,  2.6337e-02,  1.7468e-02, -7.6562e-03, -6.2260e-03],
         ...,
         [ 3.6432e-02, -1.8262e-02,  1.3772e-02,  3.4986e-02, -1.5503e-03],
         [-2.0389e-02,  2.9268e-02, -7.5903e-03,  2.1104e-02,  2.2110e-02],
         [ 7.7627e-03,  1.8584e-02,  1.3877e-02, -7.1084e-03,  2.4084e-02]],

        [[ 4.1313e-03,  7.8521e-03,  4.0560e-02,  1.8011e-02, -2.9391e-02],
         [-1.5098e-02, -1.5213e-02,  2.3782e-02,  1.1192e-02, -2.2357e-02],
         [-2.1764e-02,  1.4503e-02, -2.2784e-03, -1.3650e-02, -1.9534e-02],
         ...,
         [-1.2099e-02, -3.0849e-02,  2.0197e-02,  4.1907e-03, -5.9497e-04],
         [ 1.2567e-02, -2.3797e-02,  1.2042e-02, -5.7292e-03,  2.3272e-02],
         [ 3.0963e-02, -1.6664e-02,  2.1062e-02, -8.8696e-03, -1.1956e-03]]],
       device='cuda:0'), tensor([-1.5825e-02, -2.2956e-02,  9.0689e-03, -1.9339e-03,  2.1333e-02,
         1.6171e-02,  2.0571e-02, -2.4546e-02,  1.7075e-02, -3.1302e-03,
        -3.6533e-03, -1.0033e-02, -9.6508e-03,  2.1569e-02,  2.6848e-02,
         1.1046e-02, -7.4587e-03,  3.6046e-02, -1.6667e-02,  5.8381e-03,
        -9.3046e-03,  1.6996e-02, -2.3951e-03,  1.1384e-02,  8.3386e-03,
        -1.2006e-02,  3.4138e-03,  2.9870e-02, -1.3312e-02,  2.5908e-02,
         9.7896e-03, -2.7236e-02, -3.1681e-02, -1.3404e-02,  2.9428e-03,
        -2.7866e-02, -1.7304e-02, -5.7074e-03, -1.5988e-02, -3.7369e-03,
         7.5989e-04, -1.7509e-02, -5.2932e-03, -1.2807e-02,  7.7059e-03,
         1.6172e-02, -1.6356e-02,  1.0061e-02,  1.7017e-02,  4.0331e-02,
         5.3451e-03, -2.4832e-02,  1.1495e-02, -1.2477e-02, -1.5285e-02,
        -1.2026e-03, -2.6272e-03,  4.9354e-03, -4.5764e-04, -5.3355e-03,
         1.9286e-02,  2.3048e-02, -1.7496e-02, -1.1784e-02,  6.7892e-03,
         2.3010e-03, -1.9433e-02,  3.3460e-02,  1.8646e-02,  1.9435e-02,
        -1.2630e-02, -1.4819e-02, -1.9615e-02, -1.3838e-02,  1.8929e-02,
        -1.3687e-02,  3.7773e-02, -2.2104e-04,  2.1950e-02,  3.5763e-03,
        -1.3003e-02, -3.8114e-02,  1.5023e-02, -2.0629e-02, -2.8829e-03,
         2.8417e-03, -2.6791e-05,  2.8164e-02, -7.3110e-03, -1.5122e-02,
         2.1554e-04,  3.7294e-03, -9.2144e-03,  2.5204e-02, -2.0185e-02,
         1.1745e-02, -1.4950e-02, -1.0911e-02, -2.0712e-03,  1.5328e-02,
         1.2151e-02,  8.2827e-03, -6.5581e-03, -1.3830e-02, -2.0966e-03,
        -2.9421e-04, -4.5503e-03, -9.7396e-03, -1.5613e-02,  1.7715e-02,
        -2.6029e-02,  3.1862e-02,  6.9462e-03, -3.9401e-03, -6.9684e-03,
         3.4452e-02, -1.0858e-02,  1.6554e-03,  2.6733e-02,  1.3862e-02,
         3.3701e-04, -2.1151e-02,  2.5918e-03, -1.0886e-02,  1.8383e-02,
         2.7102e-02, -2.5652e-02,  2.7875e-02, -1.1494e-02,  2.3854e-02,
         2.2791e-02, -8.3536e-04, -9.2476e-03,  5.4526e-03,  1.4320e-02,
        -2.2354e-02, -8.4118e-03, -2.4261e-02,  2.5369e-02, -1.5746e-02,
         2.4744e-03,  1.4632e-02, -2.6997e-02,  8.0496e-03, -2.7036e-02,
        -2.3817e-02,  1.8912e-02, -1.1292e-02,  2.6216e-03, -2.7139e-02,
         6.1644e-04, -4.7521e-03, -5.8312e-03,  1.2452e-02, -3.0049e-03,
        -3.7467e-03,  3.0349e-02, -1.6143e-02, -2.0320e-02,  5.8375e-03,
         8.0152e-03,  5.9621e-03, -3.1833e-03,  2.2893e-02,  2.1673e-02,
         9.2091e-03,  3.1197e-03, -1.4468e-02, -1.2682e-02,  3.8368e-02,
         2.1879e-02, -3.5164e-02, -6.5828e-03,  7.2788e-03,  2.9006e-02,
         3.2410e-02, -4.4065e-03, -2.0167e-02, -2.0003e-02,  2.8125e-02,
         2.9201e-02,  3.8377e-03,  1.0952e-02,  2.6992e-02, -9.5677e-03,
         7.8339e-03,  2.6608e-03, -2.4124e-02, -1.0513e-02,  1.8611e-02,
         1.3954e-02,  2.6047e-02, -6.2252e-03,  1.5309e-02,  2.8168e-02,
         1.8480e-02, -7.4261e-03,  1.4507e-02,  3.2483e-02, -1.8057e-02,
         9.0859e-03, -1.5517e-02, -1.6105e-02, -8.2318e-03,  2.2905e-02,
         1.2102e-02, -6.4141e-03, -1.3326e-02,  5.5673e-03, -8.6304e-03,
         1.6876e-02,  6.7847e-03, -1.7082e-02,  2.0149e-02,  6.1313e-03,
         1.4477e-02,  2.0196e-02,  8.3342e-03,  5.8809e-03,  1.0059e-02,
        -1.9127e-03, -4.4469e-03, -1.8052e-02, -9.2052e-03,  1.0658e-02,
        -3.0824e-02,  1.7876e-02, -1.1532e-02,  9.5519e-03,  6.7578e-03,
         2.2566e-03, -1.8520e-03, -1.3171e-02, -4.4475e-04,  6.7620e-03,
        -1.7809e-02, -5.3658e-03,  1.5931e-02, -1.7997e-02,  2.7305e-02,
         1.3665e-02,  1.0722e-02,  1.9106e-02, -2.7131e-02, -1.3054e-03,
         1.4682e-03, -1.6051e-02,  1.6196e-02, -2.0179e-02, -1.6741e-02,
        -3.4717e-02, -1.7420e-02,  1.2563e-02, -2.7144e-02, -2.4207e-02,
         7.5531e-03], device='cuda:0'), tensor([0.9727, 0.9963, 0.9944, 0.9996, 0.9832, 1.0079, 0.9843, 0.9991, 0.9676,
        1.0153, 1.0398, 0.9788, 0.9786, 0.9944, 0.9905, 0.9608, 1.0010, 0.9989,
        1.0070, 1.0358, 0.9912, 0.9702, 1.0200, 1.0157, 1.0468, 1.0360, 1.0368,
        0.9786, 0.9918, 1.0037, 0.9808, 0.9392, 0.8880, 0.9370, 0.8144, 0.8802,
        0.8854, 0.9000, 0.9178, 0.8604, 0.9158, 0.8359, 0.9196, 0.7703, 0.9198,
        0.9520, 0.9343, 0.9558, 0.9210, 1.0123, 0.9363, 0.8765, 0.8967, 0.8420,
        0.8410, 0.9294, 0.8051, 0.7959, 0.8984, 0.8315, 0.7790, 0.9168, 0.8521,
        0.8876, 0.9479, 0.9974, 0.9871, 0.9467, 0.9983, 1.0102, 0.9933, 0.9532,
        1.0039, 0.9843, 1.0099, 0.9478, 1.0095, 1.0557, 0.9781, 0.9237, 1.0014,
        1.0085, 0.9946, 0.9719, 0.9964, 0.9083, 0.9927, 0.9716, 0.9825, 0.9955,
        0.9949, 0.9920, 1.0099, 0.9532, 0.9883, 1.0077, 0.9901, 0.9997, 1.0033,
        0.9839, 1.0017, 0.9713, 0.9941, 0.9944, 1.0086, 0.9755, 1.0341, 0.9620,
        0.9748, 1.0150, 1.0065, 0.9906, 0.9917, 1.0157, 0.9781, 0.9970, 0.9994,
        0.9709, 0.9646, 1.0285, 0.9794, 0.9823, 0.9630, 0.9530, 0.9317, 0.9823,
        1.0035, 0.9939, 0.9718, 0.9882, 0.9704, 1.0039, 1.0404, 0.9370, 0.9400,
        0.9772, 0.9962, 1.0293, 0.9352, 0.9690, 0.9422, 0.9922, 0.9551, 0.9747,
        0.9863, 0.9860, 0.9894, 1.0134, 0.9708, 1.0054, 0.9671, 1.0088, 0.9793,
        0.9600, 0.9919, 0.9851, 0.9813, 0.9887, 0.9817, 0.9841, 0.9942, 0.9908,
        1.0063, 0.9923, 1.0009, 0.9800, 1.0309, 0.9982, 1.0173, 0.9653, 1.0067,
        1.0123, 0.9728, 0.9716, 0.9915, 0.9868, 0.9725, 1.0464, 1.0709, 0.9967,
        0.9601, 0.9891, 1.0095, 0.9898, 0.9739, 1.0243, 1.0791, 0.9942, 0.9928,
        1.0325, 1.0024, 0.9951, 0.9636, 1.0040, 0.9837, 0.9899, 0.9649, 0.9940,
        0.9920, 0.9776, 0.9674, 0.9872, 0.9902, 0.9919, 0.9921, 1.0156, 0.9837,
        0.9995, 0.9856, 0.9740, 0.9847, 0.9881, 1.0546, 0.9762, 0.9897, 0.9934,
        0.9903, 0.9714, 1.0005, 1.0430, 1.0265, 0.9740, 1.0129, 0.9737, 0.9575,
        0.9957, 1.0023, 1.0686, 1.0037, 0.9906, 0.9735, 0.9721, 0.9490, 0.9764,
        0.9908, 0.9527, 0.9743, 0.9594, 0.9876, 1.0076, 0.9817, 0.9563, 1.0342,
        1.0609, 1.0032, 1.0031, 0.9988, 0.9573, 0.9334, 0.9995, 0.9667, 1.0114,
        0.9812, 0.9692, 0.9997, 0.9786], device='cuda:0'), tensor([-9.6649e-03, -1.3228e-02, -3.5486e-03,  4.9763e-03, -3.3398e-03,
        -1.3526e-02, -7.4918e-03, -2.1028e-02, -1.5542e-02, -8.4376e-03,
        -3.7053e-02,  4.7101e-03,  2.1363e-03,  9.3138e-04, -4.9426e-03,
        -8.1740e-03,  3.2358e-03, -1.8317e-03,  6.8745e-03, -5.7700e-03,
         2.9136e-03, -3.1481e-03, -2.0031e-02,  8.9982e-03, -1.4285e-02,
        -1.1228e-02, -2.5629e-02, -1.1452e-02, -7.0980e-03,  3.4102e-03,
         1.4363e-03, -1.3989e-02, -4.4642e-02, -2.5294e-02, -6.6918e-02,
        -3.9845e-02, -3.8371e-02, -1.3033e-02, -2.2515e-02, -4.3506e-02,
        -1.4185e-02, -4.7850e-02, -2.4362e-02, -1.0036e-01, -4.3796e-03,
        -4.7109e-02, -1.8532e-02, -2.5919e-02, -2.9400e-02,  5.6189e-03,
         1.0663e-02, -1.8996e-02, -3.2203e-02, -3.1981e-02, -5.1747e-02,
        -2.3306e-02, -4.3773e-02, -8.3605e-02, -3.4233e-02, -7.2182e-02,
        -6.7935e-02, -2.8416e-02, -3.4848e-02, -1.4018e-02, -4.6944e-04,
        -1.7866e-03, -1.1881e-02,  5.0125e-03, -2.4884e-03, -1.6591e-02,
        -6.3505e-03,  9.3963e-03,  7.6852e-03,  5.6724e-03, -4.7442e-03,
        -7.8755e-03,  1.0946e-02, -9.8426e-02, -6.9598e-03,  3.6642e-03,
        -3.4586e-03, -1.9309e-02,  1.2868e-02,  1.2776e-03, -1.0760e-02,
         5.7802e-03,  4.7564e-03, -6.0171e-05, -7.4157e-03, -2.4104e-03,
        -8.3028e-03, -2.8712e-03, -5.3387e-02, -5.5999e-03, -3.0248e-03,
        -2.0754e-02, -3.3077e-03,  1.6177e-03, -6.1162e-03, -2.0641e-03,
        -2.8321e-02, -5.5870e-05, -3.6743e-03,  3.5445e-04,  5.3066e-03,
        -6.2173e-03, -1.7172e-02, -1.5715e-02, -8.0699e-03, -1.0952e-02,
        -1.3775e-02, -1.0748e-03, -1.0599e-02, -2.2131e-02, -5.0332e-03,
        -6.1611e-03,  1.8919e-03, -5.1645e-03, -2.5612e-02, -1.7328e-02,
        -8.5106e-03, -4.0706e-03, -1.5749e-02, -2.9932e-02, -3.0382e-02,
        -9.3806e-03, -1.2385e-02, -2.0413e-03, -1.0953e-02, -8.2069e-03,
        -1.6860e-02,  1.7369e-04, -2.1431e-02, -2.0467e-02, -1.3600e-02,
        -1.3223e-02, -2.7755e-03, -2.9466e-02, -1.0110e-02, -1.3222e-02,
         1.2726e-03,  9.8114e-03, -1.1741e-02, -1.8401e-02, -7.8952e-03,
        -1.3702e-02,  4.6547e-03, -2.6939e-02, -8.8842e-03, -2.0083e-02,
        -8.7914e-03, -2.4387e-03,  1.6747e-03, -2.6232e-02, -1.0269e-02,
        -1.1912e-02, -5.0755e-03, -1.1302e-02, -1.4008e-02, -1.6002e-02,
        -7.1853e-03,  1.3722e-03,  1.2237e-02, -1.9712e-02, -1.1037e-02,
        -2.6354e-03, -1.8336e-02, -8.4767e-03, -1.2585e-02, -3.5884e-02,
        -1.6464e-02, -1.5020e-02, -9.4157e-03,  5.6951e-03, -5.4295e-03,
        -3.9247e-03, -6.2551e-03, -5.6948e-03, -1.5414e-02, -4.5876e-04,
        -3.5936e-02, -5.9830e-04,  1.0172e-02, -9.7332e-03, -1.0126e-02,
        -3.6353e-02, -1.1026e-02, -1.5761e-02, -7.6461e-03, -6.3099e-02,
         7.4218e-03, -4.1201e-03, -2.4202e-02, -1.0912e-02, -1.2890e-02,
        -8.2349e-03, -1.6686e-02, -1.7669e-02, -1.5380e-03, -1.4658e-02,
        -1.8299e-02, -1.0046e-03, -3.2635e-03, -6.9633e-03, -1.3340e-04,
        -2.3240e-02, -4.0762e-03,  3.5196e-03, -7.8853e-03, -5.0780e-03,
        -1.2187e-02, -8.0146e-03, -4.2106e-02, -1.9478e-02, -4.1591e-03,
        -8.9200e-03, -1.3787e-02, -2.6853e-02, -2.7914e-03, -4.1938e-02,
        -2.2076e-02, -1.4183e-02,  5.1230e-03, -2.1553e-02, -2.1779e-03,
         2.0667e-03,  1.6840e-03, -5.5274e-02, -6.6891e-02,  1.5792e-03,
        -2.1187e-03, -1.3865e-02, -1.9776e-03, -1.0299e-02, -1.8442e-04,
         3.5494e-03,  2.4037e-05, -8.7849e-03,  3.1127e-03,  1.0007e-02,
        -4.0615e-03, -3.4464e-03, -1.4165e-02, -3.8907e-02, -7.1053e-03,
         8.3205e-04,  2.4199e-03, -1.0808e-02, -1.2930e-04,  6.1465e-03,
        -1.6147e-02, -7.3868e-03, -5.8270e-04, -8.4883e-03,  2.7373e-03,
         3.4739e-03], device='cuda:0'), tensor([[[ 0.0080,  0.0073,  0.0187, -0.0157,  0.0283],
         [-0.0072, -0.0283,  0.0079, -0.0160, -0.0097],
         [-0.0230, -0.0043, -0.0174, -0.0166,  0.0043],
         ...,
         [-0.0288, -0.0238, -0.0185, -0.0238, -0.0145],
         [ 0.0005, -0.0091, -0.0344, -0.0147,  0.0203],
         [-0.0141, -0.0068,  0.0079, -0.0061,  0.0012]],

        [[-0.0162,  0.0017, -0.0237,  0.0060,  0.0061],
         [-0.0127, -0.0426, -0.0134, -0.0151,  0.0198],
         [ 0.0011, -0.0242, -0.0218, -0.0330,  0.0235],
         ...,
         [-0.0026,  0.0166,  0.0151, -0.0214, -0.0197],
         [-0.0183,  0.0303,  0.0094,  0.0079,  0.0162],
         [-0.0344,  0.0029, -0.0268, -0.0318,  0.0011]],

        [[-0.0131, -0.0019, -0.0075,  0.0113, -0.0218],
         [-0.0060, -0.0125,  0.0161,  0.0018, -0.0050],
         [-0.0256,  0.0096, -0.0104,  0.0014, -0.0070],
         ...,
         [ 0.0146, -0.0200,  0.0123, -0.0215, -0.0231],
         [ 0.0030, -0.0319, -0.0332, -0.0134,  0.0118],
         [-0.0225,  0.0174, -0.0067, -0.0095, -0.0072]],

        ...,

        [[-0.0166,  0.0268,  0.0098,  0.0154,  0.0045],
         [ 0.0143, -0.0103,  0.0287,  0.0364,  0.0128],
         [ 0.0359,  0.0024, -0.0062, -0.0033,  0.0010],
         ...,
         [ 0.0282,  0.0190,  0.0236,  0.0008,  0.0114],
         [ 0.0166, -0.0127,  0.0164,  0.0188, -0.0182],
         [-0.0075,  0.0085, -0.0234, -0.0258,  0.0062]],

        [[-0.0044, -0.0047,  0.0062,  0.0132,  0.0169],
         [-0.0401, -0.0022, -0.0074, -0.0177, -0.0030],
         [ 0.0055,  0.0163,  0.0250,  0.0171,  0.0246],
         ...,
         [-0.0311, -0.0027, -0.0197,  0.0023, -0.0039],
         [ 0.0040,  0.0144,  0.0197,  0.0107,  0.0130],
         [-0.0034, -0.0137, -0.0345, -0.0163, -0.0028]],

        [[ 0.0231, -0.0017,  0.0090, -0.0056, -0.0054],
         [ 0.0279, -0.0098, -0.0309,  0.0092, -0.0087],
         [ 0.0075,  0.0006,  0.0013, -0.0152, -0.0136],
         ...,
         [ 0.0195,  0.0009,  0.0244, -0.0098,  0.0290],
         [ 0.0108, -0.0178, -0.0297, -0.0074, -0.0161],
         [ 0.0240,  0.0275,  0.0046,  0.0251, -0.0234]]], device='cuda:0'), tensor([-2.0530e-02, -9.5409e-03, -1.4938e-03,  1.5552e-02, -1.0987e-02,
        -6.4345e-03,  1.8230e-02,  1.8277e-02, -2.8605e-02,  9.7413e-03,
         3.0177e-02,  2.6276e-02, -2.7756e-02,  1.4405e-02, -1.0167e-02,
        -8.2618e-02,  3.3294e-02, -2.8736e-02, -7.7600e-04, -3.9396e-03,
        -2.2412e-02,  1.6445e-02,  3.5556e-03, -6.9434e-02,  1.8606e-02,
         1.0351e-02, -1.5366e-02, -3.2304e-02, -2.9277e-02, -8.3705e-03,
        -7.8244e-03, -2.7889e-02, -9.8408e-04, -1.2825e-02,  2.4014e-02,
        -6.3071e-03,  2.4040e-02, -4.6585e-02, -2.0753e-02, -1.8357e-02,
         2.2544e-02,  2.7357e-02,  1.5239e-02, -5.6845e-03, -1.5864e-03,
         2.3462e-03,  9.0339e-03,  2.1374e-03, -4.0422e-04, -2.8667e-02,
         3.3989e-02, -2.8531e-05, -1.3288e-02, -5.3033e-03, -3.3993e-02,
         2.2573e-03, -6.7239e-02,  1.2019e-02,  1.5298e-02, -1.1179e-02,
         1.9153e-02,  1.1367e-02,  1.3556e-03, -2.3078e-02, -1.4103e-04,
         1.5565e-02,  3.0851e-03,  2.2243e-02, -2.9370e-02,  2.2368e-02,
        -1.8058e-02,  1.0277e-02, -5.3114e-03,  2.1101e-02, -2.6809e-02,
        -4.1958e-02,  1.3642e-02, -1.6944e-02, -1.1090e-02,  5.5030e-03,
        -9.3179e-03,  1.3578e-02, -1.3778e-02,  1.8779e-02, -1.0662e-02,
        -6.0129e-03, -2.4651e-03,  2.2945e-02, -2.3112e-03,  2.1663e-04,
        -7.7097e-03,  1.9911e-02, -1.3614e-02,  1.6402e-02, -1.2062e-02,
        -2.1252e-02,  2.3672e-03, -4.4086e-03, -2.7450e-03, -6.8682e-03,
         1.7100e-02,  1.0703e-02, -2.5639e-02, -8.3761e-03, -1.9459e-02,
         2.5538e-02,  1.0998e-02,  4.9294e-03,  2.0585e-02,  3.3144e-02,
        -1.3957e-02, -1.8042e-02,  1.8881e-02, -7.2543e-02,  5.1180e-03,
        -2.9312e-02,  2.6902e-02, -1.0554e-03, -7.5974e-03,  1.0190e-02,
         1.2516e-02, -3.7607e-03,  7.5592e-03,  1.0054e-02, -4.9515e-02,
        -7.5300e-03, -2.0813e-02,  1.1459e-02,  2.4066e-02,  1.6284e-02,
        -2.2966e-03, -1.0600e-02, -1.7093e-02,  2.0044e-02,  1.7171e-02,
         1.8683e-02, -2.0932e-02, -1.6785e-02, -4.5141e-02,  1.8866e-02,
         1.0780e-02,  1.0798e-02,  1.0491e-02,  2.7699e-02,  2.9298e-02,
        -1.6084e-02,  9.3488e-03,  1.3314e-02,  6.5777e-03,  1.0995e-02,
        -4.9254e-02, -2.7594e-03, -1.5091e-03,  4.1445e-03, -1.0999e-02,
        -2.0115e-02, -1.7535e-02, -4.9815e-02,  2.6022e-02,  1.3487e-02,
        -4.8528e-03,  1.9884e-02,  7.5734e-03, -6.0687e-03, -9.2453e-03,
        -1.0904e-02,  3.0426e-02,  2.9747e-02,  5.5408e-03,  3.1432e-02,
        -1.9907e-02,  2.2066e-03,  1.9920e-02,  2.6345e-02, -1.6517e-02,
        -2.8551e-02,  1.7478e-02,  2.0153e-02, -4.3729e-03, -5.2041e-02,
        -1.2143e-02, -1.4028e-02, -2.4214e-02, -1.1344e-02,  7.4071e-03,
        -2.9734e-02, -1.2001e-02, -4.5216e-03, -1.4029e-02, -5.6914e-03,
         1.1673e-02, -6.6042e-02,  2.8146e-02,  1.3230e-02,  1.7306e-02,
        -2.4356e-02, -6.3757e-03,  9.8711e-03,  6.3930e-03, -1.2830e-02,
        -1.4889e-02, -1.7014e-02,  8.7321e-03, -6.5834e-03,  1.0398e-02,
         1.7484e-02, -1.5452e-02,  1.6129e-02,  2.3335e-02,  1.2672e-02,
         2.8865e-02,  1.6097e-02, -2.2874e-02, -2.3047e-02,  5.7148e-03,
        -2.4566e-02,  2.8887e-02,  8.4202e-03,  1.4458e-02, -2.5749e-02,
        -5.7764e-02, -3.7981e-02, -1.8004e-02, -8.7186e-03,  1.9644e-02,
         1.3059e-02, -4.6961e-02, -1.6060e-02, -2.4214e-02, -2.9134e-02,
         5.5100e-03, -1.5358e-02,  6.4994e-03, -6.2700e-03,  1.9005e-02,
        -2.9147e-03, -6.7277e-04,  7.9123e-03,  6.2506e-03, -5.6872e-02,
        -2.3090e-02, -1.5202e-02, -1.2924e-04,  2.5060e-02, -1.2523e-02,
         2.8982e-02, -3.8060e-02, -2.3709e-02,  1.2567e-02,  1.8219e-02,
         7.4128e-03, -6.5206e-02, -9.3799e-03, -1.1825e-02, -1.3584e-02,
         6.4497e-03], device='cuda:0'), tensor([0.9827, 0.9886, 0.9848, 0.9733, 0.9838, 0.9973, 0.9823, 0.9945, 0.9826,
        0.9644, 0.9906, 0.9861, 0.9715, 0.9606, 0.9806, 0.9956, 0.9830, 0.9792,
        0.9938, 0.9775, 0.9801, 0.9800, 0.9844, 0.9923, 0.9743, 1.0041, 1.0016,
        0.9792, 1.0001, 0.9845, 0.9557, 0.9598, 0.9398, 0.9822, 0.9787, 0.9488,
        0.9617, 1.0007, 0.9817, 0.9983, 0.9710, 0.9653, 0.9576, 0.9869, 0.9886,
        0.9806, 0.9846, 0.9829, 0.9662, 0.9622, 0.9828, 0.9984, 0.9761, 0.9872,
        0.9962, 0.9869, 0.9999, 0.9841, 0.9632, 0.9757, 0.9766, 0.9856, 0.9965,
        0.9761, 0.9848, 1.0029, 0.9431, 0.9958, 0.9861, 0.9796, 0.9896, 1.0072,
        1.0031, 0.9815, 0.9595, 1.0061, 0.9241, 0.9760, 0.9594, 0.9762, 0.9631,
        0.9788, 0.9899, 0.9499, 0.9739, 0.9777, 0.9532, 0.9459, 0.9503, 0.9821,
        0.9633, 0.9479, 0.9949, 0.9929, 0.9746, 0.9905, 0.9739, 0.9854, 0.9696,
        1.0031, 0.9830, 0.9825, 0.9561, 0.9857, 0.9832, 0.9652, 0.9742, 0.9750,
        0.9908, 0.9880, 0.9729, 0.9708, 0.9852, 0.9947, 0.9696, 0.9947, 0.9909,
        0.9813, 0.9914, 0.9865, 0.9785, 0.9806, 0.9799, 0.9826, 1.0001, 0.9582,
        0.9964, 0.9786, 0.9692, 0.9992, 0.9519, 0.9644, 0.9977, 0.9946, 0.9970,
        0.9546, 0.9788, 0.9679, 0.9973, 0.9305, 0.9792, 0.9660, 0.9563, 0.9812,
        0.9866, 0.9803, 0.9710, 0.9870, 0.9697, 0.9838, 0.9963, 0.9665, 0.9882,
        0.9750, 0.9574, 0.9353, 0.9766, 0.9997, 0.9943, 0.9841, 0.9831, 0.9944,
        0.9766, 0.9719, 1.0028, 0.9832, 0.9730, 0.9751, 0.9858, 0.9834, 0.9890,
        0.9733, 0.9420, 0.9604, 0.9913, 0.9825, 0.9787, 0.9756, 0.9590, 1.0027,
        0.9784, 0.9716, 0.9871, 0.9695, 0.9867, 0.9738, 1.0029, 0.9835, 0.9987,
        0.9770, 0.9815, 0.9975, 0.9786, 0.9822, 0.9405, 1.0005, 0.9978, 0.9813,
        0.9707, 0.9911, 0.9873, 0.9779, 0.9827, 0.9966, 0.9508, 0.9803, 0.9703,
        0.9936, 0.9836, 0.9988, 0.9687, 0.9865, 0.9694, 0.9643, 0.9765, 0.9870,
        0.9664, 0.9990, 0.9504, 0.9548, 1.0007, 0.9966, 0.9844, 0.9953, 0.9797,
        0.9905, 0.9985, 0.9935, 0.9801, 0.9999, 0.9728, 1.0001, 0.9781, 0.9839,
        0.9836, 0.9823, 0.9884, 0.9760, 0.9833, 0.9985, 0.9793, 0.9818, 0.9607,
        0.9699, 0.9820, 0.9785, 1.0025, 0.9876, 0.9832, 0.9649, 0.9663, 0.9931,
        0.9886, 0.9978, 0.9847, 0.9861], device='cuda:0'), tensor([-0.0128, -0.0068, -0.0115, -0.0107, -0.0062,  0.0003, -0.0079, -0.0024,
        -0.0103, -0.0120, -0.0041, -0.0096, -0.0081, -0.0157, -0.0071,  0.0023,
        -0.0091, -0.0137, -0.0044, -0.0067, -0.0118, -0.0126, -0.0082,  0.0065,
        -0.0139, -0.0085, -0.0123, -0.0118, -0.0021, -0.0089, -0.0090, -0.0094,
        -0.0134, -0.0089, -0.0107, -0.0085, -0.0141, -0.0020, -0.0057, -0.0127,
        -0.0126, -0.0091, -0.0052, -0.0035, -0.0052, -0.0105, -0.0061, -0.0083,
        -0.0116, -0.0097, -0.0114, -0.0014, -0.0127, -0.0041,  0.0026, -0.0070,
        -0.0024, -0.0082, -0.0088, -0.0099, -0.0090, -0.0057, -0.0056, -0.0080,
        -0.0048, -0.0168, -0.0109, -0.0141, -0.0074, -0.0107, -0.0124, -0.0142,
         0.0011, -0.0108, -0.0120, -0.0241, -0.0074, -0.0092, -0.0119, -0.0106,
        -0.0078, -0.0108, -0.0060, -0.0094, -0.0089, -0.0102, -0.0077, -0.0210,
        -0.0080, -0.0124, -0.0092, -0.0117, -0.0034, -0.0059, -0.0085, -0.0101,
        -0.0071, -0.0087, -0.0107, -0.0127, -0.0091, -0.0048, -0.0158, -0.0116,
        -0.0111, -0.0105, -0.0103, -0.0056, -0.0035, -0.0064, -0.0077, -0.0102,
        -0.0062,  0.0032, -0.0144, -0.0050, -0.0056, -0.0086, -0.0043, -0.0080,
        -0.0107, -0.0064, -0.0066, -0.0095, -0.0015, -0.0101, -0.0075, -0.0109,
        -0.0102, -0.0022, -0.0126, -0.0123, -0.0129, -0.0050, -0.0119, -0.0097,
        -0.0086, -0.0070, -0.0016, -0.0086, -0.0080, -0.0105, -0.0120, -0.0084,
        -0.0062, -0.0122, -0.0078, -0.0033, -0.0140, -0.0100, -0.0058, -0.0120,
        -0.0093, -0.0078, -0.0114, -0.0150, -0.0065, -0.0286, -0.0043, -0.0090,
        -0.0087, -0.0129, -0.0174, -0.0115, -0.0089, -0.0061, -0.0093, -0.0073,
        -0.0068, -0.0120, -0.0061, -0.0102, -0.0144, -0.0074, -0.0061, -0.0058,
        -0.0124, -0.0091, -0.0091, -0.0062, -0.0163, -0.0089, -0.0065, -0.0104,
        -0.0057, -0.0110, -0.0080, -0.0084, -0.0109, -0.0095, -0.0168,  0.0015,
        -0.0089, -0.0068, -0.0166, -0.0146, -0.0037, -0.0070, -0.0071,  0.0049,
        -0.0063, -0.0098, -0.0118, -0.0080, -0.0155, -0.0084, -0.0079, -0.0056,
        -0.0070, -0.0013, -0.0101, -0.0080, -0.0113, -0.0116, -0.0131, -0.0067,
        -0.0106, -0.0134, -0.0159, -0.0109, -0.0057, -0.0004, -0.0060, -0.0061,
        -0.0113, -0.0056, -0.0017, -0.0152, -0.0083, -0.0048, -0.0137, -0.0018,
        -0.0081, -0.0082, -0.0073, -0.0102, -0.0049, -0.0115, -0.0058, -0.0017,
        -0.0131, -0.0106, -0.0123, -0.0122, -0.0078, -0.0086, -0.0049, -0.0073,
        -0.0057, -0.0093, -0.0082,  0.0043, -0.0055, -0.0118, -0.0076, -0.0093],
       device='cuda:0'), tensor([[-0.0048, -0.0622,  0.0473,  ..., -0.0371, -0.0389,  0.0074],
        [ 0.0308, -0.0168, -0.0397,  ...,  0.0003,  0.0357, -0.0381],
        [-0.0010, -0.0423,  0.0452,  ..., -0.0035, -0.0110, -0.0056],
        ...,
        [ 0.0124,  0.0477,  0.0268,  ..., -0.0158, -0.0475,  0.0432],
        [ 0.0059, -0.0410, -0.0534,  ..., -0.0375, -0.0101,  0.0410],
        [ 0.0047,  0.0328, -0.0274,  ...,  0.0050, -0.0085, -0.0336]],
       device='cuda:0'), tensor([ 0.0254, -0.0429, -0.0205, -0.0129,  0.0448,  0.0046,  0.0523, -0.0477,
         0.0133,  0.0067,  0.0184,  0.0242,  0.0168, -0.0587,  0.0173, -0.0553,
         0.0042, -0.0416,  0.0412, -0.0196, -0.0516, -0.0402,  0.0284, -0.0213,
        -0.0375,  0.0140, -0.0575,  0.0208,  0.0134,  0.0172,  0.0008, -0.0005,
        -0.0289, -0.0012, -0.0053,  0.0522, -0.0433,  0.0216, -0.0188, -0.0248,
        -0.0326, -0.0056, -0.0277, -0.0188, -0.0017,  0.0362, -0.0454,  0.0437,
         0.0098, -0.0381, -0.0328, -0.0323,  0.0384, -0.0246,  0.0186,  0.0445,
         0.0412,  0.0058,  0.0020,  0.0300,  0.0052,  0.0228,  0.0014, -0.0289,
        -0.0051, -0.0557,  0.0192,  0.0016,  0.0308,  0.0129, -0.0181,  0.0325,
        -0.0394, -0.0145, -0.0265,  0.0131,  0.0493, -0.0131,  0.0317, -0.0383,
         0.0561,  0.0231, -0.0326,  0.0482,  0.0543,  0.0062, -0.0678, -0.0027,
        -0.0306, -0.0010, -0.0530, -0.0558, -0.0172,  0.0408,  0.0020, -0.0356,
         0.0033,  0.0397, -0.0326, -0.0190,  0.0339, -0.0525, -0.0150,  0.0214,
         0.0177, -0.0378, -0.0211,  0.0049,  0.0204, -0.0061, -0.0600,  0.0614,
         0.0581,  0.0112, -0.0293,  0.0124, -0.0558, -0.0150, -0.0476,  0.0376,
         0.0344,  0.0269,  0.0396,  0.0460, -0.0045, -0.0054, -0.0050, -0.0147,
        -0.0269, -0.0361, -0.0196, -0.0418, -0.0275, -0.0264,  0.0510, -0.0093,
         0.0160, -0.0522, -0.0381, -0.0359,  0.0340, -0.0267,  0.0428,  0.0291,
         0.0337, -0.0131, -0.0466, -0.0125,  0.0280, -0.0111,  0.0246, -0.0636,
         0.0536,  0.0468, -0.0221,  0.0465, -0.0414,  0.0052, -0.0310,  0.0133,
        -0.0119,  0.0078,  0.0552,  0.0445,  0.0233,  0.0014, -0.0717, -0.0005,
        -0.0687, -0.0039,  0.0133,  0.0506, -0.0250, -0.0446,  0.0157,  0.0501,
         0.0177, -0.0462,  0.0597,  0.0451,  0.0383,  0.0511,  0.0254,  0.0252,
        -0.0462, -0.0442, -0.0119, -0.0055, -0.0404, -0.0003, -0.0220,  0.0209,
        -0.0177,  0.0458,  0.0214,  0.0118,  0.0215, -0.0305, -0.0302, -0.0037,
        -0.0044, -0.0144,  0.0433, -0.0344, -0.0222, -0.0488,  0.0420, -0.0111,
        -0.0119,  0.0095,  0.0272,  0.0253,  0.0005, -0.0384,  0.0247,  0.0616,
        -0.0156, -0.0236,  0.0164, -0.0067, -0.0132,  0.0454,  0.0579,  0.0308,
         0.0281,  0.0034, -0.0217, -0.0290,  0.0442,  0.0604,  0.0082, -0.0018,
         0.0170, -0.0492,  0.0347, -0.0030, -0.0291, -0.0476,  0.0451,  0.0592,
         0.0369, -0.0486, -0.0396,  0.0108,  0.0158,  0.0141, -0.0442, -0.0374,
        -0.0290,  0.0598,  0.0288, -0.0281,  0.0357, -0.0147, -0.0378,  0.0300,
         0.0173,  0.0013,  0.0045, -0.0295,  0.0049, -0.0500, -0.0203, -0.0244,
        -0.0052, -0.0512, -0.0507, -0.0003, -0.0063,  0.0444,  0.0507, -0.0031,
         0.0112,  0.0370, -0.0247, -0.0425,  0.0064, -0.0379,  0.0173, -0.0220,
        -0.0062, -0.0391,  0.0455, -0.0171,  0.0126,  0.0316, -0.0183, -0.0487,
        -0.0010, -0.0111, -0.0154, -0.0490, -0.0268,  0.0029, -0.0554,  0.0210,
        -0.0522,  0.0214,  0.0052,  0.0117,  0.0016, -0.0093, -0.0566, -0.0203,
        -0.0472, -0.0479, -0.0323,  0.0284,  0.0415, -0.0099,  0.0022, -0.0039,
        -0.0473, -0.0023,  0.0596,  0.0459, -0.0334, -0.0023,  0.0385,  0.0359,
        -0.0463, -0.0541,  0.0369, -0.0090, -0.0389, -0.0233,  0.0536,  0.0425,
         0.0402, -0.0191,  0.0335, -0.0074, -0.0453, -0.0388,  0.0003, -0.0605,
        -0.0187,  0.0284,  0.0515,  0.0054, -0.0344, -0.0040,  0.0237,  0.0274,
        -0.0444, -0.0374,  0.0012, -0.0452,  0.0011,  0.0581,  0.0571,  0.0284,
         0.0268, -0.0311, -0.0359, -0.0246,  0.0536,  0.0128,  0.0015, -0.0601,
        -0.0166,  0.0551,  0.0417, -0.0482,  0.0566, -0.0429, -0.0242,  0.0344,
         0.0278, -0.0528, -0.0460, -0.0023, -0.0580,  0.0481, -0.0005, -0.0322,
         0.0006, -0.0284,  0.0127, -0.0428, -0.0037,  0.0344, -0.0460, -0.0350,
         0.0228, -0.0393, -0.0546,  0.0431,  0.0597,  0.0128,  0.0223, -0.0329,
        -0.0615,  0.0199, -0.0201,  0.0105, -0.0510, -0.0122, -0.0245, -0.0315,
         0.0481,  0.0366,  0.0093, -0.0144, -0.0262,  0.0149, -0.0416, -0.0320,
         0.0275,  0.0032,  0.0539, -0.0485,  0.0097,  0.0480, -0.0010, -0.0020,
        -0.0447,  0.0449,  0.0366, -0.0196,  0.0212, -0.0485, -0.0256,  0.0136,
        -0.0446,  0.0490,  0.0408,  0.0124,  0.0059,  0.0064,  0.0093,  0.0070,
         0.0541, -0.0252, -0.0320,  0.0350, -0.0065,  0.0487, -0.0412, -0.0167,
        -0.0478,  0.0542, -0.0280, -0.0134,  0.0401,  0.0406, -0.0420, -0.0283,
        -0.0121, -0.0492, -0.0599, -0.0459, -0.0168, -0.0212,  0.0104,  0.0317,
         0.0392, -0.0163, -0.0072, -0.0407, -0.0620,  0.0106, -0.0339,  0.0180,
        -0.0372, -0.0552,  0.0416, -0.0095,  0.0164, -0.0020,  0.0457, -0.0234,
         0.0246, -0.0169, -0.0439,  0.0237, -0.0071,  0.0454,  0.0191, -0.0294,
         0.0237, -0.0099, -0.0330, -0.0376,  0.0276,  0.0592, -0.0505,  0.0590,
        -0.0136, -0.0378,  0.0106, -0.0310, -0.0095,  0.0353, -0.0060,  0.0440,
        -0.0149,  0.0118, -0.0003, -0.0342, -0.0304, -0.0020,  0.0421,  0.0112,
        -0.0024, -0.0044, -0.0414,  0.0402, -0.0235, -0.0452,  0.0030, -0.0559],
       device='cuda:0'), tensor([[[ 0.0119,  0.0306, -0.0129,  0.0287],
         [ 0.0182,  0.0353,  0.0234,  0.0050],
         [ 0.0156, -0.0211, -0.0272, -0.0138],
         ...,
         [-0.0213, -0.0170,  0.0021, -0.0038],
         [-0.0233,  0.0068, -0.0200,  0.0019],
         [ 0.0085,  0.0141,  0.0170,  0.0159]],

        [[-0.0041, -0.0248, -0.0329,  0.0037],
         [ 0.0220, -0.0232,  0.0279, -0.0133],
         [-0.0276, -0.0387, -0.0236,  0.0119],
         ...,
         [-0.0135,  0.0167, -0.0053, -0.0007],
         [-0.0117, -0.0079, -0.0094, -0.0188],
         [ 0.0007,  0.0397,  0.0123,  0.0203]],

        [[-0.0182, -0.0055, -0.0243,  0.0046],
         [ 0.0078,  0.0252,  0.0109, -0.0117],
         [ 0.0387, -0.0192, -0.0192, -0.0163],
         ...,
         [ 0.0176,  0.0454, -0.0071, -0.0114],
         [ 0.0268,  0.0010,  0.0352, -0.0284],
         [ 0.0121, -0.0041,  0.0165, -0.0120]],

        ...,

        [[ 0.0217,  0.0150,  0.0269, -0.0118],
         [-0.0098, -0.0224, -0.0152,  0.0380],
         [ 0.0271, -0.0248, -0.0366,  0.0105],
         ...,
         [-0.0247,  0.0110,  0.0112, -0.0190],
         [ 0.0388,  0.0090,  0.0213,  0.0075],
         [ 0.0218, -0.0057,  0.0098,  0.0440]],

        [[-0.0377, -0.0275,  0.0241,  0.0448],
         [ 0.0012, -0.0207,  0.0210,  0.0112],
         [ 0.0261,  0.0075, -0.0316, -0.0057],
         ...,
         [ 0.0140, -0.0137, -0.0022,  0.0122],
         [-0.0006, -0.0250,  0.0049,  0.0160],
         [ 0.0075,  0.0034,  0.0062, -0.0128]],

        [[ 0.0025,  0.0227,  0.0030,  0.0220],
         [-0.0076, -0.0067, -0.0259, -0.0255],
         [-0.0061,  0.0185,  0.0067,  0.0256],
         ...,
         [ 0.0208,  0.0364,  0.0068,  0.0162],
         [ 0.0065,  0.0108,  0.0380, -0.0272],
         [-0.0023, -0.0150, -0.0462,  0.0199]]], device='cuda:0'), tensor([ 1.8657e-02, -7.0816e-03, -1.1104e-02,  8.0719e-03, -2.9571e-02,
         2.3838e-02,  8.0510e-03, -1.5608e-02,  1.2335e-02,  1.8880e-02,
         2.8462e-02, -1.7629e-02,  1.1234e-02,  9.7499e-03,  2.3899e-02,
         1.3425e-03,  2.4794e-02, -2.7133e-02, -3.7539e-04, -1.4061e-02,
         3.1310e-02,  1.2908e-02, -4.0255e-03, -4.6214e-03, -1.6233e-02,
        -1.4570e-02, -2.8568e-02,  1.3068e-03,  2.1130e-02, -1.1102e-03,
         1.4338e-02,  3.0817e-02,  2.9871e-03,  1.1272e-02,  1.9483e-02,
        -6.4596e-03, -2.6604e-02, -2.8103e-02,  5.5218e-03,  1.6547e-02,
        -7.3464e-03, -2.7260e-02,  1.9374e-02,  8.3574e-03, -1.7179e-02,
        -1.0353e-02,  7.5320e-05, -2.5492e-02,  2.1849e-02, -2.3397e-03,
         1.0801e-02, -9.6993e-03, -6.7281e-03, -7.2318e-03, -3.6419e-04,
        -8.3854e-03, -1.9177e-02,  6.2018e-03, -2.4523e-02, -3.2160e-02,
         9.2618e-03,  6.1115e-03,  1.8740e-02, -3.0056e-03,  7.3599e-03,
        -2.1917e-02, -1.9959e-02,  2.1499e-02, -3.0619e-02,  1.8911e-03,
        -1.4491e-02, -6.5049e-04,  2.7277e-02,  2.4519e-02, -1.0914e-03,
         2.7896e-02, -2.3205e-02, -1.0438e-02,  1.8931e-02, -4.0034e-03,
         1.8043e-02, -5.3602e-03, -3.0303e-02, -3.4144e-03,  7.1726e-03,
         2.2957e-02,  5.1678e-03,  6.6994e-03, -2.5101e-02,  2.5371e-03,
         3.2008e-02, -1.8515e-03, -1.1120e-04, -4.9417e-03, -2.1311e-02,
         6.5409e-03,  2.4741e-02, -2.6999e-03,  1.9400e-02,  9.5014e-04,
         2.7838e-02,  5.9328e-04, -2.0450e-02,  1.6856e-02,  7.4785e-03,
         2.3416e-02, -1.6202e-02,  2.7289e-02,  6.1665e-03, -2.7494e-02,
        -3.5842e-02,  1.7312e-02,  2.4854e-02, -5.0913e-03, -2.8437e-02,
         2.9568e-02,  2.4757e-02, -1.6941e-02, -3.6287e-04, -1.6686e-02,
         1.7828e-02,  5.3319e-03,  2.0306e-02,  3.2781e-02, -1.1739e-02,
        -9.3521e-03,  2.6720e-02, -2.3675e-02, -7.3708e-03,  7.3975e-03,
         2.4900e-02,  1.1308e-02, -1.7060e-02,  2.1002e-03,  2.4383e-02,
         2.3594e-02,  2.3433e-02,  2.3130e-02, -2.5564e-02, -2.1136e-02,
        -2.6266e-02,  1.1694e-02, -2.5425e-02,  1.5427e-02,  3.8899e-02,
        -2.7007e-02,  2.3189e-02,  2.4369e-02, -8.7242e-03, -2.8863e-02,
        -1.7479e-02,  8.7015e-03, -2.5615e-02, -1.0956e-02, -5.6325e-03,
        -1.3008e-02,  4.1255e-03, -1.2066e-02, -8.6339e-03,  1.9711e-02,
         1.8883e-02, -4.3933e-03, -2.2734e-02, -2.4933e-02, -2.4912e-02,
        -1.8691e-02, -3.2884e-03, -9.2671e-03,  1.1660e-02,  2.2780e-02,
        -2.9824e-02,  1.7139e-02,  1.2914e-02,  1.6112e-02,  3.4031e-02,
         2.9094e-03,  2.0810e-02, -8.8191e-03,  1.5440e-02,  2.7401e-02,
         1.4988e-02,  2.2984e-02,  2.4506e-02, -2.8018e-02, -1.7495e-02,
        -5.4578e-03, -8.5306e-03, -1.5658e-02, -2.3896e-04,  3.0765e-02,
        -1.9921e-02, -2.7100e-02, -1.0633e-02, -2.9353e-02,  6.4357e-03,
        -1.7881e-02, -2.6323e-02,  9.8408e-03,  2.8675e-02,  9.9803e-03,
         6.6970e-04,  7.0628e-03, -1.3195e-02, -4.6898e-04,  4.6889e-03,
         1.5987e-02, -9.5715e-03,  2.2992e-02, -1.0038e-02, -4.0159e-03,
         1.2473e-02, -1.3297e-02, -2.0989e-02, -4.9044e-03,  1.3832e-02,
         2.4201e-02, -4.0475e-03,  1.4501e-02, -1.0618e-02, -2.0544e-02,
         2.8128e-02,  1.2710e-02, -1.4613e-02, -7.5045e-03, -3.2724e-02,
         6.2759e-03,  3.0487e-03,  7.8927e-03, -2.8364e-02, -3.1301e-03,
        -1.8548e-02,  3.2796e-02, -2.3288e-02, -7.3627e-03, -2.4431e-02,
        -3.0846e-02,  2.3078e-02, -4.6712e-03,  2.0880e-02,  2.9162e-02,
         3.1209e-03,  9.7522e-03,  7.4927e-03,  1.2321e-02,  1.5625e-02,
         2.1541e-02, -2.6092e-02, -2.1125e-02, -2.9684e-02, -3.3307e-02,
        -3.1252e-02, -2.6101e-03, -1.0964e-02, -6.9246e-03,  3.6004e-03,
        -2.7949e-04], device='cuda:0'), tensor([[[-0.0081, -0.0349,  0.0137,  0.0613,  0.0657],
         [-0.0720,  0.0181, -0.0711, -0.1275, -0.0292],
         [ 0.1372,  0.1400, -0.0669,  0.0097, -0.0811],
         ...,
         [ 0.1271,  0.0235,  0.0851, -0.1092,  0.0534],
         [ 0.0773,  0.0511,  0.0429,  0.0017, -0.0373],
         [-0.0706, -0.0685,  0.1709, -0.0547, -0.0560]],

        [[-0.0810,  0.0773, -0.0316, -0.0854,  0.0440],
         [-0.0673, -0.1096, -0.0617, -0.0280, -0.1118],
         [ 0.0940, -0.1059, -0.0491,  0.0872,  0.0070],
         ...,
         [ 0.1407, -0.0129,  0.0869,  0.1395,  0.0561],
         [-0.0607,  0.0100, -0.0933,  0.0262,  0.0448],
         [ 0.0407,  0.0865,  0.0836, -0.0407, -0.0723]],

        [[ 0.0904, -0.0275, -0.0046,  0.0433, -0.0109],
         [ 0.1265,  0.0481,  0.0300, -0.0216, -0.1085],
         [-0.0204,  0.0922,  0.0274, -0.0877, -0.0012],
         ...,
         [-0.0020, -0.1234, -0.0472,  0.0125, -0.0723],
         [-0.0572,  0.0098, -0.0589, -0.0708,  0.1529],
         [ 0.0436,  0.0813, -0.1184, -0.0170,  0.0970]],

        ...,

        [[-0.1000,  0.0020,  0.0471, -0.0850, -0.0683],
         [ 0.0979,  0.0989, -0.0299, -0.0792, -0.1306],
         [ 0.0616,  0.1010, -0.0346, -0.0613, -0.0300],
         ...,
         [-0.1375,  0.0483, -0.1090, -0.1159,  0.0904],
         [-0.1002, -0.0419,  0.0685, -0.0654, -0.0062],
         [ 0.0503, -0.0696, -0.0311,  0.1324, -0.0408]],

        [[ 0.0062, -0.0585,  0.0912, -0.0787, -0.0901],
         [-0.1085, -0.0474, -0.0799, -0.0838, -0.0244],
         [-0.0409, -0.0124, -0.0850,  0.0517,  0.0977],
         ...,
         [-0.0597, -0.0642, -0.0973, -0.1365,  0.0168],
         [ 0.0585,  0.0405,  0.0870, -0.0060,  0.0852],
         [-0.0250,  0.0356, -0.0414, -0.0749, -0.0999]],

        [[-0.1060, -0.0680, -0.0612, -0.0839,  0.0185],
         [ 0.0919, -0.0822, -0.0905, -0.0102, -0.0396],
         [ 0.0704,  0.0288,  0.0978, -0.0722,  0.0997],
         ...,
         [-0.0348, -0.0164, -0.0413, -0.0819, -0.0361],
         [ 0.1113,  0.0970, -0.0764, -0.0907,  0.0433],
         [-0.1169, -0.0670, -0.0446,  0.0405, -0.1741]]], device='cuda:0'), tensor([-0.0012,  0.0776,  0.0196, -0.0507, -0.0125,  0.0165,  0.0392, -0.0843,
        -0.1081, -0.1355,  0.0876, -0.1353,  0.0277,  0.0287,  0.0766,  0.1063,
         0.0758,  0.0008, -0.0796,  0.0098, -0.1097,  0.1344, -0.0346, -0.1136,
        -0.0806,  0.1090,  0.0447,  0.0025, -0.0157, -0.0661, -0.1051, -0.1107,
         0.0911, -0.1319, -0.0540,  0.0006, -0.0979, -0.0098,  0.0657, -0.0500,
        -0.1049,  0.0837,  0.0299,  0.0869,  0.1190, -0.0667,  0.0705,  0.0334,
        -0.0902,  0.0512, -0.0188, -0.0769,  0.0938, -0.1161,  0.0644, -0.0596,
        -0.0677, -0.0838,  0.1053, -0.1291,  0.1283,  0.0720, -0.1256,  0.0061,
        -0.0518, -0.0101,  0.0720,  0.0842, -0.1251,  0.1098,  0.1337,  0.0137,
        -0.0564, -0.0002,  0.0802, -0.1182, -0.0975, -0.1171, -0.1086,  0.0129,
         0.0847,  0.0698, -0.0637, -0.1083,  0.0159, -0.0992, -0.0178, -0.1371,
         0.0161,  0.0981,  0.0776, -0.0802, -0.0631, -0.0909, -0.0885,  0.0267,
        -0.0177, -0.1142,  0.0813,  0.0540,  0.0299,  0.1275,  0.0057,  0.0937,
        -0.0801, -0.0542, -0.0175, -0.1111, -0.1268, -0.0571, -0.0339, -0.1237,
        -0.0280, -0.0662, -0.1111,  0.0081,  0.1190, -0.0421,  0.0927,  0.0329,
        -0.0867, -0.1141,  0.0942, -0.0743,  0.0563, -0.0741,  0.0670, -0.0825,
        -0.0682,  0.0465,  0.0573,  0.1133,  0.0959,  0.0325, -0.0729,  0.0524,
        -0.0174, -0.1279, -0.0589,  0.0458, -0.0246,  0.0842,  0.0002,  0.1391,
        -0.1287,  0.0371,  0.1107, -0.0191,  0.0899, -0.0837, -0.1326, -0.0570,
         0.0738,  0.1133, -0.0900, -0.0938,  0.1245,  0.0906, -0.0649,  0.1007,
        -0.0226,  0.0529, -0.0780,  0.0923, -0.0472, -0.1254,  0.1172,  0.0931,
        -0.0741, -0.0137,  0.0181, -0.0573,  0.0911, -0.1343,  0.0424, -0.0594,
         0.1033,  0.1051, -0.1497, -0.0032, -0.0598,  0.1139,  0.0767, -0.0612,
        -0.0749, -0.0869,  0.0894,  0.0975, -0.0895,  0.0139,  0.0819,  0.0479,
         0.0854,  0.0748,  0.0798,  0.0955,  0.0093, -0.0866,  0.0452, -0.0744,
         0.0336,  0.0851, -0.0500,  0.0598,  0.0579,  0.0599,  0.0780, -0.1258,
         0.1109, -0.0942, -0.0880, -0.0657, -0.0523, -0.0043,  0.0928, -0.0581,
        -0.0233, -0.0643,  0.0999,  0.0667, -0.0354, -0.1040, -0.0124,  0.0771,
         0.0463, -0.0838,  0.0244,  0.0613, -0.0306, -0.0782, -0.0772, -0.0297,
         0.0431, -0.0384, -0.1390, -0.0613,  0.0158,  0.0138,  0.0768, -0.1238,
        -0.1275,  0.0329,  0.0785, -0.0876,  0.0275, -0.1120, -0.0752,  0.0396,
         0.0549, -0.0086,  0.1129,  0.1323, -0.1434,  0.1241, -0.0423, -0.1399],
       device='cuda:0'), tensor([1.0031, 0.9985, 0.9712, 0.9885, 0.9914, 0.9883, 0.9487, 0.9287, 0.9900,
        1.0089, 0.9750, 0.9385, 0.9845, 1.0062, 0.9717, 0.9909, 1.0014, 0.9620,
        0.9300, 0.9450, 0.9427, 1.0005, 0.9138, 0.8256, 0.8076, 0.9850, 0.9721,
        0.9811, 1.0001, 0.9951, 1.0108, 0.9157, 0.9963, 1.0056, 0.9174, 0.9973,
        0.9571, 0.9992, 0.9080, 0.9954, 0.9640, 0.9354, 0.9616, 0.9953, 0.9690,
        0.8660, 1.0019, 0.9895, 0.9390, 0.9783, 0.9528, 0.9516, 0.9889, 0.9497,
        0.9981, 0.9615, 0.9690, 0.9985, 0.9852, 0.9433, 0.9982, 0.9970, 1.0126,
        0.9523, 0.9206, 0.9644, 1.0006, 0.9574, 0.9971, 1.0040, 1.0014, 0.9864,
        0.9584, 1.0001, 0.9943, 0.9494, 0.9352, 0.9277, 0.9989, 0.9199, 0.9590,
        0.9889, 0.9956, 0.9983, 1.0009, 0.9985, 0.9925, 0.8561, 0.8796, 0.9854,
        1.0040, 1.0031, 0.9592, 0.9729, 0.9687, 1.0018, 0.9984, 0.9604, 0.9974,
        1.0001, 0.9967, 1.0000, 0.8381, 0.9794, 0.9853, 0.9422, 0.9851, 0.9490,
        0.9892, 0.8568, 0.9226, 0.9940, 0.9961, 0.9855, 0.9677, 0.9890, 0.7793,
        0.9787, 0.9866, 0.9959, 0.9952, 0.8821, 0.9528, 0.9936, 0.9902, 0.9439,
        0.9861, 0.9942, 0.9909, 0.9498, 0.8604, 1.0003, 1.0067, 0.9790, 0.9019,
        0.9940, 0.9870, 0.9353, 1.0010, 0.9954, 0.9868, 0.9974, 0.9958, 1.0035,
        0.9326, 0.9290, 0.9542, 0.9435, 0.9910, 0.8364, 0.8840, 0.9401, 1.0066,
        0.9708, 0.9104, 0.9136, 0.9982, 0.9394, 0.9880, 0.9881, 0.9995, 1.0011,
        0.9979, 0.9935, 0.9789, 0.8581, 0.9243, 1.0018, 0.9992, 0.9990, 0.9730,
        0.9409, 0.9756, 0.8833, 0.9651, 1.0049, 0.9968, 0.9780, 0.9388, 0.9491,
        0.9414, 0.9665, 0.9044, 0.9892, 0.9974, 0.9797, 0.9908, 0.9918, 0.8576,
        0.8940, 1.0069, 0.9834, 0.9861, 0.9982, 0.9922, 1.0043, 1.0021, 0.9874,
        0.8889, 0.9188, 0.9848, 1.0106, 0.9787, 0.9833, 0.9991, 0.8681, 0.9984,
        0.9581, 0.9812, 1.0038, 0.9379, 0.9809, 0.9519, 0.8581, 0.9585, 0.9113,
        0.9619, 0.9911, 0.9693, 1.0035, 0.9609, 0.9898, 1.0014, 0.9417, 0.9956,
        0.8956, 0.9192, 0.9974, 0.9955, 1.0050, 0.9950, 0.9866, 0.9788, 0.9964,
        0.9663, 0.9339, 0.9878, 0.9840, 0.9517, 0.9417, 0.8751, 0.9899, 1.0025,
        0.9844, 0.9967, 0.9969, 1.0056, 1.0162, 0.9896, 0.9586, 0.8034, 1.0035,
        0.9637, 0.9984, 0.9897, 0.9943], device='cuda:0'), tensor([ 9.6405e-03, -2.4602e-03, -5.1973e-03,  2.6417e-03,  4.1696e-03,
        -3.2026e-03,  1.3352e-02, -7.0566e-03,  7.0945e-03, -3.5871e-02,
        -6.3726e-03, -3.4250e-02,  3.3933e-03,  9.7564e-03, -1.8077e-03,
        -3.6902e-03, -6.2319e-03,  1.1212e-02,  9.3340e-03, -1.8321e-03,
         1.7770e-02,  6.4555e-03,  9.8730e-03, -8.4332e-03, -8.0277e-03,
        -7.0702e-03,  4.0201e-03, -1.4407e-02,  9.3526e-03,  7.3918e-03,
        -3.3736e-02, -7.5372e-03,  7.5916e-04, -5.1122e-03,  6.8308e-03,
         1.1565e-02,  2.7363e-02, -3.5426e-03, -1.1489e-02, -5.0102e-03,
         2.1848e-02, -4.4227e-02,  1.7124e-02, -5.7970e-03, -4.1944e-03,
        -2.6467e-02,  9.5150e-03,  6.5756e-03, -3.9994e-02,  1.6110e-02,
         1.0942e-02,  1.8182e-02,  3.1580e-02,  2.0488e-02, -4.5344e-04,
         1.5440e-02,  1.8791e-03,  3.0231e-03, -6.1472e-03, -4.5741e-02,
         1.0628e-04, -2.4614e-03, -2.7399e-02, -1.8112e-03, -1.5656e-02,
        -2.9511e-02,  8.1037e-03,  2.4904e-02, -4.6485e-03,  1.0784e-02,
         1.2145e-02,  1.6176e-02,  4.3665e-03, -3.8699e-04,  3.0661e-03,
         1.6956e-02,  2.3636e-02,  1.0846e-02, -2.3634e-02, -5.6077e-02,
         9.4294e-03,  1.3725e-02, -4.8248e-02,  1.1053e-02, -9.0813e-06,
         1.2673e-02,  1.4585e-02, -2.3419e-02,  1.7274e-02,  1.8512e-02,
         7.6875e-03,  2.5954e-03,  1.3584e-02, -5.8597e-02,  1.8359e-02,
         4.9877e-03,  2.5870e-03,  9.7709e-03,  1.1330e-02,  4.0994e-03,
         1.0012e-02,  9.9804e-03, -7.0320e-02, -1.1576e-02, -3.3283e-03,
         1.3724e-02,  9.4201e-03,  1.2601e-02,  4.4526e-03, -5.2557e-02,
         1.2818e-02, -1.6207e-02,  3.0522e-03, -4.8031e-03, -2.2610e-02,
         1.7423e-02, -4.1371e-03,  7.5026e-03, -3.4867e-04, -3.0802e-03,
        -3.8564e-03,  9.1357e-03, -1.0377e-02,  8.6018e-03, -5.8401e-03,
        -8.2634e-04, -1.5386e-03, -9.7177e-04,  5.8613e-03,  2.2369e-02,
        -1.9612e-02,  9.2648e-03,  4.2885e-03, -2.6002e-03,  1.4552e-02,
         9.0047e-04, -3.0225e-03, -3.9490e-02,  3.1321e-03,  4.9523e-03,
        -1.5383e-03,  4.8435e-03,  8.1814e-04,  1.4185e-02,  1.5282e-02,
         6.2986e-03,  1.3424e-02, -4.3958e-02, -6.4336e-03,  3.0984e-02,
        -9.0163e-02, -2.7221e-02,  5.2916e-03,  6.7072e-03, -3.5319e-02,
         1.3395e-02,  1.1336e-02,  2.5793e-02,  1.4571e-02, -1.6892e-03,
         4.3336e-03, -3.1019e-03, -3.1171e-02, -1.2613e-02, -5.4058e-02,
        -5.1088e-02,  5.9298e-03,  1.8150e-03,  3.2887e-03,  5.9386e-03,
         1.7304e-02, -8.1067e-03,  1.1652e-02, -1.3897e-02,  6.3540e-03,
         6.5931e-04,  9.6227e-03,  1.1754e-02, -6.3456e-02, -3.1471e-03,
         1.4495e-02,  2.7261e-02,  8.8074e-03,  3.4743e-03,  2.0131e-03,
         9.2738e-03,  1.3278e-02,  5.3013e-03, -9.1419e-02,  2.3283e-02,
         6.3203e-03,  7.0333e-03,  3.8346e-03,  1.2350e-02, -9.3566e-04,
         7.2709e-03,  5.3727e-03, -4.6418e-03, -3.2433e-02,  2.3502e-02,
        -2.7326e-02,  7.6548e-03, -7.7692e-03, -8.8859e-03,  9.0474e-04,
        -1.8039e-02,  9.9565e-05, -4.2084e-03, -3.1444e-02,  6.7638e-03,
        -3.3839e-02, -6.4757e-03,  1.4800e-02, -1.3481e-01,  7.8541e-03,
         6.1299e-02, -1.6049e-03, -1.0407e-03,  3.5730e-03,  9.0930e-04,
         8.4226e-03,  1.0081e-02,  2.5962e-03,  1.0332e-02, -9.6414e-04,
         8.1209e-04, -5.6814e-02, -6.0797e-03,  5.9215e-03, -1.8592e-02,
        -5.0924e-03, -8.2095e-03, -1.3353e-02,  1.0295e-02, -1.2776e-02,
         1.2976e-02,  5.7999e-03,  9.8017e-03, -1.7336e-03, -4.8221e-03,
         7.9546e-03,  4.6608e-03,  6.0955e-03,  4.6397e-03,  1.2111e-02,
        -1.7773e-04,  5.0284e-03,  1.2466e-03,  3.4387e-03,  9.1985e-03,
        -4.0096e-02,  4.3340e-03, -2.9430e-03,  1.3478e-02,  1.2884e-02,
         1.1129e-03], device='cuda:0'), tensor([[[ 3.9379e-03,  2.7723e-02, -1.4733e-02,  2.6556e-02,  1.0719e-02],
         [-2.0491e-02, -1.3240e-02,  2.5426e-03,  2.3598e-02,  1.4154e-02],
         [-3.0063e-02, -1.9756e-02, -1.6022e-02, -5.3322e-02, -3.2759e-02],
         ...,
         [ 1.2542e-02, -1.0361e-02, -1.7722e-02,  2.8297e-02,  2.1984e-02],
         [-3.9626e-02, -1.5312e-02, -5.7310e-03, -5.2251e-02, -1.9581e-02],
         [ 1.1340e-02,  1.7437e-02, -8.6506e-03, -4.3221e-03, -3.8426e-03]],

        [[-1.9964e-03, -3.2105e-02,  1.3864e-02, -1.8088e-02, -2.0303e-02],
         [-1.1765e-02, -2.2222e-02,  3.8417e-04,  3.2498e-03, -8.4606e-03],
         [-5.7146e-03, -1.2212e-02,  2.9884e-02, -2.4790e-02, -5.2371e-03],
         ...,
         [-9.5370e-03,  1.5749e-02,  2.0938e-02, -5.6313e-03, -1.2457e-02],
         [-1.7880e-02, -4.2949e-04, -3.7645e-02,  1.2015e-02,  9.3337e-05],
         [ 4.1404e-03, -1.8312e-02, -2.0305e-02, -1.7789e-02, -2.7122e-02]],

        [[ 6.0292e-05, -4.8997e-02, -1.7739e-02,  2.0853e-02, -5.8202e-03],
         [-1.5843e-02,  8.8041e-03, -7.1230e-03,  2.1302e-02, -2.1608e-03],
         [ 8.9410e-03,  2.6798e-02, -1.4524e-02, -6.6018e-03,  1.9874e-02],
         ...,
         [ 3.2202e-02, -1.6088e-02,  2.1280e-02,  3.2270e-02,  7.6364e-03],
         [ 4.8552e-02, -3.0844e-02,  1.0733e-02,  4.4259e-02,  1.8139e-03],
         [ 1.0512e-02,  9.2401e-03,  1.3524e-03,  1.0035e-02,  1.3770e-03]],

        ...,

        [[ 3.0922e-03,  2.2629e-02, -3.0747e-03, -2.0259e-02,  1.1539e-02],
         [-9.8202e-03,  9.9339e-03, -2.0301e-02, -4.1646e-03, -1.6622e-02],
         [ 1.6718e-02,  1.3673e-02,  3.5479e-04, -2.0371e-02, -1.2530e-02],
         ...,
         [-1.8960e-02, -2.7164e-02,  4.5802e-04, -9.2040e-03, -1.1087e-02],
         [-3.3756e-03,  7.4491e-03, -2.8610e-02,  2.4925e-04, -2.0515e-02],
         [-2.2750e-02, -1.0757e-02,  2.2084e-02, -1.1293e-03,  4.4388e-03]],

        [[-1.8705e-02, -2.7089e-03,  2.0779e-02, -1.1764e-02, -1.1286e-02],
         [ 1.4052e-02, -2.3804e-02,  2.8305e-02,  2.1113e-02,  2.0817e-02],
         [ 1.3284e-02, -1.0217e-03,  2.3083e-02,  2.9436e-03,  4.5538e-04],
         ...,
         [ 1.5056e-02,  2.5819e-02,  4.1593e-03, -1.7648e-02,  2.4543e-02],
         [-1.9689e-02,  1.0968e-02,  1.2696e-02, -1.1546e-02, -1.3876e-02],
         [ 2.1694e-02, -2.2091e-02,  1.6094e-03, -3.9849e-03,  1.9143e-02]],

        [[ 1.8349e-02,  1.7120e-02,  1.3765e-02,  3.6728e-03,  9.0330e-03],
         [-2.5555e-02,  2.2797e-02, -1.4663e-02, -2.5045e-03, -2.0000e-02],
         [ 1.5699e-03, -3.2384e-04,  4.5496e-03, -1.8420e-03, -1.2251e-02],
         ...,
         [ 3.8368e-03, -1.2351e-02, -8.6465e-05, -2.2136e-02,  1.7979e-02],
         [-2.1178e-02,  1.1844e-02,  2.2655e-02,  1.4908e-02,  1.8598e-02],
         [ 8.2496e-03, -6.7613e-04, -3.0312e-03,  5.0005e-03, -4.1767e-04]]],
       device='cuda:0'), tensor([ 8.4205e-03, -1.6093e-02,  3.3345e-02, -2.5767e-02, -1.6919e-02,
        -4.2139e-03, -2.2808e-02,  1.5453e-02, -1.8635e-02,  1.4443e-02,
         1.9853e-02, -1.4809e-02,  3.6859e-02,  2.0961e-02, -1.7681e-02,
         2.5977e-02,  2.0615e-02, -1.7140e-02,  7.5304e-03,  3.0888e-04,
         8.6813e-03,  1.8961e-03,  6.6964e-03,  5.5706e-03, -1.9070e-02,
        -5.7307e-03, -4.6217e-02,  1.2765e-02, -1.8153e-02, -2.4904e-02,
        -1.6738e-02, -9.1062e-03,  1.6549e-03,  2.5937e-02,  1.0027e-02,
         9.2610e-03,  3.0819e-03, -2.5670e-02,  2.3492e-02,  2.5111e-02,
         2.8401e-03, -9.8303e-04,  1.0045e-02,  2.4653e-02,  1.9798e-02,
        -2.5113e-02, -8.0543e-03, -3.0756e-02, -2.3000e-02, -3.0401e-03,
         2.5052e-02,  8.9393e-03, -1.9665e-03,  1.4467e-02, -2.8208e-02,
        -1.7976e-03,  2.8958e-02, -2.0753e-04,  1.8888e-02, -5.8957e-02,
         2.8647e-02,  8.5538e-03, -2.1114e-02,  2.3664e-02,  1.4346e-02,
         1.7461e-02,  2.1586e-02,  4.8069e-03, -2.1742e-02, -1.0511e-02,
        -6.4317e-02, -1.5980e-02, -2.8807e-02,  1.3768e-03,  1.4079e-02,
         2.3615e-02,  1.6548e-03,  1.2199e-02, -1.4311e-02,  2.4396e-02,
         2.5974e-02,  1.7457e-02,  2.9879e-02,  2.6356e-02, -7.7156e-03,
         7.1710e-03,  2.1736e-02,  2.7650e-03,  1.6245e-02, -2.4570e-02,
         1.7001e-02, -3.0677e-02, -4.4833e-02,  2.0229e-02, -1.1800e-02,
         1.3822e-02, -1.0634e-02,  1.2157e-02, -1.4609e-02,  1.0007e-03,
        -6.2698e-04,  1.9945e-02,  7.3331e-03, -2.2870e-02, -1.2111e-02,
        -2.4989e-02, -1.5817e-02,  2.1688e-02, -4.1352e-03, -8.1187e-02,
         2.8001e-02, -8.5418e-03, -3.1850e-02, -1.5480e-02,  2.9888e-02,
         3.0354e-02, -1.9396e-03, -2.5671e-02,  5.0666e-03,  1.9470e-02,
         2.1122e-02,  8.2941e-03, -1.3454e-02, -1.3472e-02,  5.8362e-03,
         2.1040e-02,  1.2694e-02,  2.3046e-02,  1.5845e-02, -4.0836e-03,
        -4.3684e-02,  1.9212e-02,  1.5624e-02,  1.9671e-02, -2.2793e-02,
        -2.3251e-02,  1.5839e-02,  2.6702e-02,  1.5575e-02, -8.9722e-03,
         2.0722e-02, -1.2454e-02,  3.8255e-03,  2.3447e-02, -3.3095e-03,
        -1.9331e-02, -1.2580e-04, -2.8296e-02,  2.0178e-02, -1.0294e-02,
        -1.2109e-02,  2.6384e-02,  2.1362e-02, -6.3733e-02,  1.0081e-02,
         1.9223e-02, -1.2415e-02, -1.0642e-02,  1.8038e-03, -1.2332e-02,
        -2.6662e-02, -2.7033e-02,  1.7794e-02, -1.5085e-02, -1.3457e-02,
        -3.9931e-04,  2.1390e-02, -3.5972e-03, -2.7746e-02, -2.1903e-02,
        -4.4218e-02,  1.6271e-02,  1.1447e-02,  2.8850e-03, -1.1721e-02,
         1.0721e-02,  1.9773e-02,  2.6378e-02, -5.7552e-04, -7.9474e-03,
        -1.7646e-02,  1.5337e-02,  4.2123e-03, -1.3409e-02,  1.6603e-03,
         1.7436e-02, -1.8801e-02,  9.6425e-03,  2.3095e-02, -3.3639e-02,
        -4.4300e-03, -5.2333e-03, -1.0811e-02,  1.6273e-02,  2.4371e-02,
         1.9454e-02, -1.4878e-03,  7.2656e-03, -1.0009e-02,  3.0760e-03,
         1.5154e-02, -7.5157e-03, -4.0498e-02,  8.7081e-04,  5.6786e-03,
         3.3155e-03, -1.1782e-02,  2.0556e-02,  3.2643e-03,  3.1742e-02,
        -2.3954e-02, -5.4624e-03,  4.6697e-03, -5.1822e-03, -1.5595e-02,
         2.4711e-02,  1.6501e-02,  2.5711e-02,  3.4968e-03,  3.0202e-03,
         1.9362e-02,  2.1659e-05, -1.1722e-02,  2.9168e-02, -8.8104e-03,
         2.0419e-02,  1.2083e-02, -2.8824e-02, -1.9933e-03, -1.9962e-02,
         7.3878e-06, -3.1571e-02,  1.2289e-02, -3.4173e-03, -4.9977e-03,
        -2.1942e-02,  1.4445e-02, -1.8900e-02, -5.4432e-03,  2.7712e-02,
        -1.2668e-03,  2.0342e-02,  8.7504e-03,  1.1585e-02,  1.3050e-02,
         8.7324e-03,  2.1919e-02, -2.3747e-02, -1.0439e-02, -3.6536e-03,
        -2.4838e-02,  4.5231e-03, -8.7137e-03,  5.3298e-03, -2.3201e-02,
        -5.9654e-03], device='cuda:0'), tensor([0.9802, 0.9769, 0.9712, 0.9643, 0.9869, 1.0057, 0.9490, 0.9942, 0.9620,
        0.9626, 0.9551, 0.9594, 0.9828, 0.9075, 0.9859, 0.9540, 0.9798, 1.0041,
        0.9704, 0.9666, 0.9734, 0.9706, 0.9811, 0.9819, 0.9581, 0.9395, 1.0011,
        0.9825, 0.9668, 1.0081, 1.0166, 0.9729, 0.9549, 1.0181, 0.9755, 0.9041,
        0.9662, 0.9912, 0.9791, 0.9702, 0.9691, 0.9704, 0.9470, 0.9633, 0.9819,
        0.9656, 0.9629, 1.0167, 0.9438, 1.0168, 0.9508, 0.9724, 0.9131, 0.9648,
        0.9765, 0.9690, 0.9534, 0.9598, 0.9521, 0.9987, 0.9831, 0.9667, 0.9601,
        0.9551, 0.9706, 0.9827, 0.9554, 0.9724, 1.0015, 0.9550, 1.0028, 0.9733,
        0.9817, 0.9750, 0.9746, 0.9862, 0.9885, 0.9311, 0.9727, 0.9707, 0.9653,
        0.9726, 0.9887, 0.9683, 0.9689, 0.9796, 0.9881, 0.9820, 0.9498, 0.9878,
        0.9313, 0.9767, 1.0047, 0.9807, 0.9643, 0.9677, 0.9779, 0.9734, 0.9670,
        0.9711, 0.9786, 0.9881, 0.9817, 0.9342, 1.0055, 0.9543, 0.9602, 0.9763,
        0.9733, 1.0053, 0.9790, 0.9706, 0.9990, 0.9580, 0.9705, 0.9639, 0.9610,
        0.9861, 0.9648, 0.9774, 0.9802, 0.9939, 0.9629, 0.9573, 0.9625, 0.9505,
        0.9674, 0.9638, 0.9813, 0.9713, 1.0054, 0.9576, 0.9712, 0.9770, 0.9821,
        0.9713, 0.9694, 0.9455, 0.9865, 0.9475, 0.9563, 0.9559, 0.9626, 0.9492,
        0.9740, 0.9627, 0.9825, 0.9571, 1.0043, 0.9648, 0.8952, 0.9514, 0.9782,
        1.0056, 0.9800, 0.9590, 0.9840, 1.0180, 0.9545, 0.9745, 0.9610, 0.9756,
        0.9200, 0.9652, 0.9703, 0.9778, 0.9544, 0.9896, 1.0108, 0.9201, 1.0159,
        0.9480, 0.9670, 0.9495, 0.9269, 0.9835, 0.9735, 0.9823, 0.8827, 0.9678,
        0.9495, 0.9779, 0.9814, 0.9611, 0.9545, 0.9746, 1.0125, 0.9789, 0.9620,
        1.0251, 0.9785, 0.9639, 0.9796, 0.9835, 0.9605, 0.9457, 0.9841, 0.9705,
        0.9563, 0.9686, 0.9727, 0.9858, 1.0109, 0.9512, 1.0106, 0.9650, 0.9599,
        0.9933, 0.9813, 0.9830, 0.9971, 0.9832, 0.9656, 0.9687, 0.9765, 0.9787,
        0.9756, 0.9852, 1.0127, 1.0134, 0.9856, 0.9626, 0.9658, 0.9643, 0.9718,
        0.9849, 0.9837, 0.9408, 0.9342, 0.9778, 1.0049, 1.0166, 0.9699, 0.9625,
        0.9813, 0.9919, 0.9660, 1.0019, 0.9583, 0.9587, 0.9559, 0.9616, 1.0287,
        0.9427, 0.9734, 0.9386, 0.9851, 1.0016, 0.9656, 0.9849, 0.9763, 0.9440,
        0.9810, 0.9715, 0.9707, 0.9135], device='cuda:0'), tensor([-0.0163, -0.0224, -0.0095, -0.0117, -0.0214, -0.0314, -0.0237, -0.0291,
        -0.0109, -0.0107, -0.0295, -0.0203,  0.0009, -0.0183, -0.0022, -0.0165,
        -0.0112, -0.0128, -0.0096, -0.0236, -0.0109, -0.0169, -0.0094, -0.0042,
        -0.0089, -0.0077, -0.0024, -0.0058, -0.0187, -0.0324, -0.0294, -0.0030,
        -0.0174, -0.0306, -0.0164, -0.0232, -0.0161, -0.0088, -0.0216, -0.0216,
        -0.0355, -0.0195, -0.0243, -0.0128, -0.0047, -0.0116, -0.0163, -0.0219,
        -0.0253, -0.0319, -0.0109, -0.0187, -0.0312, -0.0231, -0.0300, -0.0079,
        -0.0132, -0.0228, -0.0261, -0.0018, -0.0066, -0.0242, -0.0183, -0.0252,
        -0.0153, -0.0088, -0.0324, -0.0067, -0.0145, -0.0191, -0.0043, -0.0145,
        -0.0326, -0.0049, -0.0073, -0.0085, -0.0131, -0.0124, -0.0083, -0.0100,
        -0.0134, -0.0116, -0.0016, -0.0235, -0.0292, -0.0080, -0.0175, -0.0085,
        -0.0170, -0.0114, -0.0207, -0.0106, -0.0077, -0.0136, -0.0222, -0.0243,
        -0.0146, -0.0288, -0.0115, -0.0104, -0.0075, -0.0030, -0.0274, -0.0085,
        -0.0285, -0.0076, -0.0093, -0.0094, -0.0230, -0.0092, -0.0113, -0.0102,
        -0.0155, -0.0403, -0.0236, -0.0115, -0.0171, -0.0169, -0.0215, -0.0151,
        -0.0088, -0.0254, -0.0056, -0.0257, -0.0119, -0.0072, -0.0076, -0.0276,
        -0.0120, -0.0117, -0.0132, -0.0275, -0.0156, -0.0134, -0.0022, -0.0170,
        -0.0280, -0.0276, -0.0049, -0.0228, -0.0089, -0.0364, -0.0294, -0.0273,
        -0.0248, -0.0330, -0.0104, -0.0394, -0.0271, -0.0223, -0.0118, -0.0179,
        -0.0189, -0.0100, -0.0233, -0.0342, -0.0341, -0.0316, -0.0381, -0.0089,
        -0.0186, -0.0182, -0.0415, -0.0283, -0.0147, -0.0143, -0.0386, -0.0343,
        -0.0166, -0.0183, -0.0203, -0.0257, -0.0245, -0.0426, -0.0242, -0.0052,
        -0.0092, -0.0077, -0.0137, -0.0237, -0.0295, -0.0169, -0.0084, -0.0261,
        -0.0385, -0.0199, -0.0221, -0.0147, -0.0211, -0.0336, -0.0173, -0.0321,
        -0.0081, -0.0246, -0.0289, -0.0228, -0.0064, -0.0080, -0.0108, -0.0131,
        -0.0273, -0.0052, -0.0173, -0.0278, -0.0339, -0.0315, -0.0217, -0.0250,
        -0.0077, -0.0060, -0.0317, -0.0069, -0.0137, -0.0161, -0.0129, -0.0100,
        -0.0247, -0.0048, -0.0221, -0.0352, -0.0326, -0.0194, -0.0116, -0.0170,
        -0.0225, -0.0037, -0.0013, -0.0147, -0.0143, -0.0168, -0.0248, -0.0191,
        -0.0071, -0.0147, -0.0111,  0.0028, -0.0268, -0.0283, -0.0219, -0.0156,
        -0.0248, -0.0151, -0.0487, -0.0251, -0.0125, -0.0190, -0.0113, -0.0218,
        -0.0180, -0.0052, -0.0139, -0.0293, -0.0072, -0.0313, -0.0194, -0.0246],
       device='cuda:0'), tensor([[-0.0072, -0.0182, -0.0488,  ...,  0.0283,  0.0378, -0.0040],
        [-0.0610, -0.0251,  0.0166,  ...,  0.0113, -0.0283,  0.0190],
        [ 0.0011,  0.0211, -0.0224,  ..., -0.0305,  0.0230,  0.0311],
        ...,
        [-0.0134, -0.0334, -0.0007,  ...,  0.0210,  0.0325, -0.0184],
        [-0.0040,  0.0446,  0.0020,  ..., -0.0253,  0.0376,  0.0183],
        [ 0.0370,  0.0373, -0.0521,  ..., -0.0186, -0.0243, -0.0382]],
       device='cuda:0'), tensor([ 2.6038e-02, -1.0418e-02, -3.8904e-02,  5.3162e-02, -1.6953e-03,
        -8.7001e-03, -1.9776e-02,  4.4825e-02,  6.8941e-02,  3.0025e-02,
         2.2872e-02,  4.2070e-02, -6.2788e-02,  3.5007e-02, -1.1714e-02,
         5.3656e-02, -2.3730e-02,  3.6047e-02,  4.0775e-02,  2.1247e-02,
         1.0016e-02, -2.6264e-02, -1.5882e-02,  2.0356e-02, -1.8651e-02,
         4.0916e-02,  7.3401e-03,  1.4547e-02, -4.8651e-02,  5.8208e-02,
         3.8238e-02, -3.1373e-03, -2.8979e-02,  3.2822e-02,  5.5898e-02,
         3.4640e-02, -1.7405e-02, -2.5265e-03, -4.4282e-02, -4.1815e-03,
         5.2476e-02,  1.9038e-02, -3.0331e-02,  6.3759e-02,  3.2859e-02,
         3.7663e-02,  5.2359e-02, -5.3784e-02, -9.9336e-03, -1.4202e-02,
         4.4648e-02,  2.8499e-02, -3.1303e-03, -2.7943e-03, -4.6181e-02,
         2.2672e-02,  3.9064e-02,  2.3348e-02, -1.1165e-02, -1.1515e-02,
        -6.8280e-02, -7.9247e-04,  5.5212e-02,  3.0759e-02, -1.3979e-02,
        -2.4227e-02, -5.3504e-02,  5.9217e-02,  6.2034e-02, -4.1533e-02,
         2.1950e-02,  2.9898e-02,  4.3389e-03, -6.3890e-02, -3.7407e-03,
        -1.1269e-02, -5.8274e-03, -4.3029e-02,  3.6182e-02, -4.6418e-02,
        -6.0874e-03, -5.5192e-03,  4.0371e-02,  6.5526e-02,  3.6379e-02,
         2.0893e-02, -1.8869e-02, -7.1527e-03, -1.1293e-02, -2.2029e-02,
        -1.2016e-02,  1.1792e-02,  5.9013e-02,  5.5725e-03, -3.8256e-02,
        -2.1027e-02,  7.9226e-03, -4.7526e-02, -2.6193e-02, -5.5513e-02,
        -6.6018e-02,  5.0400e-02,  1.4974e-02, -2.0455e-02, -5.6574e-03,
        -2.6410e-02, -1.8754e-02,  5.7701e-03,  1.6202e-02,  2.1980e-02,
        -1.7176e-02,  3.5424e-02, -2.6620e-02, -1.1360e-02,  3.0099e-03,
         6.9687e-02,  1.9240e-02,  5.8488e-02, -9.1520e-03,  4.7126e-02,
         4.2194e-02, -2.1260e-02,  2.4259e-02, -6.9020e-02, -7.3663e-03,
         1.6465e-02, -2.6539e-02, -5.2556e-02, -2.6315e-02, -6.0116e-03,
         4.7250e-03, -3.2325e-02, -1.7298e-02,  5.6757e-02, -1.3453e-02,
        -7.3959e-03, -1.5121e-02,  5.3580e-03, -2.3599e-03,  5.6977e-02,
        -1.7369e-03,  2.8905e-02,  3.2056e-02, -5.7119e-02,  1.6097e-02,
        -5.1607e-02,  1.7241e-02,  1.4981e-02,  3.1790e-02, -1.2220e-02,
        -4.2774e-02, -3.0245e-02,  3.4825e-03, -2.6615e-02, -2.5707e-02,
         1.5188e-02, -3.3029e-02,  3.8198e-02, -7.1423e-02, -4.1975e-02,
         5.0944e-02, -3.4237e-02,  3.2251e-02, -1.4036e-02, -3.5104e-02,
         1.0997e-02,  4.1125e-02, -1.2155e-02,  2.9369e-03,  5.7274e-02,
        -1.8332e-02,  2.6585e-02, -1.2926e-03,  1.1568e-02, -3.6115e-02,
         2.9684e-02,  1.9488e-02,  4.3600e-02,  3.5916e-03, -8.1694e-03,
        -2.4854e-02, -2.2785e-02,  1.2617e-02,  2.9363e-02, -3.5347e-02,
         8.7448e-03, -6.6681e-03, -5.8016e-02, -2.5131e-02,  1.7108e-02,
         6.5950e-03,  7.6030e-03,  4.7737e-03,  1.3177e-02,  3.7011e-02,
         5.7955e-03, -5.2293e-02, -3.9974e-02,  4.4085e-02,  1.4620e-02,
        -2.1689e-02, -1.3701e-02,  3.7290e-02,  1.7419e-03, -2.7592e-02,
        -3.1531e-02,  2.3580e-02, -6.2689e-03,  4.5576e-02,  5.6766e-02,
         1.4652e-02, -9.3712e-03,  1.4018e-02, -1.6406e-02,  7.2829e-04,
         4.0823e-02, -1.0582e-02, -4.2360e-02,  1.2155e-02, -2.5977e-02,
        -5.4412e-03,  4.2457e-02, -5.4895e-02,  5.4158e-02, -2.1560e-02,
        -9.5506e-03,  5.1604e-02,  2.1860e-02, -2.3093e-02, -1.4705e-04,
        -2.1089e-02,  3.9205e-02,  1.8712e-02, -5.5782e-02, -4.2398e-03,
         4.7201e-02,  7.1453e-03, -2.4689e-02, -4.7633e-02, -2.6648e-02,
        -2.3871e-02, -5.4714e-03,  4.7997e-02,  1.9672e-02,  3.4538e-02,
         2.8891e-02,  5.5701e-02,  4.8819e-02, -2.0238e-03,  1.8579e-02,
        -2.9601e-02, -3.5516e-02, -2.0755e-02,  4.9640e-02,  2.0452e-03,
         4.2949e-02, -1.6158e-02,  1.0979e-02, -2.5501e-02, -4.0277e-02,
        -3.4362e-03, -4.5450e-02, -4.0940e-02,  4.1777e-02, -3.4383e-02,
         1.8810e-02,  1.2835e-02, -1.4684e-02,  2.8588e-02,  3.7787e-02,
         2.4444e-02,  3.5366e-02,  9.9124e-03, -2.7754e-05, -3.6016e-02,
         5.4637e-02, -2.5813e-02,  4.0247e-02, -3.5832e-02,  3.0730e-02,
         1.8171e-02, -2.0923e-03, -3.1228e-02, -2.0217e-02, -3.6143e-02,
         3.5453e-02,  2.5825e-02,  5.9919e-02,  1.7290e-02, -2.1435e-02,
         5.0611e-02,  3.9781e-03, -2.1131e-02,  2.7089e-03,  2.2813e-02,
        -1.1338e-02,  5.4906e-04, -4.8082e-02, -4.8246e-02,  3.2160e-02,
        -2.0110e-03,  5.2380e-02, -5.2579e-03, -3.8866e-02, -5.6940e-02,
         3.9713e-02,  4.2038e-02,  1.4666e-02, -1.8969e-02, -2.3018e-02,
        -1.4232e-02,  2.9373e-02,  5.3265e-02, -1.6153e-02, -4.1221e-02,
        -1.9888e-02, -3.4248e-02,  3.9269e-02, -5.9144e-02,  4.0734e-02,
        -3.3849e-02, -1.4586e-02,  3.6955e-02,  2.8554e-02, -4.4814e-02,
        -1.1994e-03,  8.9765e-03, -6.6151e-03, -1.3533e-02,  2.6242e-02,
         4.7366e-02,  5.0913e-02, -2.5023e-02,  1.1428e-02, -1.1345e-02,
        -3.8282e-02,  5.5287e-02, -1.2165e-02, -3.8691e-02,  9.7433e-03,
         1.4878e-02,  2.6451e-02, -1.2418e-02,  9.2311e-03,  2.2118e-02,
        -4.3958e-02,  3.2112e-02,  1.8964e-02, -4.5446e-02, -5.5564e-02,
        -3.7078e-02, -2.1380e-03,  1.1725e-02, -3.7583e-02, -4.7113e-03,
         2.4273e-02, -2.5560e-02, -5.0932e-02, -4.0665e-02, -1.6211e-02,
        -4.8904e-02, -2.7850e-03, -1.8828e-02, -1.1329e-02, -4.6251e-02,
        -4.9635e-02, -3.2578e-02, -1.7619e-02, -2.3409e-02, -1.9491e-02,
         3.0942e-02, -5.3014e-03, -3.0480e-02, -3.3356e-03, -3.1823e-02,
         2.0910e-02,  4.7519e-02,  2.7742e-02,  5.4271e-02, -3.2468e-02,
         5.6235e-03, -1.4746e-02, -2.6375e-03, -1.5411e-02, -1.0281e-03,
         2.4129e-02,  4.1965e-02,  1.0982e-02, -2.9619e-02, -1.6415e-02,
         1.8973e-02, -2.2375e-02,  5.3523e-03, -2.6529e-02,  5.1642e-02,
         1.1957e-02, -3.6773e-02, -3.7020e-02,  3.7653e-03,  1.5242e-02,
        -1.7558e-02, -7.2423e-03, -4.9940e-02, -8.1861e-03,  5.0598e-02,
        -1.0204e-02, -5.9524e-02,  1.6874e-02,  2.8169e-02, -3.2193e-02,
        -9.8066e-04,  2.8153e-02, -1.0192e-02, -2.7972e-02,  1.0432e-02,
        -3.2678e-02, -1.9695e-02, -3.6078e-02, -3.5933e-02, -4.7746e-03,
         2.0995e-02,  4.2367e-02, -5.5699e-02, -3.4732e-02,  5.4977e-02,
        -3.1839e-02, -1.9789e-02,  4.6750e-02,  4.5630e-02, -1.9449e-02,
         1.9860e-02,  1.0371e-02,  1.1891e-02, -7.8542e-03,  4.0402e-02,
         3.6664e-02,  3.5252e-02, -1.2701e-02, -3.8752e-02, -4.8887e-02,
         2.2606e-02, -1.7943e-02, -1.7463e-02, -2.7759e-02, -5.4919e-02,
        -3.0716e-02,  5.2231e-02,  4.6267e-02,  5.2997e-03, -3.5296e-02,
         3.3148e-02, -4.2999e-02, -1.4424e-02, -2.0092e-02,  1.8884e-02,
         3.4321e-03,  5.1145e-02, -7.0727e-03,  4.8522e-02,  2.8613e-02,
         3.2391e-02, -2.6208e-03, -1.5343e-02,  7.2776e-03, -2.7509e-02,
        -4.9986e-02, -2.0629e-02, -1.7453e-02,  1.9657e-02, -4.3651e-02,
         5.3009e-02,  2.6739e-02,  1.0558e-02, -3.8739e-02,  3.6742e-02,
        -3.0640e-02,  2.7048e-02, -2.9227e-03, -3.7709e-02, -3.7937e-02,
        -1.0380e-02, -1.1788e-02, -4.1094e-02,  3.6425e-02,  1.3904e-02,
        -5.2868e-02, -1.5580e-03, -5.6305e-02, -1.1918e-02,  5.2006e-02,
         5.4874e-02,  4.4823e-03,  2.7498e-02,  2.3448e-02,  2.7670e-02,
        -6.3421e-02,  6.7011e-02,  2.1604e-02,  3.4682e-03, -3.6031e-02,
        -4.5365e-02,  2.8023e-02, -2.8903e-02, -4.0182e-02, -4.3993e-02,
         3.9758e-02,  3.3128e-02,  3.7369e-02,  3.6541e-02,  1.6239e-02,
        -6.4214e-03, -1.2827e-02], device='cuda:0'), tensor([[[ 0.0595],
         [ 0.0937],
         [-0.0071],
         ...,
         [ 0.2752],
         [ 0.1278],
         [ 0.2392]],

        [[ 0.1928],
         [ 0.1723],
         [ 0.2966],
         ...,
         [ 0.0241],
         [-0.3771],
         [ 0.4023]],

        [[-0.2328],
         [-0.0510],
         [ 0.1166],
         ...,
         [ 0.2843],
         [ 0.3918],
         [ 0.3060]],

        ...,

        [[-0.1270],
         [-0.1911],
         [ 0.1943],
         ...,
         [ 0.2197],
         [ 0.1214],
         [-0.0066]],

        [[-0.2607],
         [-0.2623],
         [ 0.1731],
         ...,
         [-0.2596],
         [ 0.0154],
         [ 0.2032]],

        [[ 0.1766],
         [-0.2140],
         [ 0.1034],
         ...,
         [ 0.1973],
         [ 0.0645],
         [ 0.2967]]], device='cuda:0'), tensor([ 0.0812,  0.1267, -0.2280,  0.0451,  0.0998,  0.2389, -0.1001, -0.1014,
        -0.0094, -0.0289,  0.0426,  0.1939, -0.2495, -0.1670,  0.1513,  0.0213,
        -0.2250,  0.2105, -0.2223, -0.1617, -0.2377, -0.1214,  0.1362,  0.1770,
         0.0137,  0.2695, -0.0714, -0.2915,  0.1012,  0.0303,  0.0685, -0.1066,
        -0.1115,  0.1599, -0.2509, -0.1329, -0.1717, -0.0581,  0.1740, -0.2676,
         0.0211, -0.0962, -0.2213, -0.0696,  0.1513,  0.0341, -0.2641,  0.0771,
         0.0241,  0.0941,  0.0600,  0.1355,  0.2125,  0.1898,  0.0240, -0.0419,
        -0.0875, -0.0788,  0.2181,  0.1368, -0.2514,  0.1227, -0.0975,  0.1251,
        -0.2910, -0.2639,  0.2069, -0.2400,  0.0495,  0.1772, -0.0916,  0.1584,
         0.1042, -0.2698, -0.1333,  0.1878,  0.2350,  0.2306, -0.1428,  0.1747,
         0.1944,  0.2208, -0.2490, -0.2733,  0.0885,  0.2277,  0.1364, -0.1738,
        -0.2712, -0.1967, -0.2052,  0.2208, -0.0912,  0.1726, -0.0797, -0.0035,
        -0.1629,  0.1724,  0.0740, -0.2680,  0.0328,  0.0054,  0.1621, -0.2213,
         0.0387,  0.2045, -0.1372,  0.0958, -0.1700,  0.0196, -0.2708,  0.0270,
         0.0622,  0.1504, -0.1195, -0.1992, -0.0845, -0.1262,  0.2397,  0.1167,
        -0.0068, -0.1810, -0.0763,  0.0282,  0.1625,  0.1532,  0.1572, -0.1229,
         0.0115, -0.2327,  0.1083,  0.0554, -0.0899, -0.0146,  0.0462, -0.1077,
         0.2199,  0.0271, -0.2379,  0.2136, -0.1925,  0.1287,  0.1090,  0.2139,
         0.0159,  0.1583, -0.2548, -0.1670,  0.0081,  0.1267,  0.0908,  0.2363,
         0.1244,  0.2164,  0.2344, -0.3200,  0.1439,  0.2394, -0.0985,  0.0277,
         0.0935,  0.1719,  0.2350, -0.0175, -0.0364,  0.0913,  0.1745,  0.1997,
        -0.1304,  0.1671,  0.0656,  0.1159, -0.1522,  0.2094, -0.1935, -0.1477,
         0.0956, -0.2169, -0.2719, -0.0859, -0.0737, -0.1410,  0.2487, -0.0538,
        -0.0480,  0.0283,  0.0769,  0.1261,  0.1914,  0.0584, -0.1069,  0.0960,
         0.2506,  0.2181, -0.2131,  0.0620, -0.1317, -0.1009,  0.1356, -0.0111,
        -0.1019, -0.0600, -0.0830,  0.0859,  0.2539, -0.1410,  0.1578,  0.2311,
        -0.2014, -0.2290,  0.1062,  0.0870,  0.1809,  0.1871, -0.1651, -0.0729,
         0.0641, -0.0296,  0.2464,  0.0268,  0.2541,  0.1486, -0.0269, -0.2826,
         0.1921, -0.2817,  0.1222,  0.1557, -0.0986, -0.2629,  0.0773, -0.1333,
        -0.1113,  0.1555,  0.0670, -0.0628, -0.1725, -0.0200, -0.1100, -0.2641,
        -0.1426,  0.0245,  0.0429,  0.1934,  0.0613,  0.0107,  0.2060,  0.1114,
         0.2580, -0.2370, -0.1928,  0.0133, -0.2734,  0.0429, -0.2446,  0.1645],
       device='cuda:0'), tensor([[[-0.0049,  0.0092, -0.0174, -0.0082, -0.0025],
         [-0.0265, -0.0226,  0.0193, -0.0280,  0.0230],
         [ 0.0091, -0.0257,  0.0031, -0.0165,  0.0055],
         ...,
         [-0.0184, -0.0013,  0.0067,  0.0067, -0.0154],
         [-0.0562, -0.0257, -0.0088, -0.0242, -0.0630],
         [ 0.0003,  0.0150,  0.0236, -0.0130,  0.0059]],

        [[ 0.0200,  0.0012,  0.0259, -0.0151,  0.0200],
         [ 0.0006,  0.0226,  0.0054,  0.0196,  0.0019],
         [ 0.0035, -0.0052,  0.0015, -0.0273,  0.0141],
         ...,
         [ 0.0133,  0.0288, -0.0009,  0.0103, -0.0002],
         [ 0.0128, -0.0084, -0.0183,  0.0142,  0.0087],
         [ 0.0168, -0.0264, -0.0228, -0.0132,  0.0131]],

        [[-0.0063, -0.0214,  0.0096, -0.0057,  0.0200],
         [ 0.0211, -0.0125,  0.0285, -0.0234, -0.0158],
         [-0.0180,  0.0039,  0.0500,  0.0290,  0.0046],
         ...,
         [ 0.0034,  0.0145,  0.0340, -0.0103,  0.0015],
         [ 0.0322,  0.0326,  0.0327,  0.0584,  0.0479],
         [ 0.0200, -0.0137,  0.0114, -0.0153, -0.0286]],

        ...,

        [[-0.0076,  0.0285, -0.0185, -0.0034, -0.0028],
         [-0.0236, -0.0202, -0.0138, -0.0019, -0.0058],
         [ 0.0330, -0.0241,  0.0288, -0.0146, -0.0252],
         ...,
         [ 0.0086,  0.0138,  0.0040,  0.0008,  0.0062],
         [ 0.0448,  0.0356,  0.0449,  0.0037,  0.0266],
         [-0.0110,  0.0169, -0.0144,  0.0132,  0.0052]],

        [[ 0.0036,  0.0053, -0.0276, -0.0203, -0.0098],
         [ 0.0175,  0.0284,  0.0099,  0.0099,  0.0130],
         [ 0.0248,  0.0137,  0.0061, -0.0243,  0.0247],
         ...,
         [ 0.0285, -0.0164, -0.0011, -0.0140, -0.0217],
         [ 0.0064, -0.0227,  0.0146,  0.0226, -0.0009],
         [-0.0139,  0.0304,  0.0308,  0.0122,  0.0045]],

        [[ 0.0272, -0.0198, -0.0113,  0.0144,  0.0262],
         [ 0.0184,  0.0266,  0.0203,  0.0036,  0.0256],
         [-0.0170, -0.0049, -0.0082, -0.0161,  0.0009],
         ...,
         [-0.0046, -0.0164,  0.0223, -0.0002, -0.0008],
         [-0.0143,  0.0072, -0.0320, -0.0163, -0.0248],
         [ 0.0282,  0.0273, -0.0055,  0.0043, -0.0003]]], device='cuda:0'), tensor([ 0.0013,  0.0083, -0.0081, -0.0104,  0.0042,  0.0188, -0.0137,  0.0023,
        -0.0147,  0.0220, -0.0025, -0.0217, -0.0217, -0.0039, -0.0222, -0.0033,
        -0.0173, -0.0073, -0.0028,  0.0028,  0.0033,  0.0307,  0.0098,  0.0202,
        -0.0126, -0.0265,  0.0097, -0.0151,  0.0185,  0.0115, -0.0240,  0.0246,
         0.0074,  0.0078,  0.0101,  0.0003,  0.0123, -0.0201,  0.0287, -0.0146,
        -0.0027,  0.0132,  0.0002, -0.0105,  0.0166,  0.0180, -0.0223,  0.0155,
        -0.0130, -0.0091, -0.0282, -0.0159,  0.0275, -0.0190, -0.0054, -0.0054,
         0.0102,  0.0078,  0.0213, -0.0339, -0.0114,  0.0058, -0.0306,  0.0185,
        -0.0063, -0.0013,  0.0092,  0.0184, -0.0028,  0.0331, -0.0269,  0.0136,
         0.0050,  0.0052, -0.0061, -0.0101,  0.0117,  0.0060, -0.0034, -0.0325,
         0.0032,  0.0095, -0.0223, -0.0160, -0.0029, -0.0182,  0.0046, -0.0263,
         0.0024, -0.0232, -0.0120,  0.0045, -0.0257,  0.0169, -0.0303, -0.0199,
         0.0223,  0.0097, -0.0135,  0.0221,  0.0221,  0.0225, -0.0082,  0.0132,
        -0.0246, -0.0141,  0.0003,  0.0073,  0.0117,  0.0117,  0.0005, -0.0104,
         0.0087, -0.0116,  0.0039,  0.0169, -0.0284, -0.0083,  0.0064, -0.0074,
        -0.0008, -0.0141,  0.0064,  0.0234, -0.0044, -0.0263, -0.0012, -0.0149,
         0.0103, -0.0087,  0.0079, -0.0092,  0.0188,  0.0284, -0.0057, -0.0020,
        -0.0233,  0.0090,  0.0225,  0.0083,  0.0193,  0.0126, -0.0253, -0.0236,
        -0.0017, -0.0242, -0.0115,  0.0137,  0.0225, -0.0156,  0.0169,  0.0133,
         0.0123, -0.0072, -0.0015, -0.0188, -0.0236,  0.0077,  0.0192, -0.0023,
         0.0221,  0.0037, -0.0282, -0.0151, -0.0049, -0.0125, -0.0142,  0.0267,
        -0.0314, -0.0177,  0.0264,  0.0050, -0.0172, -0.0212, -0.0027, -0.0086,
         0.0075,  0.0015, -0.0145,  0.0205,  0.0138, -0.0295, -0.0108, -0.0294,
        -0.0087, -0.0183,  0.0008, -0.0060, -0.0029,  0.0140, -0.0053,  0.0253,
         0.0072, -0.0075, -0.0030, -0.0141,  0.0095, -0.0085,  0.0212,  0.0095,
         0.0301,  0.0027, -0.0086, -0.0047,  0.0258, -0.0159, -0.0217,  0.0147,
         0.0068, -0.0090, -0.0145, -0.0124,  0.0092, -0.0089, -0.0219,  0.0134,
        -0.0263,  0.0063, -0.0079,  0.0058, -0.0238, -0.0185,  0.0178, -0.0090,
         0.0022, -0.0142,  0.0003, -0.0106, -0.0338, -0.0146,  0.0178, -0.0293,
        -0.0314,  0.0231,  0.0265, -0.0188, -0.0221, -0.0192, -0.0037, -0.0152,
         0.0260, -0.0042, -0.0016,  0.0291,  0.0005, -0.0267, -0.0279,  0.0106,
        -0.0242, -0.0262,  0.0256,  0.0047,  0.0071,  0.0117, -0.0095,  0.0068],
       device='cuda:0'), tensor([0.9964, 1.0030, 0.9971, 1.0109, 1.0002, 0.9905, 0.9945, 0.9726, 0.9964,
        1.0086, 1.0065, 0.9679, 1.0072, 0.9850, 0.9610, 1.0081, 1.0033, 0.9834,
        1.0142, 0.9565, 0.9975, 0.9944, 1.0015, 1.0075, 0.9626, 0.9781, 1.0010,
        1.0016, 0.9944, 1.0078, 0.9843, 0.9956, 0.9610, 0.9875, 0.9778, 1.0033,
        0.9683, 0.9832, 1.0054, 0.9723, 1.0151, 0.9726, 1.0025, 0.9849, 0.9972,
        0.9963, 1.0004, 1.0036, 1.0047, 0.9890, 0.9946, 1.0002, 1.0021, 0.9841,
        0.9925, 1.0114, 0.9812, 1.0283, 0.9662, 1.0078, 0.9814, 0.9856, 0.9746,
        0.9796, 1.0231, 0.9938, 1.0052, 0.9844, 1.0294, 1.0067, 0.9971, 1.0074,
        1.0100, 0.9995, 1.0029, 0.9744, 1.0009, 0.9858, 0.9852, 1.0156, 0.9933,
        0.9994, 1.0016, 0.9996, 1.0298, 1.0071, 1.0099, 1.0052, 1.0039, 1.0229,
        0.9894, 1.0087, 0.9970, 0.9839, 0.9958, 0.9944, 1.0082, 1.0000, 1.0005,
        0.9951, 1.0060, 0.9875, 1.0071, 1.0006, 0.9820, 1.0112, 0.9848, 0.9978,
        0.9949, 0.9859, 0.9841, 0.9975, 0.9856, 1.0193, 0.9718, 0.9963, 0.9538,
        1.0079, 0.9968, 0.9861, 0.9798, 1.0000, 1.0088, 0.9956, 0.9778, 1.0037,
        0.9789, 0.9769, 1.0184, 0.9856, 1.0034, 1.0091, 1.0038, 0.9849, 1.0012,
        0.9957, 1.0516, 1.0059, 0.9896, 0.9735, 1.1026, 0.9711, 1.0012, 0.9768,
        0.9796, 0.9859, 0.9982, 0.9921, 1.0090, 0.9887, 1.0060, 0.9895, 0.9952,
        0.9720, 1.0229, 1.0065, 0.9982, 1.0160, 0.9546, 0.9680, 0.9972, 0.9734,
        1.0062, 0.9966, 0.9657, 0.9878, 1.0001, 1.0142, 0.9588, 0.9832, 0.9839,
        1.0047, 0.9717, 1.0200, 1.0115, 0.9844, 0.9930, 1.0137, 1.0100, 1.0067,
        0.9934, 0.9862, 0.9836, 0.9860, 1.0130, 0.9827, 1.0082, 0.9835, 1.0122,
        1.0083, 0.9989, 1.0049, 0.9955, 0.9493, 1.0064, 0.9940, 0.9897, 0.9808,
        0.9718, 1.0097, 1.0030, 1.0012, 1.0049, 1.0080, 0.9914, 1.0031, 0.9922,
        1.0034, 1.0012, 0.9958, 1.0082, 0.9934, 1.0012, 1.0045, 1.0031, 1.0034,
        0.9946, 0.9809, 0.9719, 0.9887, 0.9269, 1.0050, 0.9767, 0.9870, 0.9663,
        1.0037, 1.0010, 1.0372, 1.0239, 0.9967, 1.0015, 0.9945, 0.9826, 0.9918,
        0.9986, 0.9975, 1.0359, 1.0142, 0.9993, 0.9836, 0.9949, 1.0238, 1.0068,
        0.9534, 0.9683, 0.9812, 0.9687, 0.9847, 0.9901, 0.9977, 0.9674, 1.0039,
        1.0099, 1.0071, 1.0035, 1.0044], device='cuda:0'), tensor([ 1.1079e-02,  5.4643e-03,  6.7391e-03,  2.2061e-02,  4.4355e-03,
         1.4505e-02,  2.1704e-03,  1.8505e-02,  1.3179e-02,  8.2202e-03,
         6.4743e-03,  1.4865e-02,  1.0141e-02,  5.5156e-02,  1.2540e-02,
        -1.7508e-02,  8.7710e-03,  1.3547e-02, -2.2299e-02,  1.7151e-02,
         6.0297e-03,  7.6253e-03,  1.3077e-02,  4.8684e-03,  4.1329e-02,
         8.2525e-03,  2.1909e-03,  1.5428e-03,  3.8552e-03,  2.3575e-02,
         1.0027e-02,  1.0468e-02,  9.1007e-03, -4.3865e-03,  9.6458e-03,
         3.7593e-03,  4.3211e-03,  4.7282e-03,  1.0306e-02,  1.2835e-02,
         6.0109e-03,  2.3464e-03,  3.2409e-03,  6.0225e-03,  4.7370e-03,
         8.3375e-03,  5.7923e-03,  2.5685e-03,  4.7555e-03,  2.4970e-02,
         1.6173e-02,  2.0624e-02,  1.4678e-02, -1.9528e-04,  8.3043e-03,
         7.1026e-03,  1.8873e-02,  5.1380e-03,  7.0358e-03, -1.6112e-02,
         8.2103e-03,  3.0622e-02,  1.2761e-03,  1.4281e-03, -2.4393e-02,
         4.8326e-03,  1.3901e-02,  1.7430e-02, -5.2840e-02,  7.6829e-03,
         2.7451e-03,  7.1115e-03,  1.3346e-02,  4.9349e-03,  4.0179e-03,
         1.8418e-02,  2.9167e-03,  3.0198e-03,  2.4908e-03,  3.2382e-02,
         1.1414e-02,  1.8705e-03, -6.3523e-03,  5.3383e-03,  8.5355e-04,
         4.7307e-03,  8.5748e-03,  3.9681e-03,  5.9792e-03, -2.2256e-02,
         1.1488e-03,  4.7401e-03,  2.9149e-03,  2.2748e-03,  1.6105e-02,
         9.7390e-03,  1.0302e-02,  2.9265e-02,  1.0140e-02,  4.2530e-03,
         5.2801e-03,  2.3743e-03,  7.5029e-03,  1.6706e-02,  1.0829e-02,
        -1.9418e-02, -2.5231e-04, -1.5833e-03, -1.9712e-02,  1.8670e-02,
         2.6264e-02,  5.1928e-03,  5.7216e-03, -4.8207e-02,  1.1419e-02,
         3.8173e-03,  9.1279e-03,  6.2993e-03,  7.0390e-03,  1.2364e-02,
         2.4269e-02,  4.5053e-06,  9.4121e-03,  4.3521e-03,  8.7304e-03,
         1.7477e-03,  8.9169e-03,  2.6280e-03, -3.2413e-02,  5.2016e-03,
         4.9104e-03,  1.3286e-02,  1.7319e-03,  1.0619e-02,  6.4433e-03,
         5.9842e-03, -9.0532e-03,  1.0010e-02,  1.4787e-02,  1.0578e-02,
        -2.7069e-02,  6.6768e-03,  5.0185e-03,  5.0333e-03,  1.2385e-02,
        -1.5656e-03,  4.9644e-03,  6.8275e-03,  8.6090e-03,  7.1621e-03,
         1.1656e-02,  1.8738e-02, -2.5810e-02,  1.9088e-02, -2.9195e-02,
         3.8327e-03,  3.9184e-03, -8.0503e-03,  4.1598e-02,  9.9116e-04,
         1.7797e-02,  1.7712e-02, -6.6598e-03,  1.9615e-02,  1.5440e-02,
         1.2771e-02,  6.7084e-03,  1.3006e-02,  7.0715e-02,  2.4888e-02,
         3.4808e-03,  6.3962e-03,  1.2540e-02, -4.7815e-03,  2.6015e-02,
         1.0313e-02,  1.6086e-02,  1.8486e-02,  1.7079e-02,  6.2620e-03,
         5.7771e-03,  7.3430e-03, -6.1322e-03,  2.1051e-03,  1.1944e-02,
         2.6285e-02,  4.1332e-03, -1.1419e-02,  9.4032e-03,  9.5456e-03,
         1.8840e-02,  5.8494e-03,  1.5006e-02,  1.6076e-02,  6.8770e-03,
         2.3256e-02,  1.6244e-02,  1.5475e-02,  1.2418e-02,  2.3520e-02,
         6.1613e-03,  8.3522e-03,  6.0547e-03,  8.0978e-03,  1.6591e-02,
         2.8670e-03,  1.8529e-02, -2.1027e-02,  4.1187e-03, -8.3586e-04,
         8.7879e-03,  1.0772e-02,  1.3221e-03,  4.6690e-03,  2.0738e-04,
         9.4548e-03,  4.2387e-03,  5.4802e-03,  4.6548e-02,  3.5494e-02,
         1.5098e-02,  5.3418e-04,  2.7589e-03,  1.7826e-02,  1.9291e-02,
        -3.3545e-04,  6.9026e-04, -3.2207e-02, -2.6166e-02,  4.9595e-03,
         7.2734e-03,  7.6218e-03,  3.0058e-03, -3.9462e-05,  6.1479e-03,
         2.0354e-02, -3.1956e-02,  2.5519e-03,  6.5602e-03,  1.2215e-02,
         5.1522e-03, -2.4059e-02,  1.9025e-02,  6.8867e-03,  4.7094e-03,
         9.2718e-03, -1.4937e-04,  8.1740e-03,  4.3428e-03,  6.7622e-03,
         6.9860e-03,  3.8395e-03,  6.7031e-03,  5.5085e-03,  2.9414e-03,
         4.9583e-03], device='cuda:0'), tensor([[[-3.7068e-02, -1.6337e-02,  1.9521e-02,  1.1580e-02,  1.3879e-02],
         [-2.9142e-03,  3.9971e-03, -1.6161e-02, -2.3417e-02,  9.5062e-03],
         [-4.6536e-03,  1.2617e-02, -2.6456e-02, -4.1759e-03, -2.2844e-02],
         ...,
         [-1.3880e-03, -2.8169e-02,  2.4879e-02,  2.9872e-03, -1.7110e-02],
         [ 4.2737e-03,  1.9511e-02, -1.9445e-02,  1.2174e-03, -3.3920e-03],
         [-1.7219e-03, -2.9059e-02, -3.0907e-02,  1.7639e-02,  1.8137e-02]],

        [[-2.4818e-02, -2.2391e-02, -2.6387e-02,  1.9513e-03,  1.4993e-02],
         [-1.6426e-02,  1.9233e-02, -3.0852e-02, -2.6999e-02, -8.6978e-03],
         [ 9.4532e-03,  2.8354e-02,  3.7067e-03,  2.6208e-02, -2.5098e-02],
         ...,
         [ 2.9550e-02,  3.1364e-02,  2.1923e-02, -9.8569e-03, -1.3110e-02],
         [ 2.1975e-03,  9.3391e-03, -1.4750e-02, -2.2414e-03,  1.7141e-02],
         [-7.6280e-03,  1.0053e-02,  3.3914e-02, -1.6119e-02,  3.3304e-02]],

        [[ 2.4171e-02, -1.9384e-02,  1.3942e-02, -2.0431e-02, -3.7264e-03],
         [ 1.5999e-02,  2.4797e-02, -5.9616e-03, -7.1530e-03,  1.1178e-02],
         [ 2.4454e-02,  1.5036e-02, -1.2932e-02,  7.9781e-03, -1.7487e-02],
         ...,
         [ 1.6533e-02,  2.0748e-02, -5.9254e-03, -2.1750e-02, -8.4197e-03],
         [-1.5341e-02, -2.0364e-02,  1.5228e-02, -1.0767e-02,  2.1991e-02],
         [ 1.3863e-02, -2.0193e-02, -8.6644e-03,  2.1185e-02, -1.9425e-02]],

        ...,

        [[ 1.9161e-02, -2.6589e-02,  3.5965e-02, -5.6147e-03,  2.7989e-03],
         [-2.8392e-02, -2.1780e-02,  4.1301e-03,  2.5214e-02,  2.4375e-03],
         [ 1.3011e-02, -1.7644e-02, -5.8300e-03,  1.7030e-02,  3.1663e-03],
         ...,
         [ 6.6860e-03,  6.4755e-03, -2.4823e-02,  1.6405e-02, -2.4342e-02],
         [-3.8879e-04, -3.1767e-03, -1.2390e-02,  1.5452e-02, -1.6846e-02],
         [-2.6958e-02,  1.0380e-02,  1.7778e-02, -1.3438e-02,  9.2538e-03]],

        [[-3.4492e-02, -3.4427e-02, -3.0189e-02, -3.9834e-02, -1.2652e-02],
         [ 9.1437e-03, -7.7966e-03,  1.5860e-03,  2.4220e-02, -9.8377e-03],
         [ 1.1651e-02, -1.9731e-02, -2.5931e-03, -2.1080e-03, -1.1414e-02],
         ...,
         [-4.6506e-03,  6.8908e-03, -9.5521e-03, -1.1042e-03,  2.0934e-02],
         [ 2.6005e-02,  2.2235e-02, -4.6982e-03,  1.5470e-03,  2.3128e-02],
         [ 1.6618e-02,  1.1907e-02, -1.6964e-02, -1.5242e-03, -1.4475e-02]],

        [[-2.1107e-02, -7.4753e-03,  4.0673e-02,  3.7932e-02,  1.7979e-02],
         [-1.6977e-02, -3.3016e-02, -1.7795e-02,  9.7256e-03,  1.2938e-02],
         [-3.5230e-03, -1.3264e-02, -3.0832e-02,  1.1460e-02, -2.4715e-02],
         ...,
         [ 9.1966e-05, -2.3794e-03,  6.8150e-03,  1.4727e-02, -1.6199e-02],
         [ 6.7265e-03,  2.8787e-03,  1.4366e-02, -3.3344e-02, -3.6805e-02],
         [ 2.4811e-02,  7.1385e-03, -5.3905e-03,  2.0004e-02,  5.0345e-02]]],
       device='cuda:0'), tensor([-1.1860e-02, -1.2365e-02, -5.3218e-03, -2.2036e-03, -2.5875e-02,
         1.0810e-02, -1.4983e-04, -1.5720e-02, -1.6637e-02,  1.4574e-03,
        -9.6682e-04,  1.7177e-02, -2.7340e-05, -1.2375e-02,  2.1135e-02,
        -1.2781e-02, -6.8255e-02, -5.7958e-03, -1.1871e-02,  2.5293e-02,
         2.1350e-02,  3.5856e-03, -2.0909e-02, -2.1158e-02,  2.2262e-02,
         7.2918e-04,  1.7685e-02, -3.5350e-02, -1.4313e-02,  6.0577e-03,
         9.7394e-03, -1.5474e-02, -1.5905e-02, -1.1787e-02,  2.9759e-02,
         1.2097e-02, -1.0439e-02, -1.1471e-02,  1.1397e-02,  1.0849e-02,
        -2.4403e-02,  3.5962e-03, -8.3612e-04,  9.1089e-03, -4.6867e-02,
         1.3966e-03,  3.1314e-02,  1.7356e-02,  2.2451e-02,  4.4663e-03,
        -2.7860e-02,  1.1862e-02, -1.4289e-02,  2.1194e-02,  6.3794e-03,
         1.0467e-02,  2.4040e-02,  2.6510e-02, -1.3924e-02, -4.6173e-02,
        -1.4189e-03, -2.5865e-02,  1.5571e-02,  3.1629e-02,  1.0881e-02,
         1.7582e-02,  2.7466e-02,  2.1031e-02,  1.7741e-03,  6.1994e-03,
         3.1485e-02, -2.1010e-02,  9.0828e-03,  5.3401e-03,  7.3347e-03,
        -1.6424e-02, -7.7188e-03, -3.1517e-02, -1.6111e-02,  8.3819e-03,
        -3.7163e-03, -4.8193e-03,  2.7644e-03, -1.2312e-03,  2.0205e-02,
         6.0883e-03,  2.7586e-02, -7.3828e-04,  1.7455e-02,  1.0786e-02,
         2.5814e-02,  2.0605e-02, -1.3950e-02,  1.5663e-02, -2.9406e-02,
        -6.1115e-02, -2.6877e-02, -2.5918e-02, -9.3242e-02,  1.8890e-02,
         1.5617e-02, -1.9738e-02, -2.6167e-02,  2.5388e-02,  2.3753e-02,
        -1.5058e-02,  1.9724e-02,  2.0159e-02, -1.2981e-02, -1.8869e-02,
        -2.5012e-02,  1.0823e-02,  8.5064e-03,  2.1186e-03, -5.6357e-03,
        -1.6823e-02,  1.3652e-02, -1.3655e-02,  2.1425e-02,  1.7948e-02,
         7.3113e-03,  6.3409e-03, -1.7762e-03,  8.4106e-03, -1.5611e-02,
         5.5339e-04, -5.5127e-03,  1.3780e-02, -2.1086e-03,  1.4789e-02,
         2.4124e-02, -1.7208e-02, -4.6600e-02,  8.9567e-03, -7.1621e-03,
         3.1441e-03,  6.9059e-03,  2.5163e-02, -1.1894e-02,  7.1134e-03,
        -2.5988e-02, -3.9548e-03,  2.0806e-02, -3.5620e-03, -3.4248e-04,
        -1.0849e-02,  3.0160e-02,  2.2585e-02, -1.7583e-02,  2.5716e-02,
         8.0326e-03,  4.3933e-03, -4.7710e-02,  2.0995e-02,  1.7261e-02,
         1.4773e-02,  1.8910e-02, -2.4304e-02,  5.0525e-04,  1.9604e-02,
         6.6966e-03, -5.6294e-03,  9.9683e-03,  6.9946e-03, -2.7182e-02,
        -7.0364e-03,  1.7360e-02, -1.1260e-02,  3.2081e-02, -2.5645e-02,
        -7.4245e-02,  2.1565e-02,  2.2407e-02, -1.5098e-02,  1.1862e-02,
        -2.1952e-02,  1.6557e-02,  1.4980e-02,  1.1258e-02, -1.4843e-02,
        -1.6917e-02, -2.6519e-02, -2.3178e-02,  1.0706e-02, -1.5077e-03,
         2.4321e-03, -2.2460e-02, -1.0268e-02, -1.1314e-02,  2.3369e-02,
        -2.6172e-02, -2.4133e-02,  1.3513e-02, -1.0058e-02,  2.6019e-02,
         3.0025e-04, -2.1879e-02,  2.6960e-03,  2.0294e-03, -5.4312e-03,
         1.4273e-02, -4.0474e-03, -4.9844e-03,  2.2430e-02, -1.3556e-03,
        -2.3349e-02, -2.4295e-02, -9.1535e-03,  1.6694e-02, -2.1082e-02,
         1.5220e-02,  6.6957e-03,  1.8091e-02,  7.7876e-04, -2.4764e-02,
         1.3899e-02, -2.6662e-02, -2.2203e-02, -2.5470e-02,  1.4327e-02,
        -5.4487e-04,  1.3910e-02, -4.2572e-02,  1.5013e-02,  1.6470e-03,
         6.1738e-03, -2.3398e-02, -7.8127e-04,  1.3952e-03,  1.1251e-02,
        -2.8362e-02, -9.1604e-02, -3.0951e-03, -3.0355e-02, -1.8706e-02,
        -1.0426e-02,  1.2883e-02, -2.0796e-02, -4.5214e-03,  1.6452e-02,
         2.6526e-02, -3.4772e-03,  1.6259e-02, -1.8016e-02, -6.8455e-03,
        -2.8273e-02,  6.5345e-03,  1.5340e-02,  2.3885e-02,  5.1675e-04,
        -1.1347e-02,  1.5564e-02, -1.9313e-02,  2.3741e-03,  9.1373e-03,
         2.0068e-03], device='cuda:0'), tensor([0.9760, 0.9604, 0.9864, 0.9507, 0.9872, 0.9631, 0.9802, 0.9693, 0.9795,
        0.9646, 0.9782, 1.0035, 0.9841, 0.9825, 0.9656, 0.9759, 0.9977, 0.9788,
        0.9627, 0.9765, 0.9702, 0.9778, 0.9804, 0.9844, 0.9135, 0.9526, 0.9848,
        1.0072, 0.9539, 0.9734, 0.9777, 0.9857, 0.9781, 0.9778, 0.9766, 0.9796,
        0.9742, 0.9770, 0.9665, 0.9772, 0.9778, 0.9444, 0.9867, 0.9816, 0.9952,
        0.9632, 0.9789, 0.9839, 0.9723, 0.9712, 0.9980, 1.0073, 0.9676, 0.9685,
        0.9764, 0.9603, 0.9448, 0.9548, 0.9703, 0.9965, 0.9927, 0.9579, 0.9827,
        0.9720, 0.9710, 0.9825, 0.9746, 0.9622, 0.9805, 0.9655, 0.9683, 0.9762,
        0.9585, 0.9836, 0.9897, 0.9752, 0.9828, 1.0063, 0.9972, 0.9902, 0.9804,
        0.9770, 0.9826, 0.9776, 0.9770, 0.9844, 0.9810, 0.9715, 0.9817, 0.9815,
        0.9831, 0.9824, 0.9772, 0.9774, 1.0114, 0.9856, 0.9754, 0.9307, 0.9973,
        0.9725, 0.9811, 0.9942, 0.9713, 0.9275, 0.9817, 0.9367, 0.9701, 0.9810,
        0.9749, 0.9841, 0.9608, 0.9877, 0.9645, 0.9731, 0.9693, 0.9654, 0.9818,
        0.9806, 0.9669, 0.9755, 0.9703, 0.9764, 0.9725, 0.9794, 1.0119, 0.9654,
        0.9582, 0.9674, 0.9807, 0.9838, 0.9803, 0.9859, 1.0114, 0.9766, 0.9690,
        0.9707, 0.9625, 0.9802, 0.9552, 0.9743, 1.0041, 0.9693, 0.9741, 0.9709,
        0.9704, 1.0006, 0.9858, 0.9745, 0.9732, 0.9753, 0.9268, 0.9618, 0.9838,
        0.9755, 0.9795, 0.9768, 0.9726, 0.9644, 0.9558, 0.9470, 0.9964, 0.9756,
        0.9639, 0.9695, 0.9770, 0.9734, 0.9633, 0.9551, 0.9934, 1.0016, 1.0130,
        0.9764, 0.9501, 0.9484, 0.9417, 0.9804, 1.0055, 0.9739, 0.9383, 0.9752,
        0.9768, 0.9798, 0.9769, 0.9759, 0.9727, 0.9835, 0.9175, 1.0023, 1.0200,
        0.9517, 0.9851, 0.9765, 0.9726, 1.0003, 0.9714, 0.9473, 0.9877, 0.9558,
        0.9793, 0.9828, 0.9680, 0.9659, 0.9842, 0.9590, 0.9735, 0.9532, 0.9528,
        1.0080, 0.9703, 0.9768, 0.9693, 1.0117, 0.9779, 0.9448, 0.9831, 0.9882,
        1.0141, 0.9763, 0.9720, 0.9773, 0.9732, 0.9580, 0.9940, 0.9773, 0.9684,
        0.9785, 0.9434, 0.9777, 0.9767, 0.9760, 0.9773, 1.0108, 0.9800, 0.9931,
        0.9693, 0.9764, 0.9721, 0.9777, 0.9690, 0.9618, 0.9729, 0.9753, 0.9346,
        1.0057, 0.9737, 0.9620, 0.9824, 0.9001, 0.9749, 0.9733, 0.9358, 0.9626,
        0.9745, 0.9658, 0.9780, 1.0259], device='cuda:0'), tensor([-2.1927e-02, -1.8696e-02, -1.0178e-02, -9.0426e-03, -1.9206e-02,
        -1.9238e-02, -1.4935e-02, -1.9903e-02, -1.5231e-02, -1.1899e-02,
        -2.3366e-02, -2.0221e-02, -4.9741e-03, -1.3149e-02, -8.0258e-03,
        -1.6729e-02, -6.1392e-04, -1.0030e-02, -6.7301e-03, -1.9812e-02,
        -1.0087e-02, -1.8336e-02, -1.3479e-02, -9.4287e-03, -1.0014e-02,
        -6.7036e-03, -1.0618e-02, -9.5167e-03, -1.5063e-02, -2.9860e-02,
        -1.6557e-02, -4.5249e-03, -1.2402e-02, -1.7405e-02, -2.1714e-02,
        -1.9129e-02, -1.0692e-02, -1.4316e-02, -1.5087e-02, -2.2473e-02,
        -1.9587e-02, -1.9610e-02, -1.4188e-02, -1.0646e-02, -2.1333e-03,
        -9.9776e-03, -1.4683e-02, -1.1219e-02, -1.6369e-02, -1.8864e-02,
        -8.2124e-03, -8.7844e-03, -2.0083e-02, -2.0424e-02, -2.1691e-02,
        -9.3916e-03, -9.1845e-03, -1.5797e-02, -1.6881e-02,  7.4758e-04,
        -7.4629e-03, -1.7604e-02, -1.7849e-02, -2.5288e-02, -1.4829e-02,
        -1.2028e-02, -2.4158e-02, -7.1436e-03, -9.4486e-03, -1.3334e-02,
        -8.2719e-03, -8.6782e-03, -2.2695e-02, -1.0078e-02, -6.8779e-03,
        -1.8154e-02, -1.0537e-02, -1.6433e-02,  7.7723e-03, -4.6088e-03,
        -1.2783e-02, -1.0193e-02, -7.6274e-03, -1.7076e-02, -2.4224e-02,
        -1.0928e-02, -2.0697e-02, -7.7354e-03, -1.0786e-02, -1.5236e-02,
        -1.7017e-02, -1.5150e-02, -4.9611e-03, -1.8573e-02, -1.5643e-02,
         8.0142e-03, -1.1360e-02, -2.4274e-02,  3.4844e-05, -1.1341e-02,
        -7.1146e-03, -1.2387e-03, -1.7975e-02, -8.6586e-03, -1.7129e-02,
        -5.8294e-03, -8.6560e-03, -1.4440e-02, -2.4478e-02, -8.8581e-03,
        -1.2271e-02, -1.1004e-02, -5.8024e-03, -2.5349e-02, -2.0731e-02,
        -1.0732e-02, -1.6522e-02, -1.3671e-02, -1.7510e-02, -1.7056e-02,
        -7.9794e-03, -1.9484e-02, -7.6775e-03, -1.7082e-02, -1.7982e-02,
        -6.9660e-03, -9.7361e-03, -3.0592e-02, -1.3887e-02, -1.5422e-02,
        -1.2603e-02, -2.0704e-02, -2.1743e-02, -9.5833e-03, -6.3463e-03,
        -1.1531e-02, -2.0321e-02, -1.6202e-02, -6.7929e-03, -2.3781e-02,
        -8.5322e-03, -2.2892e-02, -1.9779e-02, -1.5978e-02, -1.6341e-02,
        -2.9171e-02, -1.3165e-02, -2.3889e-02, -1.5653e-02, -1.7116e-02,
        -6.1506e-03, -1.2157e-02,  1.5160e-02, -3.1833e-03, -1.7063e-02,
        -2.1370e-02, -2.0233e-02, -1.4494e-02, -2.5619e-02, -9.2517e-03,
        -2.0490e-02, -1.6015e-02, -3.0782e-02, -2.0312e-02, -1.0522e-02,
        -1.9877e-02, -2.4827e-02, -1.9475e-02,  1.5227e-03, -1.4185e-02,
        -1.4058e-02, -2.0549e-02, -1.8056e-02, -2.9071e-02, -1.6740e-02,
        -8.2444e-03, -1.0150e-02, -1.4883e-02, -8.1971e-03, -1.4023e-02,
        -2.2389e-02, -1.3766e-02, -1.6126e-02, -1.9957e-02, -2.5419e-02,
        -1.4410e-02, -1.2572e-02, -1.0037e-02, -3.3427e-02, -6.7683e-03,
        -1.7320e-02, -1.7583e-02, -1.6898e-02, -1.6534e-02, -2.6469e-02,
        -1.7649e-02, -2.6989e-03, -6.8647e-03, -1.2873e-02, -1.2159e-02,
        -1.7874e-02, -1.8854e-03, -1.0245e-02, -2.0437e-02, -1.8901e-02,
        -2.7999e-02, -1.6532e-02, -1.7408e-02, -1.0479e-02, -1.2319e-02,
        -2.0319e-02, -1.5304e-02, -1.7947e-02, -1.6911e-02, -1.4097e-02,
        -1.5126e-02, -1.8861e-02, -9.2750e-03, -2.3554e-02, -2.3196e-02,
        -2.2994e-02, -2.2039e-02,  1.8665e-03, -1.8807e-02, -2.3149e-02,
        -4.3809e-03, -6.0410e-03, -1.6093e-02, -1.6661e-02, -2.0233e-02,
        -1.8434e-02, -1.0513e-02, -7.0671e-03, -1.2373e-02, -1.2940e-02,
        -1.2324e-02, -2.0346e-02, -1.5560e-02, -1.9820e-02, -1.2047e-02,
        -1.9105e-02, -1.6169e-02, -8.2338e-04, -1.8906e-02, -9.6526e-03,
        -1.4236e-02, -1.0759e-02, -1.3040e-02, -1.7629e-02, -4.3087e-03,
        -1.2710e-02, -2.5880e-02, -1.2576e-02, -2.4684e-02, -2.1624e-02,
        -2.4049e-02], device='cuda:0'), tensor([[-0.0139, -0.0361, -0.0009,  ..., -0.0003, -0.0244, -0.0310],
        [ 0.0053,  0.0529, -0.0158,  ...,  0.0398, -0.0344, -0.0348],
        [-0.0367, -0.0543,  0.0874,  ..., -0.0586,  0.0028, -0.0375],
        ...,
        [-0.0120,  0.0104, -0.0007,  ...,  0.0275, -0.0002,  0.0264],
        [ 0.0319, -0.0117,  0.0399,  ...,  0.0125,  0.0044, -0.0163],
        [-0.0434, -0.0212,  0.0368,  ...,  0.0226,  0.0116,  0.0181]],
       device='cuda:0'), tensor([ 3.6498e-02, -3.5656e-02, -6.7517e-02, -4.7468e-02, -5.6780e-02,
        -1.1631e-02,  3.9309e-02, -5.8864e-02, -1.6222e-02, -4.4859e-02,
        -1.7466e-02,  4.2682e-02, -4.5198e-02, -3.2224e-02, -9.0118e-02,
         4.6102e-02, -4.5540e-02, -7.2120e-02, -2.0830e-02,  1.0963e-02,
         6.2120e-02, -6.0773e-02,  1.7622e-02,  2.4961e-02,  3.0206e-02,
        -2.6023e-02, -6.2861e-02, -2.9610e-02,  3.1476e-02,  6.9190e-02,
         4.1092e-02,  5.3512e-02, -6.7707e-02,  1.6981e-02,  1.1983e-02,
        -7.8642e-04,  1.6361e-04,  8.6190e-03, -4.9830e-03,  7.3563e-03,
        -4.7540e-02,  2.5145e-03, -2.7647e-02, -6.2106e-02, -2.2311e-02,
        -5.7173e-02,  5.5371e-02, -5.6164e-02,  9.4582e-03,  9.7536e-04,
        -6.5766e-02,  1.3359e-02, -1.5653e-02, -5.3416e-02,  2.3186e-02,
         4.6051e-02,  5.1259e-03, -2.7210e-02, -8.6250e-03,  5.3082e-02,
        -3.6259e-02, -4.6115e-03, -3.1362e-02,  3.0711e-03,  4.3837e-02,
        -8.0582e-02, -3.8901e-02,  5.1135e-02, -3.9044e-02,  6.5127e-03,
         4.8316e-02, -3.5398e-02,  4.6867e-02, -9.3217e-03,  1.9445e-02,
         2.9772e-03, -4.5126e-02,  5.8600e-03,  1.8921e-02,  2.5086e-02,
         6.3580e-02, -3.5774e-02,  3.3412e-02,  7.1784e-02, -4.4314e-02,
        -2.2340e-02, -4.7986e-03,  2.1169e-03,  5.9309e-02, -1.3563e-02,
         1.0385e-02, -1.8234e-02, -1.9351e-02,  6.3004e-02,  3.7988e-02,
        -6.7458e-02, -4.9267e-03, -6.7894e-02, -5.6607e-02,  4.0521e-02,
        -2.7275e-03, -2.1511e-02, -5.6911e-02, -5.6592e-02, -9.0473e-03,
        -3.0320e-02,  2.2625e-02, -4.5701e-02, -1.8114e-02,  2.0566e-02,
        -9.3179e-03,  5.0853e-02, -6.3697e-02, -2.4643e-02, -3.5653e-03,
         6.9697e-02, -5.2147e-03, -4.9123e-02, -3.7577e-02, -7.7222e-03,
         2.7147e-02,  5.6892e-02, -4.0215e-02, -1.4778e-02,  6.4300e-02,
         1.0371e-02, -6.3649e-02, -6.6989e-02, -3.3113e-02, -1.1333e-02,
         5.6204e-02, -1.9536e-02,  2.5839e-02,  2.6291e-02,  6.5430e-02,
         6.1421e-02, -1.2696e-04, -5.6032e-02,  4.4748e-02, -2.0301e-02,
        -1.0873e-02,  2.9542e-02,  6.4542e-02, -1.2413e-02, -6.0873e-02,
        -2.1256e-02,  4.6765e-02,  3.8055e-02,  9.7920e-03,  1.7795e-02,
         4.7349e-02,  3.4095e-02,  1.5386e-02,  3.8192e-02, -2.1664e-03,
        -2.2140e-02, -5.5064e-02, -4.8419e-02,  3.2939e-02,  2.6091e-02,
         1.6299e-02, -8.9446e-03,  3.6011e-02, -4.5157e-03,  2.6833e-02,
         1.9576e-02,  4.1908e-02, -5.8949e-02,  3.0718e-02,  4.4716e-02,
         2.3489e-03,  2.2908e-02, -3.7626e-03, -2.7406e-02, -6.2967e-02,
         1.5145e-02,  6.4814e-02,  5.3031e-02,  6.4220e-02, -5.1867e-02,
         6.9618e-02,  1.2956e-02,  1.8335e-02,  7.0298e-03, -5.0507e-02,
        -2.7500e-02,  6.0729e-02,  1.8488e-02,  5.6855e-02,  3.3584e-03,
         4.7285e-02,  6.5562e-02,  5.6496e-02, -2.7691e-02,  7.0118e-02,
        -3.7433e-02,  2.3244e-02,  5.1080e-02,  4.7273e-02,  6.4702e-02,
        -2.8413e-02, -6.0544e-02, -3.2449e-02,  3.7161e-02,  2.6748e-02,
        -3.2856e-02,  3.4503e-02, -2.5317e-02,  1.4166e-03, -1.7178e-02,
         2.8046e-02, -2.8836e-02, -1.6413e-02,  1.0915e-02, -3.7450e-02,
         2.6286e-02, -3.7212e-02, -5.7532e-02, -2.2388e-02, -3.3299e-02,
         5.3605e-02, -2.9739e-02, -1.8386e-02,  5.1247e-05, -1.0246e-02,
        -5.1602e-02, -1.1062e-02, -3.2297e-02,  1.8888e-02, -3.7955e-02,
         5.8177e-02,  5.4438e-02,  5.9658e-02,  3.7031e-03, -6.4314e-02,
         3.2032e-02,  1.5357e-02, -2.7326e-02,  2.1040e-02,  5.1177e-02,
        -1.6845e-02, -5.5286e-03,  6.7419e-02,  4.4365e-02,  4.5820e-02,
         6.1403e-02, -5.7658e-02, -4.8749e-02,  2.7515e-02, -1.8603e-02,
        -9.0682e-03,  2.1988e-02,  3.8919e-03,  1.1455e-02,  3.3514e-02,
        -6.6622e-02, -2.3995e-02, -2.0236e-03, -5.3571e-02,  5.6582e-02,
         3.2215e-02, -3.0371e-02,  4.3290e-02,  5.6641e-02, -9.6752e-03,
        -3.8130e-02, -3.9769e-02,  5.4880e-02, -4.0181e-02,  6.0497e-02,
         2.2004e-02, -3.6008e-02,  5.7011e-02, -3.6185e-02, -4.6069e-02,
         4.9706e-02, -1.4811e-04, -1.2814e-03, -5.8053e-02, -2.4110e-04,
        -1.7320e-02, -3.2336e-02, -3.7065e-02,  3.3130e-02,  2.8830e-02,
        -4.8567e-02, -2.5615e-02, -3.3760e-02, -4.8306e-02,  3.3060e-02,
        -2.5002e-03, -1.1120e-02, -1.5049e-02, -6.6042e-03, -1.4224e-03,
        -1.4364e-02,  4.4235e-02,  1.3137e-02,  4.8219e-02, -1.7065e-02,
        -8.8567e-03,  1.8930e-02,  2.0236e-02,  4.1910e-02, -1.2133e-02,
         2.3274e-02, -1.2088e-03, -3.0048e-03, -2.7368e-03,  2.9872e-03,
         1.6533e-02,  1.0297e-04,  1.2063e-02, -2.0944e-02,  5.2429e-02,
        -6.3419e-03, -2.5821e-02,  1.8219e-02, -4.7459e-02, -8.8118e-03,
         2.0931e-02,  1.2256e-02,  5.6422e-02, -5.2963e-02, -1.4225e-02,
        -4.1725e-02, -1.2635e-02, -1.6370e-02, -3.5322e-02,  3.2834e-02,
         3.2200e-02,  5.0870e-02, -8.8370e-03,  2.9690e-02,  1.4120e-02,
         4.7113e-03, -4.6293e-02, -1.3170e-02, -1.9639e-03,  4.0837e-02,
        -3.0728e-03,  4.0069e-02, -4.6720e-02, -2.7275e-02,  1.9082e-02,
        -2.3107e-02,  5.1554e-02,  5.1857e-02, -1.6971e-03, -2.9233e-02,
         1.3324e-02,  4.0409e-02,  4.3414e-02,  4.6396e-02, -4.1529e-02,
        -2.5211e-02, -4.8813e-02, -3.5662e-02,  8.3524e-03,  1.8837e-02,
        -4.5978e-02, -2.0187e-02, -1.1608e-02,  1.0756e-02,  3.9379e-02,
         5.1061e-02, -1.4894e-02, -4.5571e-02, -3.6156e-02, -1.7745e-02,
         4.3540e-04,  1.6327e-02,  3.3253e-02,  4.1103e-02, -2.7094e-02,
         1.4486e-02, -5.0603e-02, -3.8474e-02,  4.7429e-02, -4.7354e-02,
        -5.5454e-03,  3.3364e-02,  5.2202e-02, -4.5872e-02,  1.8268e-02,
         2.1093e-02,  1.8102e-02,  1.1715e-02, -6.0486e-03, -1.6020e-02,
         6.0109e-03,  3.1534e-02,  2.2769e-02,  4.3445e-03,  2.3293e-02,
        -4.0830e-02,  4.8368e-02,  2.1247e-02,  2.0937e-02, -4.8933e-02,
        -2.5975e-02, -4.0736e-02, -1.0993e-02, -2.9057e-02,  4.9211e-02,
         9.3540e-03,  4.6021e-02,  4.0201e-02, -2.1261e-02,  3.3496e-02,
         1.4887e-02, -1.0905e-03, -5.1941e-02,  5.4293e-02,  4.5491e-02,
         1.3250e-02, -4.0212e-02,  2.5035e-02, -3.5583e-02,  8.6576e-03,
        -9.6510e-03, -7.6304e-03, -3.8516e-02, -2.9814e-02, -3.3777e-02,
         1.8833e-02,  3.7731e-02,  4.1745e-02, -2.5736e-03,  2.8523e-02,
         3.8081e-02, -3.2029e-02,  2.7897e-02,  4.7473e-02,  2.7408e-02,
         4.0799e-02, -1.8697e-02,  4.2089e-02, -2.0368e-03,  4.7906e-03,
        -4.9968e-02,  4.2360e-02, -4.1759e-02, -1.5119e-02, -2.7461e-02,
        -3.9849e-02, -1.5845e-02, -2.6329e-02, -3.8003e-02,  3.5836e-02,
         9.4314e-03,  6.4681e-02,  3.8248e-02, -4.0675e-02, -1.8204e-02,
        -4.2079e-02,  4.4703e-02,  2.6399e-02,  4.9707e-02,  1.9470e-02,
        -5.9120e-02,  3.0090e-02, -2.6178e-02,  2.2442e-02,  3.8750e-02,
         3.7864e-02,  4.8124e-02,  1.6500e-02,  4.2174e-02,  4.4674e-02,
         3.5991e-02,  2.4317e-02, -5.4032e-03,  1.7964e-02,  3.9615e-02,
         4.5989e-03, -7.2512e-03, -3.8968e-02, -2.4605e-02, -1.3024e-03,
        -3.5001e-02,  1.5965e-02,  4.0891e-02, -1.9178e-02,  4.7967e-02,
         1.2365e-02, -1.2323e-02, -3.1911e-02,  3.4021e-02,  3.4916e-02,
        -8.2566e-03,  3.8811e-02,  2.9886e-02, -4.1338e-02,  5.4943e-03,
        -5.2647e-02, -4.7111e-02,  2.4349e-02,  3.2516e-02,  3.1904e-02,
        -3.3147e-02, -2.9409e-02, -4.4717e-02,  5.1829e-02, -2.4442e-02,
         2.1630e-02,  5.0549e-02,  9.9566e-03, -1.0380e-02, -2.6240e-02,
        -2.2866e-04,  1.9031e-02], device='cuda:0'), tensor([[[ 3.0437e-02,  1.8597e-02, -2.5753e-02],
         [-9.4413e-03, -2.8146e-02, -3.0305e-02],
         [ 1.2671e-02,  5.1914e-03, -1.3675e-02],
         ...,
         [ 2.2770e-02,  8.6010e-03, -1.0650e-03],
         [ 2.4432e-02,  4.5532e-02,  3.1577e-02],
         [ 3.0599e-03,  4.7669e-03, -7.8698e-03]],

        [[ 3.1710e-02,  2.1304e-02,  3.7981e-02],
         [ 2.0316e-02,  1.1470e-02, -2.6196e-02],
         [-3.9551e-02, -4.2831e-03, -2.9238e-02],
         ...,
         [ 9.3895e-05,  3.0775e-02,  3.9928e-03],
         [-1.2702e-02, -6.0114e-03,  3.3222e-02],
         [-2.7862e-02,  6.0196e-03,  2.2236e-02]],

        [[ 1.1622e-02,  5.6555e-03, -1.6216e-02],
         [ 3.8000e-02,  1.7894e-02, -1.1573e-02],
         [ 1.8242e-03,  3.3364e-02, -1.9807e-02],
         ...,
         [ 1.8432e-02, -7.3016e-03,  8.4174e-03],
         [ 3.0653e-04,  3.1533e-02, -1.8726e-02],
         [-4.0624e-02,  1.1649e-02, -3.5707e-02]],

        ...,

        [[-3.3410e-02, -1.9376e-02, -1.0809e-02],
         [-3.2833e-03,  1.7441e-02,  6.2690e-03],
         [-2.3823e-02,  2.5681e-02,  2.6915e-02],
         ...,
         [-2.0795e-02, -3.5792e-02,  2.3696e-02],
         [-2.8048e-02,  6.5474e-03, -1.1106e-02],
         [-1.5573e-02,  1.2349e-02,  5.9035e-03]],

        [[ 1.5412e-02,  6.6954e-03, -2.3097e-02],
         [-1.5094e-02, -2.3492e-02, -1.2968e-02],
         [ 2.7732e-02,  3.0427e-02, -2.9334e-02],
         ...,
         [ 4.5314e-03,  1.0931e-02, -3.1448e-02],
         [ 7.1882e-03, -1.9396e-02,  3.1083e-02],
         [ 2.6055e-02, -1.9655e-02, -3.4075e-03]],

        [[-1.6932e-02,  1.1379e-02,  2.5510e-02],
         [-1.8001e-02, -1.2120e-02, -4.2275e-02],
         [ 4.1567e-03,  3.4908e-03,  2.3097e-02],
         ...,
         [-1.4986e-03, -3.6849e-02, -2.0183e-02],
         [ 2.1162e-04, -1.8875e-02, -8.1428e-03],
         [ 2.8016e-02, -5.2087e-02, -5.9703e-04]]], device='cuda:0'), tensor([ 1.1140e-02, -3.0018e-02, -1.9376e-02, -1.6994e-02,  2.0970e-02,
         2.8917e-02, -1.3416e-02,  2.9641e-02,  3.2198e-02, -2.8058e-03,
         2.3021e-02,  1.0211e-02, -3.0665e-02,  4.9192e-03,  2.3294e-02,
        -1.0954e-02,  3.4468e-02, -2.4385e-02, -3.5774e-02, -2.4451e-02,
         2.3603e-02,  1.0232e-02,  1.6112e-02,  7.5560e-03, -2.9942e-02,
         9.2335e-03,  3.2048e-02, -5.6063e-03, -1.1759e-02,  2.2731e-02,
         3.3028e-02, -2.7538e-02, -7.6598e-04,  2.3542e-02,  5.5254e-03,
         1.5651e-02,  3.2106e-02, -3.7576e-02, -1.5072e-02,  2.6071e-02,
         1.3581e-02,  1.0305e-02,  3.6662e-02, -1.7244e-02,  8.0129e-03,
         3.2756e-02,  1.4280e-02,  2.0410e-02, -9.8867e-03, -3.4434e-02,
        -1.1138e-02,  2.0663e-02,  2.7391e-02,  1.9101e-02, -1.9575e-02,
         8.9903e-03,  9.6781e-03, -2.0806e-02,  3.5569e-02, -5.8113e-04,
         2.1901e-02, -3.5840e-02,  1.8660e-02, -6.3996e-03,  1.1597e-02,
         3.2860e-03,  1.5099e-02, -1.8555e-02, -2.2944e-02, -1.7399e-02,
         2.7543e-02,  2.9387e-02,  1.1206e-02,  8.5590e-03,  2.3716e-02,
        -2.3400e-02, -1.9874e-02,  1.0158e-02, -3.0788e-02,  2.3341e-02,
        -1.7510e-02, -9.1949e-04, -1.9371e-02,  2.9784e-02,  2.8382e-02,
         1.9086e-02, -1.5193e-02, -1.6913e-02,  3.2127e-02,  2.0528e-02,
         1.9479e-02, -3.2585e-02,  2.4984e-02, -3.5975e-02, -1.1351e-02,
         9.1989e-03,  1.2577e-02,  1.3363e-02, -3.8900e-03,  3.6334e-02,
         3.3383e-02,  9.9667e-03, -1.4644e-02,  7.6022e-03,  7.5374e-03,
        -2.1793e-02, -3.2829e-02,  3.0327e-02,  2.2531e-03,  2.9178e-02,
         4.0496e-03,  2.8517e-02,  1.2805e-02,  3.6249e-03, -5.8687e-04,
         2.3968e-02,  2.4522e-02,  3.4927e-02, -1.1439e-02,  3.4357e-03,
        -2.6314e-02, -1.8272e-02,  5.1936e-03, -2.5007e-02, -2.0418e-02,
         2.4670e-02,  3.1976e-03, -3.2601e-02,  2.3580e-02,  1.2649e-02,
         2.7452e-02, -4.5270e-03,  1.0895e-02,  2.5310e-02, -3.7309e-02,
         2.3132e-02,  2.1961e-02, -2.5636e-02,  1.7717e-03,  2.0436e-02,
        -2.2343e-02,  6.6912e-03, -2.1897e-02,  2.2469e-02, -9.9377e-05,
        -2.8665e-02,  1.3511e-02, -2.7261e-02, -9.2067e-03,  1.5495e-02,
         6.6865e-03, -3.3755e-02,  2.2956e-02,  5.5318e-03, -2.8458e-02,
         3.1819e-02,  3.6303e-03,  3.3070e-03,  2.2601e-02,  9.4916e-03,
        -3.1822e-02,  5.9915e-03, -3.1706e-02, -1.8891e-03, -2.5765e-02,
         2.0888e-02,  3.8154e-02,  3.0949e-02, -1.7263e-02, -2.9343e-02,
         1.8951e-02, -2.0052e-02, -3.0437e-02,  1.0255e-02,  2.7096e-02,
        -1.6614e-02, -2.4883e-02,  2.1732e-02, -7.7944e-03, -7.1078e-03,
         1.6716e-02,  2.8694e-02,  2.1711e-02, -2.5824e-04,  7.4922e-03,
        -9.5451e-03, -1.5200e-02,  1.0258e-02,  1.2839e-02,  3.1984e-02,
        -3.2760e-02,  1.4827e-02,  1.0154e-02, -3.4627e-02, -1.9531e-02,
        -4.8361e-05, -3.7936e-03,  1.2149e-02, -9.8941e-03, -9.4729e-03,
        -4.0535e-03, -1.2322e-02,  2.6428e-02,  2.0458e-02, -7.4680e-03,
        -2.6248e-02, -1.8070e-02, -1.3579e-02,  2.3465e-02, -2.1895e-02,
        -2.7739e-02, -1.2057e-02,  1.9206e-02, -3.6305e-02,  2.5371e-02,
        -1.6270e-02,  1.4211e-02, -2.5281e-02,  2.3379e-02,  6.3047e-03,
        -2.2744e-02,  3.1231e-02, -3.0919e-02,  3.1860e-02, -7.8895e-03,
        -1.2696e-02,  6.2534e-03,  1.1695e-02, -2.0700e-02,  1.4600e-02,
         3.3706e-02,  3.2695e-02, -1.0944e-02, -2.0962e-02,  2.1745e-02,
         3.5515e-02, -3.1967e-02, -3.1524e-02, -6.7091e-04,  3.1116e-02,
         1.5894e-02, -1.5284e-02,  2.7952e-02, -2.5946e-02,  3.8060e-02,
         2.8232e-02,  2.6499e-02,  1.7948e-02,  1.3711e-02,  1.5955e-02,
        -2.3700e-02, -2.1053e-02,  2.2513e-02,  1.0230e-02,  2.2371e-02,
         1.8951e-02], device='cuda:0'), tensor([[[-0.0020, -0.0155, -0.0120, -0.0114, -0.0023],
         [ 0.0106, -0.0070, -0.0148,  0.0125, -0.0064],
         [ 0.0380, -0.0178, -0.0485,  0.0179,  0.0223],
         ...,
         [ 0.0109,  0.0067, -0.0051,  0.0148, -0.0151],
         [ 0.0177,  0.0332, -0.0166, -0.0150, -0.0073],
         [ 0.0032,  0.0002, -0.0031,  0.0084, -0.0140]],

        [[ 0.0176,  0.0250, -0.0172,  0.0091, -0.0088],
         [ 0.0086, -0.0123, -0.0133,  0.0091, -0.0060],
         [ 0.0034,  0.0247, -0.0577, -0.0055, -0.0120],
         ...,
         [ 0.0210,  0.0253,  0.0073,  0.0296, -0.0333],
         [-0.0321, -0.0207,  0.0054, -0.0025,  0.0226],
         [-0.0427, -0.0313,  0.0143,  0.0531, -0.0357]],

        [[-0.0167, -0.0172,  0.0378, -0.0257,  0.0226],
         [-0.0107,  0.0104, -0.0079,  0.0219,  0.0154],
         [-0.0246, -0.0217,  0.0125, -0.0326,  0.0145],
         ...,
         [-0.0186, -0.0128,  0.0564, -0.0021, -0.0155],
         [ 0.0258, -0.0100,  0.0222, -0.0121,  0.0198],
         [-0.0069, -0.0090, -0.0465,  0.0414,  0.0257]],

        ...,

        [[ 0.0074,  0.0068, -0.0257, -0.0539, -0.0115],
         [-0.0355, -0.0175, -0.0108, -0.0327, -0.0279],
         [-0.0319,  0.0073,  0.0357,  0.0378,  0.0396],
         ...,
         [ 0.0480,  0.0283, -0.0043, -0.0057,  0.0229],
         [ 0.0189,  0.0023, -0.0223, -0.0396,  0.0087],
         [-0.0034,  0.0093, -0.0076, -0.0278, -0.0503]],

        [[-0.0138,  0.0273,  0.0163, -0.0054, -0.0123],
         [-0.0381, -0.0429, -0.0073, -0.0041, -0.0462],
         [ 0.0098, -0.0191,  0.0156,  0.0425,  0.0101],
         ...,
         [ 0.0021, -0.0058, -0.0185, -0.0004, -0.0194],
         [ 0.0270, -0.0055,  0.0134, -0.0009, -0.0130],
         [-0.0154,  0.0427,  0.0019,  0.0304, -0.0227]],

        [[ 0.0142,  0.0198,  0.0154, -0.0158, -0.0029],
         [-0.0037, -0.0229,  0.0340, -0.0309, -0.0235],
         [ 0.0118,  0.0088,  0.0013, -0.0077,  0.0021],
         ...,
         [ 0.0116, -0.0187,  0.0058,  0.0232,  0.0421],
         [ 0.0035, -0.0228, -0.0199,  0.0270,  0.0045],
         [-0.0221,  0.0515,  0.0016,  0.0348,  0.0042]]], device='cuda:0'), tensor([-2.1076e-02,  6.3532e-03,  7.8388e-03, -1.4301e-02, -1.2679e-02,
        -6.2023e-03,  1.3735e-02, -2.5579e-02,  2.7883e-02,  1.5362e-02,
        -8.5361e-03, -1.5035e-02, -2.4750e-02, -2.2092e-02,  1.5275e-02,
         2.2331e-02, -1.9521e-02, -1.5603e-02,  1.5171e-02, -8.2422e-03,
        -1.3999e-02,  3.7892e-03,  1.6815e-02,  3.4446e-03,  2.6984e-02,
        -1.4776e-02,  2.3514e-02, -2.7729e-02,  1.8992e-03, -4.0305e-02,
         2.0986e-02, -8.5620e-03,  1.5349e-02, -1.2387e-02,  4.9622e-03,
         2.3565e-02, -1.3945e-02, -2.1523e-02, -8.7221e-03, -2.9407e-03,
         4.3302e-03,  3.0124e-02,  2.0138e-02, -1.6223e-02,  3.6500e-03,
        -1.6819e-02,  2.9683e-02,  1.5183e-02, -2.4443e-02, -4.2688e-02,
         5.4081e-03,  3.4599e-03, -2.7284e-02, -3.2702e-02,  2.7833e-02,
         7.6900e-03,  9.6529e-03, -2.7506e-02, -1.0597e-02,  2.1216e-02,
        -4.3770e-02,  3.1541e-02, -2.8025e-02, -3.6944e-02, -8.9943e-03,
         3.1338e-02, -2.8151e-02,  1.7700e-02, -2.8304e-02, -1.8765e-02,
         3.9781e-04,  1.1478e-03, -5.1132e-03,  2.0074e-02, -1.8639e-02,
        -7.4789e-03,  3.2314e-03, -4.4644e-03, -2.5790e-02, -2.8168e-02,
        -2.5733e-03, -2.3386e-02,  2.0247e-02, -2.7811e-02,  2.8244e-03,
        -2.6582e-02, -3.8190e-03, -2.2064e-02,  2.0478e-03,  1.7039e-02,
         1.0018e-02,  1.2687e-02, -7.3716e-03, -8.7425e-05,  2.7819e-02,
         3.6686e-03, -1.4243e-02,  1.4329e-02,  4.0272e-03,  2.5653e-02,
        -1.6278e-02, -3.4005e-02,  2.0962e-02, -8.7680e-03, -7.0714e-03,
         9.7048e-03,  1.2850e-03,  2.9981e-02, -1.3988e-02, -3.0026e-02,
        -1.8524e-02, -1.7301e-02, -7.1297e-03, -1.7298e-02,  1.3540e-02,
        -2.2552e-02,  3.0569e-03,  2.6773e-02,  9.0489e-03,  1.2677e-02,
        -2.6830e-02, -9.6027e-03, -2.1014e-03,  2.0629e-02,  3.0823e-02,
         4.0751e-03, -3.1686e-02,  2.0799e-02,  1.9379e-02,  2.3812e-02,
         1.5792e-02, -8.6762e-03,  1.3091e-02,  3.4973e-02, -2.8750e-02,
        -2.6346e-02,  1.9838e-02, -3.1444e-03, -7.1693e-03, -3.1152e-02,
        -1.3115e-02,  1.7753e-02, -1.1089e-03,  1.1645e-02,  1.7045e-03,
        -2.9468e-02,  9.0989e-03, -1.5765e-02,  1.0307e-02, -2.5658e-02,
         2.6176e-02,  2.8502e-02,  1.0249e-02,  1.6781e-02,  1.8647e-02,
        -5.5311e-03,  2.5483e-02,  1.5251e-02,  2.3075e-02,  2.6514e-02,
        -1.2226e-02, -1.9480e-02, -3.2426e-02,  3.7088e-03,  2.2921e-02,
        -4.1704e-03, -1.0015e-02,  1.6548e-02,  1.2539e-02, -6.7365e-03,
        -4.8214e-03, -4.9634e-03,  6.6718e-03,  2.3970e-02,  3.2035e-02,
         2.8365e-02, -1.7640e-02,  8.1735e-03,  1.1734e-03, -9.7235e-03,
         5.7298e-03, -2.0789e-02, -1.0175e-02,  8.8112e-03, -1.8068e-02,
         1.8886e-02, -4.5595e-03,  2.3813e-02, -5.7396e-02,  1.6587e-02,
         3.0283e-02, -4.6599e-03,  4.0616e-03, -1.1482e-02, -6.0033e-02,
        -1.3945e-02,  7.3416e-03, -5.4618e-03, -2.5197e-04,  3.3952e-03,
        -4.4096e-03, -1.4742e-02, -2.1489e-02,  1.9117e-02,  2.4605e-02,
        -4.2989e-03, -4.1915e-03,  1.3985e-02, -3.8294e-02,  2.5446e-02,
         2.3846e-02,  8.9322e-03,  6.6079e-03, -3.0398e-02,  2.2640e-02,
        -8.5753e-03,  4.8653e-03, -1.8449e-02,  1.2385e-02, -2.7176e-02,
         3.3694e-02, -9.5264e-03, -1.5998e-02, -2.6400e-02,  3.3920e-03,
         2.5727e-02, -1.4959e-02, -2.1004e-02,  7.4490e-03, -1.0345e-02,
        -2.6398e-02, -1.8534e-02, -2.9152e-02, -2.4190e-02, -1.3255e-03,
         2.6763e-02,  9.6663e-03,  2.1933e-02,  3.7289e-02,  1.7568e-02,
         2.0361e-02,  9.2883e-03,  1.6001e-02, -5.0583e-03,  2.2946e-02,
         7.7687e-03,  2.0563e-02,  4.5669e-03,  2.0642e-02,  9.2137e-03,
         2.2639e-02,  1.3195e-02,  2.2474e-02,  1.1229e-02, -7.9099e-03,
        -2.1906e-02, -1.0447e-02,  1.2664e-02,  1.4337e-02, -3.4713e-02,
        -1.2140e-02,  1.8505e-03, -2.0919e-02,  1.5295e-02, -2.0324e-02,
         8.5028e-03, -3.5219e-03,  1.2898e-02,  3.4245e-03,  1.2192e-03,
         3.9801e-03, -5.3914e-03, -3.5327e-02, -2.9270e-02,  1.7384e-02,
        -2.1580e-02,  5.9475e-03,  1.3342e-02, -1.0972e-02,  6.4990e-03,
        -2.3081e-02,  6.9211e-03,  1.0020e-02, -2.4589e-02, -2.4517e-02,
        -3.4620e-02, -3.1801e-02, -2.1030e-02,  2.1154e-02, -1.7700e-02,
        -3.1169e-02, -1.8013e-02, -1.3951e-02, -3.2559e-02,  9.9318e-04,
        -2.7241e-02,  2.8874e-03, -1.8512e-02,  3.0396e-02,  5.0840e-03,
        -1.0814e-02, -2.0989e-02, -2.9476e-04,  4.9275e-03,  4.1230e-03,
        -1.4652e-02, -1.2086e-02,  8.7539e-04, -2.9445e-02, -1.3332e-02,
         2.1030e-02, -8.0850e-04, -2.8336e-02, -5.0771e-03,  7.2108e-03,
        -2.2943e-02,  1.6305e-02,  3.0095e-02,  1.1990e-02, -4.2846e-03,
         7.0504e-04, -4.0354e-02,  1.1264e-02, -2.0863e-02, -4.4413e-03,
        -8.8575e-03, -2.2562e-03,  2.9745e-03, -7.9082e-03, -1.4781e-02,
        -2.5371e-02,  1.3873e-02, -1.4246e-02,  1.1752e-02, -2.9627e-02,
         1.0860e-02, -1.8389e-02, -1.0032e-02,  1.5557e-02,  4.5085e-03,
        -4.9434e-02,  1.0594e-02,  3.0711e-02, -7.2470e-03,  5.2065e-04,
        -4.4704e-02, -1.5140e-02, -5.1738e-02,  8.1530e-03, -1.7845e-02,
         1.1519e-02,  1.0118e-02,  1.2301e-02, -2.7281e-02,  5.5202e-03,
        -1.3360e-02,  2.4465e-02,  2.3054e-02, -1.8881e-02,  2.1872e-02,
        -2.7069e-02,  1.4680e-02,  9.0770e-03,  6.4269e-03, -7.9970e-03,
        -2.5822e-02, -1.2904e-02, -6.6882e-03, -1.1421e-02,  1.1859e-02,
        -2.7017e-03, -1.2643e-02,  1.8643e-03,  2.4771e-02,  5.2284e-03,
         3.5912e-03, -3.7625e-03, -1.3760e-02, -2.6613e-02, -1.6878e-02,
         8.7431e-03,  1.3519e-02, -9.2808e-03,  1.7719e-02, -8.3273e-03,
        -1.3614e-02, -1.9664e-02,  2.0175e-02, -1.2532e-02,  1.4934e-02,
        -2.2337e-02, -3.3418e-02, -1.0840e-03,  9.7649e-03, -2.7250e-02,
         2.1065e-02,  8.4667e-03,  2.7697e-02, -9.0402e-03, -4.5343e-03,
        -9.3247e-03,  1.0268e-02,  2.0456e-02,  2.3008e-02,  7.6375e-03,
        -4.2956e-02,  2.1262e-02, -8.8043e-03, -7.4410e-03, -1.4030e-02,
        -1.9266e-03, -2.7371e-02,  1.5655e-02, -1.8296e-02, -1.3380e-02,
        -1.4719e-02, -9.6024e-03,  2.9677e-02, -2.2370e-02,  3.0598e-02,
        -2.4365e-02, -1.3477e-02,  2.9275e-02, -1.6687e-02,  6.5898e-03,
         1.3347e-03,  2.4505e-02, -3.1134e-02, -2.3199e-02, -1.9555e-02,
        -5.7792e-03,  2.0886e-02, -3.7803e-02, -2.0053e-02, -4.9143e-02,
        -1.1352e-02, -9.4347e-03, -5.8189e-03, -1.4598e-02,  8.3711e-03,
         1.1746e-02,  6.7119e-03,  1.9196e-02,  2.8476e-02,  2.5022e-02,
        -4.7239e-02, -1.8406e-02, -2.7037e-02, -1.9109e-02, -1.5395e-02,
         1.1161e-02,  2.2769e-02, -2.5446e-02, -1.5312e-02,  6.7119e-03,
         1.7234e-03, -3.3052e-02, -5.1494e-03,  1.2326e-02,  7.8025e-03,
         1.0630e-02,  1.9865e-02, -4.6407e-03, -1.6088e-02, -2.7592e-02,
         5.3330e-03,  4.9328e-03, -2.9059e-02,  2.6866e-02,  9.1891e-03,
        -9.6649e-03, -1.8571e-02, -3.3177e-04,  2.2740e-02, -2.3965e-02,
         7.1143e-04,  8.5588e-03, -2.5913e-02,  2.0680e-02, -9.8584e-03,
        -1.9631e-02,  3.0630e-02,  9.7711e-03, -9.0679e-03, -1.8749e-02,
        -3.4712e-02, -5.7621e-03,  3.4803e-03, -6.2902e-03,  1.1018e-02,
         1.0862e-03, -4.5418e-03,  9.6403e-03,  2.6099e-02, -3.6836e-03,
         1.7054e-02, -9.9751e-03,  3.0983e-02,  3.4689e-03, -1.8731e-02,
         3.0714e-03,  3.0241e-02, -6.2857e-03, -3.0194e-02, -4.5818e-04,
         4.5114e-03, -1.3809e-02,  1.1211e-02,  9.4198e-03, -1.4968e-02,
        -1.0476e-02,  2.2263e-02], device='cuda:0'), tensor([0.9852, 0.9949, 0.9811, 0.9826, 0.9811, 0.9800, 0.9854, 0.9959, 1.0011,
        0.9920, 0.9733, 0.9680, 1.0018, 0.9792, 1.0040, 0.9851, 0.9811, 0.9705,
        0.9981, 0.9912, 0.9787, 0.9892, 0.9957, 0.9892, 0.9910, 0.9906, 0.9909,
        0.9880, 0.9938, 1.0449, 0.9891, 0.9843, 0.9853, 0.9970, 0.9906, 0.9958,
        0.9827, 0.9765, 1.0181, 0.9888, 0.9886, 0.9982, 0.9950, 0.9884, 0.9784,
        1.0187, 0.9942, 0.9794, 0.9722, 1.0598, 0.9885, 1.0039, 1.0021, 1.0023,
        0.9938, 1.0104, 0.9863, 0.9602, 0.9930, 0.9922, 1.0052, 0.9747, 1.0033,
        1.0678, 1.0150, 0.9978, 0.9895, 0.9843, 0.9742, 1.0107, 1.0273, 0.9978,
        1.0122, 0.9938, 1.0142, 1.0056, 1.0340, 0.9800, 0.9587, 1.0470, 0.9782,
        0.9828, 1.0017, 1.0201, 0.9851, 0.9727, 0.9933, 0.9946, 0.9946, 1.0648,
        0.9934, 0.9903, 0.9996, 0.9961, 0.9933, 1.0499, 1.0061, 0.9811, 0.9804,
        0.9883, 1.0628, 1.0355, 1.0004, 1.0172, 1.0461, 1.0495, 0.9836, 0.9900,
        0.9888, 0.9941, 1.0072, 0.9944, 1.0101, 1.0724, 1.0275, 1.0848, 1.0191,
        0.9886, 0.9823, 0.9890, 1.0010, 0.9988, 0.9651, 0.9840, 1.0090, 0.9958,
        0.9700, 0.9817, 1.0076, 0.9802, 1.0125, 0.9983, 0.9915, 0.9949, 1.0382,
        1.0876, 0.9749, 0.9868, 0.9908, 1.0315, 1.0933, 1.0481, 1.0321, 0.9742,
        0.9828, 1.0227, 1.0077, 1.0027, 0.9897, 0.9949, 0.9861, 0.9981, 0.9405,
        0.9893, 1.0169, 0.9942, 0.9821, 1.0039, 0.9699, 0.9773, 0.9748, 0.9845,
        0.9916, 0.9943, 0.9652, 0.9822, 0.9922, 1.0193, 1.0384, 1.0588, 0.9806,
        0.9818, 1.0180, 0.9778, 0.9922, 0.9968, 0.9906, 0.9823, 1.0144, 0.9944,
        1.0105, 0.9925, 1.0868, 0.9739, 1.0207, 0.9765, 0.9769, 0.9775, 1.0895,
        0.9824, 0.9930, 0.9827, 1.0874, 0.9680, 1.0855, 0.9744, 1.0232, 0.9847,
        0.9900, 1.0062, 0.9863, 0.9921, 0.9939, 0.9851, 0.9845, 1.0080, 1.0110,
        0.9892, 1.0594, 0.9788, 0.9864, 0.9914, 0.9799, 1.0288, 1.0080, 0.9789,
        1.0067, 1.0322, 0.9877, 1.0589, 1.0073, 0.9923, 0.9737, 0.9464, 0.9697,
        0.9872, 1.0008, 0.9775, 1.0059, 0.9966, 0.9579, 1.0199, 1.1213, 0.9745,
        1.0099, 1.0137, 0.9930, 1.0095, 1.0216, 1.0020, 0.9885, 0.9872, 1.0334,
        0.9715, 0.9923, 1.0356, 0.9809, 0.9774, 0.9833, 0.9985, 0.9640, 1.0187,
        0.9971, 0.9879, 1.0111, 0.9723, 0.9805, 0.9958, 1.0032, 0.9840, 0.9872,
        0.9954, 1.0425, 0.9807, 0.9766, 1.0195, 0.9923, 1.0036, 1.0148, 1.0495,
        0.9777, 1.0049, 0.9853, 1.0238, 0.9765, 0.9776, 0.9883, 1.0008, 1.0197,
        1.0099, 0.9745, 1.0428, 1.0283, 0.9787, 0.9767, 1.0175, 1.0041, 0.9797,
        0.9907, 0.9942, 1.1146, 0.9849, 0.9914, 0.9887, 0.9884, 0.9931, 1.0066,
        0.9526, 0.9943, 0.9867, 0.9826, 0.9914, 1.0059, 0.9971, 0.9911, 1.0761,
        0.9269, 0.9782, 1.0022, 0.9589, 0.9956, 0.9827, 1.0122, 0.9918, 0.9459,
        1.0005, 0.9958, 0.9972, 1.0117, 1.0052, 0.9835, 1.1004, 0.9881, 0.9849,
        1.0005, 0.9948, 0.9820, 0.9802, 1.0053, 1.0021, 0.9776, 0.9828, 0.9730,
        0.9901, 0.9873, 0.9805, 0.9959, 1.0065, 0.9741, 1.0050, 1.0703, 1.0119,
        0.9939, 0.9751, 1.0381, 1.0729, 0.9891, 1.0622, 0.9891, 1.0222, 1.0132,
        0.9868, 0.9863, 0.9820, 0.9851, 0.9914, 1.0117, 0.9981, 0.9901, 0.9888,
        0.9747, 0.9944, 0.9999, 0.9854, 0.9749, 0.9818, 1.0411, 1.0143, 0.9915,
        0.9764, 0.9900, 1.0072, 1.0457, 0.9869, 0.9812, 0.9971, 0.9767, 1.0337,
        0.9909, 0.9821, 0.9879, 1.0053, 1.0835, 0.9950, 0.9929, 1.0823, 0.9962,
        0.9894, 1.0227, 0.9983, 0.9901, 0.9994, 1.0179, 0.9920, 0.9660, 0.9972,
        0.9922, 0.9856, 0.9865, 1.0033, 0.9969, 0.9870, 0.9999, 0.9908, 0.9903,
        1.0169, 1.0026, 0.9988, 1.0362, 0.9901, 0.9934, 0.9855, 0.9953, 0.9971,
        0.9828, 1.0164, 0.9800, 0.9866, 1.0232, 0.9887, 0.9880, 0.9779, 0.9950,
        1.0008, 0.9925, 0.9835, 0.9813, 0.9869, 0.9869, 0.9924, 0.9856, 0.9906,
        1.0063, 1.0008, 1.0245, 1.0028, 0.9979, 0.9946, 0.9889, 1.0323, 1.0010,
        0.9902, 0.9887, 0.9902, 0.9906, 1.0088, 0.9920, 1.0200, 0.9959, 1.0068,
        0.9724, 1.0147, 0.9806, 0.9929, 0.9925, 0.9891, 1.0473, 1.0257, 0.9820,
        1.0127, 0.9896, 0.9707, 0.9928, 0.9871, 0.9827, 0.9918, 1.0265, 1.0027,
        0.9795, 0.9809, 1.0017, 0.9887, 1.0420, 0.9783, 0.9860, 1.0776, 0.9929,
        1.0176, 0.9981, 0.9831, 1.0001, 1.0061, 1.0004, 0.9827, 0.9763, 1.0318,
        0.9888, 0.9805, 0.9981, 0.9745, 1.0076, 0.9872, 0.9971, 0.9770, 0.9777,
        0.9750, 0.9820, 0.9900, 0.9867, 0.9876, 0.9869, 0.9945, 1.0149, 1.0400,
        1.0028, 0.9677, 1.0004, 0.9751, 0.9917, 1.0430, 1.0386, 0.9740],
       device='cuda:0'), tensor([-1.0588e-02, -9.9139e-03,  5.0280e-03,  4.8583e-05, -1.0516e-02,
        -4.8569e-03, -2.3306e-03, -2.4793e-03,  3.8786e-03,  2.5109e-03,
        -1.8569e-03, -6.6715e-04, -1.3933e-02, -3.8489e-03,  3.2904e-03,
         2.4029e-03,  4.6235e-03, -2.5064e-02, -2.9413e-03, -5.6381e-03,
        -9.1627e-03,  4.1996e-03, -1.3352e-03, -8.6557e-04, -6.1169e-03,
         1.6345e-03, -1.1100e-02, -1.5467e-02, -7.5114e-04, -1.6781e-02,
        -2.4165e-03,  4.1634e-04, -9.0169e-03, -8.5691e-04, -1.6144e-03,
        -5.2644e-03, -9.2059e-03, -8.0212e-03,  3.5568e-03,  1.2492e-04,
        -5.1067e-03, -1.8629e-03,  6.8003e-03,  6.0076e-03, -7.5836e-03,
        -5.0931e-03,  1.2245e-02, -1.0129e-02,  8.1570e-03, -2.6081e-02,
         4.7425e-03, -5.8094e-03, -2.5564e-03,  7.6346e-03, -7.2454e-03,
        -2.3626e-02,  2.7725e-03, -1.2571e-02, -3.7716e-04,  2.4018e-03,
        -1.4217e-02, -7.5842e-03,  2.8278e-03,  1.9641e-03, -3.8927e-02,
         1.7179e-03, -4.6157e-04, -2.7715e-03, -4.6267e-03, -1.1149e-02,
         2.6405e-03,  5.1506e-03,  1.4188e-02,  7.7162e-03,  2.7452e-03,
         4.3716e-03,  6.5146e-03,  1.0753e-02,  9.6780e-03, -1.9524e-03,
         6.3336e-04,  5.1030e-03,  6.8975e-03, -6.9211e-03, -5.3409e-03,
        -4.7986e-04, -4.0593e-03,  2.7341e-03, -1.0248e-03, -3.7871e-02,
        -7.5468e-03, -6.9543e-03,  3.9765e-03,  1.2935e-03, -1.4143e-03,
        -1.3236e-02,  4.5457e-03, -6.4178e-03,  9.7114e-03,  3.9012e-05,
        -4.4462e-02, -3.5190e-03, -2.2082e-04, -3.5176e-03, -1.9359e-02,
        -3.3021e-02, -5.8777e-03,  3.8723e-03,  1.1504e-04, -2.3884e-03,
        -1.0408e-02,  8.3144e-03,  2.0998e-03, -3.9967e-02,  5.1066e-03,
        -5.4802e-02,  6.0662e-03, -4.7700e-04, -9.0573e-03,  2.6726e-03,
         1.1492e-02, -1.9082e-03, -7.6977e-03,  1.1414e-03,  8.5325e-03,
         9.4114e-05,  2.5380e-03, -8.3565e-03, -4.5044e-02, -1.4165e-02,
         1.5858e-02,  1.6377e-03, -6.0629e-04,  1.7652e-02, -8.0699e-03,
        -6.5083e-02,  1.6422e-03, -1.6223e-03, -2.4848e-02,  1.3936e-03,
        -4.8273e-02, -2.6583e-02, -2.2210e-02, -6.5189e-03,  8.7962e-03,
         5.1946e-03, -1.8347e-02,  5.6858e-03, -9.1201e-03,  8.2909e-03,
         4.2997e-03,  2.4678e-03, -2.3626e-02,  2.1571e-03, -4.1640e-03,
         6.4579e-03,  9.9031e-03,  2.8575e-03,  5.5043e-03, -2.4401e-03,
         1.0807e-02,  5.0470e-03, -1.0799e-02, -1.5975e-03, -6.9316e-03,
         3.6135e-03,  4.7812e-03,  1.1874e-02, -2.6080e-02,  4.9122e-03,
         1.8696e-03,  1.1458e-03, -1.6793e-03, -1.2224e-02, -1.8316e-04,
        -2.0342e-03, -2.5867e-03, -3.3827e-03,  3.1322e-03, -8.2422e-03,
         7.9201e-03, -2.9273e-04, -3.0598e-02,  3.8422e-03, -1.3879e-04,
        -3.5087e-03,  7.6970e-03,  1.1565e-02, -9.0342e-02,  2.6003e-03,
         1.1104e-03, -5.8285e-03, -3.7086e-02, -1.4913e-02, -9.4645e-02,
        -1.1831e-02,  2.1813e-03,  2.5267e-03,  6.6083e-03,  9.0550e-03,
         1.2391e-02, -1.6921e-03, -6.6809e-03, -5.7118e-03, -2.8811e-02,
        -9.2138e-03,  1.5677e-02, -1.0543e-02, -2.7359e-02, -4.8826e-03,
         2.7809e-03, -4.5157e-03, -1.3302e-03,  1.5242e-03, -2.5616e-02,
        -6.9557e-03, -8.2991e-03, -8.5425e-04,  9.0244e-03, -3.2687e-02,
         6.6108e-03, -2.1723e-02, -1.0271e-03, -2.0226e-02,  1.1052e-02,
         1.7677e-02, -5.3466e-03,  2.1948e-03,  1.1007e-02,  6.0945e-03,
        -3.4373e-02, -2.2398e-02, -7.7442e-02, -1.1636e-03,  1.5044e-02,
        -3.9349e-03,  7.1458e-03,  8.6773e-03,  3.5351e-02, -9.7085e-03,
        -1.0019e-04, -1.1269e-02, -8.8789e-04, -9.3689e-03,  5.6828e-03,
        -3.2287e-02, -1.6967e-02, -9.9852e-03,  2.3791e-02,  9.1946e-03,
        -2.5850e-02,  1.7886e-03,  2.7572e-02, -2.8008e-02,  3.7611e-02,
        -8.4010e-03,  1.1113e-02,  8.4547e-03,  7.9490e-04, -2.9486e-02,
        -5.1528e-03,  4.1534e-03, -2.6799e-02, -4.2900e-03,  7.4256e-03,
        -1.2215e-02, -1.3544e-02,  4.8795e-03,  7.9157e-03, -8.9699e-02,
        -2.4801e-03,  3.5870e-03, -3.7950e-02,  6.4966e-03,  2.2366e-02,
         6.9894e-03,  1.8577e-03,  1.9733e-02,  1.3352e-02,  8.7502e-03,
         5.4690e-03, -2.2583e-02, -3.5109e-02, -5.7125e-03,  3.1323e-03,
        -1.5531e-02, -7.2715e-05,  1.7121e-04,  7.9215e-03,  1.8341e-03,
        -6.3016e-02, -7.9349e-03, -1.5283e-02, -3.5961e-02, -7.0326e-04,
        -4.1184e-02, -2.8112e-02, -1.2529e-03,  5.5955e-03, -4.6222e-03,
         1.2259e-02, -9.6640e-04,  1.1082e-02,  7.3267e-03,  4.6279e-03,
        -9.5985e-02, -5.7309e-04,  2.9141e-03, -2.7737e-02,  5.8685e-03,
         7.9414e-03, -7.9385e-03,  2.3168e-02, -6.3187e-04,  5.9688e-03,
         3.4398e-03, -2.0297e-03,  3.7166e-03, -1.6723e-02,  4.6753e-03,
        -5.2925e-03, -8.7389e-02, -5.7012e-03, -1.1578e-02,  6.0596e-03,
         1.1070e-05, -8.6527e-05,  7.2266e-03,  6.7410e-03,  5.1203e-03,
         7.2411e-03, -2.6728e-03, -1.6181e-02, -2.0500e-03,  1.8238e-05,
        -9.0696e-04, -1.8074e-03,  6.7926e-03, -4.7204e-03,  2.1291e-03,
        -3.6329e-02, -3.5879e-03,  1.6980e-03, -1.9130e-03,  1.8532e-03,
        -6.1248e-02,  3.7889e-03, -5.6867e-02,  2.6762e-03,  7.5804e-03,
        -8.2212e-03, -1.7682e-03, -1.4191e-03, -9.7057e-03, -1.4046e-03,
         1.0320e-02,  6.6004e-03, -1.9668e-03, -2.6591e-03,  8.9501e-05,
        -4.4913e-03, -3.6546e-03, -4.5731e-02, -5.3682e-04,  1.3697e-03,
         5.0852e-03, -1.3302e-02,  1.1927e-02, -9.2694e-04,  1.5289e-02,
        -1.2742e-03,  2.0529e-03, -7.1235e-02, -1.5381e-05, -1.0933e-02,
        -9.3023e-04, -5.2752e-03, -1.3119e-03, -9.8521e-03,  1.0748e-02,
         7.9638e-03,  6.2087e-03, -3.9844e-02, -5.3405e-03, -1.0765e-02,
        -1.7178e-02, -8.2832e-03, -4.2102e-03, -7.8818e-03, -2.4785e-03,
        -4.9835e-04,  1.1603e-02, -2.4339e-02, -1.3729e-03, -1.6204e-02,
         1.9803e-04,  2.1615e-03,  1.3695e-04, -2.4250e-03,  9.1040e-03,
         5.1709e-04, -6.6990e-04,  1.0646e-03,  3.2251e-03, -1.3675e-04,
        -5.9866e-03, -5.5065e-04, -1.0839e-03, -4.4264e-03,  6.1554e-03,
        -7.8357e-03, -8.1652e-03, -5.9425e-03, -3.9490e-03, -5.7607e-03,
         5.8063e-03, -1.1503e-02,  1.6082e-04, -2.8006e-03, -3.1348e-03,
         5.4054e-03, -8.0247e-03,  1.9917e-02,  4.1949e-03, -8.6388e-04,
        -9.7220e-03, -1.7716e-03, -5.8956e-04, -1.1592e-02, -6.3640e-03,
        -1.2624e-02, -5.5555e-03, -1.4111e-02,  1.1514e-03, -2.3896e-02,
        -5.6885e-03, -1.5665e-03,  3.9833e-03, -8.2358e-03, -9.9351e-03,
        -9.1086e-03,  1.2474e-03,  2.1045e-04, -4.8067e-03, -1.1111e-02,
        -1.3622e-02, -1.2875e-03, -1.7873e-02, -1.1857e-02,  2.1962e-02,
         2.9985e-03,  5.5913e-04,  1.4431e-02,  9.9470e-04, -5.8515e-03,
        -5.8760e-03, -3.7558e-02,  2.6812e-03, -7.1054e-03, -1.4479e-02,
        -1.9739e-03, -2.3928e-04,  1.2488e-03,  9.5438e-03, -4.2696e-03,
         7.3248e-05, -2.7750e-03, -4.4129e-05,  1.8375e-03, -1.7125e-03,
         1.1459e-02, -3.5838e-02, -6.5905e-02,  9.3452e-04, -2.4686e-04,
        -2.1252e-02, -5.5484e-03, -1.6247e-02,  5.0674e-03,  2.8634e-03,
         2.4542e-03,  3.5289e-03, -3.8896e-02, -8.0163e-03,  3.4209e-03,
         8.6413e-03, -8.3382e-03, -1.5047e-03, -7.4715e-04,  5.7322e-03,
         9.4179e-04,  3.4807e-03, -2.2274e-03, -6.8176e-03,  9.0957e-03,
        -2.6872e-03,  3.5657e-03,  1.0245e-02, -6.3230e-03,  6.6379e-03,
        -9.9951e-03, -1.6684e-04,  1.2121e-02, -4.4077e-02, -6.9670e-04,
         9.0665e-03,  1.0858e-02, -1.1851e-02, -3.7422e-03, -3.6357e-02,
        -3.2704e-02,  5.1417e-03], device='cuda:0'), tensor([[[-0.0062,  0.0091,  0.0348, -0.0061, -0.0042],
         [ 0.0045, -0.0022,  0.0145, -0.0090,  0.0208],
         [-0.0125,  0.0079,  0.0174, -0.0157, -0.0132],
         ...,
         [-0.0009, -0.0191, -0.0378, -0.0370, -0.0080],
         [-0.0010, -0.0140, -0.0138,  0.0087, -0.0277],
         [-0.0260, -0.0024, -0.0181, -0.0057, -0.0018]],

        [[ 0.0048,  0.0214, -0.0066, -0.0012, -0.0063],
         [-0.0037,  0.0053,  0.0159,  0.0045, -0.0027],
         [ 0.0324, -0.0093, -0.0021, -0.0027, -0.0027],
         ...,
         [ 0.0187,  0.0287,  0.0123, -0.0136, -0.0173],
         [-0.0027, -0.0046, -0.0109,  0.0012,  0.0203],
         [-0.0260, -0.0126, -0.0039,  0.0189,  0.0003]],

        [[-0.0268, -0.0164, -0.0103,  0.0144, -0.0037],
         [ 0.0039,  0.0038,  0.0121,  0.0059,  0.0145],
         [-0.0067, -0.0018, -0.0125,  0.0006,  0.0072],
         ...,
         [-0.0346, -0.0445, -0.0269, -0.0100,  0.0013],
         [ 0.0284,  0.0136, -0.0106, -0.0092,  0.0187],
         [ 0.0182, -0.0073, -0.0292,  0.0053, -0.0037]],

        ...,

        [[-0.0051, -0.0059, -0.0057,  0.0169,  0.0018],
         [ 0.0033,  0.0072, -0.0178, -0.0019, -0.0090],
         [-0.0117,  0.0015, -0.0056, -0.0041, -0.0103],
         ...,
         [-0.0195, -0.0033, -0.0186, -0.0185,  0.0068],
         [ 0.0034,  0.0011,  0.0276,  0.0019, -0.0072],
         [-0.0066, -0.0032, -0.0044,  0.0005,  0.0132]],

        [[ 0.0031, -0.0171, -0.0286,  0.0059, -0.0223],
         [ 0.0114,  0.0123,  0.0178, -0.0110,  0.0091],
         [-0.0029,  0.0110,  0.0005, -0.0011, -0.0140],
         ...,
         [-0.0226, -0.0388, -0.0103, -0.0043, -0.0140],
         [-0.0188,  0.0109,  0.0004,  0.0113,  0.0103],
         [-0.0022, -0.0073,  0.0135, -0.0230, -0.0329]],

        [[ 0.0210, -0.0159, -0.0050,  0.0050, -0.0037],
         [-0.0162,  0.0112, -0.0135,  0.0012,  0.0004],
         [-0.0122, -0.0158,  0.0037,  0.0277,  0.0009],
         ...,
         [ 0.0029,  0.0060,  0.0260,  0.0090,  0.0102],
         [ 0.0172,  0.0063,  0.0056,  0.0070,  0.0029],
         [ 0.0219, -0.0225,  0.0071,  0.0342,  0.0041]]], device='cuda:0'), tensor([ 2.2394e-02,  6.2805e-03, -1.7383e-02,  1.9205e-02,  6.6230e-03,
        -3.1538e-03,  1.4454e-02,  1.5662e-02,  2.1980e-03, -6.0858e-05,
        -1.0291e-02,  7.1488e-03, -1.9146e-02, -5.7562e-04, -1.6370e-02,
        -1.6593e-02, -8.5435e-03, -1.5307e-02,  1.0053e-02,  5.4151e-03,
         6.9731e-03, -1.2712e-02, -7.9452e-03, -3.8781e-04, -1.3441e-02,
         1.5391e-02,  4.9672e-03,  1.4767e-02,  1.8761e-02, -7.4052e-02,
         9.4763e-03, -6.7317e-03,  8.7386e-03,  1.4070e-02, -1.6219e-02,
         2.7936e-03, -2.4531e-03, -1.2865e-02,  8.4206e-03,  1.4017e-02,
        -9.7699e-02,  1.1788e-02, -1.3655e-02, -1.2355e-02,  4.4705e-03,
        -1.2698e-02,  3.6068e-03,  1.6970e-02,  1.2052e-02,  1.4387e-03,
        -1.0342e-02,  8.0885e-03,  3.8672e-03, -1.6466e-02, -2.0610e-03,
         4.3521e-03, -1.4628e-02, -7.8271e-03, -3.3711e-03,  1.9670e-02,
         4.7239e-03, -1.2478e-02, -1.9496e-02, -1.6812e-02,  8.8008e-03,
        -5.3083e-03,  4.6016e-03,  1.2689e-02, -4.9687e-03, -6.3141e-03,
         1.1648e-02, -4.0070e-02,  6.0053e-04, -5.0837e-03, -1.8460e-02,
        -6.9587e-03,  1.4409e-02,  2.2352e-02, -8.6466e-03,  1.5411e-02,
        -7.6554e-03, -1.4469e-02,  3.4301e-03,  1.9698e-02,  1.5641e-02,
         2.2047e-02, -8.9022e-03, -4.6032e-03, -1.9670e-02, -4.5677e-03,
         1.0784e-02,  2.6911e-03,  2.5891e-03,  1.4244e-02, -1.2137e-02,
        -9.4327e-02, -2.3491e-02,  6.7193e-03,  1.1399e-02, -7.7643e-03,
         1.8252e-02,  1.1544e-02, -5.2624e-03, -1.6872e-02,  1.5800e-02,
         1.8913e-02,  2.2707e-02,  2.0365e-02,  1.9744e-02,  1.7570e-02,
        -1.0065e-02, -8.9767e-02, -9.6185e-03,  7.5850e-03,  1.7192e-02,
         8.4002e-03,  5.9364e-03,  1.2983e-02,  4.7760e-03,  1.4505e-02,
        -1.4086e-02, -9.3539e-03, -7.3437e-03, -9.4633e-03, -2.2316e-04,
        -7.2375e-03,  6.7402e-03, -2.0683e-02,  7.3354e-03, -1.1460e-02,
         1.0498e-03,  3.3910e-03, -1.2650e-02, -4.0845e-03, -1.3163e-02,
        -1.5459e-02, -8.3280e-04,  8.7432e-03,  1.6505e-03, -2.2254e-03,
        -2.9573e-02,  1.5599e-02,  5.7612e-03,  8.7628e-03, -1.0106e-01,
         5.4444e-03,  4.4934e-03,  7.3788e-03,  1.4144e-02,  4.7848e-03,
        -1.0040e-01,  5.2050e-03,  3.5393e-03,  1.0940e-02,  1.3859e-02,
         1.9605e-02, -5.8280e-02,  6.7492e-03,  4.9852e-03, -1.9790e-02,
         1.3562e-02, -6.3119e-04, -1.1587e-02,  6.0498e-03,  1.4585e-02,
        -3.1300e-03, -9.6429e-03, -1.1588e-02, -1.6144e-02, -5.7368e-04,
        -8.3253e-03,  2.6648e-03,  4.6374e-03,  1.6095e-02, -1.5304e-02,
        -4.6251e-03,  9.1937e-03, -1.3437e-02, -1.4589e-02, -1.5658e-02,
         1.2999e-02, -1.1505e-02,  9.2736e-03, -1.5487e-02,  1.1515e-03,
         8.4021e-03,  5.2734e-03,  1.2897e-02,  8.9503e-03,  7.1031e-03,
        -1.7260e-03, -8.4614e-03,  1.9757e-02, -7.8856e-04,  1.6507e-02,
        -1.8296e-02, -1.1888e-02,  2.5438e-03,  9.7128e-03,  1.0775e-02,
         5.9360e-03,  6.2315e-03, -6.8071e-03, -1.3031e-02, -8.5829e-03,
        -1.0387e-02, -5.7766e-03, -2.1978e-03, -1.5550e-02,  5.3528e-03,
         1.9628e-02,  1.4811e-02,  1.3709e-02, -1.8244e-02,  7.4991e-03,
        -1.4387e-02, -6.8491e-03, -9.1650e-03,  2.8500e-03,  6.8491e-03,
         6.4107e-03, -8.1932e-02, -9.6007e-02,  9.3095e-03, -8.9372e-03,
         1.5741e-02, -8.8933e-03,  3.9315e-03,  1.9105e-02,  1.1600e-02,
        -3.0664e-03, -1.3529e-02, -2.3922e-03,  1.6836e-02, -1.7107e-02,
         5.1663e-04, -9.4564e-02, -1.0280e-02, -2.0832e-02, -7.3911e-03,
        -1.5219e-02, -1.0633e-02,  1.5707e-02,  6.7955e-03,  4.1932e-03,
        -8.4715e-03,  1.6118e-02, -5.6144e-03, -1.3260e-02, -6.5493e-03,
        -1.1707e-03,  9.2139e-03, -8.2143e-03, -5.6512e-03, -4.8792e-03,
         9.2946e-03, -2.4093e-02,  9.4630e-03,  1.6859e-02, -9.4529e-03,
         1.1423e-02, -4.5329e-03,  1.1691e-02,  5.0351e-03, -1.6383e-02,
         1.2477e-03,  7.7090e-03, -1.1666e-02,  1.5783e-02, -5.0879e-03,
         1.4350e-03,  1.2237e-02,  7.3357e-03,  1.2818e-02,  1.7554e-02,
         2.2094e-02, -1.1472e-02, -7.9948e-03, -1.0979e-02, -1.3830e-02,
         7.5751e-03,  1.2885e-03, -8.7844e-03, -7.8993e-02,  2.4504e-02,
        -1.8273e-02, -1.1622e-01,  5.5735e-03,  7.8543e-03,  8.7225e-03,
         1.2754e-02,  1.5436e-02, -2.6532e-03,  1.1001e-02,  8.2439e-03,
        -1.3846e-02,  1.5106e-02,  1.1745e-02,  9.8252e-03, -4.3409e-03,
         1.5242e-02, -3.4676e-03,  2.0663e-02,  3.3540e-03,  2.1032e-02,
        -1.5979e-02, -6.6809e-03,  2.0809e-03, -1.0542e-02,  4.1696e-03,
         7.9488e-03,  8.2002e-03, -1.4298e-02, -1.1302e-02,  6.3564e-03,
         3.6852e-03, -1.1917e-05, -2.8060e-04,  9.7877e-03, -2.1231e-02,
         1.5252e-03,  1.9703e-02, -4.8087e-03,  1.6313e-02, -2.3720e-02,
         8.8372e-03, -7.6712e-03, -1.6621e-02, -1.3427e-02,  4.9120e-03,
         1.7392e-02, -1.1519e-02, -9.7943e-03,  2.0744e-02, -3.4548e-03,
        -1.0998e-02,  1.2173e-02, -1.1916e-02, -1.4555e-02, -1.2920e-01,
         1.2022e-02,  6.8344e-03,  9.6827e-03,  1.8798e-02,  1.6902e-02,
         8.1060e-03,  2.6009e-03,  2.0948e-02,  1.3187e-02, -2.7228e-03,
         4.6028e-03, -1.6804e-02, -8.7486e-03,  5.1311e-03, -1.0460e-03,
         2.1599e-02,  1.3240e-03, -4.1828e-03,  1.0184e-02,  7.9667e-03,
         1.5214e-02,  1.7791e-02, -9.3187e-03,  2.1026e-02, -1.0520e-02,
        -1.5810e-02,  1.9354e-03, -1.2701e-02, -1.1481e-02, -5.8128e-03,
         2.0310e-02, -8.7535e-03,  1.1754e-02,  1.3005e-03, -8.3746e-03,
        -1.6078e-02,  1.0745e-02,  5.9752e-04,  2.3851e-03,  3.2830e-03,
         7.4172e-03, -1.0771e-02,  1.2029e-03, -1.1498e-02,  1.8248e-02,
         1.5952e-02, -1.5083e-02,  1.4820e-02,  5.3290e-03, -1.4812e-02,
        -1.3792e-02,  2.4905e-03, -1.4639e-02, -1.6832e-03,  2.1988e-02,
        -7.8024e-02, -1.2183e-02,  9.2733e-03, -5.9148e-04, -9.4026e-05,
        -5.6251e-03, -7.4248e-03,  1.8742e-02, -2.9811e-03, -1.4555e-02,
        -1.5425e-02,  1.5946e-02, -2.1168e-02,  1.7343e-02, -2.9747e-03,
        -9.0422e-03, -1.0347e-02, -5.6558e-03, -1.4854e-02, -5.6877e-03,
         3.4963e-03,  1.2996e-03, -7.9314e-03,  1.1345e-02,  3.8850e-03,
        -7.4567e-02, -1.3094e-02, -1.9067e-02, -1.7882e-03,  1.4450e-02,
         4.1234e-03, -1.0434e-02,  3.9356e-03, -1.0655e-06,  2.2243e-02,
         1.1120e-02,  1.5132e-02,  8.6862e-03, -2.9807e-03, -1.8485e-03,
         1.2730e-02,  7.8518e-03, -4.6579e-03, -8.4600e-03,  4.2226e-03,
         1.0430e-02, -8.4949e-03,  4.5663e-03,  1.1372e-02, -1.3167e-01,
         1.1272e-03, -4.7004e-03, -1.6680e-03,  8.3646e-03, -1.2415e-02,
         5.2074e-03, -1.2033e-02,  1.2358e-02, -1.2670e-02, -4.4002e-03,
        -9.4532e-03,  3.0395e-04,  1.3073e-02,  1.3944e-02, -6.0975e-03,
        -1.7411e-02,  8.6251e-03,  1.1647e-02,  1.2861e-02,  2.1210e-02,
         1.0510e-02, -8.2835e-03,  1.3000e-02, -1.3230e-02,  1.9185e-02,
         8.0035e-03,  5.6108e-03, -2.0494e-02,  4.1312e-03, -1.5087e-02,
         1.7903e-02, -8.6751e-02, -1.3599e-02, -1.3999e-02,  2.3662e-03,
         4.4432e-03, -1.2095e-01, -3.5534e-03,  3.1411e-03,  1.9128e-02,
        -1.3472e-02, -9.8202e-03,  4.4191e-03, -1.2247e-02,  3.3526e-03,
         1.8336e-02, -3.7349e-03,  1.0150e-03,  1.8068e-02,  1.6313e-02,
        -4.5467e-03, -8.0328e-03,  1.1084e-02,  1.6091e-03,  1.7084e-02,
         1.7366e-02,  1.2139e-02, -1.2779e-02,  1.0779e-03,  7.3980e-03,
        -1.2114e-02,  7.6447e-03, -1.4579e-02,  1.4071e-03,  2.2958e-02,
         5.7843e-03, -2.4442e-03], device='cuda:0'), tensor([0.9762, 0.9885, 0.9874, 0.9796, 0.9772, 0.9684, 0.9811, 0.9838, 0.9793,
        0.9873, 0.9893, 0.9687, 0.9842, 0.9867, 0.9761, 0.9922, 0.9647, 0.9634,
        0.9736, 0.9561, 0.9804, 0.9850, 0.9866, 0.9777, 0.9816, 0.9829, 0.9806,
        0.9795, 0.9657, 0.9905, 0.9859, 0.9483, 0.9858, 0.9830, 0.9741, 0.9897,
        0.9857, 0.9592, 0.9717, 0.9848, 0.9986, 0.9748, 0.9870, 0.9946, 0.9806,
        1.0055, 0.9843, 0.9705, 0.9891, 0.9727, 1.0307, 0.9804, 1.0066, 0.9839,
        0.9438, 0.9885, 0.9537, 0.9465, 0.9890, 0.9679, 0.9865, 0.9821, 0.9904,
        0.9838, 0.9746, 0.9642, 0.9794, 0.9767, 0.9800, 0.9524, 0.9722, 1.0000,
        0.9825, 0.9897, 0.9484, 0.9813, 0.9928, 0.9874, 0.9842, 0.9899, 0.9723,
        0.9881, 0.9654, 0.9807, 0.9802, 0.9890, 0.9777, 0.9816, 0.9670, 0.9636,
        0.9737, 0.9903, 0.9559, 0.9749, 0.9832, 0.9955, 0.9883, 0.9902, 0.9886,
        0.9772, 0.9799, 0.9810, 0.9790, 0.9570, 0.9814, 0.9590, 0.9819, 0.9815,
        0.9779, 0.9883, 0.9749, 0.9950, 0.9792, 0.9876, 0.9713, 0.9740, 0.9857,
        0.9863, 0.9891, 0.9623, 0.9979, 0.9787, 1.0226, 0.9378, 0.9616, 0.9830,
        0.9808, 0.9802, 0.9787, 0.9844, 0.9838, 0.9739, 0.9828, 0.9895, 0.9813,
        0.9724, 0.9825, 0.9846, 0.9633, 0.9822, 0.9951, 0.9857, 0.9778, 0.9805,
        0.9893, 0.9758, 0.9764, 0.9641, 0.9862, 0.9825, 0.9958, 0.9754, 0.9793,
        0.9838, 0.9764, 0.9769, 1.0007, 0.9391, 0.9860, 0.9622, 0.9714, 0.9856,
        0.9794, 0.9588, 0.9887, 0.9592, 0.9805, 0.9833, 0.9756, 0.9599, 0.9807,
        0.9858, 0.9785, 0.9854, 0.9868, 0.9863, 0.9784, 0.9755, 0.9690, 1.0141,
        0.9823, 0.9790, 1.0006, 0.9781, 0.9888, 0.9860, 1.0005, 0.9816, 0.9724,
        0.9793, 0.9830, 0.9801, 0.9825, 0.9833, 0.9818, 0.9807, 0.9746, 0.9333,
        0.9781, 0.9837, 0.9799, 0.9891, 0.9697, 0.9813, 0.9790, 0.9883, 0.9807,
        0.9941, 0.9795, 0.9684, 0.9918, 0.9867, 0.9807, 0.9833, 0.9852, 0.9979,
        0.9814, 0.9824, 0.9653, 0.9797, 0.9689, 1.0005, 0.9947, 0.9744, 0.9823,
        0.9804, 0.9871, 0.9808, 0.9893, 0.9843, 0.9849, 0.9552, 0.9557, 0.9846,
        0.9836, 0.9633, 0.9937, 0.9783, 0.9820, 0.9753, 0.9806, 0.9846, 0.9502,
        0.9813, 0.9756, 0.9859, 0.9862, 0.9714, 0.9840, 0.9858, 0.9777, 0.9774,
        0.9784, 0.9818, 0.9828, 0.9847, 1.0252, 0.9623, 0.9641, 0.9889, 0.9593,
        0.9631, 0.9847, 0.9824, 0.9722, 0.9532, 0.9781, 0.9778, 0.9730, 0.9756,
        0.9827, 0.9764, 0.9845, 0.9757, 0.9693, 0.9775, 0.9773, 0.9852, 0.9730,
        0.9656, 0.9768, 0.9847, 0.9665, 0.9985, 0.9782, 1.0210, 0.9903, 0.9822,
        0.9784, 0.9770, 1.0131, 0.9748, 0.9268, 0.9872, 0.9825, 0.9879, 0.9929,
        0.9403, 0.9755, 0.9824, 0.9852, 0.9954, 0.9741, 0.9615, 0.9787, 0.9793,
        0.9299, 0.9715, 0.9850, 0.9664, 0.9594, 0.9577, 0.9816, 0.9619, 0.9337,
        0.9941, 0.9866, 0.9965, 0.9830, 0.9895, 0.9816, 0.9777, 0.9596, 0.9523,
        0.9985, 0.9847, 0.9865, 0.9921, 0.9775, 0.9849, 0.9832, 0.9716, 0.9788,
        0.9792, 0.9766, 0.9693, 0.9644, 0.9965, 0.9861, 0.9896, 0.9652, 0.9806,
        0.9841, 0.9814, 0.9717, 0.9886, 0.9772, 1.0068, 0.9231, 0.9522, 0.9982,
        0.9757, 0.9635, 0.9635, 0.9773, 0.9834, 0.9908, 1.0193, 0.9826, 0.9801,
        0.9865, 0.9412, 0.9827, 0.9727, 0.9856, 0.9646, 1.0159, 0.9705, 0.9927,
        0.9803, 0.9733, 0.9773, 0.9836, 0.9910, 0.9739, 0.9926, 0.9854, 0.9734,
        0.9465, 0.9771, 1.0060, 0.9890, 0.9627, 0.9830, 0.9805, 0.9927, 0.9826,
        0.9710, 0.9845, 0.9632, 0.9785, 0.9734, 0.9796, 1.0020, 0.9786, 1.0009,
        1.0077, 0.9666, 0.9830, 0.9871, 0.9803, 0.9823, 0.9825, 0.9539, 0.9918,
        0.9843, 0.9782, 0.9899, 0.9702, 0.9689, 0.9791, 0.9851, 0.9721, 0.9797,
        0.9911, 0.9656, 0.9805, 0.9737, 0.9753, 0.9824, 0.9936, 0.9843, 1.0086,
        0.9799, 0.9506, 0.9709, 1.0236, 0.9610, 0.9778, 0.9699, 0.9861, 0.9735,
        0.9965, 0.9870, 0.9780, 0.9792, 0.9927, 0.9843, 0.9805, 0.9773, 0.9817,
        0.9578, 0.9881, 0.9891, 0.9938, 0.9553, 0.9889, 0.9827, 0.9897, 0.9955,
        0.9767, 0.9764, 0.9235, 0.9902, 0.9809, 1.0139, 0.9829, 0.9792, 0.9878,
        0.9482, 0.9587, 0.9830, 0.9374, 0.9173, 0.9773, 0.9788, 0.9719, 0.9761,
        0.9749, 0.9807, 0.9823, 0.9950, 0.9819, 0.9894, 0.9480, 0.9936, 0.9995,
        0.9721, 0.9864, 0.9745, 0.9490, 0.9998, 0.9727, 0.9847, 0.9596, 0.9882,
        0.9689, 0.9446, 0.9786, 0.9727, 0.9498, 0.9757, 0.9843, 0.9626, 0.9767,
        0.9700, 1.0021, 0.9737, 0.9802, 0.9742, 0.9820, 0.9843, 0.9813, 0.9603,
        0.9773, 0.9841, 0.9366, 0.9805, 0.9826, 0.9593, 0.9868, 0.9868],
       device='cuda:0'), tensor([-0.0149, -0.0053, -0.0099, -0.0084, -0.0154, -0.0109, -0.0081, -0.0086,
        -0.0092, -0.0122, -0.0092, -0.0098, -0.0072, -0.0026, -0.0065, -0.0099,
        -0.0089, -0.0090, -0.0106, -0.0087, -0.0096, -0.0034, -0.0076, -0.0102,
        -0.0090, -0.0055, -0.0054, -0.0106, -0.0071,  0.0103, -0.0094, -0.0077,
        -0.0066, -0.0094, -0.0113, -0.0080, -0.0105, -0.0065, -0.0123, -0.0088,
        -0.0016, -0.0069, -0.0076,  0.0037, -0.0090, -0.0042, -0.0105, -0.0064,
        -0.0051, -0.0137, -0.0176, -0.0109, -0.0171, -0.0101, -0.0063, -0.0069,
        -0.0097, -0.0085, -0.0036, -0.0124, -0.0037, -0.0100, -0.0065, -0.0038,
        -0.0078, -0.0077, -0.0069, -0.0058, -0.0096, -0.0069, -0.0122, -0.0035,
        -0.0083, -0.0105, -0.0097, -0.0063, -0.0083, -0.0093, -0.0075, -0.0030,
        -0.0106, -0.0103, -0.0136, -0.0102, -0.0099, -0.0068, -0.0084, -0.0034,
        -0.0138, -0.0095, -0.0087, -0.0076, -0.0101, -0.0101, -0.0070,  0.0020,
        -0.0076, -0.0085, -0.0090, -0.0079, -0.0052, -0.0098, -0.0080, -0.0082,
        -0.0078, -0.0104, -0.0093, -0.0111, -0.0106, -0.0076, -0.0118,  0.0017,
        -0.0088, -0.0107, -0.0072, -0.0073, -0.0041, -0.0079, -0.0094, -0.0088,
        -0.0144, -0.0125, -0.0182, -0.0081, -0.0096, -0.0063, -0.0107, -0.0119,
        -0.0039, -0.0062, -0.0137, -0.0063, -0.0078, -0.0103, -0.0059, -0.0080,
        -0.0138, -0.0037, -0.0088, -0.0007, -0.0069, -0.0045, -0.0057, -0.0103,
         0.0083, -0.0132, -0.0038, -0.0082, -0.0035, -0.0110, -0.0020, -0.0079,
        -0.0112, -0.0069, -0.0094, -0.0056, -0.0035, -0.0093, -0.0020, -0.0109,
        -0.0093, -0.0026, -0.0087, -0.0124, -0.0101, -0.0110, -0.0093, -0.0115,
        -0.0088, -0.0102, -0.0082, -0.0095, -0.0038, -0.0096, -0.0024, -0.0111,
        -0.0108, -0.0112, -0.0112, -0.0150, -0.0151, -0.0086, -0.0004, -0.0050,
        -0.0048, -0.0110, -0.0103, -0.0041, -0.0090, -0.0034, -0.0069, -0.0054,
        -0.0056, -0.0094, -0.0048, -0.0132, -0.0088, -0.0084, -0.0143, -0.0114,
        -0.0081, -0.0096, -0.0066, -0.0113, -0.0083, -0.0052, -0.0052, -0.0072,
        -0.0048, -0.0089, -0.0044, -0.0110, -0.0078, -0.0095, -0.0068, -0.0035,
        -0.0100, -0.0063, -0.0054, -0.0048, -0.0097,  0.0011,  0.0023, -0.0093,
        -0.0117, -0.0119, -0.0015, -0.0119, -0.0021, -0.0110, -0.0060, -0.0079,
        -0.0089, -0.0122, -0.0088, -0.0092,  0.0042, -0.0070, -0.0084, -0.0096,
        -0.0086, -0.0053, -0.0121, -0.0111, -0.0102, -0.0141, -0.0061, -0.0106,
        -0.0041, -0.0082, -0.0047, -0.0079, -0.0048, -0.0092, -0.0063, -0.0105,
        -0.0215, -0.0159, -0.0095, -0.0082, -0.0102, -0.0093, -0.0096, -0.0085,
        -0.0035, -0.0146, -0.0085, -0.0114, -0.0073, -0.0111, -0.0100, -0.0144,
        -0.0102, -0.0083, -0.0121, -0.0088, -0.0052, -0.0094, -0.0134, -0.0112,
        -0.0089, -0.0080, -0.0103, -0.0033, -0.0086, -0.0116,  0.0100, -0.0143,
        -0.0078, -0.0050, -0.0126, -0.0098, -0.0045, -0.0028, -0.0116, -0.0086,
        -0.0125, -0.0083, -0.0067, -0.0105, -0.0079, -0.0045, -0.0040, -0.0064,
        -0.0099, -0.0108, -0.0102, -0.0091, -0.0086, -0.0100, -0.0079, -0.0055,
        -0.0072, -0.0093, -0.0140, -0.0121, -0.0084, -0.0021, -0.0104, -0.0076,
        -0.0125, -0.0099, -0.0090, -0.0098, -0.0039, -0.0081, -0.0031, -0.0035,
        -0.0037, -0.0041, -0.0052, -0.0103, -0.0046, -0.0096, -0.0063, -0.0089,
        -0.0050, -0.0065, -0.0066,  0.0037, -0.0066, -0.0106, -0.0104, -0.0066,
        -0.0079, -0.0027, -0.0066, -0.0104, -0.0067, -0.0078, -0.0091, -0.0043,
        -0.0093, -0.0090, -0.0114, -0.0074, -0.0051, -0.0137, -0.0080, -0.0068,
        -0.0109, -0.0107, -0.0081, -0.0114, -0.0078, -0.0054, -0.0147, -0.0041,
        -0.0079, -0.0076, -0.0129, -0.0068, -0.0070, -0.0065, -0.0073, -0.0050,
        -0.0059, -0.0133, -0.0075, -0.0038, -0.0137, -0.0065, -0.0088, -0.0054,
        -0.0146, -0.0027, -0.0090, -0.0157, -0.0049, -0.0081, -0.0151, -0.0068,
        -0.0094, -0.0114, -0.0112, -0.0069, -0.0125, -0.0105, -0.0082, -0.0041,
        -0.0101, -0.0087, -0.0095, -0.0104, -0.0068, -0.0096, -0.0083, -0.0084,
        -0.0113, -0.0126, -0.0099, -0.0121, -0.0056, -0.0040, -0.0073, -0.0110,
        -0.0137, -0.0091, -0.0147, -0.0087,  0.0025, -0.0041, -0.0100, -0.0052,
        -0.0127, -0.0111, -0.0143, -0.0059, -0.0113, -0.0087, -0.0069, -0.0082,
        -0.0130, -0.0087, -0.0073, -0.0064, -0.0110, -0.0094, -0.0074, -0.0055,
        -0.0074, -0.0059, -0.0084, -0.0058,  0.0033, -0.0112, -0.0079, -0.0115,
        -0.0077, -0.0110, -0.0083, -0.0114, -0.0116, -0.0106, -0.0112, -0.0143,
        -0.0136, -0.0127, -0.0060, -0.0085, -0.0034, -0.0097, -0.0079, -0.0115,
        -0.0050, -0.0144, -0.0126, -0.0055, -0.0071, -0.0121, -0.0088, -0.0021,
        -0.0117, -0.0090, -0.0137, -0.0067, -0.0054, -0.0083, -0.0127, -0.0049,
        -0.0087, -0.0057, -0.0099, -0.0046, -0.0101, -0.0073, -0.0070, -0.0120,
        -0.0134, -0.0096, -0.0111, -0.0141, -0.0054, -0.0083, -0.0098, -0.0057,
        -0.0029, -0.0062, -0.0109, -0.0066, -0.0069, -0.0070, -0.0085, -0.0084,
        -0.0099, -0.0121, -0.0067, -0.0115, -0.0064, -0.0059, -0.0050, -0.0080],
       device='cuda:0'), tensor([[ 0.0344,  0.0173,  0.0456,  ...,  0.0329,  0.0222,  0.0486],
        [-0.0224, -0.0474,  0.0148,  ...,  0.0228,  0.0272, -0.0217],
        [-0.0433,  0.0316,  0.0287,  ..., -0.0365,  0.0340,  0.0178],
        ...,
        [-0.0324,  0.0354, -0.0051,  ...,  0.0075,  0.0243, -0.0089],
        [-0.0237,  0.0255,  0.0067,  ..., -0.0099,  0.0144, -0.0060],
        [-0.0389,  0.0219, -0.0009,  ...,  0.0293, -0.0100, -0.0317]],
       device='cuda:0'), tensor([ 0.0125,  0.0407,  0.0441,  ...,  0.0159, -0.0430,  0.0242],
       device='cuda:0'), tensor([[[-0.0054],
         [-0.0127],
         [ 0.0475],
         ...,
         [-0.0101],
         [-0.0486],
         [ 0.0246]],

        [[-0.0859],
         [ 0.0361],
         [ 0.0079],
         ...,
         [-0.0700],
         [-0.0344],
         [ 0.0064]],

        [[-0.0121],
         [ 0.0278],
         [-0.0141],
         ...,
         [ 0.0651],
         [-0.0222],
         [ 0.0317]],

        ...,

        [[-0.0633],
         [-0.0529],
         [-0.0461],
         ...,
         [ 0.0509],
         [-0.0709],
         [ 0.0111]],

        [[-0.0689],
         [-0.0338],
         [-0.0383],
         ...,
         [-0.0487],
         [ 0.0617],
         [ 0.0567]],

        [[-0.0148],
         [ 0.0192],
         [-0.0002],
         ...,
         [-0.0050],
         [-0.0348],
         [ 0.0569]]], device='cuda:0'), tensor([-0.0486, -0.0623,  0.0122, -0.0234, -0.0228,  0.0218, -0.0576,  0.0373,
         0.0168, -0.0050,  0.0041,  0.0121, -0.0378,  0.0023, -0.0394, -0.0360,
         0.0105, -0.0455,  0.0484,  0.0348,  0.0084,  0.0234, -0.0214, -0.0405,
        -0.0417, -0.0474,  0.0187, -0.0271, -0.0084, -0.0427,  0.0132,  0.0460,
        -0.0255,  0.0140, -0.0609, -0.0589,  0.0402, -0.0554, -0.0178,  0.0151,
         0.0424,  0.0423,  0.0337,  0.0234, -0.0006,  0.0171,  0.0495, -0.0245,
         0.0126, -0.0118, -0.0397, -0.0431, -0.0010,  0.0323,  0.0428, -0.0268,
         0.0218,  0.0469,  0.0407, -0.0425,  0.0383, -0.0524, -0.0681,  0.0518,
         0.0174,  0.0241, -0.0573, -0.0281, -0.0404, -0.0515,  0.0465,  0.0207,
        -0.0111,  0.0379,  0.0229,  0.0389, -0.0306,  0.0034,  0.0050, -0.0158,
         0.0197, -0.0367, -0.0491, -0.0712, -0.0439,  0.0152,  0.0414,  0.0367,
        -0.0708, -0.0460, -0.0465, -0.0592, -0.0251,  0.0170, -0.0610,  0.0544,
        -0.0347, -0.0483,  0.0382, -0.0104, -0.0278,  0.0379, -0.0282, -0.0459,
        -0.0085,  0.0189, -0.0359,  0.0484,  0.0412, -0.0407,  0.0435, -0.0006,
         0.0413, -0.0442,  0.0291, -0.0010, -0.0622,  0.0342, -0.0526, -0.0360,
        -0.0193,  0.0053, -0.0316,  0.0297,  0.0316, -0.0628, -0.0076, -0.0060,
         0.0011,  0.0213,  0.0323, -0.0147, -0.0480, -0.0010, -0.0466, -0.0269,
        -0.0023,  0.0385, -0.0199, -0.0604, -0.0509,  0.0513, -0.0251, -0.0621,
        -0.0204, -0.0457, -0.0008, -0.0077, -0.0583,  0.0299, -0.0463,  0.0564,
        -0.0349,  0.0293, -0.0405, -0.0268, -0.0040, -0.0540, -0.0349, -0.0088,
        -0.0032, -0.0181,  0.0421,  0.0429,  0.0425,  0.0203,  0.0365,  0.0262,
        -0.0286, -0.0624, -0.0560, -0.0434, -0.0429,  0.0276, -0.0024, -0.0625,
         0.0282,  0.0286, -0.0181, -0.0362, -0.0143,  0.0228, -0.0031, -0.0226,
        -0.0416,  0.0169, -0.0657, -0.0045,  0.0522, -0.0254,  0.0058,  0.0163,
        -0.0454,  0.0103,  0.0080,  0.0071, -0.0290,  0.0269, -0.0690,  0.0247,
         0.0502,  0.0388,  0.0289, -0.0558,  0.0045, -0.0386,  0.0233,  0.0493,
        -0.0244, -0.0526, -0.0555,  0.0406,  0.0253, -0.0704,  0.0302, -0.0007,
        -0.0416, -0.0633,  0.0489,  0.0055, -0.0365,  0.0214, -0.0084, -0.0417,
        -0.0717, -0.0381, -0.0517,  0.0042, -0.0455, -0.0512, -0.0059, -0.0472,
         0.0132,  0.0320, -0.0212,  0.0495, -0.0149, -0.0051,  0.0110, -0.0165,
         0.0370, -0.0480, -0.0229, -0.0221, -0.0456, -0.0185, -0.0191,  0.0427,
        -0.0078,  0.0500,  0.0490, -0.0161, -0.0354, -0.0702,  0.0262,  0.0186,
         0.0504, -0.0090, -0.0505, -0.0157,  0.0166,  0.0368, -0.0025,  0.0110,
        -0.0257, -0.0101,  0.0324, -0.0330, -0.0615, -0.0471, -0.0367,  0.0475,
        -0.0673, -0.0502, -0.0654,  0.0062,  0.0557,  0.0145, -0.0465, -0.0364,
         0.0128,  0.0498,  0.0133,  0.0460,  0.0083,  0.0267, -0.0190,  0.0418,
         0.0488, -0.0195,  0.0405,  0.0355, -0.0065, -0.0538, -0.0055, -0.0627,
         0.0161, -0.0245, -0.0061, -0.0199,  0.0229,  0.0415, -0.0461, -0.0616,
        -0.0343, -0.0638, -0.0330,  0.0141,  0.0234, -0.0636,  0.0166,  0.0391,
        -0.0304, -0.0181, -0.0521, -0.0594,  0.0520, -0.0634,  0.0042,  0.0408,
        -0.0331, -0.0313, -0.0077,  0.0046, -0.0644, -0.0524, -0.0439,  0.0261,
        -0.0617, -0.0591, -0.0122, -0.0473,  0.0459,  0.0482,  0.0477, -0.0462,
        -0.0218, -0.0300, -0.0476,  0.0130,  0.0256, -0.0055, -0.0381, -0.0242,
         0.0270, -0.0296,  0.0111,  0.0475, -0.0101,  0.0221,  0.0560,  0.0569,
        -0.0455, -0.0621,  0.0207, -0.0063,  0.0457,  0.0207,  0.0359, -0.0449,
         0.0466,  0.0405,  0.0243, -0.0207,  0.0093,  0.0126, -0.0009, -0.0449,
         0.0121, -0.0420, -0.0659,  0.0326, -0.0190, -0.0647,  0.0455, -0.0211,
        -0.0347,  0.0229, -0.0193, -0.0088, -0.0355, -0.0435, -0.0177,  0.0415,
        -0.0317,  0.0367,  0.0311, -0.0310, -0.0544,  0.0112, -0.0471,  0.0107,
         0.0030, -0.0371, -0.0454,  0.0138,  0.0261,  0.0101,  0.0062,  0.0324,
        -0.0310,  0.0157, -0.0228, -0.0565, -0.0249, -0.0187, -0.0369, -0.0003,
        -0.0534, -0.0007, -0.0618, -0.0331,  0.0496,  0.0521, -0.0388, -0.0088,
         0.0198, -0.0320, -0.0202,  0.0046, -0.0513, -0.0490,  0.0522, -0.0267,
         0.0518, -0.0620,  0.0030,  0.0422,  0.0309,  0.0477,  0.0531, -0.0098,
         0.0097,  0.0513,  0.0308, -0.0428,  0.0488, -0.0197,  0.0489,  0.0042,
         0.0214, -0.0516, -0.0522,  0.0555,  0.0418,  0.0464,  0.0522,  0.0420,
        -0.0091, -0.0491, -0.0079,  0.0352, -0.0255,  0.0502,  0.0046,  0.0273,
        -0.0368,  0.0113, -0.0012,  0.0089, -0.0192, -0.0651,  0.0424,  0.0494,
         0.0311, -0.0384, -0.0237, -0.0540,  0.0568, -0.0490,  0.0059,  0.0318,
        -0.0421,  0.0405,  0.0446, -0.0620, -0.0278, -0.0497, -0.0046,  0.0119,
        -0.0231,  0.0129, -0.0365, -0.0040,  0.0284, -0.0377, -0.0534,  0.0105,
         0.0240, -0.0171, -0.0055,  0.0214, -0.0290, -0.0603,  0.0010, -0.0430,
        -0.0626, -0.0351, -0.0652, -0.0268,  0.0045, -0.0533,  0.0030,  0.0374,
         0.0355,  0.0478, -0.0536, -0.0298, -0.0528, -0.0095,  0.0339,  0.0459],
       device='cuda:0'), tensor([[[-8.8133e-03, -2.0217e-02, -1.0681e-02, -1.2991e-02,  1.6326e-02],
         [ 5.8297e-03, -1.4181e-02,  1.7534e-02, -1.1824e-02, -3.6367e-03],
         [ 3.4501e-02,  2.7156e-02,  2.1661e-02, -8.0208e-03,  2.5171e-02],
         ...,
         [ 3.6081e-04, -1.4208e-02, -1.1298e-02,  1.2862e-02,  7.2750e-05],
         [-3.1799e-02, -1.6687e-02, -2.6043e-03,  3.0472e-03, -3.5955e-03],
         [ 2.8569e-03,  2.2726e-02, -8.0783e-03, -5.4790e-03, -4.2594e-02]],

        [[-1.6712e-02, -4.6472e-03, -2.3943e-02, -2.4998e-02, -8.3915e-03],
         [ 1.3108e-03, -1.5404e-02, -1.0211e-05, -9.4493e-03,  3.4221e-03],
         [ 1.5161e-02,  1.4124e-03,  1.6464e-02,  2.8088e-03, -1.0413e-02],
         ...,
         [ 5.6925e-03, -2.6044e-02, -1.2590e-02, -2.4306e-02, -3.0754e-02],
         [-1.5439e-02, -1.4582e-02,  1.1609e-02, -4.3157e-04, -1.2534e-02],
         [ 2.0593e-03,  1.9814e-02,  1.8433e-02,  1.2168e-04, -8.8495e-03]],

        [[-1.2041e-02, -7.9248e-03, -4.0888e-02,  7.0651e-03, -1.3651e-02],
         [-1.1130e-02,  1.5872e-02,  5.2112e-03, -1.1417e-02, -9.2791e-04],
         [-7.4631e-03,  2.1260e-02,  1.2950e-02,  2.0359e-02, -4.1470e-04],
         ...,
         [ 6.3947e-03, -1.0028e-02, -2.4721e-02, -1.5261e-02,  6.8755e-04],
         [ 3.1544e-02,  5.6371e-04,  3.8803e-02,  1.1979e-03,  8.0030e-03],
         [-2.6619e-03, -1.9983e-03,  2.1637e-02, -6.1460e-03, -1.1942e-02]],

        ...,

        [[-1.3176e-02,  1.4284e-02,  6.8841e-03, -1.5101e-03,  1.3892e-02],
         [ 1.3565e-03, -2.2531e-02, -3.3419e-02, -6.0623e-03, -4.2837e-03],
         [-5.1552e-03,  2.3575e-02,  5.2138e-03,  1.8108e-02, -2.0597e-02],
         ...,
         [-2.4010e-02, -1.5591e-02, -2.6164e-03, -2.2224e-02,  7.8284e-03],
         [-1.8341e-02, -3.7723e-03, -2.4810e-03, -5.3046e-03,  1.0329e-02],
         [-1.3236e-02, -4.5464e-03, -2.6481e-02,  1.4478e-02,  1.2666e-02]],

        [[-1.9890e-03, -7.7043e-03,  2.8242e-02, -1.7122e-02, -2.3458e-02],
         [-1.6719e-02, -6.7773e-03, -1.3856e-02,  3.9112e-02,  1.0945e-02],
         [ 1.9018e-02, -2.0992e-02,  6.6092e-03, -2.4382e-02,  3.3969e-03],
         ...,
         [ 2.5892e-03,  1.8160e-02, -6.2972e-03,  1.2697e-02, -3.9468e-03],
         [ 1.3604e-03,  7.4872e-03,  1.2736e-02,  2.9765e-02,  2.8599e-02],
         [-7.5479e-04,  1.2509e-02,  1.8471e-02,  9.5562e-03, -1.5706e-03]],

        [[-7.6399e-03, -2.9268e-02, -1.5238e-02, -2.4100e-02, -2.4127e-02],
         [ 9.8063e-04, -6.7055e-03,  1.3393e-02, -2.1456e-02,  2.8799e-03],
         [ 1.2617e-02, -1.3368e-02,  2.6127e-03,  1.3495e-02,  1.6121e-03],
         ...,
         [-3.1668e-02, -1.8252e-02,  4.9607e-03, -1.1892e-02, -8.0417e-03],
         [-1.0385e-02, -6.1367e-03,  1.1879e-02,  4.7735e-03,  9.4663e-03],
         [-2.9657e-03, -3.7531e-03,  2.5529e-02, -1.5025e-02, -6.0080e-03]]],
       device='cuda:0'), tensor([-4.6190e-03,  1.3631e-02,  1.1075e-03, -1.1261e-02, -3.7539e-02,
        -6.4038e-03, -9.6468e-03,  2.3035e-02, -1.0709e-02, -1.6191e-02,
         1.2449e-02, -6.3714e-03,  3.8705e-03, -4.7948e-03, -1.3651e-02,
        -4.5677e-03, -9.4238e-04,  9.8807e-03,  1.3836e-02, -1.1390e-02,
        -4.5995e-03,  1.9865e-03,  1.8146e-02,  1.2923e-02, -1.9233e-02,
         1.5995e-02,  2.3261e-02,  4.2063e-03,  1.0301e-02,  8.3886e-03,
         1.5823e-02,  1.9809e-02, -1.1880e-02, -1.0295e-02,  2.5116e-03,
        -1.3996e-02,  1.3247e-02, -6.0332e-03,  7.8327e-03, -1.9010e-04,
        -1.6234e-02, -6.6232e-03,  1.3133e-03,  1.3218e-02, -1.1255e-04,
         5.1905e-03, -2.3201e-03,  6.8192e-03, -3.1143e-04,  8.5046e-03,
        -1.6245e-02,  2.2788e-02,  3.8286e-03, -8.2679e-03,  3.9413e-03,
         2.4543e-02, -1.8617e-02, -7.1635e-03,  2.9289e-02,  1.4569e-03,
        -1.1523e-02,  4.1893e-03, -1.8913e-02, -2.7527e-04,  2.3627e-02,
         4.6901e-03, -2.2989e-02, -7.4250e-03, -7.9913e-03, -5.4072e-03,
         2.0133e-02,  1.4912e-02, -1.7373e-02, -2.3402e-02,  1.4729e-02,
         9.8695e-03, -7.1432e-03,  8.3576e-03,  2.2582e-02, -3.4154e-03,
         8.8191e-03, -1.3460e-02, -5.5200e-02, -7.2352e-03, -4.5895e-04,
         1.2258e-02, -1.1688e-02,  8.2852e-03,  3.7836e-03, -8.8688e-03,
        -1.2536e-02,  1.5489e-02,  8.6592e-03, -1.4054e-02,  2.1755e-02,
         2.4272e-02,  2.6549e-03,  7.0221e-03,  1.1367e-03,  1.1701e-02,
         9.0612e-03, -6.6418e-03, -1.1778e-02,  9.9613e-04,  3.1925e-02,
        -2.2350e-02,  9.0084e-03,  4.6119e-03,  1.4961e-02,  7.3489e-03,
         1.7571e-03, -8.1868e-03,  1.6185e-03, -2.2776e-02, -1.8440e-02,
         1.7542e-02,  1.9352e-02,  1.0438e-02,  2.7530e-03,  1.5645e-02,
         2.3839e-02,  1.1919e-02,  8.2398e-03, -1.2527e-02,  7.6316e-04,
         7.7610e-03,  7.8624e-03,  2.0930e-02, -1.2576e-02,  1.5728e-03,
         5.0878e-03, -2.7908e-03, -2.7049e-02,  1.5747e-02, -4.8243e-03,
         1.5324e-02, -4.4419e-02, -3.4171e-03, -9.1058e-03,  1.6171e-03,
         1.0730e-03, -9.7790e-03, -1.0728e-02, -3.4755e-03, -8.6173e-03,
        -1.6158e-02, -1.3874e-02,  8.2523e-03, -6.1602e-03, -3.5339e-03,
        -1.4674e-02,  3.7141e-03,  7.9041e-03, -2.4053e-03,  5.6201e-03,
         3.9209e-03, -1.3378e-02, -6.1302e-04,  8.4198e-03,  3.8621e-03,
         1.1425e-04, -1.8016e-02,  5.9377e-03,  1.7107e-02,  9.9332e-03,
         3.8921e-03,  5.1090e-03, -4.4138e-03, -3.3559e-03, -1.6646e-02,
         1.1745e-02,  1.2666e-02,  5.1771e-04, -1.6089e-02,  4.6844e-03,
        -2.0132e-02,  1.0774e-02,  3.1565e-03,  3.9915e-03, -9.7801e-04,
        -3.4090e-05, -6.4591e-03, -4.2519e-03,  2.9916e-03,  1.3237e-04,
        -1.9323e-02, -3.7971e-02, -3.8337e-03, -1.1495e-02,  7.0452e-03,
        -1.7511e-02,  3.3241e-03, -1.0712e-02,  1.4928e-02, -9.6319e-03,
        -1.5712e-02, -1.6137e-02, -6.5739e-03,  1.1112e-02, -2.3442e-02,
         1.0853e-02,  2.6651e-03, -1.2653e-03,  1.7123e-02,  2.0240e-02,
        -2.6268e-02,  7.0920e-03,  1.2068e-02, -6.1072e-03,  2.1868e-02,
        -5.2997e-03, -1.5768e-02, -8.5555e-03, -5.3514e-03, -1.3874e-03,
         3.0206e-02, -1.0478e-02, -3.7887e-03,  1.7455e-02, -2.2008e-02,
        -1.3083e-03,  2.1482e-03, -1.4283e-02, -1.5073e-03,  1.0340e-02,
        -2.1239e-03, -7.8760e-03,  9.0187e-03, -3.4976e-03,  7.6826e-03,
        -7.3687e-03, -1.9646e-02,  2.6702e-02, -1.3103e-02, -1.1692e-02,
        -8.0439e-03, -5.6945e-03,  4.7717e-03, -9.4241e-05,  6.0450e-03,
        -9.2406e-03,  7.3779e-03,  3.3627e-03,  1.5136e-02, -4.0517e-03,
        -3.3280e-03,  6.2751e-03, -4.4377e-03, -9.1909e-03,  7.1635e-04,
         1.8618e-02, -1.3293e-02, -6.0575e-03, -3.1338e-02,  2.1185e-02,
         4.2449e-03, -4.7048e-03,  1.2959e-03,  1.6103e-03,  2.4173e-02,
         1.4732e-02,  1.4891e-02,  9.4532e-03, -3.4582e-03,  1.2742e-02,
         6.6164e-03, -4.3275e-03, -7.3020e-03,  2.5551e-02,  1.7231e-02,
         1.0594e-02, -1.8811e-02, -7.1467e-03,  1.6925e-03,  1.0746e-02,
        -1.2677e-02, -2.0789e-02,  3.3687e-03, -2.8510e-02, -1.6013e-02,
        -1.8465e-03,  2.8240e-02, -1.4621e-03, -7.5708e-03,  5.8883e-03,
         1.0204e-02, -1.4504e-02,  1.6812e-02,  9.9699e-03,  1.7764e-02,
         1.2409e-02, -5.0602e-03, -5.0206e-02,  1.2838e-02,  8.9933e-03,
        -9.3525e-04, -5.0813e-03, -3.7859e-03, -2.2409e-02,  4.4309e-03,
         1.5883e-02,  8.4158e-03,  2.5948e-02, -1.1486e-02,  1.0112e-03,
         8.4100e-04,  2.8108e-02, -1.3195e-02,  1.0943e-02, -8.9779e-03,
        -2.1404e-02,  2.3818e-02, -1.2751e-02,  1.5801e-03,  1.1601e-02,
        -1.3285e-02, -6.2298e-03, -9.0519e-03,  2.4782e-02,  2.6569e-02,
        -1.2545e-02,  8.5961e-03,  9.0181e-03,  8.5320e-03,  1.3242e-02,
        -5.9443e-03, -8.9034e-03,  1.3880e-02, -1.7463e-02,  3.2144e-02,
         9.3934e-03,  1.9036e-02, -1.0977e-02,  2.4302e-04, -1.1471e-02,
         2.1301e-02, -1.2381e-02,  1.4091e-02,  7.1674e-03,  1.0918e-02,
        -2.1822e-02, -9.5805e-03,  1.3336e-02, -2.9781e-03, -7.6119e-03,
        -1.2603e-02,  1.5540e-02,  2.0893e-02, -6.0504e-03, -1.8243e-02,
        -1.3920e-02,  5.6571e-03,  1.1899e-02,  9.1655e-03, -4.0723e-02,
         1.1844e-02,  4.2420e-03, -1.3299e-02, -1.0281e-02,  1.7206e-02,
         4.3763e-03,  8.8433e-03, -1.3861e-02,  2.4037e-03,  4.1825e-03,
        -5.2521e-03, -1.0116e-02,  3.3683e-02, -5.8550e-03,  8.2323e-03,
        -1.5454e-02, -2.2683e-02,  1.0950e-02, -1.2482e-02, -1.5512e-02,
        -7.2498e-03,  1.0642e-02,  2.8155e-03,  1.3154e-04,  4.5058e-03,
        -2.2348e-02, -2.5788e-02, -6.8520e-03,  1.3454e-02, -1.6015e-02,
         7.9227e-03,  1.5782e-02,  2.2424e-03,  1.0518e-02, -2.6231e-02,
         9.6902e-03, -2.2774e-02,  1.1432e-02, -1.1981e-03, -1.6959e-02,
        -9.9846e-03, -1.0686e-02, -2.4533e-02, -1.4174e-02,  2.3604e-02,
         2.9726e-03, -1.7642e-02, -1.3754e-02,  7.4514e-03, -2.7685e-03,
        -3.9575e-03, -2.2028e-02, -6.3730e-03, -5.4472e-03, -9.8997e-03,
        -7.0156e-03, -4.6017e-03,  1.7484e-02, -6.5174e-03,  1.4871e-02,
        -3.8004e-03,  1.8308e-02, -4.4342e-03,  1.0716e-02,  1.9669e-02,
        -7.4717e-03, -1.4256e-02, -2.7604e-02, -9.2261e-03,  1.9852e-02,
        -2.1431e-02, -1.5931e-02,  1.2729e-04, -4.8617e-03,  7.1115e-03,
        -2.0257e-02, -1.8677e-02, -2.3058e-02, -8.7369e-03,  1.1875e-02,
        -1.9997e-03,  2.0415e-02, -1.1729e-02,  2.1553e-02, -1.4441e-02,
         6.5923e-03, -3.3213e-03, -2.0269e-02, -6.2765e-03, -5.0847e-03,
        -1.4271e-02, -2.9140e-02, -5.5010e-03, -1.4247e-02,  1.8730e-04,
        -1.0086e-02,  6.1738e-03,  1.7130e-02,  1.2486e-02,  1.6704e-02,
         2.0381e-03,  1.8064e-03, -9.0433e-03,  7.2701e-03, -1.6165e-05,
         1.7514e-05,  2.3516e-02, -6.6336e-04,  5.5538e-03,  1.5361e-02,
         1.0191e-02,  2.9463e-03, -5.4127e-03,  8.4070e-04,  1.1086e-02,
         1.0085e-02, -7.9896e-03, -2.0154e-02,  2.6030e-03, -2.4790e-02,
         1.2607e-02, -1.4889e-02,  7.8824e-03,  1.3253e-02, -8.3341e-03,
         5.5529e-03,  4.0093e-03, -3.2290e-02, -1.1233e-02, -1.6009e-02,
         2.7349e-02, -1.9883e-02,  1.2835e-02,  4.6976e-03,  3.5465e-02,
        -1.3909e-02, -1.2693e-02,  2.4281e-02, -1.2218e-02, -9.4744e-03,
         1.3185e-02,  7.3143e-03, -5.8106e-03,  1.4850e-02, -1.0051e-02,
        -9.8984e-03,  4.7675e-03,  1.4153e-02, -1.1744e-03,  5.5085e-04,
        -4.2207e-03,  1.3734e-02, -1.5885e-02,  4.1868e-03, -2.2842e-02,
         1.1625e-02,  8.2290e-03], device='cuda:0'), tensor([1.0310, 0.9645, 1.0105, 0.9718, 1.1028, 1.0093, 0.9960, 0.9995, 0.9736,
        1.0180, 0.9907, 0.9907, 0.9807, 0.9682, 1.0071, 0.9902, 0.9775, 1.0065,
        1.0361, 0.9738, 0.9924, 1.0057, 0.9716, 1.0143, 1.0051, 0.9880, 0.9990,
        1.0027, 1.0098, 0.9917, 0.9902, 0.9787, 1.0166, 0.9790, 1.0001, 0.9793,
        0.9895, 0.9720, 0.9855, 0.9696, 1.0013, 1.0266, 0.9948, 1.0110, 0.9892,
        1.0010, 1.0134, 1.0368, 1.0026, 0.9777, 0.9850, 0.9936, 0.9760, 0.9754,
        0.9913, 0.9911, 0.9923, 0.9736, 1.0078, 0.9971, 1.0380, 0.9839, 1.0128,
        1.0165, 1.0001, 1.0164, 0.9775, 0.9832, 0.9807, 1.0006, 0.9848, 0.9934,
        0.9726, 1.0110, 1.0109, 0.9967, 0.9847, 0.9646, 0.9773, 1.0130, 0.9919,
        1.0387, 1.1074, 0.9816, 0.9765, 0.9954, 0.9911, 0.9740, 0.9906, 0.9791,
        1.0002, 1.0039, 0.9986, 1.0547, 1.0520, 0.9834, 0.9815, 0.9889, 1.0149,
        0.9863, 1.0015, 0.9960, 0.9800, 0.9809, 0.9820, 1.0120, 1.0066, 0.9957,
        0.9696, 1.0245, 1.0065, 1.0301, 1.0415, 1.0844, 1.0561, 0.9742, 0.9737,
        1.0451, 0.9884, 0.9824, 0.9798, 0.9781, 0.9905, 1.0539, 1.0441, 1.0264,
        0.9804, 0.9871, 1.0107, 1.0602, 1.0680, 0.9975, 1.0250, 0.9968, 0.9789,
        0.9980, 1.0361, 0.9896, 1.0198, 0.9894, 1.0020, 1.0265, 0.9629, 0.9706,
        1.0019, 0.9843, 0.9840, 0.9706, 1.0516, 0.9880, 1.0239, 0.9956, 1.0025,
        1.0036, 1.0031, 1.0336, 1.0376, 0.9938, 1.0045, 0.9864, 1.0061, 1.0258,
        1.0208, 0.9791, 0.9966, 0.9947, 1.0502, 1.0054, 1.0174, 0.9817, 1.0267,
        0.9813, 0.9823, 0.9903, 0.9668, 1.0241, 0.9877, 0.9882, 1.0101, 0.9915,
        1.0176, 0.9955, 0.9899, 1.0109, 1.0002, 0.9842, 1.0383, 0.9891, 1.0011,
        1.0222, 1.0149, 0.9967, 0.9905, 1.0603, 0.9982, 1.0019, 0.9877, 0.9927,
        0.9936, 1.0065, 1.0123, 1.0077, 1.0031, 1.0080, 0.9977, 1.0333, 1.0620,
        0.9992, 0.9924, 0.9951, 1.0252, 1.0178, 1.0157, 0.9806, 0.9886, 1.0116,
        1.0398, 0.9857, 0.9704, 1.0108, 1.0230, 0.9697, 0.9898, 1.0128, 1.0208,
        0.9857, 0.9827, 1.0172, 0.9953, 1.0166, 0.9885, 1.0208, 0.9864, 1.0538,
        1.0075, 1.0211, 0.9885, 1.0057, 1.0308, 1.0000, 1.0015, 0.9834, 1.0052,
        0.9854, 0.9852, 0.9878, 1.0127, 0.9919, 0.9869, 0.9933, 1.0025, 0.9691,
        0.9831, 1.0492, 0.9936, 1.0027, 0.9606, 0.9981, 1.0450, 0.9962, 1.0022,
        0.9833, 1.0401, 0.9650, 1.0012, 1.0409, 1.0202, 1.0042, 0.9903, 0.9943,
        0.9939, 0.9872, 0.9750, 0.9885, 0.9971, 1.0023, 1.0359, 0.9856, 1.0570,
        1.0108, 0.9996, 0.9887, 0.9981, 0.9976, 0.9702, 0.9908, 0.9780, 0.9887,
        0.9955, 0.9736, 0.9979, 0.9629, 1.0443, 0.9985, 1.0788, 1.0192, 0.9872,
        1.0295, 1.0488, 0.9890, 0.9962, 0.9863, 0.9989, 1.0094, 0.9988, 0.9739,
        1.0026, 1.0373, 1.0150, 0.9899, 1.0128, 0.9735, 0.9680, 1.0310, 0.9664,
        1.0021, 1.0064, 0.9900, 0.9905, 0.9809, 1.0033, 0.9592, 1.0158, 0.9889,
        0.9914, 0.9557, 0.9988, 0.9921, 1.0041, 1.0150, 0.9562, 0.9678, 1.0074,
        0.9918, 0.9741, 0.9774, 1.0092, 0.9930, 0.9780, 0.9968, 1.0163, 1.0269,
        0.9874, 0.9764, 0.9867, 1.0139, 0.9965, 1.0179, 0.9829, 0.9241, 0.9793,
        0.9998, 1.0098, 0.9894, 1.0655, 0.9832, 1.0029, 0.9737, 0.9657, 1.0004,
        0.9970, 1.0112, 1.0340, 0.9882, 0.9889, 0.9729, 1.0315, 1.0233, 1.0024,
        0.9986, 0.9719, 0.9906, 0.9920, 1.0096, 1.0140, 0.9812, 1.0056, 0.9380,
        0.9896, 1.0027, 1.0011, 1.0383, 1.0015, 0.9928, 1.0411, 0.9829, 1.0502,
        1.0127, 0.9505, 0.9661, 0.9677, 0.9939, 1.0048, 0.9700, 0.9523, 1.0210,
        1.0006, 1.0116, 0.9704, 1.0121, 1.0054, 0.9595, 0.9987, 0.9684, 0.9931,
        0.9952, 1.0102, 0.9765, 1.0179, 1.0045, 0.9993, 1.0012, 0.9889, 1.0019,
        0.9709, 0.9695, 0.9997, 1.0174, 1.0013, 1.0139, 1.0101, 0.9857, 1.0237,
        0.9688, 1.0020, 0.9479, 0.9752, 0.9688, 0.9776, 0.9867, 0.9695, 0.9988,
        1.0245, 1.0126, 0.9796, 0.9954, 0.9793, 1.0130, 1.0119, 1.0000, 1.0042,
        0.9270, 0.9962, 0.9959, 0.9980, 1.0030, 1.0058, 0.9972, 0.9859, 0.9971,
        1.0077, 0.9753, 1.0075, 0.9932, 1.0025, 0.9716, 0.9636, 0.9341, 1.0166,
        0.9342, 1.0033, 1.0088, 1.0569, 0.9814, 0.9984, 1.0067, 0.9798, 0.9895,
        0.9915, 1.0140, 0.9979, 1.0043, 1.0358, 0.9842, 1.0789, 0.9935, 1.0076,
        0.9909, 1.0137, 0.9935, 1.0203, 0.9663, 1.0334, 1.0385, 0.9850, 1.0142,
        1.0310, 1.0044, 0.9849, 1.0142, 0.9913, 0.9578, 0.9745, 1.0090, 0.9837,
        0.9931, 0.9971, 0.9806, 1.0158, 1.0698, 1.0406, 1.0076, 1.0002, 1.0075,
        0.9884, 1.0226, 0.9960, 0.9879, 0.9939, 1.0344, 1.0030, 0.9819],
       device='cuda:0'), tensor([-3.9199e-02, -2.5557e-02, -9.4432e-03, -1.4853e-02, -1.0843e-01,
         9.7634e-03,  6.5517e-03,  1.1818e-03, -4.5013e-03,  1.3175e-02,
        -1.8705e-03, -6.7416e-03, -4.9452e-03, -1.9799e-02, -1.9306e-02,
        -8.3905e-03,  4.6736e-03, -1.3429e-03,  9.9562e-03, -6.7844e-03,
        -1.0607e-02, -1.0635e-02, -1.2187e-02, -3.9110e-03,  7.4600e-03,
        -1.3002e-02, -4.4355e-02,  5.4946e-03,  5.0873e-03, -1.1741e-02,
        -7.9748e-03, -9.9226e-03, -2.5780e-02, -8.1592e-05, -1.1251e-02,
        -7.4511e-03,  2.2342e-03, -2.2103e-02, -1.8418e-02, -1.7798e-02,
         8.4090e-05, -1.6706e-02, -1.7555e-03,  1.1641e-02,  2.2234e-03,
         2.7060e-03, -7.2640e-02, -1.0025e-03, -2.4492e-02, -1.1451e-02,
        -1.0078e-02,  2.2153e-03, -3.7478e-02,  4.2545e-04, -7.8949e-03,
        -6.5197e-03,  7.6151e-03, -4.9774e-02,  8.5624e-03,  5.5626e-04,
        -1.3979e-02,  1.1022e-03,  7.2937e-03, -1.8614e-02,  2.9828e-03,
        -1.8940e-03, -8.8694e-03,  7.9174e-04, -2.8160e-03, -8.6442e-04,
        -1.5568e-02, -7.1598e-03, -1.0885e-02, -2.8944e-02, -1.9386e-02,
        -4.9402e-03, -1.5662e-02, -1.5947e-02,  1.7777e-03,  4.8738e-03,
        -7.3978e-03, -5.6157e-02, -7.7920e-02, -3.8560e-02, -1.3116e-02,
         2.4033e-03, -4.2779e-02, -5.1493e-03, -2.9945e-03, -3.0831e-03,
        -6.5689e-03, -7.4815e-03, -4.3709e-03, -4.3377e-02, -1.2536e-03,
         1.8810e-02, -8.6274e-03,  7.3025e-03, -6.5025e-03, -1.1208e-02,
        -8.8074e-03, -6.3923e-05,  9.4438e-04, -5.0002e-03,  5.8396e-03,
        -1.0723e-02, -7.4207e-03, -4.1251e-03,  1.5821e-03, -3.7306e-02,
        -1.3338e-02, -5.8081e-03,  4.6738e-03, -5.2322e-02, -3.8755e-02,
         5.8711e-03,  4.5320e-03, -7.0339e-03, -9.5814e-03, -1.1146e-02,
        -1.8321e-02, -9.5705e-03,  3.8188e-03, -6.7925e-03,  5.0402e-03,
         7.5322e-03, -1.2161e-02, -6.8249e-03, -2.2158e-02, -2.8645e-02,
        -5.0668e-02, -5.6760e-03,  6.2022e-03, -5.4865e-03,  8.3324e-03,
         6.6293e-03, -8.0940e-02,  7.2663e-04,  8.6712e-03,  7.6257e-03,
        -3.2775e-03,  2.2076e-03,  7.0139e-03,  1.0664e-03, -4.3771e-03,
        -5.4891e-03, -5.2819e-03, -1.1461e-02, -9.3632e-04, -1.6929e-02,
         3.1651e-03,  3.8314e-03, -2.6499e-05, -1.4095e-02, -1.0141e-02,
        -2.5829e-02,  7.0083e-03, -5.8968e-03, -1.0216e-02,  3.5936e-03,
         1.9639e-03, -8.5389e-03,  9.4098e-03, -4.5763e-04,  4.0449e-03,
        -6.0151e-03, -3.6693e-02, -6.4807e-03,  6.8574e-03,  9.5982e-03,
        -2.0983e-03, -1.7830e-02, -9.2000e-03,  2.5399e-03, -7.8357e-03,
        -1.3394e-02, -3.6750e-03,  4.5521e-03, -7.2218e-03, -8.2365e-04,
        -6.4779e-02, -9.2902e-03, -8.7565e-03,  1.0584e-03, -3.5105e-03,
         5.9893e-03,  2.7525e-03, -3.1853e-03, -2.4066e-03,  4.5938e-03,
        -5.9857e-03, -3.7031e-03,  1.2710e-02,  9.1747e-03, -1.9384e-02,
         5.3065e-03,  1.4550e-03, -5.5345e-03, -6.5248e-03, -1.9209e-03,
         4.7924e-03,  3.8459e-03,  2.4939e-03, -1.0289e-03, -9.9310e-03,
        -2.6273e-02, -4.8415e-02,  1.4882e-03, -2.6352e-03,  1.2717e-03,
        -1.0514e-03,  3.7574e-03,  1.2828e-03, -3.5402e-03,  9.9181e-04,
         7.6541e-03,  3.1217e-03,  1.3180e-02,  1.1348e-02,  7.1650e-03,
         9.3414e-05,  1.7458e-02, -6.2743e-03, -1.6943e-02, -2.5468e-02,
        -1.8241e-02, -3.4470e-03,  3.2607e-03, -7.5422e-03,  5.4408e-03,
        -1.5492e-03, -1.6900e-02,  6.0734e-04,  3.0609e-03, -4.0086e-02,
         4.2125e-03, -3.2487e-02,  5.3011e-03,  7.8361e-03, -5.1699e-03,
        -1.6033e-02, -1.2780e-02, -7.2276e-04,  4.4836e-03, -1.0778e-02,
        -7.3802e-03, -4.8245e-02, -8.4501e-03,  3.2373e-03,  1.9712e-04,
         1.2392e-03, -1.6724e-02, -2.6122e-04, -3.0032e-02, -1.0112e-02,
         1.8781e-03, -2.7073e-02, -2.1975e-03, -4.4805e-02, -3.3532e-03,
        -4.0171e-03, -2.8468e-03,  2.2156e-03, -3.8500e-03, -9.5685e-03,
        -1.7538e-02, -4.8553e-03, -1.7247e-02, -1.1683e-02, -1.6826e-03,
        -1.3968e-02, -6.4085e-03, -8.8428e-03, -3.1300e-03, -1.6989e-02,
        -1.1050e-02,  4.8954e-03,  1.4350e-02, -9.5270e-02, -1.3128e-02,
        -4.4755e-03, -6.6351e-03, -1.9310e-03, -7.1960e-03, -2.8347e-02,
        -2.4696e-03,  6.8393e-03, -1.7799e-04,  1.6099e-03, -1.1028e-03,
        -1.5914e-02, -1.9518e-02, -4.0939e-02,  9.9583e-04, -6.3890e-02,
        -4.1627e-02, -1.5879e-02, -7.7150e-03, -1.4486e-02, -9.3113e-03,
        -1.2187e-03, -1.6231e-02,  1.5710e-03, -3.4074e-02, -3.6891e-03,
        -2.0440e-02,  4.4782e-03, -2.1431e-02, -6.3191e-02, -1.1860e-02,
        -1.3716e-03,  5.9030e-03, -1.5619e-03,  5.2590e-03, -1.0472e-02,
        -5.0358e-03,  5.9230e-04, -4.0284e-03, -3.8266e-03, -2.6041e-03,
        -2.9757e-03,  1.3116e-02,  3.8123e-03, -2.6153e-02, -3.0346e-02,
         1.9188e-02,  4.4871e-04, -1.2881e-02, -4.3709e-03,  2.5216e-03,
        -3.7018e-03, -1.5207e-02,  2.9862e-04, -5.9235e-03,  4.4751e-03,
        -5.0391e-03, -1.5776e-02, -1.5701e-02, -4.9286e-02,  1.0675e-03,
         6.0115e-03,  1.0983e-02, -2.4959e-03, -1.5526e-02, -2.6511e-02,
        -2.9744e-02, -2.2111e-02,  4.2465e-03, -1.6187e-02, -5.0621e-02,
         9.4636e-03,  2.0758e-03, -6.3443e-04, -1.9454e-02, -9.2297e-02,
        -8.6713e-03,  9.1582e-03,  2.4992e-04, -1.4343e-02,  7.0382e-03,
        -1.0512e-02,  2.7907e-03,  1.9864e-03, -2.2349e-02, -2.0654e-02,
        -2.8626e-02, -9.8002e-03,  6.0352e-02, -1.9676e-04,  6.2421e-04,
        -3.1787e-02, -1.6573e-02,  2.0399e-03,  6.2479e-03,  1.1050e-03,
         1.8663e-02, -5.2477e-03, -5.5633e-03, -2.1583e-02, -1.2369e-02,
        -2.5649e-02,  2.6162e-03, -1.3749e-02, -6.6372e-03, -5.6837e-02,
        -7.3781e-03,  4.0669e-02,  7.2530e-03, -4.6145e-02, -9.3502e-03,
         1.8293e-03, -2.3482e-02,  3.2265e-03, -1.1127e-02,  8.6006e-03,
         3.2856e-03, -1.1320e-02,  9.4752e-03, -5.1404e-02,  3.3124e-03,
        -1.2201e-02, -1.3058e-03, -6.5426e-03,  1.4051e-02, -9.2330e-03,
        -2.0036e-02, -1.7778e-03, -1.6645e-02, -5.9880e-05,  3.4072e-03,
        -1.0866e-02, -1.9867e-03, -1.4681e-02,  2.0784e-02, -2.7487e-03,
        -1.9006e-02, -4.8093e-03,  3.7230e-03,  8.4588e-04,  1.4206e-03,
         7.7299e-03, -1.1320e-02,  7.7449e-03, -7.0349e-03, -2.2067e-03,
        -2.9531e-02, -6.9430e-04,  6.9818e-03,  1.3697e-02, -6.3691e-03,
         6.1008e-03, -1.2907e-03, -1.2291e-03,  9.6866e-03, -1.4906e-02,
        -8.0750e-03,  1.0951e-02, -1.0403e-02,  3.8181e-03, -1.5375e-02,
        -1.7492e-02,  1.2734e-02, -1.5541e-02, -2.6720e-03,  1.4068e-03,
         7.2808e-04,  5.5421e-03,  2.2376e-04, -1.1268e-02,  1.2004e-03,
        -2.7284e-02, -4.2194e-03, -1.0750e-02, -5.4536e-03, -4.6471e-03,
        -7.3340e-03,  3.1435e-03, -6.0951e-02,  3.2273e-03, -1.1720e-02,
        -2.4112e-02,  9.3176e-03, -5.7459e-02, -2.6311e-03, -4.4022e-05,
         4.8586e-03,  1.2967e-02, -1.8250e-02, -4.0621e-03,  6.3705e-03,
        -4.8170e-03, -1.5725e-03,  5.6635e-03, -1.2674e-02, -3.1792e-02,
         3.3202e-03, -1.0059e-02, -1.0883e-02,  1.0523e-02,  4.0873e-04,
        -1.3503e-02, -3.6398e-03, -2.7666e-02, -1.2896e-02, -2.4904e-02,
         8.6335e-03,  5.4911e-03,  1.2200e-02, -1.1026e-02,  1.4557e-02,
        -9.2269e-03, -2.8709e-02,  1.2670e-02,  1.5519e-03, -9.6328e-03,
        -1.5370e-02, -8.1863e-04, -1.4270e-02, -5.6748e-03, -3.2678e-03,
         1.6146e-02, -3.6170e-02,  2.4393e-03, -3.5570e-02, -4.4417e-03,
        -2.1873e-02, -3.6963e-02, -6.9736e-03, -9.2038e-04, -1.3082e-02,
        -1.4261e-02, -1.5193e-02], device='cuda:0'), tensor([[[ 0.0313,  0.0359,  0.0111,  0.0336,  0.0148],
         [-0.0159,  0.0041, -0.0025, -0.0268,  0.0030],
         [ 0.0175,  0.0049,  0.0073,  0.0015, -0.0076],
         ...,
         [-0.0195,  0.0053,  0.0080,  0.0137, -0.0165],
         [-0.0093,  0.0156,  0.0108,  0.0038,  0.0169],
         [ 0.0215,  0.0168, -0.0133, -0.0150,  0.0070]],

        [[ 0.0123,  0.0174,  0.0212,  0.0146,  0.0274],
         [ 0.0120,  0.0123, -0.0097, -0.0024, -0.0046],
         [ 0.0023,  0.0207, -0.0045,  0.0053,  0.0165],
         ...,
         [-0.0033,  0.0118, -0.0134, -0.0195,  0.0152],
         [ 0.0005,  0.0031,  0.0054,  0.0179, -0.0128],
         [ 0.0109,  0.0113, -0.0012, -0.0206, -0.0045]],

        [[-0.0149, -0.0250, -0.0123, -0.0094,  0.0007],
         [ 0.0255, -0.0193,  0.0188, -0.0184, -0.0157],
         [-0.0104,  0.0111,  0.0172,  0.0174,  0.0205],
         ...,
         [ 0.0175,  0.0082,  0.0045,  0.0038,  0.0088],
         [ 0.0052,  0.0134, -0.0019,  0.0052, -0.0094],
         [-0.0033, -0.0024, -0.0064,  0.0242,  0.0041]],

        ...,

        [[ 0.0123,  0.0252, -0.0059, -0.0058, -0.0190],
         [-0.0128,  0.0127, -0.0098, -0.0181,  0.0184],
         [ 0.0098,  0.0043,  0.0045, -0.0165, -0.0177],
         ...,
         [-0.0165,  0.0053, -0.0081,  0.0025, -0.0206],
         [-0.0187, -0.0003, -0.0079,  0.0118, -0.0022],
         [-0.0003, -0.0142,  0.0064,  0.0002,  0.0181]],

        [[ 0.0360,  0.0054,  0.0295,  0.0336,  0.0264],
         [-0.0246, -0.0133,  0.0211,  0.0220, -0.0131],
         [ 0.0019, -0.0093,  0.0135,  0.0078,  0.0163],
         ...,
         [-0.0057, -0.0077,  0.0063, -0.0345, -0.0009],
         [-0.0132,  0.0130, -0.0101,  0.0095, -0.0025],
         [ 0.0178,  0.0148, -0.0127, -0.0125,  0.0020]],

        [[ 0.0403,  0.0137,  0.0260,  0.0191, -0.0148],
         [ 0.0336,  0.0229,  0.0009,  0.0082, -0.0005],
         [-0.0096, -0.0150, -0.0126, -0.0165,  0.0154],
         ...,
         [-0.0311,  0.0069,  0.0299,  0.0019, -0.0136],
         [ 0.0100, -0.0024, -0.0022, -0.0163,  0.0109],
         [-0.0102,  0.0064,  0.0009,  0.0016,  0.0115]]], device='cuda:0'), tensor([ 1.4491e-02,  9.0399e-03, -9.5702e-03,  3.6052e-03, -1.1350e-02,
         1.5285e-02,  8.5167e-03,  4.3944e-03, -1.7535e-02,  1.9343e-02,
         5.5086e-03, -1.0992e-02,  5.8347e-03, -3.0920e-02, -1.7129e-03,
        -1.2445e-02,  1.6462e-02, -1.5547e-02,  1.3212e-02,  1.2995e-03,
        -4.7958e-03, -7.9151e-03, -1.7300e-02, -1.3811e-02, -1.2108e-02,
         1.0183e-02,  1.2349e-02, -7.7106e-03, -1.5901e-02,  1.1816e-02,
        -4.4029e-03,  4.1968e-03, -1.5784e-02,  5.4681e-03,  1.1275e-02,
        -1.4301e-02,  1.3781e-02,  1.1000e-02,  1.3514e-02, -1.5589e-02,
        -7.8147e-03, -1.0078e-01, -9.6273e-03, -1.2455e-02, -1.2108e-02,
         1.5915e-02,  1.3781e-02,  2.9863e-03,  7.6240e-03, -1.6340e-02,
         1.4825e-02,  1.6458e-02, -1.6178e-02, -1.5177e-02,  1.8482e-03,
        -1.1614e-01,  3.4926e-04, -2.1969e-02, -1.3158e-02, -1.0942e-02,
         1.4603e-02, -1.2435e-02, -1.1018e-02,  1.3050e-02,  6.3358e-03,
        -1.3057e-02,  1.5020e-02,  1.3954e-02,  2.4175e-02, -2.0232e-02,
         6.7960e-03,  1.8911e-02,  8.7418e-03, -2.2836e-02, -5.8607e-02,
         1.3933e-02,  3.6153e-03, -6.0451e-03, -1.1021e-01, -2.4843e-02,
        -7.2387e-03, -1.9211e-02,  1.8663e-02,  1.7390e-02,  2.2714e-03,
        -7.2120e-03, -1.9279e-03, -9.3465e-03, -1.4187e-02, -1.6198e-03,
        -1.1478e-02, -1.2544e-02,  2.0667e-02, -1.3945e-02, -1.6324e-02,
        -1.1493e-02,  7.1330e-03, -1.6969e-02, -1.2574e-02,  1.1041e-02,
        -2.8655e-02, -1.0251e-02, -6.0818e-03,  7.3900e-03, -1.1564e-02,
         2.0221e-02, -9.1814e-03,  1.4845e-03, -7.5687e-03,  2.0175e-02,
         1.7056e-02,  1.0653e-03,  7.6723e-03, -1.2459e-02,  6.7301e-03,
        -3.6970e-03, -1.2533e-02,  8.3756e-03, -1.4827e-02,  7.4198e-03,
         2.5473e-02, -1.8175e-03, -4.7790e-06, -1.1069e-02, -1.8383e-02,
        -5.5938e-03,  1.8574e-02,  4.1534e-03, -5.9634e-02,  7.8269e-03,
         2.3568e-02, -2.1134e-02,  1.5597e-02,  8.4496e-03,  1.7174e-02,
         1.7765e-02,  1.7658e-02,  7.7980e-03, -2.2110e-03, -4.3360e-02,
        -3.6420e-03,  1.7305e-03,  1.2234e-02,  1.2776e-02,  3.4750e-03,
         1.3133e-02, -1.0388e-02, -1.1962e-02, -6.0694e-02, -4.9544e-03,
         1.2734e-02,  1.2914e-02,  7.4577e-03,  1.1313e-02,  8.4604e-03,
        -6.8887e-02, -1.0479e-02,  1.3917e-02, -1.6700e-02,  1.9802e-03,
        -1.9651e-02, -1.9712e-02,  1.9227e-02,  7.8242e-03, -9.3224e-03,
         1.1648e-02, -4.1492e-03,  1.8381e-02, -9.6716e-03, -2.2115e-02,
         2.9818e-03, -1.5689e-02,  1.6286e-02, -9.9462e-03, -2.3053e-02,
         3.1539e-03,  1.2610e-02, -1.1622e-02,  2.1041e-02, -1.7115e-03,
         1.3475e-02, -7.0508e-03,  4.2178e-03,  1.1834e-02,  2.0296e-02,
        -4.8197e-03,  8.2639e-03,  1.5039e-02, -2.2540e-03,  1.9007e-02,
         7.4604e-03, -5.0158e-03,  6.1740e-03,  1.3818e-02,  4.8177e-03,
         1.7825e-02,  2.4409e-02, -7.9780e-03, -2.2175e-03, -1.5808e-02,
         9.1612e-03, -3.5405e-03, -1.3026e-02, -5.5044e-03,  1.0050e-03,
         2.1026e-02,  5.4509e-03, -2.8242e-03,  1.6051e-02, -1.2309e-02,
        -1.7322e-02,  1.1333e-02,  3.8251e-04, -2.4052e-02,  5.6679e-03,
        -1.8561e-02, -7.5710e-03, -4.2359e-03,  1.3848e-02, -1.2781e-02,
        -6.9588e-03, -9.6863e-02, -8.6961e-03, -1.1125e-02,  1.2986e-02,
        -1.7869e-02, -1.0165e-01,  5.9878e-03, -4.4588e-02, -1.1021e-02,
        -8.8393e-05, -1.5021e-03, -6.8099e-04, -1.0495e-02,  1.5122e-02,
        -4.1576e-03,  1.9874e-02,  4.8169e-03, -2.1383e-02,  1.8944e-02,
         6.2921e-03, -1.9809e-02, -1.8684e-02,  1.5962e-02,  1.8417e-02,
         1.6005e-02, -1.6272e-02,  1.8521e-02, -8.4411e-03, -6.3985e-03,
         5.9706e-04, -1.4016e-03, -9.7740e-03, -2.0723e-02,  3.7179e-03,
         1.3198e-02,  1.4734e-02, -1.9689e-02,  6.7534e-03, -1.2852e-02,
        -8.4254e-03,  7.3536e-03,  7.4626e-03,  1.2373e-02, -1.0468e-01,
         2.0023e-03, -1.5273e-02, -1.7257e-02,  2.5053e-03, -4.5513e-03,
         2.0282e-02, -2.9021e-03,  9.2669e-03,  1.5032e-02,  1.5832e-02,
        -1.1218e-02, -8.2643e-02, -6.6851e-03,  4.8803e-03, -1.1152e-02,
        -1.8428e-02,  9.7719e-03,  1.0963e-02,  2.2079e-03, -9.5004e-03,
        -1.1095e-02, -1.6749e-03, -4.8003e-03, -2.2098e-02,  2.1419e-02,
        -3.4611e-04, -2.1344e-02, -3.8826e-02, -6.1573e-03,  1.7065e-02,
         5.5111e-03, -1.0416e-02,  1.3773e-02,  8.0307e-05, -7.0053e-03,
        -1.1618e-02, -1.0483e-02, -1.5585e-02,  2.0183e-02,  8.8541e-03,
        -1.9848e-02, -5.9907e-03,  1.6459e-02, -1.1131e-02,  2.0429e-02,
        -6.7408e-03,  7.1175e-04, -2.2684e-03,  1.3847e-02, -1.3950e-02,
         1.4427e-02, -1.5006e-02,  1.8108e-03,  6.1632e-03, -5.7306e-03,
         4.9002e-03,  3.5017e-03, -4.4111e-03,  1.0239e-02, -1.3716e-02,
         1.2783e-02, -1.7422e-02, -1.3470e-02, -2.0176e-02, -5.1245e-03,
         1.8825e-02,  7.8164e-03, -9.1172e-02,  4.2584e-03,  1.4324e-02,
        -5.1929e-04, -1.7571e-02,  2.0992e-02, -1.1548e-01,  1.7471e-02,
         1.0555e-03, -1.6024e-02, -1.0151e-02,  1.2696e-03, -1.2731e-02,
         1.8060e-02,  1.5554e-02,  1.3238e-02,  4.2635e-03,  1.3785e-02,
         1.2570e-02, -6.0375e-03, -1.3988e-02,  1.0391e-02,  9.1753e-03,
        -8.1289e-03,  6.0227e-03,  3.9725e-03,  4.9815e-03, -3.6856e-03,
         2.0026e-02,  1.0106e-02, -6.8281e-03,  1.6470e-02,  1.6792e-02,
        -8.5094e-03,  2.4105e-02, -8.7447e-03,  8.5584e-03, -1.5165e-02,
        -6.9629e-03, -6.4129e-04, -2.1134e-02,  1.1061e-02,  4.9056e-03,
        -1.5848e-02, -9.4751e-03, -1.3474e-02, -1.2077e-02,  2.1679e-03,
         8.5215e-03, -3.6394e-03,  6.4666e-03,  1.2704e-02,  1.7755e-02,
        -1.1419e-03, -7.0114e-03,  2.3490e-02, -1.8689e-02, -8.8125e-03,
         2.0646e-02, -1.1902e-02,  1.0550e-02,  6.9694e-03,  3.3011e-03,
        -2.0464e-02,  3.2036e-04, -1.6459e-02, -1.3847e-02, -2.3185e-02,
         1.4375e-02, -8.6679e-03,  1.6333e-02, -1.9777e-02,  2.7559e-03,
        -1.3489e-02,  1.9110e-02,  4.7986e-03,  4.7911e-03, -2.8028e-03,
         1.4006e-03, -5.2618e-03, -4.7496e-02, -7.6059e-02, -9.4310e-03,
        -8.6012e-03, -2.0755e-03,  6.8677e-03, -1.9638e-02, -9.6973e-03,
        -1.9965e-03, -3.7010e-03, -3.6430e-03, -1.1855e-01, -1.1768e-02,
         1.5123e-02, -6.6539e-03, -2.5083e-03,  1.9947e-02,  1.2619e-02,
        -1.9112e-02,  1.3374e-02, -1.0639e-02,  1.6869e-02,  1.0808e-02,
        -1.2117e-02,  1.0771e-02,  5.0457e-03, -1.3557e-02, -1.4093e-02,
        -1.5120e-02, -3.8200e-03, -2.9901e-03,  1.6658e-02,  5.8708e-04,
        -1.7535e-02, -2.1870e-02,  8.4241e-03, -1.3218e-02,  1.0157e-02,
         1.6679e-02,  8.9949e-03, -8.6034e-03,  8.4485e-04,  1.4910e-02,
        -6.9247e-03,  4.5028e-03,  2.1267e-02,  1.2262e-02,  5.8781e-03,
        -1.8521e-02, -5.7138e-03, -1.3155e-02, -2.2155e-03,  2.7871e-04,
         2.0101e-02,  7.3617e-03,  6.2480e-03, -9.1457e-03, -1.5568e-02,
        -1.9121e-02, -3.0504e-03, -7.2842e-03,  6.8476e-03,  3.0287e-03,
         3.9521e-03, -2.1447e-02,  7.1469e-03,  2.1421e-02, -8.0487e-05,
         1.0503e-02, -9.9612e-02,  5.8244e-03,  2.2829e-02,  8.2327e-04,
        -9.5494e-03, -1.9759e-03, -1.5970e-02,  1.1788e-02,  2.3584e-02,
         1.3803e-02,  1.8352e-02, -2.4794e-03,  6.1400e-03, -4.2941e-04,
        -1.3293e-02,  1.6038e-02, -5.0266e-03,  1.8726e-02, -9.7942e-02,
        -1.8786e-02, -3.3598e-03, -1.8837e-02, -1.6389e-02, -2.3814e-03,
         1.9755e-02,  1.1798e-02,  3.8899e-04,  1.0437e-02,  8.7747e-03,
        -1.5264e-02, -1.5185e-02], device='cuda:0'), tensor([0.9841, 0.9848, 0.9829, 0.9663, 0.9714, 0.9770, 0.9787, 0.9449, 0.9751,
        0.9727, 0.9749, 0.9673, 0.9777, 0.9989, 0.9733, 0.9604, 0.9624, 0.9822,
        0.9746, 0.9531, 0.9712, 0.9807, 0.9760, 0.9842, 0.9721, 0.9864, 0.9626,
        0.9803, 0.9625, 0.9901, 0.9528, 0.9706, 0.9802, 0.9719, 0.9800, 0.9774,
        0.9808, 0.9826, 0.9716, 0.9731, 0.9798, 0.9992, 0.9811, 0.9852, 0.9786,
        0.9888, 0.9616, 0.9744, 0.9782, 0.9787, 0.9857, 0.9791, 0.9814, 0.9864,
        0.9648, 0.9922, 0.9636, 0.9469, 1.0089, 0.9859, 0.9874, 0.9824, 0.9651,
        0.9644, 0.9805, 0.9551, 0.9627, 0.9482, 0.9761, 0.9423, 0.9878, 0.9792,
        0.9703, 0.9602, 0.9940, 0.9491, 0.9825, 0.9816, 0.9932, 1.0033, 0.9697,
        0.9794, 0.9727, 0.9774, 0.9679, 0.9703, 0.9599, 0.9980, 0.9863, 0.9862,
        0.9733, 0.9879, 0.9842, 0.9806, 0.9839, 0.9677, 0.9857, 0.9612, 0.9851,
        0.9821, 1.0049, 0.9752, 0.9803, 0.9568, 0.9859, 0.9810, 0.9804, 0.9822,
        0.9801, 0.9767, 0.9793, 0.9805, 0.9714, 0.9763, 0.9522, 0.9407, 0.9925,
        0.9826, 0.9827, 0.9775, 0.9884, 0.9782, 0.9843, 0.9535, 0.9708, 0.9676,
        0.9802, 0.9797, 0.9991, 0.9139, 0.9839, 0.9602, 0.9837, 0.9824, 0.9761,
        0.9619, 0.9770, 1.0170, 0.9733, 1.0004, 0.9829, 0.9510, 0.9935, 0.9835,
        0.9866, 0.9729, 0.9922, 0.9789, 0.9984, 0.9843, 0.9749, 0.9351, 0.9762,
        0.9558, 0.9068, 0.9968, 0.9681, 0.9445, 0.9742, 0.9813, 0.9590, 0.9726,
        0.9673, 0.9751, 0.9625, 0.9827, 0.9788, 0.9788, 0.9696, 0.9776, 0.9724,
        0.9673, 0.9387, 0.9665, 0.9528, 0.9861, 0.9705, 0.9816, 0.9777, 0.9668,
        0.9789, 0.9467, 1.0103, 0.9687, 0.9653, 0.9653, 0.9845, 0.9232, 0.9567,
        0.9799, 0.9603, 0.9784, 0.9658, 0.9822, 0.9961, 0.9820, 0.9849, 0.9640,
        0.9862, 0.9769, 0.9805, 0.9711, 0.9800, 0.9733, 0.9570, 0.9734, 0.9968,
        0.9875, 0.9830, 0.9398, 1.0002, 0.9855, 0.9654, 0.9872, 0.9742, 0.9880,
        0.9656, 0.9647, 0.9145, 0.9505, 0.9736, 1.0011, 0.9726, 0.9829, 0.9771,
        0.9749, 0.9983, 0.9811, 0.9981, 0.9884, 0.9583, 0.9696, 0.9829, 0.9810,
        0.9755, 0.9637, 0.9655, 0.9833, 0.9748, 0.9819, 0.9510, 0.9640, 0.9835,
        0.9761, 0.9533, 0.9757, 1.0010, 0.9818, 0.9377, 0.9534, 0.9602, 0.9612,
        0.9671, 0.9836, 0.9696, 0.9828, 0.9741, 0.9844, 0.9646, 0.9657, 0.9818,
        0.9714, 0.9618, 0.9725, 1.0009, 0.9714, 0.9622, 0.9778, 0.9721, 0.9810,
        0.9816, 0.9800, 0.9777, 0.9681, 0.9851, 0.9758, 0.9969, 0.9623, 0.9802,
        0.9826, 0.9627, 0.9649, 0.9810, 0.9408, 0.9751, 0.9746, 0.9878, 0.9696,
        1.0019, 0.9477, 0.9691, 0.9699, 0.9999, 1.0017, 0.9814, 0.9685, 0.9818,
        0.9832, 0.9827, 0.9678, 0.9765, 0.9432, 1.0060, 0.9344, 0.9913, 0.9818,
        0.9784, 0.9704, 0.9836, 0.9811, 0.9738, 1.0098, 0.9663, 0.9787, 0.9844,
        0.9836, 0.9846, 0.9917, 0.9668, 0.9517, 0.9819, 0.9676, 0.9764, 0.9544,
        0.9860, 0.9871, 1.0030, 0.9846, 0.9660, 0.9570, 0.9705, 0.9837, 0.9958,
        0.9836, 0.9588, 0.9782, 0.9190, 0.9830, 0.9942, 0.9849, 0.9391, 0.9823,
        0.9788, 0.9619, 0.9615, 0.9705, 0.9553, 0.9825, 0.9864, 0.9755, 0.9780,
        0.9848, 0.9734, 0.9778, 0.9772, 0.9753, 0.9465, 0.9699, 0.9850, 0.9773,
        0.9545, 0.9735, 0.9853, 0.9856, 0.9725, 0.9772, 0.9915, 1.0061, 0.9894,
        0.9832, 0.9867, 0.9799, 0.9655, 0.9862, 0.9838, 0.9702, 0.9681, 0.9845,
        0.9749, 0.9802, 0.9929, 0.9760, 0.9783, 0.9413, 0.9776, 1.0100, 0.9756,
        0.9868, 0.9122, 0.9872, 0.9799, 0.9688, 0.9545, 0.9912, 0.9897, 0.9844,
        0.9755, 0.9369, 0.9554, 0.9698, 0.9806, 0.9151, 0.9389, 0.9481, 0.9862,
        0.9562, 0.9813, 0.9864, 0.9812, 0.9807, 0.9573, 0.9760, 0.9997, 0.9954,
        0.9671, 0.9741, 0.9731, 0.9754, 0.9710, 0.9550, 0.9888, 0.9835, 0.9962,
        0.9998, 0.9774, 0.9758, 0.9859, 0.9698, 0.9545, 0.9820, 0.9872, 0.9535,
        0.9690, 0.9772, 0.9550, 0.9393, 0.9565, 0.9820, 0.9491, 0.9962, 0.9573,
        0.9606, 0.9761, 0.9812, 0.9852, 0.9659, 0.9891, 0.9776, 0.9432, 0.9785,
        0.9648, 0.9658, 0.9814, 0.9656, 0.9828, 0.9557, 0.9823, 0.9877, 0.9761,
        0.9734, 0.9508, 0.9465, 0.9745, 0.9288, 0.9583, 0.9789, 0.9842, 0.9508,
        0.9844, 0.9738, 0.9531, 0.9807, 0.9799, 0.9895, 0.9828, 0.9843, 0.9765,
        0.9843, 0.9763, 1.0045, 0.9647, 1.0034, 0.9779, 0.9621, 0.9471, 0.9682,
        0.9418, 0.9829, 0.9807, 0.9803, 0.9821, 0.9668, 0.9849, 0.9761, 0.9738,
        0.9859, 0.9529, 0.9881, 0.9716, 0.9976, 0.9520, 0.9697, 0.9608, 0.9599,
        0.9737, 0.9776, 0.9786, 0.9807, 0.9765, 0.9562, 0.9841, 0.9806],
       device='cuda:0'), tensor([-1.3052e-02, -8.2831e-03, -8.7544e-03, -1.2705e-02, -1.7825e-02,
        -1.1522e-02, -1.0354e-02, -1.1213e-02, -1.0617e-02, -1.6977e-02,
        -9.4132e-03, -9.9832e-03, -7.3266e-03, -1.9331e-03, -8.2729e-03,
        -1.8238e-02, -9.4799e-03, -7.3135e-03, -1.2178e-02, -1.1017e-02,
        -1.2396e-02, -7.4019e-03, -1.0912e-02, -8.2760e-03, -1.0694e-02,
        -5.5549e-03, -8.3525e-03, -1.1486e-02, -8.0573e-03, -6.3141e-03,
        -1.7595e-02, -6.0378e-03, -8.5052e-03, -1.1192e-02, -1.3150e-02,
        -1.0840e-02, -1.3393e-02, -6.1575e-03, -1.4199e-02, -1.0833e-02,
        -6.5026e-03, -3.5508e-03, -8.3665e-03, -8.6339e-03, -1.1707e-02,
        -1.8959e-03, -1.4817e-02, -7.2017e-03, -7.5687e-03, -1.4631e-02,
        -2.6282e-03, -1.3282e-02, -8.1772e-03, -5.6848e-03, -5.8647e-03,
         8.6463e-03, -9.4726e-03, -8.4773e-03, -9.9557e-03, -1.0352e-02,
        -8.2166e-03, -1.0567e-02, -1.2020e-02, -6.3003e-03, -8.7922e-03,
        -9.6390e-03, -9.9791e-03, -9.5646e-03, -1.2006e-02, -9.8462e-03,
        -9.3310e-03, -4.6996e-03, -1.2195e-02, -1.5339e-02,  9.1753e-03,
        -9.2299e-03, -1.1173e-02, -1.2826e-02,  3.4204e-03, -7.6968e-03,
        -1.2430e-02, -1.4533e-02, -9.1671e-03, -1.0731e-02, -1.2199e-02,
        -9.3633e-03, -1.1751e-02, -3.0628e-03, -1.1857e-02, -7.7742e-03,
        -8.4338e-03, -1.0187e-02, -8.6694e-03, -9.9917e-03, -7.8416e-03,
        -5.3036e-03, -1.0025e-02, -1.3378e-02, -9.4937e-03, -9.3682e-03,
        -7.9560e-03, -1.3081e-02, -9.7039e-03, -9.2511e-03, -7.8362e-03,
        -8.6920e-03, -1.0401e-02, -1.1419e-02, -1.1978e-02, -1.0380e-02,
        -1.1230e-02, -7.9775e-03, -1.0045e-02, -1.5686e-02, -1.1091e-02,
        -8.5598e-03, -9.6444e-03, -1.0740e-02, -1.3477e-02, -7.8746e-03,
        -7.0876e-03, -1.5227e-02, -3.4742e-03, -8.4644e-03, -1.0252e-02,
        -9.0740e-03, -1.1538e-02, -1.3704e-02,  4.3373e-05, -1.1023e-02,
        -1.3231e-02, -9.1874e-03, -8.0180e-03, -1.4619e-02, -7.4670e-03,
        -1.0674e-02, -1.6625e-02, -1.8333e-02, -1.0164e-02, -2.7237e-03,
        -1.3852e-02, -1.1167e-02, -8.0660e-03, -1.0880e-02, -9.0802e-03,
        -1.5918e-02, -1.0836e-02, -1.3328e-02, -1.1888e-03, -1.0819e-02,
        -6.6040e-03, -1.1207e-02, -1.5506e-02, -1.2148e-02, -1.1325e-02,
         1.9451e-03, -5.2457e-03, -1.0113e-02, -8.2329e-03, -1.0196e-02,
        -1.2356e-02, -8.4949e-03, -1.1188e-02, -1.2313e-02, -1.5695e-02,
        -1.2353e-02, -1.2906e-02, -7.8747e-03, -1.1746e-02, -8.4083e-03,
        -1.0079e-02, -1.2421e-02, -6.4304e-03, -1.6002e-02, -9.5301e-03,
        -1.2026e-02, -1.2504e-02, -1.1757e-02, -1.2829e-02, -9.1259e-03,
        -1.7159e-02, -1.2959e-02, -9.7772e-03, -1.2085e-02, -8.2910e-03,
        -1.4820e-02, -9.8322e-03, -9.5209e-03, -1.0414e-02, -6.9546e-03,
        -1.1837e-02, -1.0952e-02, -7.9141e-03, -1.1053e-02, -1.2720e-02,
        -1.0878e-02, -6.7548e-03, -7.5206e-03, -1.2450e-02, -1.5188e-02,
        -8.3512e-03, -1.5199e-02, -7.4174e-03, -1.9805e-02, -1.0196e-02,
        -8.6661e-03, -1.4057e-02, -9.6055e-03, -6.4700e-03, -1.0394e-02,
        -1.3910e-02, -1.2118e-02, -9.7492e-03, -8.8354e-03, -1.0502e-02,
        -6.0600e-03, -1.2564e-02, -1.0392e-02, -7.8407e-03, -7.9444e-03,
        -7.2055e-03, -3.7999e-03, -1.0282e-02, -7.8037e-03, -1.6162e-02,
        -1.5916e-02, -4.7095e-03, -1.4270e-02, -2.1049e-04, -8.7701e-03,
        -8.4597e-03, -8.3175e-03, -7.2362e-03, -1.5452e-02, -1.2712e-02,
        -9.5876e-03, -7.0389e-03, -8.7610e-03, -1.1765e-02, -1.0088e-02,
        -1.0642e-02, -9.0912e-03, -9.1835e-03, -1.2583e-02, -1.3255e-02,
        -1.1510e-02, -1.3113e-02, -1.3486e-02, -8.1706e-03, -1.3283e-02,
        -7.0980e-03, -9.6307e-03, -6.0296e-03, -8.5515e-03, -8.0637e-03,
        -1.1838e-02, -8.3078e-03, -1.1856e-02, -1.0561e-02, -1.2223e-02,
        -9.0641e-03, -8.6713e-03, -1.5202e-02, -1.1427e-02, -4.0358e-03,
        -1.4285e-02, -1.1342e-02, -1.4369e-02, -9.0434e-03, -1.2655e-02,
        -1.1490e-02, -1.5013e-02, -1.3723e-02, -8.9988e-03, -1.1513e-02,
        -9.6586e-03, -2.3822e-03, -1.3973e-02, -1.3048e-02, -8.6717e-03,
        -1.0772e-02, -1.4940e-02, -1.1049e-02, -9.4741e-03, -1.0158e-02,
        -7.8975e-03, -5.7049e-03, -1.6223e-02, -1.2245e-02, -9.8343e-03,
        -7.0829e-03, -1.1979e-02, -2.9953e-03, -1.3763e-02, -1.3266e-02,
        -1.3973e-02, -9.4622e-03, -2.9787e-03, -1.4156e-02, -1.4046e-02,
        -1.3725e-02, -8.4031e-03, -1.4946e-02, -7.5082e-03, -8.2367e-03,
        -1.0366e-02, -7.9162e-03, -1.0152e-02, -1.0161e-02, -7.5150e-03,
        -7.9076e-03, -1.7805e-02, -9.5272e-03, -1.0919e-02, -1.0188e-02,
        -7.8698e-03, -1.0143e-02, -6.7257e-03, -1.1833e-02, -1.7499e-02,
        -1.3020e-02, -8.6207e-03, -9.7280e-03, -1.0531e-02, -4.9277e-03,
        -1.0589e-02, -4.5984e-03, -6.6730e-03, -9.0888e-03, -9.0904e-03,
        -8.0838e-03, -8.7867e-03,  2.8464e-03, -9.5797e-03, -7.5537e-03,
        -1.1644e-02, -6.9192e-03, -1.0140e-02, -1.1305e-03, -6.6451e-03,
        -9.3208e-03, -7.6125e-03, -1.3230e-02, -1.0010e-02, -8.7837e-03,
        -9.3678e-03, -1.0288e-02, -5.3251e-03, -4.4896e-03, -6.8524e-03,
        -5.3668e-03, -8.7361e-03, -1.1290e-02, -1.0277e-02, -1.0646e-02,
        -9.7786e-03, -1.0551e-02, -4.4154e-03, -8.4695e-03, -7.3143e-03,
        -1.6230e-02, -8.6974e-03, -1.0015e-02, -1.0087e-02, -1.0389e-02,
        -5.1425e-03, -1.7518e-03, -1.0189e-02, -9.7441e-03, -7.3556e-03,
        -1.0745e-02, -1.1236e-02, -1.0746e-02, -1.1897e-02, -7.5228e-03,
        -1.2562e-02, -9.6385e-03, -1.2318e-02, -8.4284e-03, -5.6423e-03,
        -3.4671e-03, -1.1231e-02, -7.6334e-03, -7.7781e-03, -1.7041e-02,
        -8.7425e-03, -1.0138e-02, -1.1738e-02, -9.2426e-03, -6.7512e-03,
        -1.4218e-02, -7.7344e-03, -1.2443e-02, -4.8078e-03, -1.3206e-02,
        -2.4260e-03, -6.8994e-03, -1.5488e-02, -1.1003e-02, -1.1076e-02,
        -1.2128e-02, -1.5059e-02, -1.2565e-02, -1.0485e-02, -9.8076e-03,
        -1.4754e-02, -1.0278e-02, -1.1558e-02, -1.1449e-02, -1.1101e-02,
        -1.2706e-02, -1.1449e-02, -1.2841e-03,  2.9033e-03, -1.3756e-02,
        -1.0485e-02, -1.6264e-02, -1.1252e-02, -1.5853e-02, -1.2751e-02,
        -5.6369e-03, -1.2733e-02, -2.5452e-03, -4.9072e-03, -1.0969e-02,
        -1.1285e-02, -9.4681e-03, -7.5553e-03, -1.6628e-02, -9.0116e-03,
        -1.0582e-02, -1.1719e-02, -1.7954e-02, -5.5719e-03, -9.1669e-03,
        -1.0087e-02, -1.8120e-02, -8.7255e-03, -1.1951e-02, -1.2198e-02,
        -1.0286e-02, -6.9940e-03, -1.3057e-02, -1.0135e-02, -4.6636e-03,
        -1.0507e-02, -9.5960e-03, -1.4260e-02, -1.1381e-02, -6.0499e-03,
        -8.8328e-03, -1.3097e-02, -7.9745e-03, -1.6354e-02, -1.3337e-02,
        -4.0141e-03, -1.4092e-02, -8.8039e-03, -8.5836e-03, -7.7194e-03,
        -7.6265e-03, -1.4319e-02, -5.3277e-03, -1.2154e-02, -9.1801e-03,
        -1.5162e-02, -1.1860e-02, -6.5631e-03, -1.2060e-02, -1.4395e-02,
        -1.1312e-02, -6.2936e-03, -1.4116e-02, -5.8795e-03, -1.3229e-02,
        -1.0674e-02, -3.8636e-03, -5.3233e-03, -1.7664e-02, -1.2018e-02,
        -8.1730e-03, -7.1603e-03, -1.1294e-02, -7.8095e-03, -1.1986e-02,
        -1.1031e-02, -1.0301e-02, -7.4502e-03, -1.3022e-02, -1.0421e-02,
        -8.2435e-03, -1.8367e-02, -8.3645e-03, -9.0954e-03, -9.9517e-03,
        -5.0940e-03, -9.9939e-03, -4.3818e-03, -1.1749e-02,  3.4863e-03,
        -9.4246e-03, -1.0265e-02, -1.2930e-02, -9.3730e-03, -1.0364e-02,
        -1.6326e-02, -6.0384e-03, -1.1753e-02, -9.5873e-03, -7.4485e-03,
        -7.7598e-03, -1.4272e-02], device='cuda:0'), tensor([[ 0.0020,  0.0775,  0.0491,  ..., -0.0437, -0.0405,  0.0303],
        [-0.0054, -0.0431, -0.0113,  ..., -0.0458, -0.0183, -0.0475],
        [-0.0389, -0.0099,  0.0429,  ..., -0.0005,  0.0068,  0.0076],
        ...,
        [-0.0372,  0.0431, -0.0286,  ...,  0.0431,  0.0189,  0.0419],
        [ 0.0428, -0.0469,  0.0311,  ...,  0.0092, -0.0117, -0.0353],
        [ 0.0480, -0.0052,  0.0233,  ...,  0.0330, -0.0426,  0.0101]],
       device='cuda:0'), tensor([-0.0260, -0.0251, -0.0002,  ..., -0.0498,  0.0370, -0.0471],
       device='cuda:0'), tensor([[[-0.0113, -0.0104, -0.0145],
         [-0.0086, -0.0326, -0.0303],
         [-0.0131,  0.0101, -0.0289],
         ...,
         [ 0.0192, -0.0286,  0.0094],
         [-0.0009, -0.0067,  0.0054],
         [-0.0205,  0.0212, -0.0109]],

        [[ 0.0144, -0.0124, -0.0024],
         [-0.0032,  0.0220,  0.0259],
         [-0.0187,  0.0092, -0.0093],
         ...,
         [-0.0067, -0.0227,  0.0129],
         [-0.0030, -0.0023, -0.0050],
         [-0.0159, -0.0187, -0.0108]],

        [[-0.0156,  0.0229,  0.0060],
         [-0.0004, -0.0060, -0.0057],
         [ 0.0191, -0.0274,  0.0135],
         ...,
         [-0.0183,  0.0282, -0.0171],
         [ 0.0054,  0.0060, -0.0245],
         [ 0.0006,  0.0250, -0.0073]],

        ...,

        [[ 0.0207,  0.0043,  0.0107],
         [ 0.0048,  0.0059,  0.0119],
         [-0.0103, -0.0071,  0.0075],
         ...,
         [ 0.0187, -0.0081,  0.0296],
         [ 0.0179, -0.0120, -0.0041],
         [-0.0110, -0.0173,  0.0177]],

        [[-0.0218,  0.0157,  0.0126],
         [ 0.0319, -0.0058, -0.0335],
         [ 0.0155, -0.0177,  0.0197],
         ...,
         [ 0.0116,  0.0043,  0.0202],
         [ 0.0069,  0.0243,  0.0051],
         [-0.0064, -0.0134,  0.0043]],

        [[-0.0133,  0.0127, -0.0118],
         [ 0.0144,  0.0024, -0.0046],
         [ 0.0138,  0.0006,  0.0141],
         ...,
         [ 0.0045,  0.0106,  0.0171],
         [ 0.0120,  0.0024, -0.0181],
         [ 0.0179,  0.0115, -0.0151]]], device='cuda:0'), tensor([-2.2884e-02, -3.5908e-03,  1.3492e-02, -2.4097e-02, -1.0822e-02,
        -1.0507e-02,  1.5885e-02, -1.4283e-02, -2.1773e-02, -1.4946e-02,
         8.3045e-03, -1.0142e-02, -2.1086e-02,  1.3094e-02, -2.1164e-02,
         8.2189e-03,  2.1008e-03, -4.1609e-02,  1.9918e-04,  1.2298e-02,
        -1.7243e-02, -1.3539e-03, -2.7801e-03, -4.5089e-03, -3.5834e-02,
        -5.0254e-03,  1.6850e-02, -3.4831e-02,  1.7236e-02, -2.6522e-02,
        -3.2291e-03,  1.8869e-02, -1.8589e-02, -7.4076e-03, -1.2001e-03,
        -1.0391e-02,  1.4151e-02, -1.3648e-02,  1.9018e-02, -2.2613e-02,
        -7.5351e-03,  1.0016e-02,  5.2245e-03, -2.5503e-03, -4.1750e-03,
         2.5512e-03,  1.1061e-02,  8.0120e-04,  2.2295e-02, -1.5713e-02,
        -2.5984e-02, -1.4023e-02, -1.0814e-03,  1.1934e-02,  2.0644e-02,
        -7.5623e-03, -2.9865e-03, -1.4176e-02,  7.6807e-03,  2.4131e-02,
         1.7405e-02,  5.1621e-06, -2.0989e-02,  2.3640e-02, -5.8470e-03,
        -1.5206e-03, -2.8725e-02, -1.8734e-02, -5.7485e-03,  2.3051e-02,
        -4.1720e-02,  1.8577e-02, -7.0476e-03,  2.9099e-03, -1.8058e-02,
        -1.7403e-02, -1.1119e-02, -2.2261e-02,  2.1590e-03,  7.4009e-03,
         5.5718e-03, -1.9530e-02,  2.0789e-02,  1.1042e-02, -9.9261e-03,
         8.4938e-03,  2.5848e-02,  1.9855e-02, -6.3734e-03, -2.3843e-02,
         9.9933e-04,  1.4180e-02,  2.3513e-02, -8.2194e-04,  1.2073e-02,
        -4.8479e-03,  1.4776e-02,  1.9181e-02, -1.2098e-02, -1.0796e-02,
        -1.5604e-02,  2.2388e-02,  7.0786e-03, -2.2413e-02, -2.4551e-02,
         9.9324e-03, -2.3313e-02, -2.5877e-02, -2.6438e-03,  2.3318e-02,
        -5.2260e-03, -1.8094e-02,  2.1545e-02, -1.3007e-03,  1.8911e-02,
        -1.6742e-03, -9.5219e-03, -2.4908e-02, -1.3263e-03, -3.5712e-02,
        -2.8172e-02,  1.2671e-02,  1.2823e-02, -1.4357e-02, -5.4603e-03,
         1.2897e-02, -1.3963e-02,  1.7146e-02,  1.1977e-03,  1.7611e-02,
         1.6336e-02, -2.0276e-02, -2.9061e-02,  8.4748e-03,  1.4895e-02,
         3.4324e-03,  1.7978e-02, -2.6064e-03,  3.5858e-03, -7.1005e-03,
        -1.5397e-02,  8.5722e-03,  3.8906e-02, -2.7500e-02,  5.3524e-03,
         1.7644e-02, -1.9368e-02, -5.8792e-03, -3.8277e-03,  2.5268e-02,
         1.6080e-02,  1.1602e-02,  1.5932e-03,  1.6775e-02, -8.4731e-03,
        -1.6099e-02,  2.2646e-02,  2.2648e-02, -3.2282e-02,  3.1033e-02,
        -1.2548e-02,  5.6761e-03, -3.7759e-03,  1.8718e-02,  1.5974e-02,
         1.1509e-02,  1.4333e-02, -6.2919e-03, -1.1187e-02, -1.2606e-02,
         2.8385e-02,  1.6733e-02, -1.8983e-02,  3.1567e-02, -1.0417e-02,
        -2.3924e-02, -9.5101e-03, -1.2682e-02, -1.4771e-02,  7.1986e-03,
         1.1170e-02,  2.3967e-02, -2.1399e-02, -7.3925e-03, -2.3360e-02,
        -2.9626e-02, -1.0834e-02,  6.0677e-03,  9.4328e-03, -8.6035e-03,
        -2.8889e-03, -2.5429e-03,  1.4270e-03, -4.3981e-03,  1.0795e-02,
         1.8538e-02,  2.1532e-02,  1.7468e-02, -6.6637e-03,  3.9508e-02,
         2.4176e-02, -1.3699e-02, -6.6296e-03, -1.5557e-02,  2.2077e-02,
         5.4004e-03, -7.1404e-03, -2.6291e-02,  1.2414e-02,  1.4104e-02,
        -1.2503e-02, -1.4925e-02, -1.2146e-02,  2.0538e-02,  1.7199e-02,
         1.2465e-02, -2.4632e-02,  2.2476e-02, -2.4786e-02, -1.9248e-02,
         2.6480e-03, -1.6415e-04,  2.5113e-02,  1.1126e-02, -2.4353e-02,
         2.1924e-02,  1.6041e-03, -2.2669e-02, -5.1279e-03, -2.3276e-02,
        -1.9850e-02,  1.6229e-02, -2.3291e-02, -4.2372e-03,  3.2297e-02,
        -6.9983e-03, -1.9718e-02, -3.0457e-02,  2.8978e-03,  6.9398e-03,
         1.2506e-02,  1.4158e-02, -1.8993e-02,  7.9062e-03,  1.7319e-02,
         2.3923e-02,  6.7247e-03, -1.2243e-02, -1.0408e-02,  2.0417e-02,
         7.9447e-03,  1.2370e-03, -2.0635e-02,  1.5657e-02, -2.7965e-03,
         1.0743e-02,  2.6963e-02,  1.6747e-02,  9.1064e-03,  7.6898e-03,
         1.7252e-02, -2.0630e-02,  7.2173e-04, -6.9228e-03, -1.3536e-02,
         1.6743e-02,  1.5329e-02,  1.1003e-02,  2.1868e-04,  1.4199e-03,
         1.9472e-02,  6.1415e-03, -1.3245e-03,  1.0704e-02, -9.5271e-03,
        -2.4666e-02, -1.5972e-02, -1.2450e-03, -2.9797e-02,  1.2441e-02,
         1.2009e-02, -2.1507e-02,  1.5235e-02,  2.2260e-03, -6.4242e-03,
         6.0771e-03, -9.7117e-03, -1.7592e-03, -2.2643e-02,  2.0561e-02,
         1.0047e-02,  3.3287e-02, -2.0040e-02,  8.4761e-03, -2.5990e-02,
         1.7895e-02,  2.3998e-02,  3.2725e-02, -1.4328e-02,  2.1628e-02,
        -2.2289e-02, -2.2161e-02,  8.3621e-03, -1.2953e-02,  1.9421e-02,
        -7.0639e-03, -1.8031e-02,  1.4155e-03, -3.4411e-03, -1.0251e-02,
         2.7744e-02, -1.3513e-02,  1.8261e-02, -1.3042e-03, -4.5372e-03,
         8.0956e-03,  1.3447e-02, -1.0261e-02,  1.9148e-02, -7.3796e-03,
        -9.5077e-03, -2.6059e-02, -4.5118e-03, -9.0110e-03, -5.4146e-03,
        -1.6249e-02, -1.1893e-02, -1.4358e-03, -1.2025e-02,  1.3481e-02,
        -1.8436e-02, -1.6071e-02, -1.1037e-02,  1.0580e-02, -4.4027e-03,
         2.9652e-04, -1.3444e-02, -8.8155e-03,  1.2599e-02, -4.7517e-03,
        -5.8838e-03, -2.0031e-02,  1.8470e-02, -3.7320e-03, -1.3231e-02,
         6.9485e-03, -2.2529e-02, -1.1816e-02,  2.1245e-02, -1.3933e-02,
         3.3613e-03, -7.2979e-03, -1.4305e-02, -2.8815e-02,  1.0444e-02,
        -1.2403e-02,  7.9472e-03, -4.3342e-03, -2.1922e-02, -2.0367e-02,
        -2.3181e-02,  1.5435e-02, -1.6837e-02,  2.0357e-02,  5.7448e-03,
        -1.9309e-02,  1.2821e-02,  1.3450e-02, -6.8439e-03, -1.8704e-02,
        -2.4708e-03, -1.1607e-03, -6.8186e-03,  1.0381e-02,  1.2140e-02,
        -1.0817e-03, -1.1133e-02, -4.6521e-03,  6.2212e-03,  1.6894e-02,
        -3.0325e-02,  1.9527e-02, -2.3520e-02,  1.3020e-02,  8.0578e-03,
         1.8232e-02,  2.3060e-02, -2.5656e-02, -1.8418e-04,  1.7364e-02,
         2.4281e-03,  1.4055e-03, -6.3249e-03, -1.1813e-02, -1.5215e-02,
        -1.3751e-02, -3.6377e-02,  1.0567e-02, -1.9329e-02,  3.1433e-02,
         1.2203e-02, -2.4769e-03,  1.4178e-02,  2.6001e-02,  8.3833e-03,
         7.1916e-03, -2.8460e-02,  2.3277e-02, -1.0160e-02,  3.7887e-03,
        -2.1270e-02,  1.2986e-03,  3.7341e-02,  3.1490e-04, -2.7742e-03,
        -4.8457e-03, -1.4664e-02, -2.1698e-02,  1.4831e-02,  2.6511e-02,
         1.0769e-02,  2.1796e-02, -9.5562e-03,  1.3190e-03,  1.1562e-02,
         2.0167e-03, -6.3352e-03, -1.6378e-03, -2.2921e-02,  1.7712e-02,
         2.1982e-02,  8.1461e-03,  1.8073e-04, -3.9818e-03, -1.3568e-03,
        -2.2557e-02,  2.4591e-02, -4.3857e-03, -2.2802e-02,  2.2615e-02,
        -3.6117e-03, -5.2874e-03, -2.4946e-02,  3.8824e-03, -1.0204e-02,
         6.1955e-03,  1.9748e-02,  9.4521e-03, -5.2195e-03,  3.6559e-03,
        -2.4196e-02,  9.5389e-03,  1.0284e-02, -2.1519e-02, -1.6206e-02,
        -2.5231e-03,  2.7842e-02,  2.0260e-02, -3.1052e-03,  1.1352e-02,
         1.5634e-02, -1.3069e-02,  8.0380e-03, -2.1180e-03, -1.0899e-02,
        -1.8528e-02,  9.9104e-03,  2.0698e-02, -2.1281e-02,  4.7413e-03,
         1.6137e-02,  1.5772e-02,  2.0182e-03, -1.8617e-02,  9.6826e-03,
        -9.5666e-03, -2.3304e-02,  1.9593e-02,  2.8219e-02,  1.9406e-03,
         2.1538e-03,  1.9762e-02, -1.6806e-02,  1.9049e-02, -2.1281e-02,
        -5.2009e-03, -1.2530e-02, -2.2254e-02,  9.3205e-03, -8.0086e-03,
         1.6671e-02, -2.8139e-02,  6.5929e-03, -1.5815e-02, -1.9444e-02,
         2.0069e-02, -1.1918e-02, -1.9865e-02, -3.9162e-03, -8.1217e-03,
        -1.4074e-02, -5.7265e-03, -1.6929e-02,  1.9304e-02,  2.2862e-02,
         2.2904e-02, -1.0373e-02, -1.7248e-02,  1.2011e-02, -1.1834e-02,
        -1.3581e-02, -5.9287e-03], device='cuda:0'), tensor([[[-0.0033,  0.0259,  0.0214, -0.0079, -0.0070],
         [ 0.0197,  0.0079,  0.0253, -0.0086,  0.0267],
         [-0.0061,  0.0028, -0.0213, -0.0294, -0.0302],
         ...,
         [ 0.0080, -0.0093,  0.0070, -0.0029, -0.0069],
         [ 0.0299, -0.0158, -0.0074,  0.0031,  0.0141],
         [-0.0191, -0.0022, -0.0088, -0.0313,  0.0048]],

        [[-0.0116, -0.0025, -0.0004, -0.0144,  0.0298],
         [ 0.0333,  0.0039, -0.0265, -0.0121,  0.0014],
         [-0.0046, -0.0093,  0.0072, -0.0377, -0.0160],
         ...,
         [ 0.0016, -0.0051,  0.0260,  0.0154, -0.0095],
         [ 0.0190,  0.0109,  0.0097,  0.0194,  0.0202],
         [-0.0029,  0.0143, -0.0208, -0.0083, -0.0140]],

        [[ 0.0004, -0.0122, -0.0264, -0.0036,  0.0173],
         [ 0.0171, -0.0244,  0.0134, -0.0181,  0.0081],
         [ 0.0050,  0.0142, -0.0162,  0.0140,  0.0212],
         ...,
         [-0.0063, -0.0330,  0.0048, -0.0303, -0.0299],
         [-0.0278,  0.0114, -0.0150, -0.0336, -0.0187],
         [-0.0229, -0.0249,  0.0170,  0.0215, -0.0130]],

        ...,

        [[-0.0096, -0.0229, -0.0086, -0.0287, -0.0045],
         [-0.0011,  0.0048,  0.0195,  0.0095,  0.0016],
         [ 0.0045,  0.0013, -0.0020,  0.0058, -0.0172],
         ...,
         [ 0.0038,  0.0210,  0.0034, -0.0165, -0.0004],
         [ 0.0068, -0.0041, -0.0002, -0.0163, -0.0035],
         [ 0.0043,  0.0278,  0.0107,  0.0008,  0.0282]],

        [[-0.0380, -0.0101,  0.0283,  0.0422,  0.0267],
         [-0.0142, -0.0071,  0.0368,  0.0316,  0.0044],
         [-0.0188,  0.0027, -0.0068,  0.0069, -0.0054],
         ...,
         [ 0.0007, -0.0079, -0.0202, -0.0482, -0.0766],
         [ 0.0073, -0.0298,  0.0090, -0.0061, -0.0213],
         [-0.0056, -0.0095,  0.0197,  0.0004,  0.0096]],

        [[ 0.0095, -0.0119, -0.0116, -0.0060, -0.0014],
         [ 0.0002,  0.0134, -0.0027,  0.0134, -0.0078],
         [-0.0197,  0.0101,  0.0027,  0.0289,  0.0004],
         ...,
         [ 0.0067, -0.0115,  0.0099,  0.0008,  0.0020],
         [ 0.0046,  0.0053,  0.0094,  0.0055,  0.0078],
         [-0.0036, -0.0193,  0.0090,  0.0146, -0.0031]]], device='cuda:0'), tensor([-0.0050,  0.0179,  0.0249,  ...,  0.0184,  0.0045, -0.0148],
       device='cuda:0'), tensor([1.0444, 0.9123, 1.0015,  ..., 0.8587, 0.9461, 0.7132], device='cuda:0'), tensor([-0.0364, -0.0148,  0.0135,  ..., -0.0266, -0.0172, -0.0184],
       device='cuda:0'), tensor([[[-1.0514e-02,  1.8788e-02,  2.4377e-02,  2.9620e-02,  3.5638e-02],
         [ 8.9881e-03, -3.0438e-03,  7.5787e-04, -8.4064e-03, -1.0937e-02],
         [ 5.5974e-03,  8.3114e-03, -1.6244e-03, -6.3286e-03,  1.1216e-02],
         ...,
         [ 2.2473e-02,  4.5309e-03, -9.2964e-03, -4.7650e-02, -7.4285e-02],
         [-1.2262e-03,  1.8412e-02, -3.1861e-03, -8.1364e-04, -7.4907e-03],
         [-1.4332e-02, -2.3969e-02, -2.6474e-02, -2.5442e-02, -1.6311e-02]],

        [[ 2.8738e-02,  5.6309e-03,  6.2493e-03, -3.4792e-03, -5.4901e-03],
         [-1.3956e-02, -5.2752e-03, -1.2586e-02, -2.3148e-02, -2.1450e-02],
         [ 8.6205e-03, -1.0756e-02, -6.9080e-03,  1.0979e-02,  2.5118e-03],
         ...,
         [ 7.1209e-03, -9.7735e-04, -5.0360e-03, -8.6299e-03, -1.7910e-02],
         [-2.9206e-02,  3.3325e-03,  5.7667e-03,  1.6804e-02,  3.2391e-03],
         [ 2.2107e-03,  4.4477e-03,  1.3329e-03,  2.2029e-02,  4.0259e-02]],

        [[ 8.5330e-03,  9.7079e-04,  2.6971e-02,  5.1513e-03,  3.8212e-03],
         [-1.9455e-02, -2.1191e-03, -1.6037e-02, -1.5491e-02, -1.6148e-02],
         [-6.2105e-03, -1.4922e-02, -1.1308e-02, -1.0359e-02, -2.3868e-02],
         ...,
         [ 1.8960e-02,  3.0556e-02,  2.9621e-02,  3.1097e-02,  3.4081e-02],
         [-5.2770e-03, -1.4901e-02, -2.8027e-03,  7.7087e-03, -1.2911e-02],
         [ 2.7392e-02, -4.7156e-03,  3.0368e-03, -9.8318e-03, -7.4481e-03]],

        ...,

        [[-1.6887e-02, -1.0254e-02,  7.5026e-03, -2.4567e-02, -1.0823e-02],
         [ 1.2408e-02,  2.1716e-02,  1.8116e-02,  1.5131e-02,  2.0346e-03],
         [-6.5017e-03, -9.4654e-04, -3.1678e-02, -1.7571e-02, -2.5007e-02],
         ...,
         [ 1.5237e-03,  2.6250e-02,  4.2223e-02,  4.6959e-02,  6.9584e-02],
         [-7.1303e-05,  1.8906e-02, -6.3843e-03, -3.7077e-03,  1.5737e-02],
         [ 1.5226e-02,  7.7993e-04, -1.1899e-02, -1.6750e-02, -3.7997e-02]],

        [[ 2.5510e-03, -1.3987e-02, -3.5243e-03, -1.9611e-02, -1.3512e-02],
         [-1.6141e-02, -4.2650e-03, -1.2163e-04,  3.3005e-03, -3.9028e-04],
         [ 3.8671e-04, -9.0336e-03, -4.5344e-03, -8.7932e-03,  1.8189e-03],
         ...,
         [ 1.2995e-02, -1.0161e-02,  6.7901e-03,  1.3327e-02,  3.8556e-03],
         [ 1.3197e-02,  1.1266e-02,  6.6942e-03,  1.6141e-03, -1.2089e-02],
         [ 1.5957e-02,  1.2746e-02,  5.3597e-03,  2.5193e-02,  1.9608e-02]],

        [[-1.6440e-02, -1.0343e-02, -9.1193e-03, -4.7292e-02, -2.0036e-02],
         [-1.2473e-02, -2.1043e-02, -1.5304e-02, -2.1663e-02, -5.9193e-03],
         [-1.3323e-03, -9.7603e-03,  2.8451e-02,  1.2115e-02,  3.8965e-02],
         ...,
         [-3.4471e-02, -4.0009e-02, -1.8917e-02, -2.6936e-02, -7.6853e-03],
         [-2.8591e-02,  1.1253e-02,  5.1127e-03, -3.8471e-03,  1.1262e-02],
         [ 1.1659e-02,  1.6893e-02,  3.2954e-02,  4.9568e-02,  4.2528e-02]]],
       device='cuda:0'), tensor([-0.0074, -0.0005,  0.0259,  ..., -0.0021,  0.0300,  0.0132],
       device='cuda:0'), tensor([1.0416, 0.9888, 0.9653,  ..., 0.9694, 0.9720, 0.9986], device='cuda:0'), tensor([-0.0404, -0.0186, -0.0279,  ..., -0.0226, -0.0218, -0.0204],
       device='cuda:0'), tensor([[ 4.4309e-02,  3.9371e-02, -5.1609e-02,  ..., -1.1257e-02,
          3.3692e-05,  3.5064e-02],
        [ 2.1661e-02, -1.6134e-02,  1.4806e-02,  ..., -3.5443e-02,
         -1.8742e-03,  6.6259e-03],
        [-3.3125e-02,  9.9184e-03,  1.6615e-03,  ...,  3.6538e-02,
          2.6535e-02, -1.0541e-02],
        ...,
        [-1.1870e-03, -3.5578e-02, -4.7557e-02,  ..., -5.3403e-02,
          1.2425e-02,  1.0008e-02],
        [ 5.4938e-03, -2.4617e-02, -6.1044e-02,  ..., -1.5111e-02,
         -4.1240e-02,  3.1770e-02],
        [ 1.9410e-02,  5.2592e-02, -9.9516e-03,  ...,  2.0318e-02,
         -4.0070e-02, -1.7682e-03]], device='cuda:0'), tensor([-0.0345, -0.0225, -0.0326,  ...,  0.0118, -0.0077, -0.0004],
       device='cuda:0'), tensor([[[-0.0061],
         [-0.0376],
         [ 0.0428],
         ...,
         [-0.0221],
         [-0.0057],
         [-0.0221]],

        [[-0.0165],
         [-0.0051],
         [-0.0173],
         ...,
         [ 0.0097],
         [-0.0247],
         [-0.0253]],

        [[-0.0294],
         [-0.0234],
         [ 0.0270],
         ...,
         [ 0.0316],
         [ 0.0091],
         [ 0.0442]],

        ...,

        [[ 0.0147],
         [-0.0469],
         [-0.0290],
         ...,
         [ 0.0217],
         [ 0.0180],
         [-0.0150]],

        [[ 0.0242],
         [ 0.0186],
         [ 0.0407],
         ...,
         [ 0.0048],
         [ 0.0224],
         [-0.0306]],

        [[ 0.0085],
         [-0.0296],
         [-0.0072],
         ...,
         [-0.0175],
         [ 0.0285],
         [ 0.0216]]], device='cuda:0'), tensor([-0.0583,  0.0055,  0.0278,  ..., -0.0050,  0.0335,  0.0040],
       device='cuda:0'), tensor([[[-0.0310, -0.0208, -0.0161, -0.0028, -0.0099],
         [ 0.0038,  0.0063,  0.0224, -0.0072,  0.0025],
         [ 0.0064,  0.0020, -0.0149, -0.0051, -0.0112],
         ...,
         [-0.0015, -0.0109, -0.0096, -0.0154, -0.0380],
         [-0.0163,  0.0109,  0.0243,  0.0175,  0.0153],
         [-0.0046, -0.0094, -0.0199, -0.0229, -0.0095]],

        [[-0.0042,  0.0014,  0.0044,  0.0058, -0.0085],
         [-0.0073, -0.0105, -0.0240, -0.0145, -0.0033],
         [-0.0251,  0.0115,  0.0187,  0.0193,  0.0139],
         ...,
         [-0.0225,  0.0213,  0.0206, -0.0114, -0.0008],
         [ 0.0185,  0.0154,  0.0145, -0.0151, -0.0024],
         [ 0.0092,  0.0027,  0.0292,  0.0051,  0.0103]],

        [[-0.0202, -0.0034, -0.0215, -0.0264, -0.0071],
         [-0.0059,  0.0031, -0.0163, -0.0293,  0.0001],
         [ 0.0291, -0.0017, -0.0108,  0.0312, -0.0244],
         ...,
         [-0.0229,  0.0038,  0.0169,  0.0230,  0.0164],
         [-0.0259, -0.0099, -0.0114,  0.0161, -0.0104],
         [ 0.0057,  0.0067,  0.0101,  0.0126, -0.0059]],

        ...,

        [[-0.0147, -0.0339, -0.0203, -0.0022, -0.0079],
         [ 0.0068, -0.0052, -0.0154,  0.0076,  0.0318],
         [ 0.0118,  0.0112,  0.0240,  0.0052,  0.0050],
         ...,
         [-0.0369, -0.0130, -0.0328, -0.0116,  0.0161],
         [ 0.0021,  0.0148,  0.0138, -0.0030,  0.0024],
         [ 0.0008, -0.0141, -0.0084,  0.0053,  0.0007]],

        [[-0.0091,  0.0179,  0.0144,  0.0127, -0.0083],
         [ 0.0652,  0.0506,  0.0317,  0.0331,  0.0293],
         [-0.0315, -0.0206, -0.0254, -0.0321, -0.0174],
         ...,
         [ 0.0101,  0.0094,  0.0207,  0.0108,  0.0359],
         [-0.0255,  0.0040,  0.0130,  0.0100,  0.0218],
         [ 0.0332,  0.0106,  0.0080, -0.0155, -0.0177]],

        [[ 0.0042,  0.0049, -0.0039,  0.0010,  0.0260],
         [ 0.0099,  0.0055, -0.0173,  0.0058,  0.0025],
         [-0.0058, -0.0151, -0.0100,  0.0064,  0.0047],
         ...,
         [-0.0083, -0.0327, -0.0089, -0.0395, -0.0482],
         [ 0.0205,  0.0216,  0.0235,  0.0048, -0.0124],
         [-0.0106, -0.0118, -0.0212, -0.0343, -0.0339]]], device='cuda:0'), tensor([ 0.0071,  0.0033,  0.0382,  ...,  0.0026, -0.0262,  0.0141],
       device='cuda:0'), tensor([0.9486, 0.9385, 1.0108,  ..., 1.0076, 1.0495, 0.9793], device='cuda:0'), tensor([-0.0580, -0.0431,  0.0298,  ..., -0.0318, -0.0399, -0.0171],
       device='cuda:0'), tensor([[[-8.3191e-03, -3.8672e-03, -2.8202e-02, -1.7424e-02, -9.6373e-03],
         [ 6.9335e-03,  1.3212e-02,  7.5499e-03,  1.0975e-02,  4.5511e-02],
         [ 7.2716e-03,  3.0909e-02,  1.6831e-02,  7.2857e-03,  1.0943e-02],
         ...,
         [-2.1146e-02, -1.1391e-02,  3.3571e-03, -9.8933e-03, -2.9745e-02],
         [ 2.5773e-02,  1.4845e-03, -8.4909e-04,  7.0825e-03,  5.4153e-03],
         [ 8.0605e-04,  1.9133e-02,  7.0442e-03, -4.8900e-03,  2.3089e-03]],

        [[-1.3639e-02, -1.8170e-02, -5.3011e-03, -1.6134e-02,  1.3638e-02],
         [ 1.1032e-02,  4.3804e-02, -9.6689e-03,  7.7409e-03,  2.0397e-02],
         [ 1.2014e-03, -2.1697e-02,  4.4446e-03,  5.5264e-03, -7.0902e-04],
         ...,
         [ 1.4815e-03,  3.3407e-03, -3.7350e-03,  2.5306e-03, -2.5559e-03],
         [-7.0863e-03,  1.2321e-02,  8.4211e-03,  1.9713e-02,  6.3518e-03],
         [ 2.4236e-03,  1.4554e-02, -1.4212e-02, -1.3450e-02, -2.4300e-02]],

        [[ 8.3749e-03,  1.6059e-02,  9.2098e-03,  1.7544e-02,  6.0219e-03],
         [ 1.6555e-02, -2.9485e-03,  2.8245e-02,  2.5210e-02,  1.1221e-02],
         [-1.3433e-02,  6.9576e-03, -2.9792e-02, -2.8783e-02, -2.3868e-03],
         ...,
         [-2.6607e-02,  1.4989e-03,  6.9750e-03,  3.4307e-03,  1.1972e-02],
         [ 1.5253e-02, -1.0191e-02, -3.0866e-02, -1.2721e-02,  7.6711e-06],
         [ 2.7950e-02, -2.7288e-03,  2.8412e-02,  1.5718e-02, -3.4353e-03]],

        ...,

        [[-1.8799e-04,  1.3961e-03, -1.2754e-02, -3.5109e-03, -6.3037e-03],
         [ 1.2657e-02, -2.1205e-03, -1.8837e-02, -8.1460e-03,  2.0482e-02],
         [-2.8233e-02, -3.2601e-02, -2.5299e-02, -4.1830e-02, -6.2675e-02],
         ...,
         [-7.6521e-03, -1.1087e-02, -4.5692e-03,  6.6626e-03,  5.7224e-03],
         [-4.5799e-03, -6.2888e-03,  1.3430e-02,  2.1649e-02,  1.3747e-02],
         [ 4.6888e-02,  2.8341e-02, -3.6423e-03,  2.1915e-02,  6.9296e-03]],

        [[ 1.7070e-02, -4.5827e-03,  1.3062e-02,  5.1413e-03,  2.5360e-03],
         [ 1.3370e-02,  3.0325e-03,  7.3627e-03,  2.8757e-02,  1.3210e-02],
         [ 3.2896e-02,  3.1448e-02,  3.0831e-02,  1.1497e-02, -2.3750e-02],
         ...,
         [-6.5419e-03, -4.6544e-04, -1.1006e-02,  6.2904e-03,  2.3271e-02],
         [ 9.9936e-03,  4.6532e-03, -2.9018e-04, -1.5867e-03, -6.4450e-03],
         [-1.7729e-02,  4.6408e-03, -2.5822e-02, -1.3838e-02,  1.9980e-02]],

        [[-2.2033e-03,  3.5200e-02,  1.5181e-02, -5.1098e-03, -1.6544e-02],
         [-2.8190e-03, -1.0351e-02, -1.7058e-02,  3.8991e-03,  2.3497e-02],
         [ 1.7224e-02,  8.8760e-04, -1.0176e-02,  3.3625e-03,  7.3474e-03],
         ...,
         [-1.6078e-02, -3.3560e-02,  4.3181e-03, -5.5291e-03,  6.3422e-03],
         [ 2.3554e-02, -2.5787e-03, -9.8483e-03, -5.5423e-03, -3.1073e-03],
         [-6.0165e-03, -1.6920e-02, -3.3901e-03, -6.1887e-03, -8.4859e-03]]],
       device='cuda:0'), tensor([ 0.0011, -0.0159,  0.0039,  ...,  0.0054, -0.0007,  0.0085],
       device='cuda:0'), tensor([0.9569, 0.9976, 0.9742,  ..., 0.9668, 1.0324, 0.9517], device='cuda:0'), tensor([-0.0427, -0.0399, -0.0606,  ..., -0.0438, -0.0547, -0.0468],
       device='cuda:0'), tensor([[ 0.0175, -0.0168, -0.0388,  ...,  0.0367,  0.0493,  0.0049],
        [ 0.0471, -0.0245, -0.0054,  ..., -0.0197, -0.0202, -0.0358],
        [-0.0498, -0.0366, -0.0179,  ...,  0.0107, -0.0426,  0.0377],
        ...,
        [-0.0307, -0.0454, -0.0248,  ...,  0.0461, -0.0102,  0.0131],
        [ 0.0114, -0.0420, -0.0404,  ...,  0.0470,  0.0402, -0.0115],
        [ 0.0510, -0.0110, -0.0044,  ...,  0.0351, -0.0137,  0.0030]],
       device='cuda:0'), tensor([-0.0419, -0.0287,  0.0217,  ..., -0.0507,  0.0512,  0.0326],
       device='cuda:0'), tensor([[[ 0.0169, -0.0137, -0.0180, -0.0011, -0.0025],
         [ 0.0272,  0.0141, -0.0025,  0.0072, -0.0200],
         [ 0.0268, -0.0040, -0.0104, -0.0194, -0.0231],
         ...,
         [-0.0350, -0.0389,  0.0353,  0.0038, -0.0069],
         [-0.0136, -0.0150, -0.0378, -0.0135, -0.0078],
         [ 0.0007,  0.0026,  0.0397,  0.0016, -0.0175]],

        [[ 0.0100, -0.0012, -0.0070,  0.0009, -0.0041],
         [-0.0121,  0.0078,  0.0136,  0.0200, -0.0159],
         [-0.0259, -0.0042, -0.0124, -0.0059,  0.0072],
         ...,
         [ 0.0121,  0.0015,  0.0266,  0.0394,  0.0326],
         [ 0.0167, -0.0073,  0.0240, -0.0110,  0.0023],
         [ 0.0083, -0.0076, -0.0129, -0.0099, -0.0290]],

        [[-0.0088,  0.0012,  0.0023, -0.0088, -0.0017],
         [-0.0106,  0.0342,  0.0004, -0.0137, -0.0079],
         [-0.0121,  0.0275, -0.0182,  0.0146,  0.0064],
         ...,
         [ 0.0196,  0.0351, -0.0051,  0.0165,  0.0232],
         [-0.0134, -0.0201,  0.0151, -0.0256,  0.0104],
         [ 0.0180, -0.0115, -0.0107,  0.0096, -0.0048]],

        ...,

        [[ 0.0066,  0.0174, -0.0262, -0.0275, -0.0050],
         [-0.0134, -0.0011,  0.0047, -0.0154, -0.0220],
         [ 0.0091,  0.0209,  0.0179,  0.0300,  0.0191],
         ...,
         [ 0.0112,  0.0101, -0.0545,  0.0122,  0.0104],
         [ 0.0057,  0.0002,  0.0184,  0.0014, -0.0198],
         [ 0.0010, -0.0259, -0.0038, -0.0002, -0.0049]],

        [[-0.0147, -0.0126, -0.0029, -0.0156, -0.0222],
         [-0.0030,  0.0177,  0.0368,  0.0113, -0.0085],
         [-0.0064,  0.0061, -0.0211, -0.0071,  0.0137],
         ...,
         [-0.0045, -0.0024,  0.0327,  0.0165,  0.0307],
         [ 0.0174,  0.0238,  0.0011,  0.0209, -0.0034],
         [ 0.0257, -0.0041, -0.0227,  0.0025, -0.0127]],

        [[ 0.0039, -0.0154,  0.0361, -0.0049, -0.0257],
         [-0.0013,  0.0230, -0.0218,  0.0090, -0.0035],
         [-0.0175,  0.0193, -0.0194, -0.0228, -0.0079],
         ...,
         [-0.0300, -0.0085,  0.0166, -0.0239,  0.0120],
         [ 0.0173, -0.0018,  0.0200, -0.0212,  0.0165],
         [-0.0134, -0.0006,  0.0135, -0.0073,  0.0155]]], device='cuda:0'), tensor([-0.0234,  0.0115, -0.0300, -0.0045,  0.0026,  0.0338, -0.0078,  0.0020,
         0.0242,  0.0105,  0.0132,  0.0078,  0.0033, -0.0010, -0.0175, -0.0610,
        -0.0177, -0.0148,  0.0180, -0.0269,  0.0107, -0.0224, -0.0178,  0.0230,
        -0.0188,  0.0091, -0.0070, -0.0303, -0.0237, -0.0216, -0.0012,  0.0145,
         0.0136,  0.0150, -0.0140, -0.0281,  0.0229,  0.0165,  0.0167, -0.0099,
         0.0217, -0.0080, -0.0017,  0.0191, -0.0023,  0.0263,  0.0038,  0.0013,
        -0.0461,  0.0206,  0.0185,  0.0159, -0.0044, -0.0434, -0.0132, -0.0088,
        -0.0155, -0.0274, -0.0145,  0.0214, -0.0140,  0.0258,  0.0172,  0.0067,
         0.0310,  0.0111,  0.0256, -0.0094, -0.0121, -0.0242,  0.0108,  0.0004,
         0.0350,  0.0076, -0.0107, -0.0167,  0.0097, -0.0301, -0.0212, -0.0200,
        -0.0024,  0.0164, -0.0078,  0.0243, -0.0124, -0.0060,  0.0275,  0.0107,
        -0.0207, -0.0030, -0.0516, -0.0131, -0.0316, -0.0067, -0.0031,  0.0107,
        -0.0026, -0.0030,  0.0301,  0.0168,  0.0099, -0.0178,  0.0035, -0.0041,
         0.0131,  0.0295,  0.0147,  0.0294,  0.0084, -0.0092,  0.0070, -0.0111,
        -0.0244,  0.0100,  0.0023, -0.0112,  0.0111,  0.0236, -0.0228,  0.0173,
         0.0154,  0.0103, -0.0147, -0.0239, -0.0254, -0.0057, -0.0220, -0.0167,
         0.0148, -0.0063,  0.0204,  0.0072,  0.0172,  0.0115, -0.0267,  0.0068,
         0.0271, -0.0264, -0.0065, -0.0218, -0.0282,  0.0130,  0.0069, -0.0267,
        -0.0299, -0.0049, -0.0104, -0.0247, -0.0077,  0.0081, -0.0023, -0.0166,
        -0.0122, -0.0362, -0.0192,  0.0005,  0.0089,  0.0006,  0.0008, -0.0180,
        -0.0291, -0.0356,  0.0118,  0.0013,  0.0097,  0.0130,  0.0054,  0.0209,
         0.0222,  0.0107, -0.0158, -0.0064, -0.0237, -0.0320, -0.0518,  0.0082,
        -0.0271,  0.0060, -0.0198,  0.0017, -0.0292,  0.0183,  0.0315,  0.0063,
        -0.0007, -0.0017, -0.0289,  0.0114,  0.0258,  0.0097,  0.0090, -0.0074,
        -0.0084,  0.0153, -0.0130,  0.0029,  0.0294,  0.0154,  0.0253, -0.0284,
        -0.0226, -0.0229,  0.0192,  0.0158, -0.0212,  0.0028, -0.0227,  0.0170,
        -0.0010,  0.0063,  0.0328,  0.0190, -0.0085,  0.0003, -0.0143,  0.0102,
         0.0175, -0.0321,  0.0114,  0.0331,  0.0032, -0.0053,  0.0130, -0.0030,
         0.0040,  0.0223,  0.0152,  0.0302,  0.0001, -0.0127,  0.0007,  0.0156,
        -0.0092,  0.0115, -0.0174, -0.0240, -0.0297, -0.0244, -0.0017, -0.0123,
        -0.0152,  0.0187, -0.0135,  0.0048,  0.0238,  0.0124, -0.0302, -0.0127,
         0.0018, -0.0072,  0.0206, -0.0032,  0.0211,  0.0181,  0.0071, -0.0063],
       device='cuda:0'), tensor([1.0023, 1.0099, 1.0014, 0.9957, 1.0073, 1.0137, 1.0133, 1.0041, 1.0054,
        0.9928, 1.0024, 1.0163, 1.0105, 1.0178, 1.0092, 1.0003, 0.9956, 1.0058,
        1.0078, 0.9996, 1.0049, 0.9977, 0.9978, 1.0022, 0.9988, 1.0155, 0.9955,
        0.9957, 0.9925, 0.9995, 0.9872, 1.0091, 0.9930, 1.0139, 1.0080, 1.0012,
        1.0025, 0.9844, 1.0038, 0.9923, 0.9880, 0.9943, 0.9953, 1.0131, 0.9957,
        1.0090, 0.9955, 0.9989, 1.0016, 0.9970, 0.9934, 1.0100, 0.9946, 0.9882,
        1.0181, 0.9957, 1.0000, 0.9926, 0.9959, 1.0054, 1.0054, 1.0121, 0.9807,
        1.0067, 1.0050, 1.0018, 1.0131, 1.0083, 1.0121, 0.9853, 1.0138, 1.0056,
        1.0129, 1.0075, 1.0053, 0.9992, 0.9997, 1.0191, 1.0197, 1.0285, 0.9958,
        1.0080, 1.0266, 1.0132, 1.0034, 1.0029, 1.0115, 0.9948, 1.0020, 1.0087,
        0.9960, 0.9951, 1.0166, 1.0071, 0.9845, 1.0043, 0.9851, 0.9984, 1.0031,
        0.9952, 0.9971, 0.9937, 0.9981, 1.0067, 1.0013, 1.0032, 0.9950, 1.0048,
        0.9986, 1.0001, 1.0015, 1.0007, 1.0010, 1.0021, 1.0015, 1.0037, 0.9979,
        1.0002, 0.9790, 1.0051, 1.0080, 1.0034, 0.9997, 0.9974, 0.9982, 1.0333,
        1.0058, 0.9918, 1.0108, 0.9875, 1.0026, 1.0101, 1.0003, 0.9972, 1.0089,
        1.0076, 1.0049, 1.0010, 1.0127, 0.9992, 1.0065, 0.9954, 1.0118, 0.9887,
        1.0073, 0.9978, 1.0253, 1.0094, 1.0235, 1.0028, 1.0114, 0.9920, 0.9887,
        1.0157, 0.9982, 1.0064, 1.0094, 1.0043, 1.0047, 0.9964, 0.9989, 0.9870,
        1.0036, 1.0081, 1.0079, 0.9987, 1.0044, 1.0094, 1.0054, 0.9802, 0.9914,
        1.0011, 0.9967, 0.9924, 0.9838, 1.0108, 1.0163, 0.9966, 0.9941, 1.0099,
        1.0000, 1.0031, 1.0067, 1.0084, 1.0168, 0.9945, 0.9967, 0.9872, 1.0108,
        1.0020, 1.0150, 0.9958, 0.9988, 1.0093, 1.0042, 0.9870, 1.0037, 1.0130,
        0.9955, 1.0010, 1.0051, 1.0146, 1.0098, 1.0087, 0.9900, 1.0008, 1.0000,
        1.0111, 0.9993, 0.9746, 1.0044, 1.0085, 1.0122, 1.0081, 1.0043, 1.0032,
        1.0045, 1.0025, 1.0089, 1.0023, 1.0090, 1.0078, 1.0077, 1.0158, 0.9957,
        1.0117, 1.0082, 1.0069, 1.0024, 1.0146, 1.0106, 1.0087, 0.9959, 1.0001,
        0.9861, 1.0043, 0.9955, 1.0063, 1.0035, 1.0095, 0.9966, 1.0154, 1.0068,
        1.0077, 0.9880, 1.0038, 0.9914, 1.0109, 0.9991, 1.0095, 1.0210, 0.9867,
        0.9988, 1.0272, 0.9973, 1.0095], device='cuda:0'), tensor([ 2.1290e-03, -9.7056e-06,  3.2757e-03,  5.2694e-03,  5.6271e-03,
         1.8463e-02,  1.1601e-02,  7.2331e-03,  1.2939e-02,  1.0157e-02,
         6.9751e-03,  2.8860e-03,  7.2172e-03,  2.1671e-02,  5.5436e-03,
         1.2236e-02,  2.0047e-03,  1.6458e-02,  1.6299e-02,  5.6554e-03,
         6.9488e-03,  1.3834e-02,  5.9719e-03,  7.4337e-03,  6.4963e-03,
         5.7362e-03,  1.3454e-02,  4.0392e-03,  3.9729e-03,  6.6149e-03,
         8.2795e-03,  5.2825e-03,  7.9730e-03,  1.7383e-02,  5.2388e-03,
         2.2486e-03,  9.8062e-03,  3.1188e-03,  8.0457e-03,  5.9741e-03,
         3.8342e-03,  1.8287e-03,  6.0929e-03,  1.1434e-04,  4.4997e-03,
         1.2180e-02,  5.1845e-03,  6.1311e-03,  7.0267e-03,  9.4329e-03,
         1.2743e-02,  1.9468e-03,  2.5144e-03,  1.8776e-02,  2.1898e-02,
         7.1818e-03,  6.0729e-03,  7.8030e-03,  1.1730e-04,  2.7840e-03,
         5.5033e-03,  5.5357e-03,  3.1589e-03,  1.9570e-03,  1.3341e-02,
         1.8837e-02,  1.5354e-02,  1.8354e-02,  4.0189e-03,  1.3461e-02,
         6.6288e-03,  7.1222e-03,  1.4805e-02,  7.1105e-03,  1.4511e-02,
         6.6416e-03,  8.3131e-03,  9.2313e-03,  6.6505e-03,  1.4215e-02,
         5.8267e-03,  1.3884e-02,  5.1863e-03,  1.1199e-02,  6.3049e-03,
         6.2346e-03,  1.8987e-02,  9.4397e-03,  4.4409e-03,  1.4244e-02,
         1.1941e-02,  1.7573e-02,  5.9289e-03,  7.3703e-03,  8.3006e-03,
         1.0230e-02,  1.3708e-02,  5.8546e-03,  1.1468e-02,  4.2284e-03,
         4.5727e-03,  7.2159e-03,  4.3035e-03,  8.4441e-03,  9.7158e-03,
         1.5480e-02,  1.0758e-02,  9.2441e-03,  6.9361e-03,  4.6694e-03,
         6.1856e-03,  5.4607e-03,  3.1530e-03,  2.5751e-03,  5.2787e-03,
         4.8410e-03,  9.7514e-03,  1.3741e-02,  1.5328e-02,  1.2763e-02,
         1.6481e-03,  9.3918e-03,  1.8857e-03,  1.3416e-02,  8.1266e-03,
         1.4888e-03,  3.6472e-03,  4.8827e-03, -4.0244e-03,  5.6825e-03,
         7.8057e-03,  5.6647e-03,  9.2289e-03,  1.7942e-03,  3.8096e-03,
         1.2796e-02,  8.7248e-03,  1.1802e-04,  2.3151e-03,  7.7710e-03,
         2.8665e-03,  5.6457e-03,  7.9348e-03,  3.9503e-02,  3.5687e-03,
         5.0270e-03,  5.5668e-03,  6.5016e-03,  3.2846e-04,  1.0813e-02,
         1.4598e-02,  6.9270e-03,  4.8038e-03,  1.4829e-02,  1.1825e-03,
         1.5120e-02,  1.2769e-02,  2.2688e-03,  2.0995e-03,  3.3735e-03,
         7.5733e-03,  1.4705e-02,  1.9497e-03, -4.0065e-04,  1.2138e-02,
         3.5603e-03,  2.9942e-03,  1.1920e-02,  2.3981e-03,  1.3370e-02,
         5.8465e-03,  1.2311e-02,  2.8951e-03,  1.3191e-02,  1.0738e-02,
         4.2020e-03,  3.9759e-03,  3.2693e-03,  2.6399e-03,  9.1801e-03,
         1.1378e-02,  6.6346e-03,  1.5919e-02,  1.1656e-02,  7.2069e-03,
         4.8392e-03,  8.1671e-03,  1.0901e-02,  1.6719e-02,  6.2406e-03,
         6.2228e-03,  1.4720e-03,  2.8227e-03,  3.9798e-03,  6.4350e-03,
         6.4052e-03,  9.7825e-03,  3.2057e-03,  6.8691e-03,  6.2266e-03,
         1.7510e-02,  1.5146e-03,  7.1044e-03,  1.1117e-02,  1.0491e-02,
         3.4613e-03,  7.9678e-03,  1.3300e-02,  7.6685e-03,  1.7994e-02,
         1.0323e-02,  1.9367e-03, -7.0771e-04, -3.4760e-04,  6.7437e-03,
         1.1690e-02,  5.3811e-03,  4.4114e-03,  1.1487e-02,  1.0931e-02,
         6.0073e-03,  3.5156e-03,  1.5211e-02,  7.1657e-03,  4.3862e-03,
         2.6241e-03,  1.5332e-03,  1.3992e-02,  1.0716e-02,  6.5310e-03,
         3.3708e-04,  1.5652e-02,  6.6981e-03,  7.4561e-03,  4.7555e-03,
         5.4290e-03,  5.6992e-03,  8.0003e-04,  1.1590e-02,  1.5635e-03,
         4.5262e-03,  2.5043e-03,  3.4175e-03,  5.6596e-03,  9.3144e-03,
         8.3775e-03,  1.1622e-02,  2.2986e-03,  6.5386e-03,  1.7386e-02,
         3.8024e-03,  3.0455e-03,  6.0630e-03,  7.8288e-03,  1.0293e-02,
         5.3081e-03], device='cuda:0'), tensor([[[-0.0313],
         [-0.0863],
         [-0.0254],
         ...,
         [ 0.0812],
         [-0.0737],
         [ 0.0234]],

        [[-0.0909],
         [ 0.0839],
         [ 0.0247],
         ...,
         [-0.1559],
         [-0.0204],
         [-0.0031]],

        [[-0.0167],
         [ 0.0272],
         [ 0.0458],
         ...,
         [ 0.0738],
         [-0.0168],
         [ 0.0272]],

        ...,

        [[-0.0063],
         [ 0.1177],
         [ 0.1501],
         ...,
         [ 0.0152],
         [-0.0559],
         [-0.0471]],

        [[-0.1192],
         [-0.0540],
         [ 0.0378],
         ...,
         [ 0.0055],
         [-0.0496],
         [ 0.1035]],

        [[-0.0954],
         [ 0.1207],
         [ 0.0912],
         ...,
         [-0.0936],
         [ 0.0087],
         [ 0.0292]]], device='cuda:0'), torch.tensor([ 0.0006,  0.0280,  0.0390,  0.0524,  0.0584, -0.0103,  0.0102, -0.0233,
         0.0048,  0.0016, -0.0586, -0.0373], device='cuda:0')]}


NameError: name 'tensor' is not defined

In [6]:
checkpoint_path = '/home/cam/Documents/diffusion_policy_cam/no_sync/checkpoints/T_checkpoint_(SV)(SV)SS_epoch_139.pth'

checkpoint = torch.load(checkpoint_path)



In [8]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])

In [11]:
# Load model state
noise_pred_net.load_state_dict(checkpoint['model_state_dict'])

# Load optimizer state
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Load LR scheduler state (if applicable)
lr_scheduler.load_state_dict({'base_lrs': [0.0001],
 'last_epoch': 99800,
 'verbose': False,
 '_step_count': 99801,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0],
 'lr_lambdas': [None]})

# Load EMA model state
ema.load_state_dict(ema

# Optionally, retrieve other information such as epoch and loss
epoch_loaded = checkpoint['epoch']
loss_loaded = checkpoint['loss']